This note book is mainly build by Zongyue ZHANG and Yifei Li

# Environment perparing

In [ ]:
# %matplotlib inline Notice: the import may take a longtime.
from init_nitrogen import *

In [ ]:
fpei37 = r"C:\Users\DELL\学术\ecoinvent 3.7.1_consequential_ecoSpold02\datasets"
if 'ecoinvent 3.7.1 consequential' in bw.databases:
    print("Database has already been imported")
else:
    ei371consequentia = bw.SingleOutputEcospold2Importer(fpei37, 'ecoinvent 3.7.1 consequential')
    ei371consequential.apply_strategies()
    ei371consequential.statistics()
    ei371consequential.drop_unlinked(i_am_reckless=True)
    ei371consequential.statistics()
    ei371consequential.write_database()

In [ ]:
#ei371consequential=bw.SingleOutputEcospold2Importer('C:/Users/DELL/学术/ecoinvent 3.7.1_consequential_ecoSpold02/datasets','ecoinvent 3.7.1 consequential')
#ei371consequential.apply_strategies()
#ei371consequential.statistics()
#ei371consequential.drop_unlinked(i_am_reckless=True)

In [ ]:
#ei371consequential.statistics()
#ei371consequential.write_database()

In [ ]:
from brightway2 import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
%matplotlib inline

In [ ]:
import brightway2 as bw

In [ ]:
import pyprind

In [ ]:
def CA_Elem_Flow(lca, ratio = 0.8, length_max = 10):
    #EF analysis
    CAe = bwa.ContributionAnalysis().annotated_top_emissions(lca)
    df_CAe = pd.DataFrame(CAe, columns = ['lca_score','inventory_amount', 'activity'])
    index = [act[2]['name'] for act in CAe]
    df_CAe['activity'] = index
    
    df_CAe = df_CAe.groupby('activity').sum()
    df_CAe = df_CAe.sort_values(by = 'lca_score', ascending = False)
    
    impact = lca.score
    impact_i = 0
    i = 0
    while impact_i < ratio * impact:
        impact_i += df_CAe.iloc[i]['lca_score']
        i+= 1
        
    i = np.min([i, length_max])
    df_CAe = df_CAe.iloc[0:i]
    #display(df_CAe)
    df_CAe.loc['Other'] = [impact - df_CAe.lca_score.sum(), None]
    df_CAe['PCT(%)'] = df_CAe.lca_score / impact * 100
    #df_CAe.index = df_CAe.activity    
    return df_CAe

In [ ]:
def CA_Process(lca, ratio = 0.8, length_max = 10):
    #Process analysis
    CAp = bwa.ContributionAnalysis().annotated_top_processes(lca)
    df_CAp = pd.DataFrame(CAp, columns = ['lca_score','inventory_amount', 'activity'])
    index = [act[2]['name'] for act in CAp]
    df_CAp['activity'] = index
    
    df_CAp = df_CAp.groupby('activity').sum()
    df_CAp = df_CAp.sort_values(by = 'lca_score', ascending = False)
    
    impact = lca.score
    impact_i = 0
    i = 0
    while impact_i < ratio * impact:
        impact_i += df_CAp.iloc[i]['lca_score']
        i+= 1
        if i >= np.min([length_max, df_CAp.shape[0]]):
            break
        
    i = np.min([i, length_max])
    df_CAp = df_CAp.iloc[0:i]
    #display(df_CAe)
    df_CAp.loc['Rest'] = [impact - df_CAp.lca_score.sum(), None]
    df_CAp['PCT(%)'] = df_CAp.lca_score / impact * 100
    #df_CAe.index = df_CAe.activity    
    return df_CAp



In [ ]:
def Contribution_Analysis_one_impact_category_noact(act, method, ratio = 0.8, length_max = 10):
    lca = bw.LCA({act:1}, method)
    lca.lci()
    lca.lcia()    
    
    df_CAe = CA_Elem_Flow(lca = lca, ratio = ratio, length_max = length_max)
    df_CAp = CA_Process(lca = lca, ratio = ratio, length_max = length_max)

    fig, ax  = plt.subplots(2,1, figsize = (16, 7), sharex = True)
    df_CAe[['PCT(%)']].T.plot(ax = ax[0],
                              kind = 'barh', 
                              stacked = True, 
                              alpha = 0.6, rot = 0,fontsize=22) #,align='edge'
    ax[0].set_title('Contribution of biosphere elementary flows',fontsize=22,fontname="Times New Roman")
    #ax[0].set_xlabel('Pourcentage (%)',fontsize=22,fontname="Times New Roman")
    ax[0].legend(bbox_to_anchor = (1,1),fontsize=18,loc='best', frameon=False)
    #ax[0].set_ylabel('Substances')
    
    df_CAp[['PCT(%)']].T.plot(ax = ax[1], 
                                  kind = 'barh', 
                                  stacked = True, 
                                  alpha = 0.6, rot  = 0,fontsize=22)
    ax[1].legend(bbox_to_anchor = (1,1),fontsize=18,loc='best', frameon=False)
    ax[1].set_title('Contribution of technosphere activities',fontsize=22,fontname="Times New Roman")
    #ax[1].set_xlabel('Pourcentage (%)',fontsize=22)
    #ax[1].set_xlabel('Procédés')

    #plt.tight_layout()
    m = bw.Method(method)
    plt.suptitle(' Categories analyzed : %s\n Total score = %0.2g %s/MJ bio-jet fuel'%(str(lca.method),
                                                                   lca.score,
                                                                   m.metadata['unit']), 
                 fontsize = 22,
                 y = 1.07)
    return fig       

In [ ]:
def Contribution_Analysis_one_impact_category_noact_longlegend(act, method, ratio = 0.8, length_max = 10):
    lca = bw.LCA({act:1}, method)
    lca.lci()
    lca.lcia()    
    
    df_CAe = CA_Elem_Flow(lca = lca, ratio = ratio, length_max = length_max)
    df_CAp = CA_Process(lca = lca, ratio = ratio, length_max = length_max)

    fig, ax  = plt.subplots(2,1, figsize = (16, 7), sharex = True)
    df_CAe[['PCT(%)']].T.plot(ax = ax[0],
                              kind = 'barh', 
                              stacked = True, 
                              alpha = 0.6, rot = 0,fontsize=22) #,align='edge'
    ax[0].set_title('Contribution of biosphere elementary flows',fontsize=22,fontname="Times New Roman")
    #ax[0].set_xlabel('Pourcentage (%)',fontsize=22,fontname="Times New Roman")
    ax[0].legend(bbox_to_anchor = (1,1.1),fontsize=16,loc='best')
    #ax[0].set_ylabel('Substances')
    
    df_CAp[['PCT(%)']].T.plot(ax = ax[1], 
                                  kind = 'barh', 
                                  stacked = True, 
                                  alpha = 0.6, rot  = 0,fontsize=22)
    ax[1].legend(bbox_to_anchor = (1,1),fontsize=16,loc='best')
    ax[1].set_title('Contribution of technosphere activities',fontsize=22)
    #ax[1].set_xlabel('Pourcentage (%)',fontsize=22)
    #ax[1].set_xlabel('Procédés')
    mpl.rc('font',family='Times New Roman')
    #plt.tight_layout()
    m = bw.Method(method)
    plt.suptitle(' Categories analyzed : %s\n Total score = %0.2g %s/MJ bio-jet fuel'%(str(lca.method),
                                                                   lca.score,
                                                                   m.metadata['unit']), 
                 fontsize = 22,
                 y = 1.07)
    return fig       

In [ ]:
def Contribution_Analysis_one_impact_category(act, method, ratio = 0.8, length_max = 10):
    lca = bw.LCA({act:1}, method)
    lca.lci()
    lca.lcia()    
    
    df_CAe = CA_Elem_Flow(lca = lca, ratio = ratio, length_max = length_max)
    df_CAp = CA_Process(lca = lca, ratio = ratio, length_max = length_max)

    fig, ax  = plt.subplots(2,1, figsize = (16, 7), sharex = True)
    df_CAe[['percentage']].T.plot(ax = ax[0],
                              kind = 'barh', 
                              stacked = True, 
                              alpha = 0.6, rot = 90)
    ax[0].set_title('Contribution par substances',fontsize=22,fontname="Times New Roman")
    ax[0].set_xlabel('Pourcentage (%)')
    ax[0].legend(bbox_to_anchor = (1.173,1))
    #ax[0].set_ylabel('Substances')
    
    df_CAp[['PCT(%)']].T.plot(ax = ax[1], 
                                  kind = 'barh', 
                                  stacked = True, 
                                  alpha = 0.6, rot  = 90)
    ax[1].legend(bbox_to_anchor = (1,1))
    ax[1].set_title('Contribution par activités')
    ax[1].set_xlabel('Pourcentage (%)')
    #ax[1].set_xlabel('Procédés')

    #plt.tight_layout()
    m = bw.Method(method)
    plt.suptitle('Analyse de contribution de l\'activité "%s" \n pour la catégorie : %s\n Impact total = %0.2g %s/MJ H2'%(act['name'],
                                                                                        str(lca.method),
                                                                   lca.score,
                                                                   m.metadata['unit']), 
                 fontsize = 20,
                 y = 1.07)
    return fig       

In [ ]:
def Contribution_Analysis_all_impact_category(act, methods, ratio = 0.8, length_max = 10):
    for method in methods:
        Contribution_Analysis_one_impact_category(act = act,
                                                  method = method,
                                                  ratio = ratio,
                                                  length_max = length_max)
        
    lca = bw.LCA({act:1}, method)
    lca.lci()
    lca.lcia()    
    
    df_CAe = CA_Elem_Flow(lca = lca, ratio = ratio, length_max = length_max)
    df_CAp = CA_Process(lca = lca, ratio = ratio, length_max = length_max)

    fig, ax  = plt.subplots(2,1, figsize = (16, 7), sharex = True)
    df_CAe[['percentage']].T.plot(ax = ax[0],
                              kind = 'barh', 
                              stacked = True, 
                              alpha = 0.6, rot = 90)
    ax[0].set_title('Contribution par substances')
    ax[0].set_xlabel('Pourcentage (%)')
    ax[0].legend(bbox_to_anchor = (1.173,1))
    #ax[0].set_ylabel('Substances')
    
    df_CAp[['percentage']].T.plot(ax = ax[1], 
                                  kind = 'barh', 
                                  stacked = True, 
                                  alpha = 0.6, rot  = 90)
    ax[1].legend(bbox_to_anchor = (1,1))
    ax[1].set_title('Contribution par activités')
    ax[1].set_xlabel('Pourcentage (%)')
    #ax[1].set_xlabel('Procédés')

    #plt.tight_layout()
    m = bw.Method(method)
    plt.suptitle('Analyse de contribution de l\'activité "%s" \n pour la catégorie : %s\n Impact total = %0.3g %s/MJ H2'%(act['name'],
                                                                                        str(lca.method),
                                                                   lca.score,
                                                                   m.metadata['unit']), 
                 fontsize = 20,
                 y = 1.07)
    return fig       

In [ ]:
import plotly.graph_objects as go
from bw2calc import GraphTraversal

def plot_sankey(act, method, cutoff,
                #i = 0, n = 1
               ):
    
    lca = bw.LCA({act:1}, method)
    lca.lci()
    lca.lcia()
    impact = lca.score
    unit = bw.Method(method).metadata['unit']
    
    gt = GraphTraversal().calculate({act:1}, method = method, cutoff=cutoff)

    acts = gt["lca"].activity_dict
    id_to_key = {v:k for k, v in acts.items()}
    #labels = {k: bw.get_activity(v)["name"] for k in gt["nodes"].values()}
    ids = list(gt["nodes"].keys())
    labels = [bw.get_activity(id_to_key[id])["name"] for id in ids[1:]]
    labels = ["root"] + labels
    id_to_idx = {id : idx for idx, id in enumerate(ids)}
    edges = gt["edges"]
    edges_plot = dict(
        target = [id_to_idx[edge["to"]] for edge in edges],
        source = [id_to_idx[edge["from"]] for edge in edges],
        value = [edge["impact"]/impact * 100 for edge in edges])
    
    fig = go.Figure(data = [go.Sankey(valueformat = '.0f',
                                valuesuffix = ' %',
                                node = dict(label=labels),
                                link = edges_plot,
                                #domain = {'y': [(n-i-1)/n, (n-i)/n]}
                                     ),
                           ],
                   )
    
    if True:
        fig.update_layout(title_text = "%s : %0.3g %s/MJ H2"%(str(method), 
                                                              impact, 
                                                              unit),
                  title_x = 0.5,
                  font_size = 12,
                  width = 800,
                  height = 500)
    
    return fig

In [ ]:
def list_add(a,b):
    c = []
    for i in range(len(a)):
        c.append(a[i]+b[i])
    return c

In [ ]:
def list_minus(a,b):
    c = []
    for i in range(len(a)):
        c.append(a[i]-b[i])
    return c

In [ ]:
def list_divide(a,b):
    c = []
    for i in range(len(a)):
        c.append(a[i]/b[i])
    return c

In [ ]:
def list_plus(a,b):
    c = []
    for i in range(len(a)):
        c.append(a[i]*b[i])
    return c

In [ ]:
def list_xy(a,b):
    c = []
    for i in range(len(a)):
        c.append((a[i],b[i]))
    return c

In [ ]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [ ]:
SET_USER_DB('ENFI_jetfuel')

In [ ]:
USER_DB

In [ ]:
databases

In [ ]:
#bw.methods

In [ ]:
#list(bw.methods)

In [ ]:
#bdb = bw.Database('ecoinvent 3.7.1 apos')
#bdb.deregister()

# Zoom in the bioshpere3

In [ ]:
bio = bw.Database('biosphere3')

In [ ]:
bio.metadata

In [ ]:
#biodf = pd.DataFrame(bio)
#biodf.to_excel(r'C:\Users\DELL\学术\bioshpere3_v3.xlsx', sheet_name='bioshpere3', index = True)

In [ ]:
random_biosphere = my_bio.random()
random_biosphere

In [ ]:
random_biosphere.as_dict()

In [ ]:
test= [act for act in bio if 'Laterite' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            #and 'urban air close to ground' in str(act['categories'])
      ][0]
test.as_dict()

# Explaining of the GNFA thoery

In [ ]:
import pandas as pd
import numpy as np
from holoviews import opts

In [ ]:
import holoviews as hv
import plotly.graph_objects as go
import plotly.express as pex
import bokeh

In [ ]:
hv.extension('bokeh')

In [ ]:
N_flow = pd.read_excel("C:/Users/DELL/brightway2/example_N_flow.xlsx", 'Sheet1')
N_flow.head()

In [ ]:
%%opts Sankey (edge_color="Source"  edge_line_width=2 node_cmap="tab20")
%%opts Sankey (node_alpha=1.0 edge_hover_fill_color="red")
%%opts Sankey [node_sort=False label_position='right' node_width=30 node_sort=True ]
%%opts Sankey [title="Life cycle N flow, Case 1, (100gN per function unit) ", width=1000, height=500, fontsize=14]
%%opts Sankey [margin=0 padding=0 bgcolor="white"]
#%%opts Sankey [node_width=30]
#%%opts Sankey [label_text_font_size=15]
# label_text_font_size
#

def hook(plot, element):
    plot.handles['text_1_glyph'].text_font = 'helvetica'
    plot.handles['text_1_glyph'].text_font_size = '12pt'
    #plot.handles['text_2_glyph'].text_font = 'times'
    #plot.handles['text_2_glyph'].text_font_size = '12pt'
    #plot.handles['label_text_font'].text_font = 'Times New Roman'
    
tt = hv.Sankey(N_flow, kdims=["Source", "Dest"], vdims=["Count"])

tt.opts(opts.Sankey(edge_color=hv.dim('A').str(),label_position='inner'))
tt.opts(height=600,width=800)
tt.opts(hooks=[hook])

#tt.opts(opts.Sankey(edge_color=hv.dim('A').str(),label_position='inner'))
#tt.opts.title.text_font = "times"


## plotly method

In [ ]:
import plotly.graph_objects as go
import kaleido 

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = ["Input from Soil-lithosphere (20)", "Input from Hydrosphere (12)", "Input from Atmosphere (68)", 
               "Reactive N Input (20)", "Inert N Input (80)", "System Life Cycle Boundary (100)",
               "Reactive N Onput (15)", "Inert N Output (15)", "Output to Soil-lithosphere (10)",
              "Output to Hydrosphere (9)", "Output to Atmosphere (11)","System N Accumulation (70)"],
      color = "paleturquoise"
    ),
    link = dict(
      source = [0, 1, 2, 0, 1, 2, 3, 4, 5, 5, 6, 6, 6, 7, 7, 7, 5], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = [3, 3, 3, 4, 4, 4, 5, 5, 6, 7, 8, 9,10, 8, 9,10,11],
      value = [10, 2, 8,10,10,60,20,80,15, 15, 5, 5,5,5,4,6,70]
  ))])

fig.update_layout(title_text="GNFA theory-based life cycle N flow, h-Case 1 (100 gN per functional unit)", font_size=30, font_family='Times New Roman',
                 autosize=True, width=750*2.2, height=450*1.45)
#mpl.rcParams["figure.dpi"] = 900

#import plotly.io as pio
#save a figure of 300dpi, with 1.5 inches, and  height 0.75inches
#pio.write_image(fig, "test.png", width=1.5*450, height=0.75*450, scale=1)

fig.show()


In [ ]:
#fig.write_image(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\test1.png', scale=3, engine='kaleido')

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = ["Input from Soil-lithosphere (25)", "Input from Hydrosphere (14)", "Input from Atmosphere (61)", 
               "Reactive N Input (30)", "Inert N Input (70)", "System Life Cycle Boundary (100)",
               "Reactive N Onput (15)", "Inert N Output (5)", "Output to Soil-lithosphere (7.5)",
              "Output to Hydrosphere (6.5)", "Output to Atmosphere (6)","System N Accumulation (80)"],
      color = "paleturquoise"
    ),
    link = dict(
      source = [0, 1, 2, 0, 1, 2, 3, 4, 5, 5, 6, 6, 6, 7, 7, 7, 5], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = [3, 3, 3, 4, 4, 4, 5, 5, 6, 7, 8, 9,10, 8, 9,10,11],
      value = [15, 4, 11,10,10,50,30,70,15, 5, 5, 5, 5,2.5,1.5,1,80]
  ))])

fig.update_layout(title_text="GNFA theory-based life cycle N flow, h-Case 2 (100 gN per functional unit)", font_size=30, font_family='Times New Roman',
                 autosize=True, width=750*2.2, height=450*1.45)
#mpl.rcParams["figure.dpi"] = 900

#import plotly.io as pio
#save a figure of 300dpi, with 1.5 inches, and  height 0.75inches
#pio.write_image(fig, "test.png", width=1.5*450, height=0.75*450, scale=1)

fig.show()


##  ternary N flow sustainability indicators demonstration

In [ ]:
#N_human= newFloatParam(
#    "N_human", 
#    distrib=DistributionType.FIXED,
#    default=62*0.03, 
#    #min=0.301, max=0.49,
#    label_fr="nitrogen content in global average human body mass", 
#    group="N sustainability", 
#    unit="kg N") 

In [ ]:
N_human=62*0.03

In [ ]:
df_NFI_IC = pd.DataFrame()

df_NFI_IC['NIF_IN'] = [80/1000,70/1000]
df_NFI_IC['NIF_ON'] = [20/1000,30/1000]

df_NFI_IC['NOF_IN'] = [15/1000,5/1000]
df_NFI_IC['NOF_ON'] = [15/1000,15/1000]

#df_NFI_IC['NFI_Nacc_x(per)'] = [80/1000]
#df_NFI_IC['NFI_Nacc_y(per)'] = [80/1000]

#df_NFI_IC['GWP_x_m'] = MC_GWP_scores
#df_NFI_IC['WF_y_m'] = MC_water_scores

df_NFI_IC

In [ ]:
#test = df_NFI_IC.values.tolist()

#GWP_SV=test[0]
#WF_SV =test[1]
NIF_IN_SV= df_NFI_IC.iloc[:,0].values.tolist()
NIF_ON_SV=df_NFI_IC.iloc[:,1].values.tolist()
NOF_IN_SV=df_NFI_IC.iloc[:,2].values.tolist()
NOF_ON_SV=df_NFI_IC.iloc[:,3].values.tolist()
NIF_ON_SV

In [ ]:
NIF_SV=list_add(NIF_IN_SV,NIF_ON_SV)
NIF_SV

In [ ]:
NOF_SV=list_add(NOF_IN_SV,NOF_ON_SV)
NOF_SV

In [ ]:
Nacc_SV=list_minus(NIF_SV,NOF_SV)
Nacc_SV

##### SV_inert N flow index

###### x indices

In [ ]:
NFI_IN_x_N_SV=list_minus(NOF_IN_SV,NIF_IN_SV)
NFI_IN_x_N_SV

In [ ]:
NFI_IN_x_SV=list_divide(NFI_IN_x_N_SV,NIF_SV)
NFI_IN_x_SV

In [ ]:
percentage_list = [100]*(len(NFI_IN_x_SV))
#print(cf_key_list)
len(percentage_list)
NFI_IN_x_per_SV=list_plus(NFI_IN_x_SV,percentage_list)
NFI_IN_x_per_SV

In [ ]:
#NFI_IN_x_per_CI=mean_confidence_interval(NFI_IN_x_per, confidence=0.95)
#NFI_IN_x_per_CI

In [ ]:
#NFI_IN_x_per_CI[0]

In [ ]:
#NFI_IN_x_per95 = []

In [ ]:
#i = 0

#while NFI_IN_x_per[i] >= NFI_IN_x_per_CI[1] and NFI_IN_x_per[i] <= NFI_IN_x_per_CI[2]:
#  NFI_IN_x_per95.append(NFI_IN_x_per[i])
##  if NFI_IN_x_per[i] >= NFI_IN_x_per_CI[1] and NFI_IN_x_per[i] <= NFI_IN_x_per_CI[2]:
##    print(NFI_IN_x_per[i])
#  i += 1
#  if i > len(NFI_IN_x_per):
#    break

#print(NFI_IN_x_per95)

###### y indices

In [ ]:
NFI_IN_y_N_SV=list_minus(NOF_IN_SV,NIF_IN_SV)
NFI_IN_y_N_SV

In [ ]:
Nhuman_list = [N_human]*(len(NFI_IN_y_N_SV))
#print(cf_key_list)
len(Nhuman_list)
NFI_IN_y_SV=list_divide(NFI_IN_y_N_SV,Nhuman_list)
NFI_IN_y_SV

In [ ]:
percentage_list = [100]*(len(NFI_IN_y_SV))
#print(cf_key_list)
len(percentage_list)
NFI_IN_y_per_SV=list_plus(NFI_IN_y_SV,percentage_list)
NFI_IN_y_per_SV

##### reacive N flow index

###### x indices

In [ ]:
NFI_ON_x_N_SV=list_minus(NOF_ON_SV,NIF_ON_SV)
NFI_ON_x_N_SV

In [ ]:
NFI_ON_x_SV=list_divide(NFI_ON_x_N_SV,NIF_SV)
NFI_ON_x_SV

In [ ]:
percentage_list = [100]*(len(NFI_ON_x_SV))
#print(cf_key_list)
len(percentage_list)
NFI_ON_x_per_SV=list_plus(NFI_ON_x_SV,percentage_list)
NFI_ON_x_per_SV

###### y indices

In [ ]:
NFI_ON_y_N_SV=list_minus(NOF_ON_SV,NIF_ON_SV)
NFI_ON_y_N_SV

In [ ]:
Nhuman_list = [N_human]*(len(NFI_ON_y_N_SV))
#print(cf_key_list)
len(Nhuman_list)
NFI_ON_y_SV=list_divide(NFI_ON_y_N_SV,Nhuman_list)
NFI_ON_y_SV

In [ ]:
percentage_list = [100]*(len(NFI_ON_y_SV))
#print(cf_key_list)
len(percentage_list)
NFI_ON_y_per_SV=list_plus(NFI_ON_y_SV,percentage_list)
NFI_ON_y_per_SV

In [ ]:
#test=list_xy(NFI_IN_x_per,NFI_IN_y_per)
#test

##### N accumulation

###### x indices

In [ ]:
NFI_Nacc_x_N_SV=Nacc_SV
NFI_Nacc_x_N_SV

In [ ]:
NFI_Nacc_x_SV=list_divide(NFI_Nacc_x_N_SV,NIF_SV)
NFI_Nacc_x_SV

In [ ]:
percentage_list = [100]*(len(NFI_Nacc_x_SV))
#print(cf_key_list)
len(percentage_list)
NFI_Nacc_x_per_SV=list_plus(NFI_Nacc_x_SV,percentage_list)
NFI_Nacc_x_per_SV

###### y indices

In [ ]:
NFI_Nacc_y_N_SV=Nacc_SV
NFI_Nacc_y_N_SV

In [ ]:
Nhuman_list = [N_human]*(len(NFI_Nacc_y_N_SV))
#print(cf_key_list)
len(Nhuman_list)
NFI_Nacc_y_SV=list_divide(NFI_Nacc_y_N_SV,Nhuman_list)
NFI_Nacc_y_SV

In [ ]:
percentage_list = [100]*(len(NFI_Nacc_y_SV))
#print(cf_key_list)
len(percentage_list)
NFI_Nacc_y_per_SV=list_plus(NFI_Nacc_y_SV,percentage_list)
NFI_Nacc_y_per_SV

In [ ]:
#test=list_xy(NFI_IN_x_per,NFI_IN_y_per)
#test

In [ ]:
df_SV = pd.DataFrame()

#df_SV['GWP_SV'] = GWP_SV
#df_SV['WF_SV'] = WF_SV

df_SV['NFI_IN_x_per_SV'] = NFI_IN_x_per_SV
df_SV['NFI_IN_y_per_SV'] = NFI_IN_y_per_SV

df_SV['NFI_ON_x_per_SV'] = NFI_ON_x_per_SV
df_SV['NFI_ON_y_per_SV'] = NFI_ON_y_per_SV

df_SV['NFI_Nacc_x_per_SV'] = NFI_Nacc_x_per_SV
df_SV['NFI_Nacc_y_per_SV'] = NFI_Nacc_y_per_SV

df_SV

In [ ]:
#df_NFI.to_excel(r'C:\Users\DELL\学术\N sustainability\demo_panneau1_NFI.xlsx', sheet_name='panneau1_NFI', index = True)

In [ ]:
import scipy

In [ ]:
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
import matplotlib as mpl 
from scipy.stats import gmean

In [ ]:
def eigsorted(cov):
    vals, vecs = np.linalg.eigh(cov)
    order = vals.argsort()[::-1]
    return vals[order], vecs[:,order]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def mscatter(x,y,ax=None, m=None, **kw):
    import matplotlib.markers as mmarkers
    if not ax: ax=plt.gca()
    sc = ax.scatter(x,y,**kw)
    if (m is not None) and (len(m)==len(x)):
        paths = []
        for marker in m:
            if isinstance(marker, mmarkers.MarkerStyle):
                marker_obj = marker
            else:
                marker_obj = mmarkers.MarkerStyle(marker)
            path = marker_obj.get_path().transformed(
                        marker_obj.get_transform())
            paths.append(path)
        sc.set_paths(paths)
    return sc

In [ ]:
randomlist = random.sample(range(10, 30), 5)
randomlist

In [ ]:
import numpy
plt.figure(figsize=(9,7))

import matplotlib as mpl
#myfont1 = mpl.font_manager.FontProperties(fname='/System/Library/Fonts/Supplemental/Arial Italic.ttf')     
#myfont2 = mpl.font_manager.FontProperties(fname='C:/Users/DELL/AppData/Local/Microsoft/Windows/Fonts/TimesNewRomanPSBoldMT.ttf')
#test_x=random.sample(range(0, 1000), 12)
#test_y=random.sample(range(0, 1000), 12)

#inert nitrogen flow index perpare
NFI_IN_x_plotC1 = NFI_IN_x_per_SV[0]
NFI_IN_y_plotC1 = NFI_IN_y_per_SV[0]

NFI_IN_x_plotC2 = NFI_IN_x_per_SV[1]
NFI_IN_y_plotC2 = NFI_IN_y_per_SV[1]

#NFI_IN_x_plotMV = NFI_IN_x_per_SV[2]
#NFI_IN_y_plotMV = NFI_IN_y_per_SV[2]


#reactive nitrogen flow index perpare
NFI_ON_x_plotC1 = NFI_ON_x_per_SV[0]
NFI_ON_y_plotC1 = NFI_ON_y_per_SV[0]

NFI_ON_x_plotC2 = NFI_ON_x_per_SV[1]
NFI_ON_y_plotC2 = NFI_ON_y_per_SV[1]

#NFI_ON_x_plotMV = NFI_ON_x_per_SV[2]
#NFI_ON_y_plotMV = NFI_ON_y_per_SV[2]


# nitrogen accumulation index perpare
NFI_Nacc_x_plotC1 = NFI_Nacc_x_per_SV[0]
NFI_Nacc_y_plotC1 = NFI_Nacc_y_per_SV[0]

NFI_Nacc_x_plotC2 = NFI_Nacc_x_per_SV[1]
NFI_Nacc_y_plotC2 = NFI_Nacc_y_per_SV[1]

#NFI_Nacc_x_plotMV = NFI_Nacc_x_per_SV[2]
#NFI_Nacc_y_plotMV = NFI_Nacc_y_per_SV[2]

#carbon footprint and waterfootprint perpare
#GWP_SV_x_plotMM = GWP_SV[0]
#WF_SV_y_plotMM  =  WF_SV[0]

#GWP_SV_x_plotME = GWP_SV[1]
#WF_SV_y_plotME  =  WF_SV[1]

#GWP_SV_x_plotMV = GWP_SV[2]
#WF_SV_y_plotMV  =  WF_SV[2]

Size_legend1=50
# plotting point shape legend 
plt.scatter(x=NFI_IN_x_plotC1, y=NFI_IN_y_plotC1, c = 'black', marker = 'o', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotC2, y=NFI_IN_y_plotC2, c = 'black', marker = 'x', s = Size_legend1)
#plt.scatter(x=NFI_IN_x_plotMV, y=NFI_IN_y_plotMV, c = 'black', marker = 'v', s = Size_legend1)

#plottingg colour legend
p1=plt.scatter(x=NFI_IN_x_plotC1, y=NFI_IN_y_plotC1, c = 'green', marker = 'o', s = 10)
p2=plt.scatter(x=NFI_IN_x_plotC1, y=NFI_IN_y_plotC1, c = 'blue', marker = 'o', s = 10)
p3=plt.scatter(x=NFI_IN_x_plotC1, y=NFI_IN_y_plotC1, c = 'red', marker = 'o', s = 10)
p4=plt.scatter(x=NFI_IN_x_plotC1, y=NFI_IN_y_plotC1, c = 'fuchsia', marker = 'o', s = 10)

#plotting inert nitrogen flow index
plt.scatter(x=NFI_IN_x_plotC1, y=NFI_IN_y_plotC1, c = 'green', marker = 'o', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotC2, y=NFI_IN_y_plotC2, c = 'green', marker = 'x', s = Size_legend1+60)
#plt.scatter(x=NFI_IN_x_plotMV, y=NFI_IN_y_plotMV, c = 'green', marker = 'v', s = Size_legend1)

#plotting reactive nitrogen flow index
plt.scatter(x=NFI_ON_x_plotC1, y=NFI_ON_y_plotC1, c = 'blue', marker = 'o', s = Size_legend1)
plt.scatter(x=NFI_ON_x_plotC2, y=NFI_ON_y_plotC2, c = 'blue', marker = 'x', s = Size_legend1+60)
#plt.scatter(x=NFI_ON_x_plotMV, y=NFI_ON_y_plotMV, c = 'blue', marker = 'v', s = Size_legend1)

#plotting nitrogen accumulation flow index
plt.scatter(x=NFI_Nacc_x_plotC1, y=NFI_Nacc_y_plotC1, c = 'red', marker = 'o', s = Size_legend1)
plt.scatter(x=NFI_Nacc_x_plotC2, y=NFI_Nacc_y_plotC2, c = 'red', marker = 'x', s = Size_legend1+60)
#plt.scatter(x=NFI_Nacc_x_plotMV, y=NFI_Nacc_y_plotMV, c = 'red', marker = 'v', s = Size_legend1)

#plotting carbon footprint and waterfootprint
#plt.scatter(x=GWP_SV_x_plotC1, y=WF_SV_y_plotC1, c = 'fuchsia', marker = '1', s = Size_legend1)
#plt.scatter(x=GWP_SV_x_plotME, y=WF_SV_y_plotME, c = 'fuchsia', marker = 'x', s = Size_legend1+60)
#plt.scatter(x=GWP_SV_x_plotMV, y=WF_SV_y_plotMV, c = 'fuchsia', marker = 'v', s = Size_legend1)

#col_g=["green","green","green","green","green","green","green","green","green","green","green","green"]
markers = ["1","x","v","2","^","o","s","p","P","*","h","+"]

#size = [10,15,20,30,40,50,60,70,80,85,90,95]  
#for m in enumerate(np.unique(markers)):  #c=col_g[col_g==c],
    #plt.scatter(NFI_IN_x_per_SV[:],NFI_IN_y_per_SV[:],c=col_g[col_g==c], marker=markers[i])
#    plt.scatter(test_x[:],test_y[:],marker=markers[markers==m],color='green')


#sns.scatterplot(data=df_SV, x='NFI_IN_x_per_SV', y='NFI_IN_y_per_SV',color='green',s=25)
#sns.scatterplot(data=df_SV, x='GWP_SV', y='WF_SV',color='white',s=1, marker=',')
#sns.scatterplot(data=df_SV, x='NFI_ON_x_per_SV', y='NFI_ON_y_per_SV',color='black',s=25, marker='>')
#sns.scatterplot(data=df_SV, x='NFI_Nacc_x_per_SV', y='NFI_Nacc_y_per_SV',color='red',s=25, marker=',')
#sns.scatterplot(data=df_SV, x='GWP_SV', y='WF_SV',color='fuchsia',s=25, marker=',')
mpl.rc('font',family='Times New Roman')

legend1=plt.legend(["h-Case1", "h-Case2",#"Basline scenario", "Mass maize, Economic value fuel",
            #"Energy maize, Mass fuel", "Energy maize, Energy fuel", "Energy maize, Economic value fuel",
            #"Economic value maize, Mass fuel", "Economic value maize, Energy fuel", "Economic value maize, Economic value fuel",
           #"Maize straw from organic cultivation","Maize straw from rainfed cultivation", "Maize straw environmental burdern free","Maize straw bruder all environmental impact during cultivation stage"
                   ], 
                   fontsize=18, bbox_to_anchor=(1.43,0.9),title="Shape",title_fontsize=18, frameon=False)

#legend1.set_title('Shape',prop=myfont2) title="$\\bf{Shape}$"
#'X axis: EF v3.0 climate change total \nY aixs EF v3.0 deprivation-weighted water consumption'

plt.legend([p1, p2, p3], ['Inert nitrogen flow index', 'Reactive nitrogen flow index', 'Nitrogen accumulation index'], loc='lower right',
           scatterpoints=1, bbox_to_anchor=(1.6, 0.28),fontsize=18, title="Color",title_fontsize=18, frameon=False)
plt.gca().add_artist(legend1)

#'Analyse de contribution de l\'activité "%s" \n pour la catégorie : %s\n Impact total = %0.2g %s/MJ H2'
plt.title('N flow sustainability index', fontsize=22,fontname="Times New Roman")
plt.xlabel('Relative percentage of N flow change or N accumulation (%)', fontsize=20,fontname="Times New Roman")
plt.ylabel('Magnitude of N flow change or N accumulation (%)', fontsize=20,fontname="Times New Roman")

# , Carbon footprint, Water footprint
# \n Carbon footprint(CO2 eq.)
# \n Water footprint(m3 water eq. deprived)

# rotation=45,horizontalalignment="right"
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
mpl.rc('font',family='Times New Roman')

# Benchmark Mean values          
          
plt.axvline(x=0, color='k',linestyle='--', linewidth=1) 
plt.axhline(y=0, color='k', linestyle='--', linewidth=1) 


plt.yscale('log')
plt.yscale('symlog')
plt.xscale('log')
plt.xscale('symlog')

mpl.rcParams["figure.dpi"] = 780

#Quadrant Marker          
plt.text(x=1500, y=-4.2, s="Quadrant 4: NIF<0",fontsize=18, color='black') #alpha=0.7,
plt.text(x=1500, y=-1.95, s="Quadrant 3: NIF>0",fontsize=18, color='black')
plt.text(x=1500, y=-3, s="Quadrant 2: NIF<0",fontsize=18, color='black')
plt.text(x=1500, y=-1.7, s="Quadrant 1: NIF>0",fontsize=18, color='black')   

#plt.ylim(ymin = 0)



#panel.ellipse(NFI_IN_x_per, NFI_IN_y_per, lwd = 2, col="green", robust=FALSE,  level=0.65)

#nstd = 1
#ax = plt.subplot(111)

#x1=NFI_IN_x_per
#y1=NFI_IN_y_per

#cov = np.cov(x1, y1)
#vals, vecs = eigsorted(cov)
#theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
#w, h = 2 * nstd * np.sqrt(vals)
#ell = Ellipse(xy=(np.mean(x1), np.mean(y1)),
#              width=w, height=h,
#              angle=theta, color='grey')
#ell.set_facecolor('none')
#ax.add_artist(ell)

#x2=NFI_ON_x_per
#y2=NFI_ON_y_per

#cov = np.cov(x2, y2)
#vals, vecs = eigsorted(cov)
#theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
#w, h = 2 * nstd * np.sqrt(vals)
#ell = Ellipse(xy=(np.mean(x2), np.mean(y2)),
#              width=w, height=h,
#              angle=theta, color='blue')
#ell.set_facecolor('none')
#ax.add_artist(ell)

#sns.scatterplot(x='-7.81376479400426', y='-987.4604094144358',color='yellow',s=100)


#x3=NFI_Nacc_x_per
#y3=NFI_Nacc_y_per

#cov = np.cov(x3, y3)
#vals, vecs = eigsorted(cov)
#theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
#w, h = 2 * nstd * np.sqrt(vals)
#ell = Ellipse(xy=(np.mean(x3), np.mean(y3)),
#              width=w, height=h,
#              angle=theta, color='yellow',lw=1.5)
#ell.set_facecolor('none')
#ax.add_artist(ell)

#ax = sns.regplot(x=x1, y=y1, ci=95,scatter=False,color='lime')
#ax = sns.regplot(x=x2, y=y2, ci=95,scatter=False,color='blue')
#ax = sns.regplot(x=x3, y=y3, ci=95,scatter=False,color='red')



#x1_ave = np.average(x1)
#y1_ave = np.average(y1)
#plt.scatter(x1_ave, y1_ave, c = 'lime', marker = '*', s = 300)

#x2_ave = np.average(x2)
#y2_ave = np.average(y2)
#plt.scatter(x2_ave, y2_ave, c = 'cyan', marker = '*', s = 300)

#x3_ave = np.average(x3)
#y3_ave = np.average(y3)
#plt.scatter(x3_ave, y3_ave, c = 'yellow', marker = '*', s = 300)


plt.show()

In [ ]:
import numpy
plt.figure(figsize=(9,7))

import matplotlib as mpl
#myfont1 = mpl.font_manager.FontProperties(fname='/System/Library/Fonts/Supplemental/Arial Italic.ttf')     
#myfont2 = mpl.font_manager.FontProperties(fname='C:/Users/DELL/AppData/Local/Microsoft/Windows/Fonts/TimesNewRomanPSBoldMT.ttf')
#test_x=random.sample(range(0, 1000), 12)
#test_y=random.sample(range(0, 1000), 12)

#inert nitrogen flow index perpare
NFI_IN_x_plotC1 = NFI_IN_x_per_SV[0]
NFI_IN_y_plotC1 = NFI_IN_y_per_SV[0]

NFI_IN_x_plotC2 = NFI_IN_x_per_SV[1]
NFI_IN_y_plotC2 = NFI_IN_y_per_SV[1]

#NFI_IN_x_plotMV = NFI_IN_x_per_SV[2]
#NFI_IN_y_plotMV = NFI_IN_y_per_SV[2]


#reactive nitrogen flow index perpare
NFI_ON_x_plotC1 = NFI_ON_x_per_SV[0]
NFI_ON_y_plotC1 = NFI_ON_y_per_SV[0]

NFI_ON_x_plotC2 = NFI_ON_x_per_SV[1]
NFI_ON_y_plotC2 = NFI_ON_y_per_SV[1]

#NFI_ON_x_plotMV = NFI_ON_x_per_SV[2]
#NFI_ON_y_plotMV = NFI_ON_y_per_SV[2]


# nitrogen accumulation index perpare
NFI_Nacc_x_plotC1 = NFI_Nacc_x_per_SV[0]
NFI_Nacc_y_plotC1 = NFI_Nacc_y_per_SV[0]

NFI_Nacc_x_plotC2 = NFI_Nacc_x_per_SV[1]
NFI_Nacc_y_plotC2 = NFI_Nacc_y_per_SV[1]

#NFI_Nacc_x_plotMV = NFI_Nacc_x_per_SV[2]
#NFI_Nacc_y_plotMV = NFI_Nacc_y_per_SV[2]

#carbon footprint and waterfootprint perpare
#GWP_SV_x_plotMM = GWP_SV[0]
#WF_SV_y_plotMM  =  WF_SV[0]

#GWP_SV_x_plotME = GWP_SV[1]
#WF_SV_y_plotME  =  WF_SV[1]

#GWP_SV_x_plotMV = GWP_SV[2]
#WF_SV_y_plotMV  =  WF_SV[2]

Size_legend1=50
# plotting point shape legend 
plt.scatter(x=NFI_IN_x_plotC1, y=NFI_IN_y_plotC1, c = 'black', marker = 'o', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotC2, y=NFI_IN_y_plotC2, c = 'black', marker = 'x', s = Size_legend1)
#plt.scatter(x=NFI_IN_x_plotMV, y=NFI_IN_y_plotMV, c = 'black', marker = 'v', s = Size_legend1)

#plottingg colour legend
p1=plt.scatter(x=NFI_IN_x_plotC1, y=NFI_IN_y_plotC1, c = 'green', marker = 'o', s = 10)
p2=plt.scatter(x=NFI_IN_x_plotC1, y=NFI_IN_y_plotC1, c = 'blue', marker = 'o', s = 10)
p3=plt.scatter(x=NFI_IN_x_plotC1, y=NFI_IN_y_plotC1, c = 'red', marker = 'o', s = 10)
p4=plt.scatter(x=NFI_IN_x_plotC1, y=NFI_IN_y_plotC1, c = 'fuchsia', marker = 'o', s = 10)

#plotting inert nitrogen flow index
plt.scatter(x=NFI_IN_x_plotC1, y=NFI_IN_y_plotC1, c = 'green', marker = 'o', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotC2, y=NFI_IN_y_plotC2, c = 'green', marker = 'x', s = Size_legend1+60)
#plt.scatter(x=NFI_IN_x_plotMV, y=NFI_IN_y_plotMV, c = 'green', marker = 'v', s = Size_legend1)

#plotting reactive nitrogen flow index
plt.scatter(x=NFI_ON_x_plotC1, y=NFI_ON_y_plotC1, c = 'blue', marker = 'o', s = Size_legend1)
plt.scatter(x=NFI_ON_x_plotC2, y=NFI_ON_y_plotC2, c = 'blue', marker = 'x', s = Size_legend1+60)
#plt.scatter(x=NFI_ON_x_plotMV, y=NFI_ON_y_plotMV, c = 'blue', marker = 'v', s = Size_legend1)

#plotting nitrogen accumulation flow index
plt.scatter(x=NFI_Nacc_x_plotC1, y=NFI_Nacc_y_plotC1, c = 'red', marker = 'o', s = Size_legend1)
plt.scatter(x=NFI_Nacc_x_plotC2, y=NFI_Nacc_y_plotC2, c = 'red', marker = 'x', s = Size_legend1+60)
#plt.scatter(x=NFI_Nacc_x_plotMV, y=NFI_Nacc_y_plotMV, c = 'red', marker = 'v', s = Size_legend1)

#plotting carbon footprint and waterfootprint
#plt.scatter(x=GWP_SV_x_plotC1, y=WF_SV_y_plotC1, c = 'fuchsia', marker = '1', s = Size_legend1)
#plt.scatter(x=GWP_SV_x_plotME, y=WF_SV_y_plotME, c = 'fuchsia', marker = 'x', s = Size_legend1+60)
#plt.scatter(x=GWP_SV_x_plotMV, y=WF_SV_y_plotMV, c = 'fuchsia', marker = 'v', s = Size_legend1)

#col_g=["green","green","green","green","green","green","green","green","green","green","green","green"]
markers = ["1","x","v","2","^","o","s","p","P","*","h","+"]

#size = [10,15,20,30,40,50,60,70,80,85,90,95]  
#for m in enumerate(np.unique(markers)):  #c=col_g[col_g==c],
    #plt.scatter(NFI_IN_x_per_SV[:],NFI_IN_y_per_SV[:],c=col_g[col_g==c], marker=markers[i])
#    plt.scatter(test_x[:],test_y[:],marker=markers[markers==m],color='green')


#sns.scatterplot(data=df_SV, x='NFI_IN_x_per_SV', y='NFI_IN_y_per_SV',color='green',s=25)
#sns.scatterplot(data=df_SV, x='GWP_SV', y='WF_SV',color='white',s=1, marker=',')
#sns.scatterplot(data=df_SV, x='NFI_ON_x_per_SV', y='NFI_ON_y_per_SV',color='black',s=25, marker='>')
#sns.scatterplot(data=df_SV, x='NFI_Nacc_x_per_SV', y='NFI_Nacc_y_per_SV',color='red',s=25, marker=',')
#sns.scatterplot(data=df_SV, x='GWP_SV', y='WF_SV',color='fuchsia',s=25, marker=',')
mpl.rc('font',family='Times New Roman')

legend1=plt.legend(["Case1", "Case2",#"Basline scenario", "Mass maize, Economic value fuel",
            #"Energy maize, Mass fuel", "Energy maize, Energy fuel", "Energy maize, Economic value fuel",
            #"Economic value maize, Mass fuel", "Economic value maize, Energy fuel", "Economic value maize, Economic value fuel",
           #"Maize straw from organic cultivation","Maize straw from rainfed cultivation", "Maize straw environmental burdern free","Maize straw bruder all environmental impact during cultivation stage"
                   ], 
                   fontsize=18, bbox_to_anchor=(0.8, -0.15),title="Shape",title_fontsize=18, frameon=False)

#legend1.set_title('Shape',prop=myfont2) title="$\\bf{Shape}$"
#'X axis: EF v3.0 climate change total \nY aixs EF v3.0 deprivation-weighted water consumption'

plt.legend([p1, p2, p3], ['Inert nitrogen flow index', 'Reactive notrogen flow index', 'Nitrogen accumulation index'], loc='lower right',
           scatterpoints=1, bbox_to_anchor=(0.5, -0.48),fontsize=18, title="Color",title_fontsize=18, frameon=False)
plt.gca().add_artist(legend1)

#'Analyse de contribution de l\'activité "%s" \n pour la catégorie : %s\n Impact total = %0.2g %s/MJ H2'
plt.title('N flow sustainability index', fontsize=22,fontname="Times New Roman")
plt.xlabel('Relative percentage of N flow change or N accumulation (%)', fontsize=20,fontname="Times New Roman")
plt.ylabel('Magnitude of N flow change or N accumulation (%)', fontsize=20,fontname="Times New Roman")

# , Carbon footprint, Water footprint
# \n Carbon footprint(CO2 eq.)
# \n Water footprint(m3 water eq. deprived)

# rotation=45,horizontalalignment="right"
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
mpl.rc('font',family='Times New Roman')

# Benchmark Mean values          
          
plt.axvline(x=0, color='k',linestyle='--', linewidth=1) 
plt.axhline(y=0, color='k', linestyle='--', linewidth=1) 


plt.yscale('log')
plt.yscale('symlog')
plt.xscale('log')
plt.xscale('symlog')

mpl.rcParams["figure.dpi"] = 780
#plt.ylim(ymin = 0)



#panel.ellipse(NFI_IN_x_per, NFI_IN_y_per, lwd = 2, col="green", robust=FALSE,  level=0.65)

#nstd = 1
#ax = plt.subplot(111)

#x1=NFI_IN_x_per
#y1=NFI_IN_y_per

#cov = np.cov(x1, y1)
#vals, vecs = eigsorted(cov)
#theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
#w, h = 2 * nstd * np.sqrt(vals)
#ell = Ellipse(xy=(np.mean(x1), np.mean(y1)),
#              width=w, height=h,
#              angle=theta, color='grey')
#ell.set_facecolor('none')
#ax.add_artist(ell)

#x2=NFI_ON_x_per
#y2=NFI_ON_y_per

#cov = np.cov(x2, y2)
#vals, vecs = eigsorted(cov)
#theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
#w, h = 2 * nstd * np.sqrt(vals)
#ell = Ellipse(xy=(np.mean(x2), np.mean(y2)),
#              width=w, height=h,
#              angle=theta, color='blue')
#ell.set_facecolor('none')
#ax.add_artist(ell)

#sns.scatterplot(x='-7.81376479400426', y='-987.4604094144358',color='yellow',s=100)


#x3=NFI_Nacc_x_per
#y3=NFI_Nacc_y_per

#cov = np.cov(x3, y3)
#vals, vecs = eigsorted(cov)
#theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
#w, h = 2 * nstd * np.sqrt(vals)
#ell = Ellipse(xy=(np.mean(x3), np.mean(y3)),
#              width=w, height=h,
#              angle=theta, color='yellow',lw=1.5)
#ell.set_facecolor('none')
#ax.add_artist(ell)

#ax = sns.regplot(x=x1, y=y1, ci=95,scatter=False,color='lime')
#ax = sns.regplot(x=x2, y=y2, ci=95,scatter=False,color='blue')
#ax = sns.regplot(x=x3, y=y3, ci=95,scatter=False,color='red')



#x1_ave = np.average(x1)
#y1_ave = np.average(y1)
#plt.scatter(x1_ave, y1_ave, c = 'lime', marker = '*', s = 300)

#x2_ave = np.average(x2)
#y2_ave = np.average(y2)
#plt.scatter(x2_ave, y2_ave, c = 'cyan', marker = '*', s = 300)

#x3_ave = np.average(x3)
#y3_ave = np.average(y3)
#plt.scatter(x3_ave, y3_ave, c = 'yellow', marker = '*', s = 300)


plt.show()

# Définition des impacts

In [ ]:
#list(bw.methods)

In [ ]:
[m for m in bw.methods if 'midpoint' in str(m)]

In [ ]:
CC_EN15804 = ('EF v3.0 EN15804', 'climate change', 'global warming potential (GWP100)')

In [ ]:
CC_EF3 = ('EF v3.0', 'climate change', 'global warming potential (GWP100)')

In [ ]:
CC_EF3_B = ('EF v3.0', 'climate change: biogenic', 'global warming potential (GWP100)')

In [ ]:
water_ReCiPe_v =  ('ReCiPe Midpoint (H) V1.13', 'water depletion', 'WDP')

In [ ]:
water_EF =('EF v3.0', 'water use', 'user deprivation potential (deprivation-weighted water consumption)')

In [ ]:
NIF_IN = ('N flow sustainability', 'N input footprint', 'inert N resource input, biosphere traversal(total)')

In [ ]:
NIF_ON = ('N flow sustainability', 'N input footprint', 'reactive N resource input, biosphere traversal(total)')

In [ ]:
NIF_ON_air = ('N flow sustainability', 'N input footprint', 'reactive N resource input from atmosphere')

In [ ]:
NIF_ON_water = ('N flow sustainability', 'N input footprint', 'reactive N resource input from hydrosphere')

In [ ]:
NIF_ON_soil = ('N flow sustainability', 'N input footprint', 'reactive N resource input from soil-lithosphere')

In [ ]:
NOF_IN = ('N flow sustainability', 'N output footprint', 'inert N emission, biosphere traversal(total)')

In [ ]:
NOF_ON = ('N flow sustainability', 'N output footprint', 'reactive N emission, biosphere traversal(total)')

In [ ]:
CCEF_method = bw.Method(water_EF)
CCEF_method.metadata

In [ ]:
#################################
#CC_M = bw.Method(water_EF)

#EF=pd.Series(
#    (str(bw.get_activity(ef[0])['name']) +';'+ str(bw.get_activity(ef[0])['categories']) +';'+str(ef[1])) for ef in CC_M.load()
#)
#EF
#EF.to_csv(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\Water_EF.csv', index = True)

# nitrogen flow sustainability impact categories creation

## nitrogen flow on the level of biosphere traversal

In [ ]:
## all code under this section only need to run once

### preparing for the demo version

In [ ]:
## the demo version code only need to run once

In [ ]:
ILCDM=bw2io.ExcelLCIAImporter(r'C:\Users\DELL\学术\ILCD EF\importable\N_EF_test.xlsx',name=('N flow sustainability', 'N input footprint', 'inert N resource input, biosphere traversal(total)'),unit='kg N',description='inert N input calculated from Extensive nitrogen flow accounting method based on biosphere material flow traversal')
ILCDM.extractor()

ILCDM.apply_strategies()
list(ILCDM.unlinked)

#ILCDM.write_methods()
#ExcelLCIAImporter.
ILCDM.write_methods(overwrite=True)

In [ ]:
ILCDM=bw2io.ExcelLCIAImporter(r'C:\Users\DELL\学术\ILCD EF\importable\N_EF_test.xlsx',name=('N flow sustainability', 'N input footprint', 'reactive N resource input, biosphere traversal(total)'),unit='kg N',description='reactive N input calculated from Extensive nitrogen flow accounting method based on biosphere material flow traversal')
ILCDM.extractor()

ILCDM.apply_strategies()
list(ILCDM.unlinked)

#ILCDM.write_methods()
#ExcelLCIAImporter.
ILCDM.write_methods(overwrite=True)

In [ ]:
ILCDM=bw2io.ExcelLCIAImporter(r'C:\Users\DELL\学术\ILCD EF\importable\N_EF_test.xlsx',name=('N flow sustainability', 'N output footprint', 'inert N emission, biosphere traversal(total)'),unit='kg N',description='inert N emission calculated from Extensive nitrogen flow accounting method based on biosphere material flow traversal')
ILCDM.extractor()

ILCDM.apply_strategies()
list(ILCDM.unlinked)

#ILCDM.write_methods()
#ExcelLCIAImporter.
ILCDM.write_methods(overwrite=True)

In [ ]:
ILCDM=bw2io.ExcelLCIAImporter(r'C:\Users\DELL\学术\ILCD EF\importable\N_EF_test.xlsx',name=('N flow sustainability', 'N output footprint', 'reactive N emission, biosphere traversal(total)'),unit='kg N',description='reactive N emission calculated from Extensive nitrogen flow accounting method based on biosphere material flow traversal')
ILCDM.extractor()

ILCDM.apply_strategies()
list(ILCDM.unlinked)

#ILCDM.write_methods()
#ExcelLCIAImporter.
ILCDM.write_methods(overwrite=True)

### clip to natural resource CF inly

In [ ]:
# please import the excel accroding to the name, the path depend on your storage of the file
EF_cfRrs = pd.read_excel (r'C:\Users\DELL\学术\bioshpere3_resource_0316.xlsx', sheet_name='bioshpere_resource_only') #place "r" before the path string to address special character, such as '\'. Don't forget to put the file name at the end of the path + '.xlsx'
#aaa=emission_cf.iloc[1,0]
#df1.drop(df1.index[[0,3]])
#aaa1=[df1.iloc[3,0],df1.iloc[3,1],df1.iloc[3,2]]
#aaa2=[df1.iloc[3,0],df1.iloc[3,1],df1.iloc[3,2]]
EF_cfRrs

### finding the insert nitrogen (N2-N) resource Cf 

In [ ]:
resource_cf_N2 = EF_cfRrs

In [ ]:
resource_cf_N2i = resource_cf_N2[resource_cf_N2['N type A'].isin(['N2-N'])]
resource_cf_N2i

### insert nitrogen natural resource combining

In [ ]:
# in all kinds of cf to natural resource elementary flow in bioshpere db, only the dinitrogen molecule (N2) is an accurate number without uncertainty  
Ncfi_NR=resource_cf_N2i[resource_cf_N2i['Molecular Formula'].isin(['N2'])]
Ncfi_NR_value=Ncfi_NR.loc[:,"N content(kg/unit)"]

Ncfi_NR_value.iloc[0]

In [ ]:
Ncfi_NR.key

In [ ]:
Ncfi_NR.key=Ncfi_NR.key.apply(ast.literal_eval)

In [ ]:
Ncfi_NR.key.iloc[0]

In [ ]:
Ncfi_NR['Ncf_key'] = list(zip(Ncfi_NR.key, Ncfi_NR_value))
print(Ncfi_NR.Ncf_key)

In [ ]:
Ncfi_NR.Ncf_key.iloc[0]

In [ ]:
keyNR_CFN2i_SUM = [(Ncfi_NR.Ncf_key.iloc[0])]
print(keyNR_CFN2i_SUM)

In [ ]:
Off_NG_CMM = [(('biosphere3','3ed5f377-344f-423a-b5ec-9a9a1162b944'),
    {"amount": 0.0679194960798357,  # This is the only required field
    "uncertainty type": 2, # lognormal
    "loc": 2.0758174699,
    "scale": 1.3682453896,
    #"shape": 14.4536158091,
    'minimum': 0.0503334043,
    #'maximum':0.00000002718801,
    "negative": False,
    }
     )]
   
Off_NG_CMM

In [ ]:
Natural_gas = [(('biosphere3','7c337428-fb1b-45c7-bbb2-2ee4d29e17ba'),
    {"amount": 0.0679194960798357,  # This is the only required field
    "uncertainty type": 9, # Gamma
    "loc": -0.0020669959,
    "scale": 0.0048421442,
    "shape": 14.4536158091,
    #'minimum': 0,
    #'maximum':0.00000002718801,
    "negative": False,
    }
     )]
   
Natural_gas

In [ ]:
Olivine= [(('biosphere3','d7cadc9a-f42c-4711-a8e2-0b626c0a4c7a'),
    {"amount": 0.00000001250598 ,  # This is the only required field
    "uncertainty type": 4, # Weibull
    #"loc": 0.0011140606,
    #"scale": 0.0016447225,
    #"shape": 3.6196924196,
    'minimum': 0,
    'maximum':0.00000002718801,
    "negative": False,
    }
     )]
   
Olivine

In [ ]:
Water_ocean_N2i= [(('biosphere3','629ffbca-ca71-4e4b-a006-ca9bdd9cd1df'),
    {"amount": 0.01120536 ,  # This is the only required field
    "uncertainty type": 0 , # No uncertainty
    #"loc": 0.0011140606,
    #"scale": 0.0016447225,
    #"shape": 3.6196924196,
    #'minimum': 0,
    #'maximum':0.00000002718801,
    #"negative": False,
    }
     )]
   
Water_ocean_N2i

In [ ]:
keyNR_CFN2i_SUM += Off_NG_CMM+Natural_gas+Olivine+Water_ocean_N2i
print(keyNR_CFN2i_SUM)

In [ ]:
NIF_Ni = bw.Method(('N flow sustainability', 'N input footprint', 'inert N resource input, biosphere traversal(total)'))

In [ ]:
NIF_Ni.write(keyNR_CFN2i_SUM)
Method(("N flow sustainability", "N input footprint", "inert N resource input, biosphere traversal(total)")).load()

### finding the reactive nitrogen (Nr-N) resource Cf 

In [ ]:
resource_cf_N2 = EF_cfRrs

In [ ]:
resource_cf_N2org = resource_cf_N2[resource_cf_N2['N type A'].isin(['Nr-N'])]
resource_cf_N2org

### reactive nitrogen natural resource combining

In [ ]:
# in all kinds of natural resource elementary flow in bioshpere db, only the nitrogen content in NaNO3 is an accurate number without uncertainty  
NaNO3 = resource_cf_N2[resource_cf_N2['Molecular Formula'].isin(['NNaO3'])]
NaNO3

In [ ]:
NaNO3_Ncf=NaNO3.loc[:,"N content(kg/unit)"]
NaNO3_Ncf.iloc[0]

In [ ]:
NaNO3.key=NaNO3.key.apply(ast.literal_eval)

In [ ]:
NaNO3.key.iloc[0]

In [ ]:
NaNO3['Ncf_key'] = list(zip(NaNO3.key, NaNO3_Ncf))
print(NaNO3.Ncf_key)

In [ ]:
NaNO3.Ncf_key.iloc[0]

In [ ]:
NaNO3_key_list = [0]*(len(NaNO3.Ncf_key))
#print(cf_key_list)
len(NaNO3_key_list)

In [ ]:
i = 0
while i < len(NaNO3.Ncf_key):
  NaNO3_key_list[i] = [(NaNO3.Ncf_key.iloc[i])]
  if i >= len(NaNO3.Ncf_key):
    break
  i += 1
print(NaNO3_key_list)

In [ ]:
NaNO3_CF_SUM =[]

In [ ]:
i = 0

while i < len(NaNO3.Ncf_key):
  NaNO3_CF_SUM += NaNO3_key_list[i]
  i += 1
  if i >= len(NaNO3.Ncf_key):
    break
  
print(NaNO3_CF_SUM)

In [ ]:
Basalt = [(('biosphere3','ac3a8914-35f0-4c34-a956-f26b3a053e4a'),
    {"amount": 0.000004,  # This is the only required field
    "uncertainty type": 4, # linear
    #"loc": 0.0255633927,
    #"scale": 0.0007362381,
    #"shape": 3.6152250317,
    "minimum": 0,
    "maximum": 0.0001,
     "negative": False,
    }
     )]
   
Basalt

In [ ]:
clay = [(('biosphere3','f7519ca9-5ffc-41c3-a33e-806da82cfc0e'),
    {"amount": 0.0004,  # This is the only required field
    "uncertainty type": 5, # trangile
    "loc": 0.0004,
    #"scale": 0.0000037381,
    #"shape": 177.9506430874,
    'minimum': 0.00002,
    'maximum': 0.007000,
    "negative": False,
    }
     )]
   
clay

In [ ]:
brown_coal= [(('biosphere3','024c9722-1e88-412b-8c4b-10c532be8dca'),
    {"amount": 0.0114844238495064,  # This is the only required field
    "uncertainty type": 9, # gamma
    "loc": -0.0100749996,
    "scale": 0.0000296933,
    "shape": 726.069485603,
    #'minimum': 0.00002,
    #'maximum': 0.007000,
    "negative": False,
    }
     )]
   
brown_coal

In [ ]:
hard_coal= [(('biosphere3','b6d0042d-0ef8-49ed-9162-a07ff1ccf750'),
    {"amount": 0.00972339214243983,  # This is the only required field
    "uncertainty type": 9, # gamma
    "loc": 0.0011854329,
    "scale": 0.0002074237,
    "shape": 41.1619311618,
    #'minimum': 0.00002,
    #'maximum': 0.007000,
    "negative": False,
    }
     )]
   
hard_coal

In [ ]:
fish_demersal_NR= [(('biosphere3','55219a64-a6ed-4675-9210-7ed14c82a05f'),
    {"amount": 0.0262269726656303,  # This is the only required field
    "uncertainty type": 8, # Weibull
    "loc": 0.0255633927,
    "scale": 0.0007362381,
    "shape": 3.6152250317,
    #'minimum': 0.00002,
    #'maximum': 0.007000,
    "negative": False,
    }
     )]
   
fish_demersal_NR

In [ ]:
fish_pelagic_NR= [(('biosphere3','d22719d0-8afd-45d3-b9eb-222112f84cc7'),
    {"amount": 0.0262269726656303,  # This is the only required field
    "uncertainty type": 9, # Gamma
    "loc": 0.0273163110,
    "scale": 0.0000037381,
    "shape": 177.9506430874,
    #'minimum': 0.00002,
    #'maximum': 0.007000,
    "negative": False,
    }
     )]
   
fish_pelagic_NR

In [ ]:
Granite= [(('biosphere3','a4375a18-172c-4f82-90b7-bca972f75548'),
    {"amount": 0.000125,  # This is the only required field
    "uncertainty type": 4, # linear
    #"loc": 0.0273163110,
    #"scale": 0.0000037381,
    #"shape": 177.9506430874,
    'minimum': 0,
    'maximum': 0.000250,
    "negative": False,
    }
     )]
   
Granite

In [ ]:
Laterite= [(('biosphere3','86fb18d4-a425-407a-94bc-194254e4d7d7'),
    {"amount": 0.0005,  # This is the only required field
    "uncertainty type": 4, # linear
    #"loc": 0.0273163110,
    #"scale": 0.0000037381,
    #"shape": 177.9506430874,
    'minimum': 0.0001,
    'maximum': 0.001,
    "negative": False,
    }
     )]
   
Laterite

In [ ]:
Metamorphous_rock= [(('biosphere3','5666353e-2db2-41d3-8414-404709151422'),
    {"amount": 0.00000127,  # This is the only required field
    "uncertainty type": 4, # linear
    #"loc": 0.0273163110,
    #"scale": 0.0000037381,
    #"shape": 177.9506430874,
    'minimum': 0.00000027,
    'maximum': 0.00000227,
    "negative": False,
    }
     )]
   
Metamorphous_rock

In [ ]:
crude_oil= [(('biosphere3','88d06db9-59a1-4719-9174-afeb1fa4026a'),
    {"amount": 0.00259656745503983,  # This is the only required field
    "uncertainty type": 8, # Weibull
    "loc": 0.0011140606,
    "scale": 0.0016447225,
    "shape": 3.6196924196,
    #'minimum': 0.00000027,
    #'maximum': 0.00000227,
    'negative': False,
    }
     )]
   
crude_oil

In [ ]:
Peat= [(('biosphere3','c5035ce2-5ee5-431f-a287-4b25da42be74'),
    {"amount": 0.0175138856030076,  # This is the only required field
    "uncertainty type": 8, # Weibull
    "loc": 0.0089787173,
    "scale": 0.0092266822,
    "shape": 5.698422798,
    #'minimum': 0.00000027,
    #'maximum': 0.00000227,
    "negative": False,
    }
     )]
   
Peat

In [ ]:
Shale= [(('biosphere3','9e9b6792-40e6-4d62-a3e0-ebebc0c65166'),
    {"amount": 0.001,  # This is the only required field
    "uncertainty type": 5, # Trangile
    "loc": 0.001,
    #"scale": 0.0092266822,
    #"shape": 5.698422798,
    'minimum': 0.0003,
    'maximum': 0.0032,
    "negative": False,
    }
     )]
   
Shale

In [ ]:
Water_cooling= [(('biosphere3','fc1c42ce-a759-49fa-b987-f1ec5e503db1'),
    {"amount": 0.001,  # This is the only required field
    "uncertainty type": 5, # Trangile
    "loc":0.001,
    #"scale": 0.0092266822,
    #"shape": 5.698422798,
    'minimum': 0,
    'maximum': 0.01,
    "negative": False,
    }
     )]
   
Water_cooling

In [ ]:
Water_lake = [(('biosphere3','1acb026e-9de6-48fe-9e0d-be4d24125bbc'),
    {"amount": 0.002,  # This is the only required field
    "uncertainty type": 5, # Trangile
    "loc":0.002,
    #"scale": 0.0092266822,
    #"shape": 5.698422798,
    'minimum': 0.0002,
    'maximum': 0.02,
    "negative": False,
    }
     )]
   
Water_lake

In [ ]:
Water_river = [(('biosphere3','8c75e7ab-8ab8-41e4-b394-c166ff5b050d'),
    {"amount": 0.002,  # This is the only required field
    "uncertainty type": 5, # Trangile
    "loc":0.002,
    #"scale": 0.0092266822,
    #"shape": 5.698422798,
    'minimum': 0.0002,
    'maximum': 0.015,
    "negative": False,
    }
     )]
   
Water_river

In [ ]:
Water_ocean = [(('biosphere3','629ffbca-ca71-4e4b-a006-ca9bdd9cd1df'),
    {"amount": 0.000285196410476319,  # This is the only required field
    "uncertainty type": 8, # Weibull
    "loc":0.0000809630,
    "scale": 0.0002286376,
    "shape": 1.6631169834,
    #'minimum': 0.0002,
    #'maximum': 0.015,
    "negative": False,
    }
     )]
   
Water_ocean

In [ ]:
Water_well = [(('biosphere3','67c40aae-d403-464d-9649-c12695e43ad8'),
    {"amount": 0.0005,  # This is the only required field
    "uncertainty type": 5, # Trangile
    "loc":0.0005,
    #"scale": 0.0092266822,
    #"shape": 5.698422798,
    'minimum': 0.00002,
    'maximum': 0.001,
    "negative": False,
    }
     )]
   
Water_well

In [ ]:
Wood_soft = [(('biosphere3','b073ec00-a5bf-4b64-bda0-ef366a3ac9bb'),
    {"amount": 1.6730478040,  # This is the only required field
    "uncertainty type": 2, # lognormal
    "loc":1.6730478040,
    "scale": 0.0079793010,
    #"shape": 5.698422798,
    #'minimum': 0.00002,
    #'maximum': 0.001,
    "negative": False,
    }
     )]
   
Wood_soft

In [ ]:
Wood_hard = [(('biosphere3','bac875f4-75fb-4dde-841a-b07d3a41bcd1'),
    {"amount": 1.6730478040,  # This is the only required field
    "uncertainty type": 2, # lognormal
    "loc":1.6730478040,
    "scale": 0.0079793010,
    #"shape": 5.698422798,
    #'minimum': 0.00002,
    #'maximum': 0.001,
    "negative": False,
    }
     )]
   
Wood_hard

In [ ]:
Wood_primary = [(('biosphere3','28528881-7154-48d5-9cc3-5c13ddcdc47a'),
    {"amount": 2.43092398925838,  # This is the only required field
    "uncertainty type": 9, # gamma
    "loc":1.8445082364,
    "scale": 0.0062673107,
    "shape": 93.5680636548,
    #'minimum': 0.00002,
    #'maximum': 0.001,
    "negative": False,
    }
     )]
   
Wood_primary

In [ ]:
NaNO3_CF_SUM += Basalt+clay+brown_coal+hard_coal+fish_demersal_NR+fish_pelagic_NR+Granite+Laterite+Metamorphous_rock+crude_oil+Peat+Shale+Water_cooling+Water_lake+Water_river+Water_ocean+Water_well+Wood_soft+Wood_hard+Wood_primary
print(NaNO3_CF_SUM)
len(NaNO3_CF_SUM)

In [ ]:
# in all kinds of natural resource elementary flow in bioshpere db, only the nitrogen content in NaNO3 is an accurate number without uncertainty  
N_deposition_list = resource_cf_N2[resource_cf_N2['comment'].isin(['land occupation, surface'])]
N_deposition_list

In [ ]:
N_deposition_list.key=N_deposition_list.key.apply(ast.literal_eval)

In [ ]:
N_deposition_list.key.iloc[59]

In [ ]:
SurfaceN_deposition = {
    "amount": 0.2697553849,  # This is the only required field
    "uncertainty type": 2, # lognormal
    "loc":0.2697553849,
    "scale": 1.7349488438,
    #"shape": 93.5680636548,
    #'minimum': 0.00002,
    #'maximum': 0.001,
    "negative": False,
    }

   
SurfaceN_deposition

In [ ]:
landD_cf_list = [0]*(len(N_deposition_list.key))
#print(cf_key_list)
len(landD_cf_list)

In [ ]:
i = 0
while i < len(N_deposition_list.key):
  landD_cf_list[i] = [(SurfaceN_deposition)]
  if i >= len(N_deposition_list.key):
    break
  i += 1
print(landD_cf_list)

In [ ]:
landD_cf_list_combine =[]

In [ ]:
i = 0

while i < len(N_deposition_list.key):
  landD_cf_list_combine += landD_cf_list[i]
  i += 1
  if i >= len(N_deposition_list.key):
    break
  
print(landD_cf_list_combine)

In [ ]:
N_deposition_list['landD_Ncf_key'] = list(zip(N_deposition_list.key, landD_cf_list_combine))
print(N_deposition_list.landD_Ncf_key)

In [ ]:
N_deposition_list.landD_Ncf_key.iloc[0]

In [ ]:
landD_keycf_list_one = [0]*(len(N_deposition_list.key))
#print(cf_key_list)
len(landD_keycf_list_one)

In [ ]:
i = 0
while i < len(N_deposition_list.key):
  landD_keycf_list_one[i] = [(N_deposition_list.landD_Ncf_key.iloc[i])]
  if i >= len(N_deposition_list.key):
    break
  i += 1
print(landD_keycf_list_one)

In [ ]:
landD_cfkey_SUM =[]

In [ ]:
i = 0

while i < len(N_deposition_list.key):
  landD_cfkey_SUM += landD_keycf_list_one[i]
  i += 1
  if i >= len(N_deposition_list.key):
    break
  
print(landD_cfkey_SUM)

In [ ]:
Total_SUM =[]
Total_SUM = NaNO3_CF_SUM + landD_cfkey_SUM
len(Total_SUM)

In [ ]:
NIF_Norg_air = bw.Method(('N flow sustainability', 'N input footprint', 'reactive N resource input, biosphere traversal(total)'))

In [ ]:
NIF_Norg_air.write(Total_SUM)
Method(('N flow sustainability', 'N input footprint', 'reactive N resource input, biosphere traversal(total)')).load()

### clip to emission CF inly

In [ ]:
# please import the excel accroding to the name, the path depend on your storage of the file
EF_cf = pd.read_excel (r'C:\Users\DELL\学术\bioshpere3_emission_0316.xlsx', sheet_name='bioshpere3_emission') #place "r" before the path string to address special character, such as '\'. Don't forget to put the file name at the end of the path + '.xlsx'
#aaa=emission_cf.iloc[1,0]
#df1.drop(df1.index[[0,3]])
#aaa1=[df1.iloc[3,0],df1.iloc[3,1],df1.iloc[3,2]]
#aaa2=[df1.iloc[3,0],df1.iloc[3,1],df1.iloc[3,2]]

In [ ]:
nan_value = float("NaN")
EF_cf.replace("", nan_value, inplace=True)
EF_cf.dropna(subset = ["name"], inplace=True)

EF_cf

In [ ]:
emission_cf=EF_cf

emission_cf.replace("economic", nan_value, inplace=True)
emission_cf.dropna(subset = ["type"], inplace=True)

emission_cf

In [ ]:
emission_cf.replace("inventory indicator", nan_value, inplace=True)
emission_cf.dropna(subset = ["type"], inplace=True)

emission_cf

In [ ]:
emission_cf.replace("natural resource", nan_value, inplace=True)
emission_cf.dropna(subset = ["type"], inplace=True)

emission_cf

In [ ]:
emission_cf.replace("social", nan_value, inplace=True)
emission_cf.dropna(subset = ["type"], inplace=True)

emission_cf

In [ ]:
# for resource whose nitrogen content embodied with uncertainty distribution, their nitrogen characteristic factor should be imported separately

In [ ]:
emission_cf.replace("Discarded fish, demersal, to ocean", nan_value, inplace=True)
emission_cf.dropna(subset = ["name"], inplace=True)

emission_cf

In [ ]:
# for resource whose nitrogen content embodied with uncertainty distribution, their nitrogen characteristic factor should be imported separately
emission_cf.replace("Discarded fish, pelagic, to ocean", nan_value, inplace=True)
emission_cf.dropna(subset = ["name"], inplace=True)

emission_cf

### finding the insert nitrogen (N2-N) emission Cf 

In [ ]:
emission_cf_N2 = emission_cf

In [ ]:
emission_cf_N2i = emission_cf_N2[emission_cf_N2['name'].isin(['Nitrogen'])]
emission_cf_N2i

### insert nitrogen emission combining

In [ ]:
Ncfi=emission_cf_N2i.loc[:,"N content(kg/unit)"]
Ncfi.iloc[9]

In [ ]:
emission_cf_N2i.key=emission_cf_N2i.key.apply(ast.literal_eval)

In [ ]:
emission_cf_N2i.key.iloc[1]

In [ ]:
emission_cf_N2i['Ncf_key'] = list(zip(emission_cf_N2i.key, Ncfi))
print(emission_cf_N2i.Ncf_key)

In [ ]:
emission_cf_N2i.Ncf_key.iloc[0]

In [ ]:
cfN2i_key_list = [0]*(len(emission_cf_N2i.Ncf_key))
#print(cf_key_list)
len(cfN2i_key_list)

In [ ]:
i = 0
while i < len(emission_cf_N2i.Ncf_key):
  cfN2i_key_list[i] = [(emission_cf_N2i.Ncf_key.iloc[i])]
  if i >= len(emission_cf_N2i.Ncf_key):
    break
  i += 1
print(cfN2i_key_list)

In [ ]:
key_CFN2i_SUM =[]

In [ ]:
i = 0

while i < len(emission_cf_N2i.Ncf_key):
  key_CFN2i_SUM += cfN2i_key_list[i]
  i += 1
  if i >= len(emission_cf_N2i.Ncf_key):
    break
  
print(key_CFN2i_SUM)

In [ ]:
NOF_Ni = bw.Method(('N flow sustainability', 'N output footprint', 'inert N emission, biosphere traversal(total)'))

In [ ]:
NOF_Ni.write(key_CFN2i_SUM)
Method(("N flow sustainability", "N output footprint", "inert N emission, biosphere traversal(total)")).load()

### finding the reactive nitrogen (N2-N) emission Cf 

In [ ]:
#emission_cf_N2Org = emission_cf_N2
#emission_cf_N2Org.replace("N2", nan_value, inplace=True)
#emission_cf_N2Org.dropna(subset = ["Molecular Formula"], inplace=True)

#emission_cf_N2Org

#filterinfDataframe = dfObj[(dfObj['Sale'] > 30) & (dfObj['Sale'] < 33) ]

emission_cf_N2Org = emission_cf_N2[~emission_cf_N2['name'].isin(['Nitrogen'])]
emission_cf_N2Org

### reactive nitrogen emission combining

In [ ]:
Ncforg=emission_cf_N2Org.loc[:,"N content(kg/unit)"]
Ncforg.iloc[2]

In [ ]:
emission_cf_N2Org.key=emission_cf_N2Org.key.apply(ast.literal_eval)

In [ ]:
emission_cf_N2Org.key.iloc[0]

In [ ]:
emission_cf_N2Org['Ncf_key'] = list(zip(emission_cf_N2Org.key, Ncforg))
print(emission_cf_N2Org.Ncf_key)

In [ ]:
emission_cf_N2Org.Ncf_key.iloc[0]

In [ ]:
cforg_key_list = [0]*(len(emission_cf_N2Org.Ncf_key))
#print(cf_key_list)
len(cforg_key_list)

In [ ]:
i = 0
while i < len(emission_cf_N2Org.Ncf_key):
  cforg_key_list[i] = [(emission_cf_N2Org.Ncf_key.iloc[i])]
  if i >= len(emission_cf_N2Org.Ncf_key):
    break
  i += 1
print(cforg_key_list)

In [ ]:
keyorg_CF_SUM =[]

In [ ]:
i = 0

while i < len(emission_cf_N2Org.Ncf_key):
  keyorg_CF_SUM += cforg_key_list[i]
  i += 1
  if i >= len(emission_cf_N2Org.Ncf_key):
    break
  
print(keyorg_CF_SUM)

In [ ]:
fish_demersal = [(('biosphere3','61505469-d82e-4f73-9ac6-bb6910038c26'),
    {"amount": 0.0262269726656303,  # This is the only required field
    "uncertainty type": 8, # Weibull
    "loc": 0.0255633927,
    "scale": 0.0007362381,
    "shape": 3.6152250317,
    #"minimum": 0,
    #"maximum": 1,
     "negative": False,
    }
     )]
   
fish_demersal

In [ ]:
fish_pelagic = [(('biosphere3','68e3130c-7c4b-4825-a383-c54410568d5d'),
    {"amount": 0.0279815062509324,  # This is the only required field
    "uncertainty type": 9, # Gamma
    "loc": 0.0273163110,
    "scale": 0.0000037381,
    "shape": 177.9506430874,
    #'minimum': 0,
    #'maximum': 1,
    "negative": False,
    }
     )]
   
fish_pelagic

In [ ]:
keyorg_CF_SUM += fish_demersal+fish_pelagic
print(keyorg_CF_SUM)

In [ ]:
NOF_Norg = bw.Method(('N flow sustainability', 'N output footprint', 'reactive N emission, biosphere traversal(total)'))

In [ ]:
NOF_Norg.write(keyorg_CF_SUM)
Method(('N flow sustainability', 'N output footprint', 'reactive N emission, biosphere traversal(total)')).load()

## sub nitrogen flow on atmosphere, hydrosphere, soil-lithosphere level

### preparing for the demo version

In [ ]:
## the demo version code only need to run once 

In [ ]:
ILCDM=bw2io.ExcelLCIAImporter(r'C:\Users\DELL\学术\ILCD EF\importable\N_EF_test.xlsx',name=('N flow sustainability', 'N input footprint', 'inert N resource input from atmosphere'),unit='kg N',description='atmospheric inert N input calculated from Extensive nitrogen flow accounting method based on biosphere material flow traversal')
ILCDM.extractor()

ILCDM.apply_strategies()
list(ILCDM.unlinked)

#ILCDM.write_methods()
#ExcelLCIAImporter.
ILCDM.write_methods(overwrite=True)

In [ ]:
ILCDM=bw2io.ExcelLCIAImporter(r'C:\Users\DELL\学术\ILCD EF\importable\N_EF_test.xlsx',name=('N flow sustainability', 'N input footprint', 'inert N resource input from hydrosphere'),unit='kg N',description='hydrospheric inert N input calculated from Extensive nitrogen flow accounting method based on biosphere material flow traversal')
ILCDM.extractor()

ILCDM.apply_strategies()
list(ILCDM.unlinked)

#ILCDM.write_methods()
#ExcelLCIAImporter.
ILCDM.write_methods(overwrite=True)

In [ ]:
ILCDM=bw2io.ExcelLCIAImporter(r'C:\Users\DELL\学术\ILCD EF\importable\N_EF_test.xlsx',name=('N flow sustainability', 'N input footprint', 'inert N resource input from soil-lithosphere'),unit='kg N',description='soil-lithosphere origionated inert N input calculated from Extensive nitrogen flow accounting method based on biosphere material flow traversal')
ILCDM.extractor()

ILCDM.apply_strategies()
list(ILCDM.unlinked)

#ILCDM.write_methods()
#ExcelLCIAImporter.
ILCDM.write_methods(overwrite=True)

In [ ]:
ILCDM=bw2io.ExcelLCIAImporter(r'C:\Users\DELL\学术\ILCD EF\importable\N_EF_test.xlsx',name=('N flow sustainability', 'N input footprint', 'reactive N resource input from atmosphere'),unit='kg N',description='atmospheric reactive N input calculated from Extensive nitrogen flow accounting method based on biosphere material flow traversal')
ILCDM.extractor()

ILCDM.apply_strategies()
list(ILCDM.unlinked)

#ILCDM.write_methods()
#ExcelLCIAImporter.
ILCDM.write_methods(overwrite=True)

In [ ]:
ILCDM=bw2io.ExcelLCIAImporter(r'C:\Users\DELL\学术\ILCD EF\importable\N_EF_test.xlsx',name=('N flow sustainability', 'N input footprint', 'reactive N resource input from hydrosphere'),unit='kg N',description='hydrospheric reactive N input calculated from Extensive nitrogen flow accounting method based on biosphere material flow traversal')
ILCDM.extractor()

ILCDM.apply_strategies()
list(ILCDM.unlinked)

#ILCDM.write_methods()
#ExcelLCIAImporter.
ILCDM.write_methods(overwrite=True)

In [ ]:
ILCDM=bw2io.ExcelLCIAImporter(r'C:\Users\DELL\学术\ILCD EF\importable\N_EF_test.xlsx',name=('N flow sustainability', 'N input footprint', 'reactive N resource input from soil-lithosphere'),unit='kg N',description='soil-lithosphere origionated reactive N input calculated from Extensive nitrogen flow accounting method based on biosphere material flow traversal')
ILCDM.extractor()

ILCDM.apply_strategies()
list(ILCDM.unlinked)

#ILCDM.write_methods()
#ExcelLCIAImporter.
ILCDM.write_methods(overwrite=True)

In [ ]:
ILCDM=bw2io.ExcelLCIAImporter(r'C:\Users\DELL\学术\ILCD EF\importable\N_EF_test.xlsx',name=('N flow sustainability', 'N output footprint', 'inert N emission to atmosphere'),unit='kg N',description='inert N emission to atmosphere calculated from Extensive nitrogen flow accounting method based on biosphere material flow traversal')
ILCDM.extractor()

ILCDM.apply_strategies()
list(ILCDM.unlinked)

#ILCDM.write_methods()
#ExcelLCIAImporter.
ILCDM.write_methods(overwrite=True)

In [ ]:
ILCDM=bw2io.ExcelLCIAImporter(r'C:\Users\DELL\学术\ILCD EF\importable\N_EF_test.xlsx',name=('N flow sustainability', 'N output footprint', 'inert N emission to hydrosphere'),unit='kg N',description='inert N emission to hydrosphere calculated from Extensive nitrogen flow accounting method based on biosphere material flow traversal')
ILCDM.extractor()

ILCDM.apply_strategies()
list(ILCDM.unlinked)

#ILCDM.write_methods()
#ExcelLCIAImporter.
ILCDM.write_methods(overwrite=True)

In [ ]:
ILCDM=bw2io.ExcelLCIAImporter(r'C:\Users\DELL\学术\ILCD EF\importable\N_EF_test.xlsx',name=('N flow sustainability', 'N output footprint', 'inert N emission to soil-lithosphere'),unit='kg N',description='inert N emission to soil-lithosphere calculated from Extensive nitrogen flow accounting method based on biosphere material flow traversal')
ILCDM.extractor()

ILCDM.apply_strategies()
list(ILCDM.unlinked)

#ILCDM.write_methods()
#ExcelLCIAImporter.
ILCDM.write_methods(overwrite=True)

In [ ]:
ILCDM=bw2io.ExcelLCIAImporter(r'C:\Users\DELL\学术\ILCD EF\importable\N_EF_test.xlsx',name=('N flow sustainability', 'N output footprint', 'reactive N emission to atmosphere'),unit='kg N',description='reactive N emission to atmosphere calculated from Extensive nitrogen flow accounting method based on biosphere material flow traversal')
ILCDM.extractor()

ILCDM.apply_strategies()
list(ILCDM.unlinked)

#ILCDM.write_methods()
#ExcelLCIAImporter.
ILCDM.write_methods(overwrite=True)

In [ ]:
ILCDM=bw2io.ExcelLCIAImporter(r'C:\Users\DELL\学术\ILCD EF\importable\N_EF_test.xlsx',name=('N flow sustainability', 'N output footprint', 'reactive N emission to hydrosphere'),unit='kg N',description='reactive N emission to hydrosphere calculated from Extensive nitrogen flow accounting method based on biosphere material flow traversal')
ILCDM.extractor()

ILCDM.apply_strategies()
list(ILCDM.unlinked)

#ILCDM.write_methods()
#ExcelLCIAImporter.
ILCDM.write_methods(overwrite=True)

In [ ]:
ILCDM=bw2io.ExcelLCIAImporter(r'C:\Users\DELL\学术\ILCD EF\importable\N_EF_test.xlsx',name=('N flow sustainability', 'N output footprint', 'reactive N emission to soil-lithosphere'),unit='kg N',description='reactive N emission to soil-lithosphere calculated from Extensive nitrogen flow accounting method based on biosphere material flow traversal')
ILCDM.extractor()

ILCDM.apply_strategies()
list(ILCDM.unlinked)

#ILCDM.write_methods()
#ExcelLCIAImporter.
ILCDM.write_methods(overwrite=True)

### clip to natural resource CF inly

In [ ]:
# please import the excel accroding to the name, the path depend on your storage of the file
EF_cfRrs = pd.read_excel (r'C:\Users\DELL\学术\bioshpere3_resource_0316.xlsx', sheet_name='bioshpere_resource_only') #place "r" before the path string to address special character, such as '\'. Don't forget to put the file name at the end of the path + '.xlsx'
#aaa=emission_cf.iloc[1,0]
#df1.drop(df1.index[[0,3]])
#aaa1=[df1.iloc[3,0],df1.iloc[3,1],df1.iloc[3,2]]
#aaa2=[df1.iloc[3,0],df1.iloc[3,1],df1.iloc[3,2]]
EF_cfRrs

### finding the insert nitrogen (N2-N) resource Cf 

####  insert nitrogen (N2-N) resource Cf from atmosphere

In [ ]:
resource_cf_N2 = EF_cfRrs

In [ ]:
resource_cf_N2i = resource_cf_N2[resource_cf_N2['N type A'].isin(['N2-N'])]
resource_cf_N2i

In [ ]:
resource_cf_N2i_air = resource_cf_N2i[resource_cf_N2i['categories'].isin(["('natural resource', 'in air')"])]
resource_cf_N2i_air

#### atmosphere insert nitrogen natural resource combining

In [ ]:
# in all kinds of cf to natural resource elementary flow in bioshpere db, only the dinitrogen molecule (N2) is an accurate number without uncertainty  
Ncfi_NR=resource_cf_N2i[resource_cf_N2i['Molecular Formula'].isin(['N2'])]
Ncfi_NR_value=Ncfi_NR.loc[:,"N content(kg/unit)"]

Ncfi_NR_value.iloc[0]

In [ ]:
Ncfi_NR.key

In [ ]:
Ncfi_NR.key=Ncfi_NR.key.apply(ast.literal_eval)

In [ ]:
Ncfi_NR.key.iloc[0]

In [ ]:
Ncfi_NR['Ncf_key'] = list(zip(Ncfi_NR.key, Ncfi_NR_value))
print(Ncfi_NR.Ncf_key)

In [ ]:
Ncfi_NR.Ncf_key.iloc[0]

In [ ]:
NIFNi_air_list=[Ncfi_NR.Ncf_key.iloc[0]]
NIFNi_air_list

In [ ]:
NIF_Ni_air = bw.Method(('N flow sustainability', 'N input footprint', 'inert N resource input from atmosphere'))

In [ ]:
NIF_Ni_air.write(NIFNi_air_list)
Method(('N flow sustainability', 'N input footprint', 'inert N resource input from atmosphere')).load()

#### hydrosphere insert nitrogen natural resource combining

In [ ]:
Water_ocean_N2i= [(('biosphere3','629ffbca-ca71-4e4b-a006-ca9bdd9cd1df'),
    {"amount": 0.01120536 ,  # This is the only required field
    "uncertainty type": 0 , # No uncertainty
    #"loc": 0.0011140606,
    #"scale": 0.0016447225,
    #"shape": 3.6196924196,
    #'minimum': 0,
    #'maximum':0.00000002718801,
    #"negative": False,
    }
     )]
   
Water_ocean_N2i

In [ ]:
NIF_Ni_water = bw.Method(('N flow sustainability', 'N input footprint', 'inert N resource input from hydrosphere'))

In [ ]:
NIF_Ni_water.write(Water_ocean_N2i)
Method(('N flow sustainability', 'N input footprint', 'inert N resource input from hydrosphere')).load()

#### soil-lithosphere insert nitrogen natural resource combining

In [ ]:
# 陆地中惰性氮含量，留待更新

In [ ]:
keyNR_CFN2i_SUM_land =[]
keyNR_CFN2i_SUM_land += Off_NG_CMM+Natural_gas+Olivine
print(keyNR_CFN2i_SUM_land)

In [ ]:
NIF_Ni_land = bw.Method(('N flow sustainability', 'N input footprint', 'inert N resource input from soil-lithosphere'))

In [ ]:
NIF_Ni_land.write(keyNR_CFN2i_SUM_land)
Method(('N flow sustainability', 'N input footprint', 'inert N resource input from soil-lithosphere')).load()

### finding the reactive nitrogen (Nr-N) resource Cf 

####  reactive nitrogen (Nr-N) resource Cf from atmosphere

In [ ]:
#None. 地球大气中活性氮的合成几乎全部来自于闪电固氮。However,the lightning nitrogen-fixing process is out of direct human control. And rarely of the industrial processes taken the Nr-N in atmosphere as an direct industrial raw material or resource.  From the biosphere resource repository angle as applied in ecoinvent, Nr-N sedimented from air will be either dissloved in the waterbody or absorbed by soil and the above-grown biomass hereafter. Thus, to avoid double accounting, keeping in consist with the eco-lca N inventory methodology, the reactive nitrogen input from atmosphere is not directly accounted in extensive nitrogen flow accounting method.     

In [ ]:
len(landD_cfkey_SUM)

In [ ]:
NIF_Norg_air = bw.Method(('N flow sustainability', 'N input footprint', 'reactive N resource input from atmosphere'))

In [ ]:
NIF_Norg_air.write(landD_cfkey_SUM)
Method(('N flow sustainability', 'N input footprint', 'reactive N resource input from atmosphere')).load()

####  reactive nitrogen (Nr-N) resource Cf from hydrosphere

In [ ]:
resource_cf_N2 = EF_cfRrs

In [ ]:
resource_cf_N2org = resource_cf_N2[resource_cf_N2['N type A'].isin(['Nr-N'])]
resource_cf_N2org

In [ ]:
resource_cf_N2org_water = resource_cf_N2org[resource_cf_N2org['categories'].isin(["('natural resource', 'in water')"])]
resource_cf_N2org_water

In [ ]:
NIForg_key_list_water = []

In [ ]:
NIForg_key_list_water += fish_demersal_NR+fish_pelagic_NR+Water_cooling+Water_lake+Water_river+Water_ocean+Water_well
print(NIForg_key_list_water)

In [ ]:
NIF_Norg_water = bw.Method(('N flow sustainability', 'N input footprint', 'reactive N resource input from hydrosphere'))

In [ ]:
NIF_Norg_water.write(NIForg_key_list_water)
Method(('N flow sustainability', 'N input footprint', 'reactive N resource input from hydrosphere')).load()

####  reactive nitrogen (Nr-N) resource Cf from soil-lithosphere

In [ ]:
NaNO3_CF_SUM =[]

In [ ]:
i = 0

while i < len(NaNO3.Ncf_key):
  NaNO3_CF_SUM += NaNO3_key_list[i]
  i += 1
  if i >= len(NaNO3.Ncf_key):
    break
  
print(NaNO3_CF_SUM)

In [ ]:
# fish_demersal_NR+fish_pelagic_NR+Water_cooling+Water_lake+Water_river+Water_ocean+Water_well+Basalt+clay+brown_coal+hard_coal+Granite+Laterite+Metamorphous_rock+crude_oil+Peat+Shale+Wood_soft+Wood_hard+Wood_primary
NaNO3_CF_SUM += Basalt+clay+brown_coal+hard_coal+Granite+Laterite+Metamorphous_rock+crude_oil+Peat+Shale+Wood_soft+Wood_hard+Wood_primary
print(NaNO3_CF_SUM)
len(NaNO3_CF_SUM)

In [ ]:
NIF_Norg_soil = bw.Method(('N flow sustainability', 'N input footprint', 'reactive N resource input from soil-lithosphere'))

In [ ]:
NIF_Norg_soil.write(NaNO3_CF_SUM)
Method(('N flow sustainability', 'N input footprint', 'reactive N resource input from soil-lithosphere')).load()

### clip to emission CF inly

In [ ]:
# please import the excel accroding to the name, the path depend on your storage of the file
EF_cf = pd.read_excel (r'C:\Users\DELL\学术\bioshpere3_emission_0316.xlsx', sheet_name='bioshpere3_emission') #place "r" before the path string to address special character, such as '\'. Don't forget to put the file name at the end of the path + '.xlsx'
#aaa=emission_cf.iloc[1,0]
#df1.drop(df1.index[[0,3]])
#aaa1=[df1.iloc[3,0],df1.iloc[3,1],df1.iloc[3,2]]
#aaa2=[df1.iloc[3,0],df1.iloc[3,1],df1.iloc[3,2]]

In [ ]:
nan_value = float("NaN")
EF_cf.replace("", nan_value, inplace=True)
EF_cf.dropna(subset = ["name"], inplace=True)

EF_cf

In [ ]:
emission_cf=EF_cf

emission_cf.replace("economic", nan_value, inplace=True)
emission_cf.dropna(subset = ["type"], inplace=True)

emission_cf

In [ ]:
emission_cf.replace("inventory indicator", nan_value, inplace=True)
emission_cf.dropna(subset = ["type"], inplace=True)

emission_cf

In [ ]:
emission_cf.replace("natural resource", nan_value, inplace=True)
emission_cf.dropna(subset = ["type"], inplace=True)

emission_cf

In [ ]:
emission_cf.replace("social", nan_value, inplace=True)
emission_cf.dropna(subset = ["type"], inplace=True)

emission_cf

In [ ]:
# for resource whose nitrogen content embodied with uncertainty distribution, their nitrogen characteristic factor should be imported separately

In [ ]:
emission_cf.replace("Discarded fish, demersal, to ocean", nan_value, inplace=True)
emission_cf.dropna(subset = ["name"], inplace=True)

emission_cf

In [ ]:
# for resource whose nitrogen content embodied with uncertainty distribution, their nitrogen characteristic factor should be imported separately
emission_cf.replace("Discarded fish, pelagic, to ocean", nan_value, inplace=True)
emission_cf.dropna(subset = ["name"], inplace=True)

emission_cf

### finding the insert nitrogen (N2-N) emission Cf 

####  insert nitrogen emission (N2-N) Cf to atmosphere

In [ ]:
emission_cf_N2 = emission_cf

In [ ]:
emission_cf_N2i = emission_cf_N2[emission_cf_N2['name'].isin(['Nitrogen'])]
emission_cf_N2i

In [ ]:
emission_cf_N2i = emission_cf_N2i[emission_cf_N2i['Nr emission type B'].isin(['air'])]
emission_cf_N2i

In [ ]:
Ncfi=emission_cf_N2i.loc[:,"N content(kg/unit)"]
Ncfi.iloc[0]

In [ ]:
emission_cf_N2i.key=emission_cf_N2i.key.apply(ast.literal_eval)

In [ ]:
emission_cf_N2i.key.iloc[0]

In [ ]:
emission_cf_N2i['Ncf_key'] = list(zip(emission_cf_N2i.key, Ncfi))
print(emission_cf_N2i.Ncf_key)

In [ ]:
emission_cf_N2i.Ncf_key.iloc[0]

In [ ]:
key_CFN2i_SUM =[emission_cf_N2i.Ncf_key.iloc[0]]
key_CFN2i_SUM

In [ ]:
NOF_Ni_air = bw.Method(('N flow sustainability', 'N output footprint', 'inert N emission to atmosphere'))

In [ ]:
NOF_Ni_air.write(key_CFN2i_SUM)
Method(('N flow sustainability', 'N output footprint', 'inert N emission to atmosphere')).load()

####  insert nitrogen emission (N2-N) Cf to hydrosphere

In [ ]:
emission_cf_N2 = emission_cf

In [ ]:
emission_cf_N2i = emission_cf_N2[emission_cf_N2['name'].isin(['Nitrogen'])]
emission_cf_N2i

In [ ]:
emission_cf_N2i = emission_cf_N2i[emission_cf_N2i['Nr emission type B'].isin(['water'])]
emission_cf_N2i

In [ ]:
Ncfi=emission_cf_N2i.loc[:,"N content(kg/unit)"]
Ncfi.iloc[0]

In [ ]:
emission_cf_N2i.key=emission_cf_N2i.key.apply(ast.literal_eval)

In [ ]:
emission_cf_N2i.key.iloc[0]

In [ ]:
emission_cf_N2i['Ncf_key'] = list(zip(emission_cf_N2i.key, Ncfi))
print(emission_cf_N2i.Ncf_key)

In [ ]:
emission_cf_N2i.Ncf_key.iloc[0]

In [ ]:
cfN2i_key_list = [0]*(len(emission_cf_N2i.Ncf_key))
#print(cf_key_list)
len(cfN2i_key_list)

In [ ]:
i = 0
while i < len(emission_cf_N2i.Ncf_key):
  cfN2i_key_list[i] = [(emission_cf_N2i.Ncf_key.iloc[i])]
  if i >= len(emission_cf_N2i.Ncf_key):
    break
  i += 1
print(cfN2i_key_list)

In [ ]:
key_CFN2i_SUM =[]

In [ ]:
i = 0

while i < len(emission_cf_N2i.Ncf_key):
  key_CFN2i_SUM += cfN2i_key_list[i]
  i += 1
  if i >= len(emission_cf_N2i.Ncf_key):
    break
  
print(key_CFN2i_SUM)

In [ ]:
NOF_Ni_water = bw.Method(('N flow sustainability', 'N output footprint', 'inert N emission to hydrosphere'))

In [ ]:
NOF_Ni_water.write(key_CFN2i_SUM)
Method(('N flow sustainability', 'N output footprint', 'inert N emission to hydrosphere')).load()

####  insert nitrogen emission (N2-N) Cf to soil-lithosphere

In [ ]:
emission_cf_N2 = emission_cf

In [ ]:
emission_cf_N2i = emission_cf_N2[emission_cf_N2['name'].isin(['Nitrogen'])]
emission_cf_N2i

In [ ]:
emission_cf_N2i = emission_cf_N2i[emission_cf_N2i['Nr emission type B'].isin(['soil'])]
emission_cf_N2i

In [ ]:
Ncfi=emission_cf_N2i.loc[:,"N content(kg/unit)"]
Ncfi.iloc[0]

In [ ]:
emission_cf_N2i.key=emission_cf_N2i.key.apply(ast.literal_eval)

In [ ]:
emission_cf_N2i.key.iloc[0]

In [ ]:
emission_cf_N2i['Ncf_key'] = list(zip(emission_cf_N2i.key, Ncfi))
print(emission_cf_N2i.Ncf_key)

In [ ]:
emission_cf_N2i.Ncf_key.iloc[0]

In [ ]:
cfN2i_key_list = [0]*(len(emission_cf_N2i.Ncf_key))
#print(cf_key_list)
len(cfN2i_key_list)

In [ ]:
i = 0
while i < len(emission_cf_N2i.Ncf_key):
  cfN2i_key_list[i] = [(emission_cf_N2i.Ncf_key.iloc[i])]
  if i >= len(emission_cf_N2i.Ncf_key):
    break
  i += 1
print(cfN2i_key_list)

In [ ]:
key_CFN2i_SUM =[]

In [ ]:
i = 0

while i < len(emission_cf_N2i.Ncf_key):
  key_CFN2i_SUM += cfN2i_key_list[i]
  i += 1
  if i >= len(emission_cf_N2i.Ncf_key):
    break
  
print(key_CFN2i_SUM)

In [ ]:
NOF_Ni_water = bw.Method(('N flow sustainability', 'N output footprint', 'inert N emission to soil-lithosphere'))

In [ ]:
NOF_Ni_water.write(key_CFN2i_SUM)
Method(('N flow sustainability', 'N output footprint', 'inert N emission to soil-lithosphere')).load()

### finding the reactive nitrogen (Nr-N) emission Cf 

####  reactive nitrogen emission (Nr-N) Cf to atmosphere

In [ ]:
#emission_cf_N2Org = emission_cf_N2
#emission_cf_N2Org.replace("N2", nan_value, inplace=True)
#emission_cf_N2Org.dropna(subset = ["Molecular Formula"], inplace=True)

#emission_cf_N2Org

#filterinfDataframe = dfObj[(dfObj['Sale'] > 30) & (dfObj['Sale'] < 33) ]

emission_cf_N2Org = emission_cf_N2[~emission_cf_N2['name'].isin(['Nitrogen'])]
emission_cf_N2Org

In [ ]:
emission_cf_N2Org = emission_cf_N2Org[emission_cf_N2Org['Nr emission type B'].isin(['air'])]
emission_cf_N2Org

In [ ]:
Ncforg=emission_cf_N2Org.loc[:,"N content(kg/unit)"]
Ncforg.iloc[2]

In [ ]:
emission_cf_N2Org.key=emission_cf_N2Org.key.apply(ast.literal_eval)

In [ ]:
emission_cf_N2Org.key.iloc[0]

In [ ]:
emission_cf_N2Org['Ncf_key'] = list(zip(emission_cf_N2Org.key, Ncforg))
print(emission_cf_N2Org.Ncf_key)

In [ ]:
emission_cf_N2Org.Ncf_key.iloc[0]

In [ ]:
cforg_key_list = [0]*(len(emission_cf_N2Org.Ncf_key))
#print(cf_key_list)
len(cforg_key_list)

In [ ]:
i = 0
while i < len(emission_cf_N2Org.Ncf_key):
  cforg_key_list[i] = [(emission_cf_N2Org.Ncf_key.iloc[i])]
  if i >= len(emission_cf_N2Org.Ncf_key):
    break
  i += 1
print(cforg_key_list)

In [ ]:
keyorg_CF_SUM =[]

In [ ]:
i = 0

while i < len(emission_cf_N2Org.Ncf_key):
  keyorg_CF_SUM += cforg_key_list[i]
  i += 1
  if i >= len(emission_cf_N2Org.Ncf_key):
    break
  
print(keyorg_CF_SUM)

In [ ]:
NOF_Norg_air = bw.Method(('N flow sustainability', 'N output footprint', 'reactive N emission to atmosphere'))

In [ ]:
NOF_Norg_air.write(keyorg_CF_SUM)
Method(('N flow sustainability', 'N output footprint', 'reactive N emission to atmosphere')).load()

####  reactive nitrogen emission (Nr-N) Cf to hydrosphere

In [ ]:
#emission_cf_N2Org = emission_cf_N2
#emission_cf_N2Org.replace("N2", nan_value, inplace=True)
#emission_cf_N2Org.dropna(subset = ["Molecular Formula"], inplace=True)

#emission_cf_N2Org

#filterinfDataframe = dfObj[(dfObj['Sale'] > 30) & (dfObj['Sale'] < 33) ]

emission_cf_N2Org = emission_cf_N2[~emission_cf_N2['name'].isin(['Nitrogen'])]
emission_cf_N2Org

In [ ]:
emission_cf_N2Org = emission_cf_N2Org[emission_cf_N2Org['Nr emission type B'].isin(['water'])]
emission_cf_N2Org

In [ ]:
Ncforg=emission_cf_N2Org.loc[:,"N content(kg/unit)"]
Ncforg.iloc[2]

In [ ]:
emission_cf_N2Org.key=emission_cf_N2Org.key.apply(ast.literal_eval)

In [ ]:
emission_cf_N2Org.key.iloc[0]

In [ ]:
emission_cf_N2Org['Ncf_key'] = list(zip(emission_cf_N2Org.key, Ncforg))
print(emission_cf_N2Org.Ncf_key)

In [ ]:
emission_cf_N2Org.Ncf_key.iloc[0]

In [ ]:
cforg_key_list = [0]*(len(emission_cf_N2Org.Ncf_key))
#print(cf_key_list)
len(cforg_key_list)

In [ ]:
i = 0
while i < len(emission_cf_N2Org.Ncf_key):
  cforg_key_list[i] = [(emission_cf_N2Org.Ncf_key.iloc[i])]
  if i >= len(emission_cf_N2Org.Ncf_key):
    break
  i += 1
print(cforg_key_list)

In [ ]:
keyorg_CF_SUM =[]

In [ ]:
i = 0

while i < len(emission_cf_N2Org.Ncf_key):
  keyorg_CF_SUM += cforg_key_list[i]
  i += 1
  if i >= len(emission_cf_N2Org.Ncf_key):
    break
  
print(keyorg_CF_SUM)

In [ ]:
keyorg_CF_SUM += fish_pelagic+fish_demersal
print(keyorg_CF_SUM)

In [ ]:
NOF_Norg_water = bw.Method(('N flow sustainability', 'N output footprint', 'reactive N emission to hydrosphere'))

In [ ]:
NOF_Norg_water.write(keyorg_CF_SUM)
Method(('N flow sustainability', 'N output footprint', 'reactive N emission to hydrosphere')).load()

####  reactive nitrogen emission (Nr-N) Cf to soil-lithosphere

In [ ]:
#emission_cf_N2Org = emission_cf_N2
#emission_cf_N2Org.replace("N2", nan_value, inplace=True)
#emission_cf_N2Org.dropna(subset = ["Molecular Formula"], inplace=True)

#emission_cf_N2Org

#filterinfDataframe = dfObj[(dfObj['Sale'] > 30) & (dfObj['Sale'] < 33) ]

emission_cf_N2Org = emission_cf_N2[~emission_cf_N2['name'].isin(['Nitrogen'])]
emission_cf_N2Org

In [ ]:
emission_cf_N2Org = emission_cf_N2Org[emission_cf_N2Org['Nr emission type B'].isin(['soil'])]
emission_cf_N2Org

In [ ]:
Ncforg=emission_cf_N2Org.loc[:,"N content(kg/unit)"]
Ncforg.iloc[2]

In [ ]:
emission_cf_N2Org.key=emission_cf_N2Org.key.apply(ast.literal_eval)

In [ ]:
emission_cf_N2Org.key.iloc[0]

In [ ]:
emission_cf_N2Org['Ncf_key'] = list(zip(emission_cf_N2Org.key, Ncforg))
print(emission_cf_N2Org.Ncf_key)

In [ ]:
emission_cf_N2Org.Ncf_key.iloc[0]

In [ ]:
cforg_key_list = [0]*(len(emission_cf_N2Org.Ncf_key))
#print(cf_key_list)
len(cforg_key_list)

In [ ]:
i = 0
while i < len(emission_cf_N2Org.Ncf_key):
  cforg_key_list[i] = [(emission_cf_N2Org.Ncf_key.iloc[i])]
  if i >= len(emission_cf_N2Org.Ncf_key):
    break
  i += 1
print(cforg_key_list)

In [ ]:
keyorg_CF_SUM =[]

In [ ]:
i = 0

while i < len(emission_cf_N2Org.Ncf_key):
  keyorg_CF_SUM += cforg_key_list[i]
  i += 1
  if i >= len(emission_cf_N2Org.Ncf_key):
    break
  
print(keyorg_CF_SUM)

In [ ]:
NOF_Norg_soil = bw.Method(('N flow sustainability', 'N output footprint', 'reactive N emission to soil-lithosphere'))

In [ ]:
NOF_Norg_soil.write(keyorg_CF_SUM)
Method(('N flow sustainability', 'N output footprint', 'reactive N emission to soil-lithosphere')).load()

In [ ]:
#Method(('N flow sustainability', 'N input footprint', 'reactive N resource input from atmosphere')).deregister()

In [ ]:
m_list=[m for m in bw.methods if 'N flow sustainability' in str(m)]
m_list

In [ ]:
len(m_list)

# bio-jet fuel proudction life cycle definition

### straw feedstock production

#### allocation ratios under different decision

In [ ]:
eidb = bw.Database('ecoinvent 3.7.1 apos')
bio = bw.Database('biosphere3')

In [ ]:
# mass base

In [ ]:
R_straw= newFloatParam(
    "R_straw", 
    distrib=DistributionType.NORMAL,
    default=0.84,
    std=0.23,
    min=0.5, max=1.55,
    label_fr="ratio of maize straw to grain Chinese average", 
    group="feed stock production", 
    unit="%") 

In [ ]:
R_cob= newFloatParam(
    "R_cob", 
    distrib=DistributionType.NORMAL,
    default=0.2105,
    std=0.056,
    #min=0.301, max=0.49,
    label_fr="ratio of maize cob to grain Chinese average", 
    group="feed stock production", 
    unit="%") 

In [ ]:
R_Straw_return= newFloatParam(
    "R_Straw_return", 
    distrib=DistributionType.TRIANGLE,
    default=0.30,   # Virtual nitrogen factors and nitrogen footprints associated with nitrogen loss and food wastage of China’s main food crops
    min=0.176, max=0.50,
    label_fr="maize straw field-returning ratio after harvest", 
    group="feed stock production", 
    unit="%") 

In [ ]:
AC_straw = R_straw/(1+R_straw+R_cob)

In [ ]:
# environmental burden free

In [ ]:
# energy base

In [ ]:
H_grain= newFloatParam(
    "H_grain", 
    distrib=DistributionType.FIXED,
    default=16.86, 
    #min=0.301, max=0.49,
    label_fr="low heating value of maize seed in China", 
    group="feed stock production", 
    unit="MJ/kg") 

In [ ]:
H_straw= newFloatParam(
    "H_straw", 
    distrib=DistributionType.FIXED,
    default=16.36, 
    #min=0.301, max=0.49,
    label_fr="low heating value of maize straw in China", 
    group="feed stock production", 
    unit="MJ/kg") 

In [ ]:
H_cob= newFloatParam(
    "H_cob", 
    distrib=DistributionType.FIXED,
    default=16.90, 
    #min=0.301, max=0.49,
    label_fr="low heating value of maize cob in China", 
    group="feed stock production", 
    unit="MJ/kg") 

In [ ]:
# economic value base

In [ ]:
V_grain= newFloatParam(
    "V_grain", 
    distrib=DistributionType.FIXED,
    default=2100, 
    #min=0.301, max=0.49,
    label_fr="market proice of maize seed in China", 
    group="feed stock production", 
    unit="MJ/kg") 

In [ ]:
V_cob= newFloatParam(
    "V_cob", 
    distrib=DistributionType.FIXED,
    default=450, 
    #min=0.301, max=0.49,
    label_fr="market proice of maize cob in China", 
    group="feed stock production", 
    unit="RMB") 

In [ ]:
V_straw= newFloatParam(
    "V_straw", 
    distrib=DistributionType.FIXED,
    default=450, 
    #min=0.301, max=0.49,
    label_fr="market proice of maize straw in China", 
    group="feed stock production", 
    unit="RMB") 

In [ ]:
# Undertake-all environmental burden

#### resource input and emission

In [ ]:
TM_corngrain= newFloatParam(
    "TM_corngrain", 
    distrib=DistributionType.NORMAL,
    default=4824, 
    std=1860,
    min=0,
    label_fr="cron grain production per 100 KgN applied", 
    group="feed stock production", 
    unit="kg straw/100 kgN") 

In [ ]:
N_Straw= newFloatParam(
    "N_Straw", 
    distrib=DistributionType.LINEAR,
    default=0.0061,   # Virtual nitrogen factors and nitrogen footprints associated with nitrogen loss and food wastage of China’s main food crops
    min=0.004, max=0.0077,
    label_fr="nitrogen content in maize straw Chinese average", 
    group="feed stock production", 
    unit="%") 

In [ ]:
N_strawFR_SM_100= newFloatParam(
    "N_strawFR_SM_100", 
    distrib=DistributionType.FIXED,
    default=16.5/0.74,   # Virtual nitrogen factors and nitrogen footprints associated with nitrogen loss and food wastage of China’s main food crops
    #min=0.004, max=0.0077,
    label_fr="nitrogen content in wheat straw havested after winter wheat production in China", 
    group="feed stock production", 
    unit="kg N/100 kg N applied") 

In [ ]:
R_SM = newFloatParam(
    "R_SM", 
    distrib=DistributionType.FIXED,
    default= 62.7 / (62.7+160.2),   # Virtual nitrogen factors and nitrogen footprints associated with nitrogen loss and food wastage of China’s main food crops
    #min=0.004, max=0.0077,
    label_fr="ratio of grain production from WWSM rotation to grain production from other production system", 
    group="feed stock production", 
    unit="%") 

In [ ]:
ER_NH3 = newFloatParam(
    "ER_NH3", 
    distrib=DistributionType.LINEAR,
    default= 0.870,   
    min=0.743, max=0.946,
    label_fr="ratio of agricultural soil NH3 emission under total straw field return to NH3 emission without straw field return", 
    group="feed stock production", 
    unit="%") 

In [ ]:
ER_N2O = newFloatParam(
    "ER_N2O", 
    distrib=DistributionType.TRIANGLE,
    default= 1.260,   
    min=1.120, max=1.411,
    label_fr="ratio of agricultural soil N2O emission under total straw field return to N2O emission without straw field return", 
    group="feed stock production", 
    unit="%") 

In [ ]:
ER_NO3 = newFloatParam(
    "ER_NO3", 
    distrib=DistributionType.TRIANGLE,
    default= 1.780,     
    min=1.364, max=2.173,
    label_fr="ratio of agricultural soil NO3- leaking under total straw field return to NO3- leaking without straw field return", 
    group="feed stock production", 
    unit="%") 

In [ ]:
ER_NO2_N2 = newFloatParam(
    "ER_NO2_N2", 
    distrib=DistributionType.NORMAL,
    default= 0.375,     
    std=0.035,
    label_fr="ratio of agricultural soil N20 emission to (N2O+N2) emission global average", 
    group="feed stock production", 
    unit="%") 

In [ ]:
CO2_maizep_total = newFloatParam(
    "CO2_maizep_total", 
    distrib=DistributionType.TRIANGLE,
    default= 638.12/1000,     
    min=450/1000, max=750/1000,
    label_fr="CO2 emission of maize filed per kg maize grain", 
    group="feed stock production", 
    unit="kg") 

In [ ]:
# ammonia, anhydrous, liquid 
# NH3   14.0067/17.03 = kg N/ kg substance   0.00844 kg

# ammonium nitrate
# NH4NO3  14.0067/80.04 = kg N/ kg substance  0.014 kg

# urea
# CH4N2O 14.0067/60.06 = kg N/ kg substance   0.0076 kg

# in ecoinvent they allocate all the environmental burdern, i.e., all the product/resource input and emissions during the maize growing process to maize grain.

# N_KGcgrain_eidb N fertilizer applized per kg grain unallocated
N_KGcgrain_eidb = 14.0067/17.03*0.00844 + 2*14.0067/80.04*0.014 + 2*14.0067/60.06*0.0076
N_KGcgrain_eidb

In [ ]:
N_hacgrain_eidb = N_KGcgrain_eidb*9315
N_hacgrain_eidb

In [ ]:
P2O5_hacgrain_eidb = 0.00585*9315
P2O5_hacgrain_eidb

In [ ]:
# kg N sedimentation per kg grain unallocated base on Zhang et al., used in bachelor thesis

N_self_D = 16/4824
N_self_D

In [ ]:
# kg N sedimentation per kg grain unallocated base on ENFI detabase
test = 0.626*0.2697553849
test

In [ ]:
# N in wheat straw returned to field at the begining of summer maize production during WWSM production
FRN_WWSM_eidbgrain = N_KGcgrain_eidb*N_strawFR_SM_100/100

In [ ]:
FRN_OM_eidbgrain = R_straw*N_Straw*R_Straw_return

In [ ]:
# 农田N沉降按照现有氮数据版本计算，具体强度留待后续更新
#Nr_Adep = 16/TM_corngrain
    
#Nr_Adep =Sur_dep * land_occup


In [ ]:
eidb_cgrain = findActivity(name='maize grain production', loc='RoW', db_name='ecoinvent 3.7.1 apos')

In [ ]:
eidb_cgrain_v2 = findActivity(name='maize grain production', loc='RoW', db_name='ecoinvent 3.7.1 apos')

In [ ]:
CO2_air = [act for act in bio if 'Carbon dioxide, from soil or biomass stock' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'low population density, long-term' in str(act['categories'])
         ][0]
CO2_air

In [ ]:
# only need and can only run once！！
Nsoil_node= bio.new_activity(code='reactive_nitrogen_in_agricultural_soil',
        name='Nitrogen, reactive form, in agricultural soil',
        categories=('natural resource', 'in ground'),
        unit = 'kilogram',
        type="natural resource",
        
)

Nsoil_node.save()

In [ ]:
len(bio)

In [ ]:
N_recycling = [act for act in bio if 'Nitrogen, reactive form, in agricultural soil' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'natural resource' in str(act['categories'])
                                            and 'in ground' in str(act['categories'])
         ][0]
N_recycling

In [ ]:
NO3 = [act for act in bio if 'Nitrate' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'water' in str(act['categories'])
                                            and 'ground' not in str(act['categories'])
                                            and 'ocean' not in str(act['categories'])
                                            and 'surface' not in str(act['categories'])
         ][0]
NO3

In [ ]:
N2_air = [act for act in bio if 'Nitrogen' in act['name'] 
                                            and 'fluoride' not in act['name']
                                            and 'oxides' not in act['name']
                                            and 'air' in str(act['categories'])
                                            and 'natural resource' not in str(act['categories'])
                                            #and 'ocean' not in str(act['categories'])
                                            #and 'surface' not in str(act['categories'])
         ][0]
N2_air

In [ ]:
#P2O5_eidbgrain = 0.00585

In [ ]:
#P2O5_CN = 60.00/6110.30920540579
#P2O5_CN

In [ ]:
T_infield = [act for act in eidb if 'market for transport, tractor and trailer, agricultural' in act['name'] 
                                            
                                            and 'RoW' in str(act['location'])
                                           # and 'in ground' in str(act['categories'])
         ][0]
T_infield

In [ ]:
KCl_RoW = findActivity(name='market for potassium chloride', loc='RoW', db_name='ecoinvent 3.7.1 apos')
KCl_RoW

In [ ]:
lime_RoW = findActivity(name='market for lime', loc='RoW', db_name='ecoinvent 3.7.1 apos')
lime_RoW

In [ ]:
P2O5_RoW_LCI = findActivity(name='market for inorganic phosphorus fertiliser, as P2O5', loc='RoW', db_name='ecoinvent 3.7.1 apos')
P2O5_RoW_LCI

In [ ]:
P2O5_CN_LCI = findActivity(name='market for inorganic phosphorus fertiliser, as P2O5', loc='CN', db_name='ecoinvent 3.7.1 apos')
P2O5_CN_LCI

In [ ]:
ammonium_nitrate_RoW = findActivity(name='market for ammonium nitrate', loc='RoW', db_name='ecoinvent 3.7.1 apos')
ammonium_nitrate_RoW

In [ ]:
ammonium_nitrate_CN = findActivity(name='market for ammonium nitrate', loc='CN', db_name='ecoinvent 3.7.1 apos')
ammonium_nitrate_CN

In [ ]:
ammonium_nitrate_CN2 = findActivity(name='market for ammonium nitrate', loc='CN', db_name='ecoinvent 3.7.1 apos')
ammonium_nitrate_CN2

In [ ]:
NH3_RoW = findActivity(name='market for ammonia, anhydrous, liquid', loc='RoW', db_name='ecoinvent 3.7.1 apos')
NH3_RoW

In [ ]:
NH3_CN = findActivity(name='market for ammonia, anhydrous, liquid', loc='CN', db_name='ecoinvent 3.7.1 apos')
NH3_CN

##### update the inventory to Chinese context

In [ ]:
ammonium_CN_m=0.004182+0.007568+0.002229
ammonium_CN_m

In [ ]:
urea_CN_m=0.006417+0.001178
urea_CN_m

In [ ]:
ammonia_CN_m=0.001833+0.001053+0.000223+0.005276
ammonia_CN_m

In [ ]:
irr_water = 0.000034+0.181342+0.001339+0.002972+0.000149+0.008438+0.001373+0.006699+0.002289+0.006138+0.033970
irr_water

In [ ]:
water_MJfuel_test=0.244743*11.76*85.03/((1+0.2105+0.84)*(43.07*85.03+46*19.83+43.1*19.83+7.99*44.857))
water_MJfuel_test

In [ ]:
list_parameters()

In [ ]:
P2O5_CN_m = 0.000068+0.000054+0.000001+0.000014+0.000002+0.000032+0.000030+0.000028+0.000008+0.000011+0.000062+0.000043+0.000004+0.000063+0.000067+0.000277+0.003927+0.000010+0.001006+0.000018+0.000002+0.000000+0.000004+0.000116

In [ ]:
maize_all = copyActivity(
    USER_DB, # base utilisateur
    eidb_cgrain, # activité initiale 
    'maize production in china, unallocated whole plant with grain, straw, and cob, flow localizated') # Nouveau nom
#printAct(MonoSi_ingot_CN)

#MonoSi_ingot_CN.updateExchanges({
#   'market group for electricity, medium voltage' : Elec_MonoSi_ingot})
#printAct(MonoSi_ingot_CN, MonoSi_ingot_init_CN)

maize_all.addExchanges({ 
    
    CO2_air : CO2_maizep_total,
    N_recycling : (R_SM*FRN_WWSM_eidbgrain +(1-R_SM)*FRN_OM_eidbgrain),
    N2_air : 10 # demo value 反硝化作用
    #NO3 : (R_SM*N_KGcgrain_eidb*26.9/100 + (1-R_SM)*N_KGcgrain_eidb*(11.8/(1+(ER_NO3-1)*0.3)*(1+(ER_NO3-1)*R_Straw_return))/100)/(14.0067/62.004),
    #P2O5_RoW_LCI : 100
    #iron : Q_iron_gear
})

In [ ]:
maize_all_CN = copyActivity(
    USER_DB, # base utilisateur
    maize_all, # activité initiale 
    'maize production in china, unallocated whole plant')

N2O_KGcgrain_eidb = (R_SM*N_KGcgrain_eidb*0.9/100 + (1-R_SM)*N_KGcgrain_eidb*(0.9/(1+(ER_N2O-1)*0.3)*(1+(ER_N2O-1)*R_Straw_return))/100)/(14.0067/44.013)

maize_all_CN.updateExchanges({ 
    
    'transport, tractor and trailer, agricultural' : T_infield,
    'potassium chloride': KCl_RoW,
    'lime': lime_RoW,
    
    'inorganic phosphorus fertiliser, as P2O5''#CN':P2O5_CN_m,
    'inorganic phosphorus fertiliser, as P2O5''#AU':0,
    'inorganic phosphorus fertiliser, as P2O5''#BD':0,
    'inorganic phosphorus fertiliser, as P2O5''#IL':0,
    'inorganic phosphorus fertiliser, as P2O5''#KE':0,
    'inorganic phosphorus fertiliser, as P2O5''#LK':0,
    'inorganic phosphorus fertiliser, as P2O5''#MX':0,
    'inorganic phosphorus fertiliser, as P2O5''#MY':0,
    'inorganic phosphorus fertiliser, as P2O5''#NZ':0,
    'inorganic phosphorus fertiliser, as P2O5''#PE':0,
    'inorganic phosphorus fertiliser, as P2O5''#PH':0,
    'inorganic phosphorus fertiliser, as P2O5''#RU':0,
    'inorganic phosphorus fertiliser, as P2O5''#TH':0,
    'inorganic phosphorus fertiliser, as P2O5''#CI':0,
    'inorganic phosphorus fertiliser, as P2O5''#TR':0,
    'inorganic phosphorus fertiliser, as P2O5''#VN':0,
    'inorganic phosphorus fertiliser, as P2O5''#RER':0,
    'inorganic phosphorus fertiliser, as P2O5''#RoW':0,
    'inorganic phosphorus fertiliser, as P2O5''#CL':0,
    'inorganic phosphorus fertiliser, as P2O5''#CO':0,
    'inorganic phosphorus fertiliser, as P2O5''#CR':0,
    'inorganic phosphorus fertiliser, as P2O5''#CY':0,
    'inorganic phosphorus fertiliser, as P2O5''#EC':0,
    'inorganic phosphorus fertiliser, as P2O5''#ID':0,
    
    'ammonium nitrate''#CN' : ammonium_CN_m,
    'ammonium nitrate''#RoW' : 0, # for China, the demostic ammonium nitrate comspumption is self-satified, the ammonium nitrate import is less than 1% on comsumption side, which is ignored. 
    'ammonium nitrate''#RER' : 0,
    
    'urea''#RER':0,
    'urea''#CN':urea_CN_m,
    
    #'irrigation''#!CN':0,
    
    #'ammonium nitrate''#RoW' : 0.01175，
    'ammonia, anhydrous, liquid''#CN' : ammonia_CN_m,
    'ammonia, anhydrous, liquid''#RoW' : 0, 
    'ammonia, anhydrous, liquid''#UN-SEASIA' :0,
    'ammonia, anhydrous, liquid''#RER' :0,
    
    'drying of maize straw and whole-plant' : 0.409840*(1+R_straw+R_cob), # to onclude the drying of maize straw and maize cob
    'Ammonia' : (R_SM*N_KGcgrain_eidb*7.4/100 + (1-R_SM)*N_KGcgrain_eidb*(5.2/(1-(1-ER_NH3)*0.3)*(1-(1-ER_NH3)*R_Straw_return))/100)/(14.0067/17.0305),
    'Dinitrogen monoxide' : N2O_KGcgrain_eidb,
    'Nitrate': (R_SM*N_KGcgrain_eidb*26.9/100 + (1-R_SM)*N_KGcgrain_eidb*(11.8/(1+(ER_NO3-1)*0.3)*(1+(ER_NO3-1)*R_Straw_return))/100)/(14.0067/62.004),
    'Nitrogen' : (1-ER_NO2_N2)/ER_NO2_N2*N2O_KGcgrain_eidb,
    #iron : Q_iron_gear
    
    'irrigation''#CH':irr_water,
    'irrigation''#CN':0,
    'irrigation''#TN':0,
    'irrigation''#CO':0,
    'irrigation''#DE':0,
    'irrigation''#ES':0,
    'irrigation''#FR':0,
    'irrigation''#MA':0,
    'irrigation''#MY':0,
    'irrigation''#PE':0,
    'irrigation''#PH':0,
    
    
})

printAct(maize_all, maize_all_CN)

#### inventory mass allocation

In [ ]:
M_strawp_LCI = newActivity(USER_DB, "maize straw production, harvest,transported from field to farm, mass allocation", "kg straw")

M_strawp_LCI.addExchanges({ 
    

    maize_all_CN:R_straw/(1+R_straw+R_cob)/R_straw,
    
})

#### inventory energy allocation

In [ ]:
H_strawp_LCI = newActivity(USER_DB, "maize straw production, harvest,transported from field to farm, enery allocation", "kg straw")

H_strawp_LCI.addExchanges({ 
    

    maize_all_CN:R_straw*H_straw/(1*H_grain+R_straw*H_straw+R_cob*H_cob)/R_straw,
    
})

#### inventory market value allocation

In [ ]:
V_strawp_LCI = newActivity(USER_DB, "maize straw production, harvest,transported from field to farm, market value allocation", "kg straw")

V_strawp_LCI.addExchanges({ 
    

    maize_all_CN:R_straw*V_straw/(1*V_grain+R_straw*V_straw+R_cob*V_cob)/R_straw,
    
})

### straw feedstock preparation and transport

In [ ]:
DR_feed = newFloatParam(
    "DR_feed", #R_daily
    distrib=DistributionType.TRIANGLE,
    default= 500,   
    min=305.556, max=638.889,
    label_fr="daily maize straw consumption ratio of the plant gasification FT bio-jet plant", 
    group="feed stock preparation and transport", 
    unit="t/day") 

In [ ]:
CR_straw = newFloatParam(
    "CR_straw", 
    distrib=DistributionType.TRIANGLE,
    default= 0.83,   
    min=0.6, max=0.9,
    label_fr="collect ratio of maize straw Chinese average", 
    group="feed stock preparation and transport", 
    unit="%") 

In [ ]:
Diesel_C = newFloatParam(
    "Diesel_C", 
    distrib=DistributionType.TRIANGLE,
    default= 0.67,   
    min=0.5, max=2.4,
    label_fr="diesel consumption ratio of maize straw shredding and grinding Chinese average", 
    group="feed stock preparation and transport", 
    unit="kg/ t straw") 

In [ ]:
Elec_C = newFloatParam(
    "Elec_C", 
    distrib=DistributionType.TRIANGLE,
    default= 18.2,   
    min=7, max=22,
    label_fr="electricity consumption of maize straw shredding and grinding Chinese average", 
    group="feed stock preparation and transport", 
    unit="kWh/ t straw") 

In [ ]:
TD_straw = 26*sqrt(DR_feed)/sqrt(500) *sqrt(574.75)/sqrt(611.031) *sqrt(1.2)/sqrt(R_straw)*sqrt(0.83)/sqrt(1-R_Straw_return)
TD_straw

In [ ]:
Elec_crush_pack = findActivity(name='market group for electricity, low voltage', loc='CN', db_name='ecoinvent 3.7.1 apos')
Diesel_crush_pack = findActivity(name='market for diesel, burned in agricultural machinery', loc='GLO', db_name='ecoinvent 3.7.1 apos')
T_straw_lorry = findActivity(name='market for transport, freight, lorry 7.5-16 metric ton, EURO5', loc='RoW', db_name='ecoinvent 3.7.1 apos')

In [ ]:
M_strawT_LCI = newActivity(USER_DB, "maize straw production, harvest,transport to gasification plant, mass allocation", "kg straw")

M_strawT_LCI.addExchanges({ 
    
    M_strawp_LCI : 1,
    
    Elec_crush_pack : Elec_C/1000,
    Diesel_crush_pack : Diesel_C/1000*(1/0.0222), # diesel density 1/0.0222 
    
    T_straw_lorry : TD_straw*1/1000
    
})

### straw gasification and FT synthesis bio-jet production

In [ ]:
R_biojet_straw = newFloatParam(
    "R_biojet_straw", 
    distrib=DistributionType.TRIANGLE,
    default= 11.76,   
    min=10.8, max=15,
    label_fr=" maize straw consumption ratio per metric tonne bio-jet produced", 
    group="bio-jet production", 
    unit="t straw/t bio-jet") 

In [ ]:
Life_FTplant = newFloatParam(
    "Life_FTplant", 
    distrib=DistributionType.TRIANGLE,
    default= 15,   
    min=13, max=17,
    label_fr=" life time of the bio-jet fuel plant", 
    group="bio-jet production", 
    unit="year") 

In [ ]:
Elec_milling = newFloatParam(
    "Elec_milling", 
    distrib=DistributionType.TRIANGLE,
    default= 42,   
    min=8.87, max=0.0794/1.51*1000,
    label_fr="electricity consumption of maize straw shreding and grinding before gasification", 
    group="bio-jet production", 
    unit="Kwh/ t straw") 

In [ ]:
P_steel = newFloatParam(
    "P_steel", 
    distrib=DistributionType.TRIANGLE,
    default= 484,   
    min=1877*(484/2016), max=2089*(484/2016),
    label_fr="steel consumed during bio-jet plant construction covering the replacement need during plant infracture maintenance", 
    group="bio-jet production", 
    unit="metric ton") 

In [ ]:
P_iron = newFloatParam(
    "P_iron", 
    distrib=DistributionType.TRIANGLE,
    default= 3,   
    min=1877*(3/2016), max=2089*(3/2016),
    label_fr="iron consumed during bio-jet plant construction covering the replacement need during plant infracture maintenance", 
    group="bio-jet production", 
    unit="metric ton") 

In [ ]:
P_Al = newFloatParam(
    "P_Al", 
    distrib=DistributionType.TRIANGLE,
    default= 6,   
    min=1877*(6/2016), max=2089*(6/2016),
    label_fr="aluminum consumed during bio-jet plant construction covering the replacement need during plant infracture maintenance", 
    group="bio-jet production", 
    unit="metric ton") 

In [ ]:
P_cement = newFloatParam(
    "P_cement", 
    distrib=DistributionType.TRIANGLE,
    default= 1523,   
    min=1877*(1523/2016), max=2089*(1523/2016),
    label_fr="aluminum consumed during bio-jet plant construction covering the replacement need during plant infracture maintenance", 
    group="bio-jet production", 
    unit="metric ton") 

In [ ]:
CoRu_catalyst = newFloatParam(
    "CoRu_catalyst", 
    distrib=DistributionType.TRIANGLE,
    default= 3,   
    min=1, max=5,
    label_fr="Co-Ru ZrO2 /γ-Al2O3 catalyst comsumption during gasification syngas FT synthesis", 
    group="bio-jet production", 
    unit="kg loss / t biojet fuel produced") 

In [ ]:
NiW_catalyst = newFloatParam(
    "NiW_catalyst", 
    distrib=DistributionType.TRIANGLE,
    default= 3,   
    min=1, max=5,
    label_fr="NiW-HY+Al2O3 catalyst comsumption during hydrocracking of paraffin produced from FT synthesis", 
    group="bio-jet production", 
    unit="kg loss / t biojet fuel produced") 

In [ ]:
H_biojet = newFloatParam(
    "H_biojet", 
    distrib=DistributionType.FIXED,
    default= 43.07,   
    #min=1, max=5,
    label_fr="heating value of bio-jet fuel produced from FT synthesis", 
    group="bio-jet production", 
    unit="MJ / kg biojet fuel") 

In [ ]:
# Total jet fuel production in kg during the entire life span of the plant
jetfuelp_total = DR_feed/R_biojet_straw*365*Life_FTplant*1000

# Total maize straw comsumed in kg during the entire life span of the plant
strawc_total = DR_feed*365*Life_FTplant*1000

In [ ]:
Steel_building = findActivity(name='market for steel, low-alloyed, hot rolled', loc='GLO', db_name='ecoinvent 3.7.1 apos')
Iron_building  = findActivity(name='market for cast iron', loc='GLO', db_name='ecoinvent 3.7.1 apos')
Al_building = findActivity(name='market for aluminium, wrought alloy', loc='GLO', db_name='ecoinvent 3.7.1 apos')
cement_building = findActivity(name='market for cement, Portland', loc='RoW', db_name='ecoinvent 3.7.1 apos')

# end of life treatment 
Steel_EoL = findActivity(name='market for scrap steel', loc='RoW', db_name='ecoinvent 3.7.1 apos')
Al_EoL = findActivity('market for scrap aluminium',loc='RoW', db_name='ecoinvent 3.7.1 apos')
cement_EoL =  findActivity('treatment of inert waste, inert material landfill',loc='RoW', db_name='ecoinvent 3.7.1 apos')
# Under Chinese context, rarely of the waste cement is recycled around year 2016. 
#And we consider the landfill treatment as the EoL treatment for all cement used in the plant

Si_sand = findActivity(name='market for silica sand', loc='GLO', db_name='ecoinvent 3.7.1 apos')
Water_p = findActivity(name='water production, deionised', loc='RoW', db_name='ecoinvent 3.7.1 apos')
Elec_millp = findActivity(name='market group for electricity, low voltage', loc='CN', db_name='ecoinvent 3.7.1 apos')

Co_catalyst = findActivity(name='market for cobalt', loc='GLO',db_name='ecoinvent 3.7.1 apos')
# There is no LCI inventory to any kind of ruthenium related materials production, e.g., reuthenium ore or reuthenium chloride,ruthenium menta, in any version of ecoinvent database,
# which means the environemntal bourden of the ruthenium metal production can not be considered.
# Considering that the usage of catalyst during bio-jet fuel production is very rare, and the mass contribution of Ru in the Co-Ru/γ-Al2O3 catalyst is already very rare, 
# Ru resourcce in Ground is cited as the natural resource from in our LCI. 

Ru_catalyst = [act for act in bio if 'Ruthenium, in ground' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            #and 'urban air close to ground' in str(act['categories'])
      ][0]

ZrO2_catalyst = findActivity(name='zirconium oxide production', loc='RoW',db_name='ecoinvent 3.7.1 apos')
Al2O3_catalyst = findActivity(name='market for aluminium oxide, metallurgical', loc='RoW',db_name='ecoinvent 3.7.1 apos')

HY_catalyst = findActivity(name='market for zeolite, powder', loc='GLO',db_name='ecoinvent 3.7.1 apos')
NiO_catalyst = findActivity(name='market for nickel, class 1', loc='GLO',db_name='ecoinvent 3.7.1 apos')
W_catalyst = findActivity(name='market for ammonium paratungstate', loc='GLO',db_name='ecoinvent 3.7.1 apos')


EoL_catalyst = findActivity(name='market for municipal solid waste', loc='RoW',db_name='ecoinvent 3.7.1 apos')


In [ ]:
Ru_catalyst = [act for act in bio if 'Ruthenium, in ground' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            #and 'urban air close to ground' in str(act['categories'])
      ][0]
Ru_catalyst

In [ ]:
Elec_millpCA = copyActivity(
    USER_DB, # base utilisateur
    Elec_millp, # activité initiale 
    'market group for electricity, low voltage CA') # Nouveau nom
#printAct(MonoSi_ingot_CN)

In [ ]:
CO2_e = [act for act in bio if 'Carbon dioxide, from soil or biomass stock' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'low population density, long-term' in str(act['categories'])
      ][0]
CO2_e

In [ ]:
CH4_e = [act for act in bio if 'Methane, from soil or biomass stock' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'low population density, long-term' in str(act['categories'])
      ][0]
CH4_e

In [ ]:
N2O_e = [act for act in bio if 'Dinitrogen monoxide' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'low population density, long-term' in str(act['categories'])
      ][0]
N2O_e

In [ ]:
NOx_e = [act for act in bio if 'Nitrogen oxides' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'low population density, long-term' in str(act['categories'])
      ][0]
NOx_e

In [ ]:
SO2_e = [act for act in bio if 'Sulfur dioxide' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'low population density, long-term' in str(act['categories'])
      ][0]
SO2_e

In [ ]:
CO_e = [act for act in bio if 'Carbon monoxide, from soil or biomass stock' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'low population density, long-term' in str(act['categories'])
      ][0]
CO_e

In [ ]:
HC_e = [act for act in bio if 'Hydrocarbons, unspecified' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'air' in str(act['categories'])
                                            #and 'non-urban air or from high stacks' not in str(act['categories'])
      
       ][0]
HC_e

In [ ]:
PM10_e = [act for act in bio if 'Particulates, > 2.5 um, and < 10um' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'low population density, long-term' in str(act['categories'])
                                            #and 'non-urban air or from high stacks' not in str(act['categories'])
      
       ][0]
PM10_e

In [ ]:
COD_e = [act for act in bio if 'COD, Chemical Oxygen Demand' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'surface water' in str(act['categories'])
                                            #and 'non-urban air or from high stacks' not in str(act['categories'])
      
       ][0]
COD_e

In [ ]:
solidwaste_EoL =  findActivity('treatment of inert waste, inert material landfill',loc='RoW', db_name='ecoinvent 3.7.1 apos')

In [ ]:
TN_e = [act for act in bio if 'Nitrogen, organic bound' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'surface water' in str(act['categories'])
                                            #and 'non-urban air or from high stacks' not in str(act['categories'])
      
       ][0]
TN_e

In [ ]:
NH4_e = [act for act in bio if 'Ammonium, ion' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'surface water' in str(act['categories'])
                                            #and 'non-urban air or from high stacks' not in str(act['categories'])
      
       ][0]
NH4_e

In [ ]:
Q_catalyst_Co = 0.2*CoRu_catalyst/R_biojet_straw/1000
Q_catalyst_Ru = 0.005*CoRu_catalyst/R_biojet_straw/1000
Q_catalyst_ZrO2 = 0.115*CoRu_catalyst/R_biojet_straw/1000
Q_catalyst_Al2O3 = 0.68*CoRu_catalyst/R_biojet_straw/1000 + 0.64*NiW_catalyst/R_biojet_straw/1000

Q_catalyst_NiO = 58.69/74.69*0.07*NiW_catalyst/R_biojet_straw/1000
Q_catalyst_W = 11634.64/48/231.84*(0.23*NiW_catalyst/R_biojet_straw/1000)
Q_catalyst_HY = 0.06*NiW_catalyst/R_biojet_straw/1000

In [ ]:
#Ru_catalyst = [act for act in bio if 'Ruthenium, in ground' in act['name'] 
#                                            #and 'fossil' in act['name']
#                                            #and 'non' not in act['name']
#                                            #and 'urban air close to ground' in str(act['categories'])
#      ][0]
#Ru_catalyst

In [ ]:
# kg / kg maize straw consumed without allocation
CO2_pe = (1086.85 - 940.1/1000 *1156.10)/0.37/11.76/1000  
CH4_pe = (3.4 - 2.72/1000 *1156.10)/0.37/11.76/1000 
N2O_pe = (0.02 - 0.014/1000 *1156.10)/0.37/11.76/1000
NOx_pe = (4.29 - 2.63/1000 *1156.10)/0.37/11.76/1000
SO2_pe = (4.71 - 3.21/1000 *1156.10)/0.37/11.76/1000
CO_pe  = (0.21 - 0.175/1000 *1156.10)/0.37/11.76/1000
HC_pe  = (0.04  - 0.034/1000 *1156.10)/0.37/11.76/1000
PM10_pe= (1.48  - 1.034/1000 *1156.10)/0.37/11.76/1000
COD_pe= (0.32  - 0.084/1000 *1156.10)/0.37/11.76/1000
soildw_pe= (38.51  - 194.69/1000 *1156.10)/0.37/11.76/1000
TN_pe= 0.05/0.37/11.76/1000
NH3N_pe= 0.04/0.37/11.76/1000

In [ ]:
M_fuelp_LCI = newActivity(USER_DB, "maize straw gasification and Fischer-Tropsch bio-jet fuel production", "kg straw")

M_fuelp_LCI.addExchanges({ 
    
    Steel_building: P_steel*1000/strawc_total,
    Iron_building: P_iron*1000/strawc_total,
    Al_building: P_Al*1000/strawc_total,
    cement_building: P_cement*1000/strawc_total,
    Steel_EoL: -(P_steel+P_iron)*1000/strawc_total,
    Al_EoL: -P_Al*1000/strawc_total,
    cement_EoL: -P_cement*1000/strawc_total,
    
    M_strawT_LCI: 1,
    Elec_millp: Elec_milling/1000,
    
    Si_sand: 500/((7200/8760)*24)*5*1000/strawc_total,
    Water_p: (5292.23/11760)*1,          
    
    Co_catalyst: Q_catalyst_Co,
    Ru_catalyst: Q_catalyst_Ru,
    ZrO2_catalyst: Q_catalyst_ZrO2,
    Al2O3_catalyst: Q_catalyst_Al2O3,
    
    NiO_catalyst: Q_catalyst_NiO, # represent NiO, same reaseon as stated for Ru catalyst
    W_catalyst: Q_catalyst_W, # use ammonium paratungstate in ecoinvent to represent WO3 based on W atomic mass conservation, as same reaseon as stated for Ru catalyst 
    HY_catalyst: Q_catalyst_HY,
    
    EoL_catalyst: Q_catalyst_Co+Q_catalyst_Ru+Q_catalyst_ZrO2+Q_catalyst_Al2O3+Q_catalyst_NiO+Q_catalyst_W+Q_catalyst_HY,
    
    CO2_e: CO2_pe,
    CH4_e: CH4_pe,
    N2O_e: N2O_pe,
    NOx_e: NOx_pe,
    SO2_e: SO2_pe,
    CO_e : CO_pe,
    HC_e : HC_pe,
    PM10_e: PM10_pe,
    COD_e: COD_pe,
    solidwaste_EoL: -soildw_pe,
    TN_e: TN_pe,
    NH4_e: NH3N_pe/14.0067*118.04,
    
    
})

In [ ]:
M_fuelpMJ_LCI = newActivity(USER_DB, "bio-jet fuel production from maize straw gasification and Fischer-Tropsch, unallocated", "MJ fuel")

M_fuelpMJ_LCI.addExchanges({ 
    
    M_fuelp_LCI: R_biojet_straw/H_biojet,
    
    })
# calcualted based on 1 kg bio-jet produced from corn straw
#printAct(M_fuelpMJ_LCI)

#### allocation ratios under different decision

##### mass allocation

In [ ]:
m_biojet_tb = newFloatParam(
    "m_biojet_tb", 
    distrib=DistributionType.FIXED,
    default= 85.03,   
    #min=1, max=5,
    label_fr="mass of bio-jet fuel generated from one metric tonne maize straw gasification and FT synthesis", 
    group="bio-jet production", 
    unit="kg / tonne maize straw") 

In [ ]:
m_gasoline_tb = newFloatParam(
    "m_gasoline_tb", 
    distrib=DistributionType.FIXED,
    default= 43.79,   
    #min=1, max=5,
    label_fr="mass of bio-gasoline generated from one metric tonne maize straw gasification and FT synthesis", 
    group="bio-jet production", 
    unit="kg / tonne maize straw") 

In [ ]:
m_diesel_tb = newFloatParam(
    "m_diesel_tb", 
    distrib=DistributionType.FIXED,
    default= 19.83,   
    #min=1, max=5,
    label_fr="mass of bio-diesel generated from one metric tonne maize straw gasification and FT synthesis", 
    group="bio-jet production", 
    unit="kg / tonne maize straw") 

In [ ]:
m_paraffin_tb = newFloatParam(
    "m_paraffin_tb", 
    distrib=DistributionType.FIXED,
    default= 7.99,   
    #min=1, max=5,
    label_fr="mass of bio-paraffin generated from one metric tonne maize straw gasification and FT synthesis", 
    group="bio-jet production", 
    unit="kg / tonne maize straw") 

In [ ]:
m_hotsteam_tb = newFloatParam(
    "m_hotsteam_tb", 
    distrib=DistributionType.FIXED,
    default= 15.6,   # Property: saturated steam at 1.6MPa and 201.372  degC 
    #min=1, max=5,
    label_fr="mass of hot steam generated from one metric tonne maize straw gasification and FT synthesis", 
    group="bio-jet production", 
    unit="kg / kg bio-jet fuel produced") 

In [ ]:
m_elecsteam_tb = newFloatParam(
    "m_elecsteam_tb", 
    distrib=DistributionType.FIXED,
    default= 5292/1000,   
    #min=1, max=5,
    label_fr="mass of super heated steam generated and used to electricity generation from one metric tonne maize straw gasification and FT synthesis", 
    group="bio-jet production", 
    unit="kg / kg bio-jet fuel produced") 

##### energy allocation

In [ ]:
H_gasoline_tb = newFloatParam(
    "H_gasoline_tb", 
    distrib=DistributionType.FIXED,
    default= 43.1,   
    #min=1, max=5,
    label_fr="LHV of bio-gasoline generated from one metric tonne maize straw gasification and FT synthesis", 
    group="bio-jet production", 
    unit="MJ / kg fuel") 

In [ ]:
H_diesel_tb = newFloatParam(
    "H_diesel_tb", 
    distrib=DistributionType.FIXED,
    default= 46,   
    #min=1, max=5,
    label_fr="LHV of bio-diesel generated from one metric tonne maize straw gasification and FT synthesis", 
    group="bio-jet production", 
    unit="MJ / kg fuel") 

In [ ]:
H_paraffin_tb = newFloatParam(
    "H_paraffin_tb", 
    distrib=DistributionType.FIXED,
    default= 44.857,   
    #min=1, max=5,
    label_fr="LHV of bio-paraffin generated from one metric tonne maize straw gasification and FT synthesis", 
    group="bio-jet production", 
    unit="MJ / kg fuel") 

In [ ]:
H_hotsteam_tb = newFloatParam(
    "H_hotsteam_tb", 
    distrib=DistributionType.FIXED,
    default= 2.79*0.874,   # considerinng the average heat transfer efficiency of heating pipe network in China as 87.4%.
    #min=1, max=5,
    label_fr="LHV of hot steam generated from one metric tonne maize straw gasification and FT synthesis", 
    group="bio-jet production", 
    unit="MJ / kg steam") 

##### market value

In [ ]:
V_biojet_tb = newFloatParam(
    "V_biojet_tb", 
    distrib=DistributionType.FIXED,
    default= 6820,   
    #min=1, max=5,
    label_fr="Market value of bio-jet generated from one metric tonne maize straw gasification and FT synthesis in China", 
    group="bio-jet production", 
    unit="RMB / kg fuel") 

In [ ]:
V_gasoline_tb = newFloatParam(
    "V_gasoline_tb", 
    distrib=DistributionType.FIXED,
    default= 8050,   
    #min=1, max=5,
    label_fr="Market value of bio-gasoline generated from one metric tonne maize straw gasification and FT synthesis in China", 
    group="bio-jet production", 
    unit="RMB / kg fuel") 

In [ ]:
V_diesel_tb = newFloatParam(
    "V_diesel_tb", 
    distrib=DistributionType.FIXED,
    default= 7030,   
    #min=1, max=5,
    label_fr="Market value of bio-diesel generated from one metric tonne maize straw gasification and FT synthesis in China", 
    group="bio-jet production", 
    unit="RMB / kg fuel") 

In [ ]:
V_paraffin_tb = newFloatParam(
    "V_paraffin_tb", 
    distrib=DistributionType.FIXED,
    default= 6200,   
    #min=1, max=5,
    label_fr="Market value of bio-paraffin generated from one metric tonne maize straw gasification and FT synthesis in China", 
    group="bio-jet production", 
    unit="RMB / kg fuel") 

In [ ]:
V_hotsteam_tb = newFloatParam(
    "V_hotsteam_tb", 
    distrib=DistributionType.FIXED,
    default= 200,   # national average value estimated based on the nationalwide hot steam price survey in China conducted by Ye,D and the avergae 2019 hot steam price for comparatively developed area in China, e.g., the east costal areas, collected from Chinese government offical websites. 
    #min=1, max=5,
    label_fr="market value of hot steam generated from one metric tonne maize straw gasification and FT synthesis", 
    group="bio-jet production", 
    unit="RMB / kg fuel") 

In [ ]:
V_elecsteam_tb = newFloatParam(
    "V_hotsteam_tb", 
    distrib=DistributionType.FIXED,
    default= 0.66,   # price of business aimed electricity for year 2021 in China.
    #min=1, max=5,
    label_fr="market value of electricity generated by superheated steam generated from one metric tonne maize straw gasification and FT synthesis", 
    group="bio-jet production", 
    unit="RMB / Kwh") 

#### base case - energy allocation and electricity substitution

In [ ]:
Elec_subst = findActivity(name='market group for electricity, low voltage', loc='CN', db_name='ecoinvent 3.7.1 apos')

In [ ]:
Elec_substCA = copyActivity(
    USER_DB, # base utilisateur
    Elec_subst, # activité initiale 
    'market group for electricity, low voltage substCA') # Nouveau nom
#printAct(MonoSi_ingot_CN)

In [ ]:
Water_gen = findActivity(name='water production, deionised', loc='RoW', db_name='ecoinvent 3.7.1 apos')

In [ ]:
Water_genCA = copyActivity(
    USER_DB, # base utilisateur
    Water_gen, # activité initiale 
    'water production, deionised CA') # Nouveau nom
#printAct(MonoSi_ingot_CN)

In [ ]:
Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
   
ME_fuelpMJ_LCI = newActivity(USER_DB, "bio-jet fuel production from maize straw gasification and Fischer-Tropsch, energy allocation and electricity substitution", "MJ fuel")

ME_fuelpMJ_LCI.addExchanges({ 
    
    M_fuelpMJ_LCI: Allocate_RE_fuelp_B,
    Elec_subst: -(4221-2644)/1000*Allocate_RE_fuelp_B/H_biojet,
    Water_gen: m_elecsteam_tb*Allocate_RE_fuelp_B/H_biojet
    })
#printAct(M_fuelpMJ_LCI)

### bio-jet fuel delivery and combustion

In [ ]:
D_lorry_jetfuel = newFloatParam(
    "D_lorry_jetfuel", 
    distrib=DistributionType.LINEAR,
    default= 50,   
    min=20, max=100,
    label_fr="transportion distance of bio-jet fuel through lorry in China", 
    group="bio-jet delivery", 
    unit="km") 

In [ ]:
D_train_jetfuel = newFloatParam(
    "D_train_jetfuel", 
    distrib=DistributionType.LINEAR,
    default= 749,   
    min=716, max=810,
    label_fr="transportion distance of bio-jet fuel through train in China", 
    group="bio-jet delivery", 
    unit="km") 

In [ ]:
Elec_fuel_delivery = newFloatParam(
    "Elec_fuel_delivery", 
    distrib=DistributionType.FIXED,
    default= 0.0056,   
    #min=716, max=810,
    label_fr="eleectricity consumption during the blending operations, the transportation of train-terminal and terminal-truck process, bio-jet fuel airport tank farm storage and distribution through the airport hydrant system into the plane in China", 
    group="bio-jet delivery", 
    unit="kwh/kg fuel") 

In [ ]:
CH4_fuelem = newFloatParam(
    "CH4_fuelem", 
    distrib=DistributionType.TRIANGLE,
    default= 1.064E-07,   
    min=3.518E-13, max=1.656E-07,
    label_fr="CH4 emission during the bio-jet fuel combustion in China, cover entire plane operation cycle", 
    group="bio-jet delivery", 
    unit="kg/ MJ fuel") 

In [ ]:
N2O_fuelem = newFloatParam(
    "N2O_fuelem", 
    distrib=DistributionType.TRIANGLE,
    default= 2.088E-07,   
    min=5.000E-08, max=3.250E-07,
    label_fr="N2O emission during the bio-jet fuel combustion in China, cover entire plane operation cycle", 
    group="bio-jet delivery", 
    unit="kg/ MJ fuel") 

In [ ]:
CO2_fuelem = newFloatParam(
    "CO2_fuelem", 
    distrib=DistributionType.TRIANGLE,
    default= 7.030E-02,   
    min=6.940E-02, max=7.044E-02,
    label_fr="CO2 emission during the bio-jet fuel combustion in China, cover entire plane operation cycle", 
    group="bio-jet delivery", 
    unit="kg/ MJ fuel") 

In [ ]:
VOC_fuelem = newFloatParam(
    "VOC_fuelem", 
    distrib=DistributionType.TRIANGLE,
    default= 1.299E-05,   
    min=3.958E-06, max=8.587E-05,
    label_fr="VOC emission during the bio-jet fuel combustion in China, cover entire plane operation cycle", 
    group="bio-jet delivery", 
    unit="kg/ MJ fuel") 

In [ ]:
CO_fuelem = newFloatParam(
    "CO_fuelem", 
    distrib=DistributionType.TRIANGLE,
    default= 8.968E-05,   
    min=3.425E-05, max=4.343E-04,
    label_fr="CO emission during the bio-jet fuel combustion in China, cover entire plane operation cycle", 
    group="bio-jet delivery", 
    unit="kg/ MJ fuel") 

In [ ]:
NOx_fuelem = newFloatParam(
    "NOx_fuelem", 
    distrib=DistributionType.TRIANGLE,
    default= 3.295E-04,   
    min=2.564E-04, max=4.087E-04,
    label_fr="NOx emission during the bio-jet fuel combustion in China, cover entire plane operation cycle", 
    group="bio-jet delivery", 
    unit="kg/ MJ fuel") 

In [ ]:
PM10_fuelem = newFloatParam(
    "PM10_fuelem", 
    distrib=DistributionType.TRIANGLE,
    default= 3.411E-07,   
    min=3.269E-07, max=1.805E-06,
    label_fr="PM10 emission during the bio-jet fuel combustion in China, cover entire plane operation cycle", 
    group="bio-jet delivery", 
    unit="kg/ MJ fuel") 

In [ ]:
CH4_fuele = [act for act in bio if 'Methane, non-fossil' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'lower stratosphere + upper troposphere' in str(act['categories'])
      ][0]
CH4_fuele

In [ ]:
CH4_fueleCA = copyActivity(
    USER_DB, # base utilisateur
    CH4_fuele, # activité initiale 
    'Methane, non-fossil, jetfuel CA') # Nouveau nom
#printAct(MonoSi_ingot_CN)
CH4_fueleCA.addExchanges({ 
    
    CH4_fuele : CH4_fuelem,
    
})
printAct(CH4_fueleCA)

In [ ]:
N2O_fuele = [act for act in bio if 'Dinitrogen monoxide' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'lower stratosphere + upper troposphere' in str(act['categories'])
      ][0]
N2O_fuele


In [ ]:
N2O_fueleCA = copyActivity(
    USER_DB, # base utilisateur
    N2O_fuele, # activité initiale 
    'Dinitrogen monoxide, jetfuel CA') # Nouveau nom
#printAct(MonoSi_ingot_CN)
N2O_fueleCA.addExchanges({ 
    
    N2O_fuele : N2O_fuelem,
    
})
printAct(N2O_fueleCA)

In [ ]:
CO2_fuele = [act for act in bio if 'Carbon dioxide, non-fossil' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'low population density, long-term' in str(act['categories'])
      ][0]
CO2_fuele

In [ ]:
CO2_fueleCA = copyActivity(
    USER_DB, # base utilisateur
    CO2_fuele, # activité initiale 
    'Carbon dioxide, non-fossil, jetfuel CA') # Nouveau nom
#printAct(MonoSi_ingot_CN)
CO2_fueleCA.addExchanges({ 
    
    CO2_fuele : CO2_fuelem,
    
})
printAct(CO2_fueleCA)

In [ ]:
VOC_fuele = [act for act in bio if 'VOC, volatile organic compounds, unspecified origin' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'air' in str(act['categories'])
      ][0]
VOC_fuele

In [ ]:
VOC_fueleCA = copyActivity(
    USER_DB, # base utilisateur
    VOC_fuele, # activité initiale 
    'VOC, volatile organic compounds, unspecified origin, jetfuel CA') # Nouveau nom
#printAct(MonoSi_ingot_CN)
VOC_fueleCA.addExchanges({ 
    
    VOC_fuele : VOC_fuelem,
    
})
printAct(VOC_fueleCA)

In [ ]:
CO_fuele = [act for act in bio if 'Carbon monoxide, non-fossil' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'lower stratosphere + upper troposphere' in str(act['categories'])
      ][0]
CO_fuele

In [ ]:
CO_fueleCA = copyActivity(
    USER_DB, # base utilisateur
    CO_fuele, # activité initiale 
    'Carbon monoxide, non-fossil, jetfuel CA') # Nouveau nom
#printAct(MonoSi_ingot_CN)
CO_fueleCA.addExchanges({ 
    
    CO_fuele : CO_fuelem,
    
})
printAct(CO_fueleCA)

In [ ]:
NOx_fuele = [act for act in bio if 'Nitrogen oxides' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'lower stratosphere + upper troposphere' in str(act['categories'])
      ][0]
NOx_fuele

In [ ]:
NOx_fueleCA = copyActivity(
    USER_DB, # base utilisateur
    NOx_fuele, # activité initiale 
    'Nitrogen oxides, jetfuel CA') # Nouveau nom
#printAct(MonoSi_ingot_CN)
NOx_fueleCA.addExchanges({ 
    
    NOx_fuele : NOx_fuelem,
    
})
printAct(NOx_fueleCA)

In [ ]:
PM10_fuele = [act for act in bio if 'Particulates, > 2.5 um, and < 10um' in act['name'] 
                                            #and 'fossil' in act['name']
                                            #and 'non' not in act['name']
                                            and 'lower stratosphere + upper troposphere' in str(act['categories'])
                                            #and 'non-urban air or from high stacks' not in str(act['categories'])
      
       ][0]
PM10_fuele

In [ ]:
PM10_fueleCA = copyActivity(
    USER_DB, # base utilisateur
    PM10_fuele, # activité initiale 
    'Particulates, > 2.5 um, and < 10um, jetfuel CA') # Nouveau nom


PM10_fueleCA.addExchanges({ 
    
    PM10_fuele : PM10_fuelem,
    
})
printAct(PM10_fueleCA)

In [ ]:
T_jetfuel_lorry = findActivity(name='market for transport, freight, lorry 7.5-16 metric ton, EURO5', loc='RoW', db_name='ecoinvent 3.7.1 apos')

In [ ]:
T_jetfuel_lorryCA = copyActivity(
    USER_DB, # base utilisateur
    T_jetfuel_lorry, # activité initiale 
    'market for transport, freight, lorry 7.5-16 metric ton, EURO5, jetfuel CA') # Nouveau nom
#printAct(MonoSi_ingot_CN)

In [ ]:
T_jetfuel_train = findActivity(name='market for transport, freight train', loc='CN', db_name='ecoinvent 3.7.1 apos')

In [ ]:
Elec_delivery = findActivity(name='market group for electricity, low voltage', loc='CN', db_name='ecoinvent 3.7.1 apos')

In [ ]:
Elec_deliveryCA = copyActivity(
    USER_DB, # base utilisateur
    Elec_delivery, # activité initiale 
    'market group for electricity, low voltage deliveryCA') # Nouveau nom
#printAct(Elec_deliveryCA)

In [ ]:
#Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
   
ME_fueltMJ_LCI = newActivity(USER_DB, "bio-jet fuel delivery and combustion in aircraft", "MJ fuel")

ME_fueltMJ_LCI.addExchanges({ 
    
    ME_fuelpMJ_LCI:1,
    T_jetfuel_lorry: 1/H_biojet*D_lorry_jetfuel/1000,
    T_jetfuel_train: 1/H_biojet*D_train_jetfuel/1000,
    Elec_delivery: Elec_fuel_delivery/H_biojet,
    CH4_fuele:CH4_fuelem,
    N2O_fuele:N2O_fuelem,
    CO2_fuele:CO2_fuelem,
    VOC_fuele:VOC_fuelem,
    CO_fuele:CO_fuelem,
    NOx_fuele:NOx_fuelem,
    PM10_fuele:PM10_fuelem,
    })
#printAct(M_fuelpMJ_LCI)

### scenario variation - allocation method

#### mass maize - mass fuel

In [ ]:
Allocate_RE_fuelp_M = m_biojet_tb/(m_biojet_tb+m_gasoline_tb+m_diesel_tb+m_paraffin_tb)
   
MM_fuelpMJ_LCI = newActivity(USER_DB, "bio-jet fuel production from maize straw gasification and Fischer-Tropsch, mass allocation and electricity substitution", "MJ fuel")

MM_fuelpMJ_LCI.addExchanges({ 
    
    M_fuelpMJ_LCI: Allocate_RE_fuelp_M,
    Elec_subst: -(4221-2644)/1000*Allocate_RE_fuelp_M/H_biojet,
    Water_gen: m_elecsteam_tb*Allocate_RE_fuelp_M/H_biojet
    })
#printAct(M_fuelpMJ_LCI)

In [ ]:
#Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
   
MM_fueltMJ_LCI = newActivity(USER_DB, "bio-jet fuel delivery and combustion in aircraft, mass allocation and electricity substitution", "MJ fuel")

MM_fueltMJ_LCI.addExchanges({ 
    
    MM_fuelpMJ_LCI:1,
    T_jetfuel_lorry: 1/H_biojet*D_lorry_jetfuel/1000,
    T_jetfuel_train: 1/H_biojet*D_train_jetfuel/1000,
    Elec_delivery: Elec_fuel_delivery/H_biojet,
    CH4_fuele:CH4_fuelem,
    N2O_fuele:N2O_fuelem,
    CO2_fuele:CO2_fuelem,
    VOC_fuele:VOC_fuelem,
    CO_fuele:CO_fuelem,
    NOx_fuele:NOx_fuelem,
    PM10_fuele:PM10_fuelem,
    })
#printAct(M_fuelpMJ_LCI)

#### mass maize - marketing value fuel

In [ ]:
Allocate_RE_fuelp_V = m_biojet_tb*V_biojet_tb/(m_biojet_tb*V_biojet_tb+m_gasoline_tb*V_gasoline_tb+m_diesel_tb*V_diesel_tb+m_paraffin_tb*V_paraffin_tb)
   
MV_fuelpMJ_LCI = newActivity(USER_DB, "bio-jet fuel production from maize straw gasification and Fischer-Tropsch, market value allocation and electricity substitution", "MJ fuel")

MV_fuelpMJ_LCI.addExchanges({ 
    
    M_fuelpMJ_LCI: Allocate_RE_fuelp_V,
    Elec_subst: -(4221-2644)/1000*Allocate_RE_fuelp_V/H_biojet,
    Water_gen: m_elecsteam_tb*Allocate_RE_fuelp_V/H_biojet
    })
#printAct(M_fuelpMJ_LCI)

In [ ]:
#Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
   
MV_fueltMJ_LCI = newActivity(USER_DB, "bio-jet fuel delivery and combustion in aircraft, market value allocation and electricity substitution", "MJ fuel")

MV_fueltMJ_LCI.addExchanges({ 
    
    MV_fuelpMJ_LCI:1,
    T_jetfuel_lorry: 1/H_biojet*D_lorry_jetfuel/1000,
    T_jetfuel_train: 1/H_biojet*D_train_jetfuel/1000,
    Elec_delivery: Elec_fuel_delivery/H_biojet,
    CH4_fuele:CH4_fuelem,
    N2O_fuele:N2O_fuelem,
    CO2_fuele:CO2_fuelem,
    VOC_fuele:VOC_fuelem,
    CO_fuele:CO_fuelem,
    NOx_fuele:NOx_fuelem,
    PM10_fuele:PM10_fuelem,
    })
#printAct(M_fuelpMJ_LCI)

#### heating value maize - mass fuel 

In [ ]:
H_strawp_LCI = newActivity(USER_DB, "maize straw production, harvest,transported from field to farm, enery allocation", "kg straw")

H_strawp_LCI.addExchanges({ 
    

    maize_all_CN:R_straw*H_straw/(1*H_grain+R_straw*H_straw+R_cob*H_cob)/R_straw,
    
})

In [ ]:
H_strawT_LCI = newActivity(USER_DB, "maize straw production, harvest,transport to gasification plant, heating value allocation", "kg straw")

H_strawT_LCI.addExchanges({ 
    
    H_strawp_LCI : 1,
    
    Elec_crush_pack : Elec_C/1000,
    Diesel_crush_pack : Diesel_C/1000*(1/0.0222), # diesel density 1/0.0222 
    
    T_straw_lorry : TD_straw*1/1000
    
})

In [ ]:
H_fuelp_LCI = newActivity(USER_DB, "maize straw gasification and Fischer-Tropsch bio-jet fuel production, heating value based straw allocation", "kg straw")

H_fuelp_LCI.addExchanges({ 
    
    Steel_building: P_steel*1000/strawc_total,
    Iron_building: P_iron*1000/strawc_total,
    Al_building: P_Al*1000/strawc_total,
    cement_building: P_cement*1000/strawc_total,
    Steel_EoL: -(P_steel+P_iron)*1000/strawc_total,
    Al_EoL: -P_Al*1000/strawc_total,
    cement_EoL: -P_cement*1000/strawc_total,
    
    H_strawT_LCI: 1,
    Elec_millp: Elec_milling/1000,
    
    Si_sand: 500/((7200/8760)*24)*5*1000/strawc_total,
    Water_p: (5292.23/11760)*1,          
    
    Co_catalyst: Q_catalyst_Co,
    Ru_catalyst: Q_catalyst_Ru,
    ZrO2_catalyst: Q_catalyst_ZrO2,
    Al2O3_catalyst: Q_catalyst_Al2O3,
    
    NiO_catalyst: Q_catalyst_NiO, # represent NiO, same reaseon as stated for Ru catalyst
    W_catalyst: Q_catalyst_W, # use ammonium paratungstate in ecoinvent to represent WO3 based on W atomic mass conservation, as same reaseon as stated for Ru catalyst 
    HY_catalyst: Q_catalyst_HY,
    
    EoL_catalyst: Q_catalyst_Co+Q_catalyst_Ru+Q_catalyst_ZrO2+Q_catalyst_Al2O3+Q_catalyst_NiO+Q_catalyst_W+Q_catalyst_HY,
    
    CO2_e: CO2_pe,
    CH4_e: CH4_pe,
    N2O_e: N2O_pe,
    NOx_e: NOx_pe,
    SO2_e: SO2_pe,
    CO_e : CO_pe,
    HC_e : HC_pe,
    PM10_e: PM10_pe,
    COD_e: COD_pe,
    solidwaste_EoL: -soildw_pe,
    TN_e: TN_pe,
    NH4_e: NH3N_pe/14.0067*118.04,
    
    
})

In [ ]:
H_fuelpMJ_LCI = newActivity(USER_DB, "bio-jet fuel production from maize straw gasification and Fischer-Tropsch, heating vale straw, unallocated", "MJ fuel")

H_fuelpMJ_LCI.addExchanges({ 
    
    H_fuelp_LCI: R_biojet_straw/H_biojet,
    
    })
# calcualted based on 1 kg bio-jet produced from corn straw
#printAct(M_fuelpMJ_LCI)

In [ ]:
Allocate_RE_fuelp_M = m_biojet_tb/(m_biojet_tb+m_gasoline_tb+m_diesel_tb+m_paraffin_tb)
   
HM_fuelpMJ_LCI = newActivity(USER_DB, "bio-jet fuel production from maize straw gasification and Fischer-Tropsch, heating value maize, fuel mass allocation and electricity substitution", "MJ fuel")

HM_fuelpMJ_LCI.addExchanges({ 
    
    H_fuelpMJ_LCI: Allocate_RE_fuelp_M,
    Elec_subst: -(4221-2644)/1000*Allocate_RE_fuelp_M/H_biojet,
    Water_gen: m_elecsteam_tb*Allocate_RE_fuelp_M/H_biojet
    })
#printAct(M_fuelpMJ_LCI)

In [ ]:
#Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
   
HM_fueltMJ_LCI = newActivity(USER_DB, "bio-jet fuel delivery and combustion in aircraft, heating value maize, mass fuel and electricity substitution", "MJ fuel")

HM_fueltMJ_LCI.addExchanges({ 
    
    HM_fuelpMJ_LCI:1,
    T_jetfuel_lorry: 1/H_biojet*D_lorry_jetfuel/1000,
    T_jetfuel_train: 1/H_biojet*D_train_jetfuel/1000,
    Elec_delivery: Elec_fuel_delivery/H_biojet,
    CH4_fuele:CH4_fuelem,
    N2O_fuele:N2O_fuelem,
    CO2_fuele:CO2_fuelem,
    VOC_fuele:VOC_fuelem,
    CO_fuele:CO_fuelem,
    NOx_fuele:NOx_fuelem,
    PM10_fuele:PM10_fuelem,
    })
#printAct(M_fuelpMJ_LCI)

#### heating value maize - energy fuel 

In [ ]:
Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
   
HH_fuelpMJ_LCI = newActivity(USER_DB, "bio-jet fuel production from maize straw gasification and Fischer-Tropsch, heating value maize, heating value fuel allocation and electricity substitution", "MJ fuel")

HH_fuelpMJ_LCI.addExchanges({ 
    
    H_fuelpMJ_LCI: Allocate_RE_fuelp_B,
    Elec_subst: -(4221-2644)/1000*Allocate_RE_fuelp_B/H_biojet,
    Water_gen: m_elecsteam_tb*Allocate_RE_fuelp_B/H_biojet
    })
#printAct(M_fuelpMJ_LCI)

In [ ]:
#Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
   
HH_fueltMJ_LCI = newActivity(USER_DB, "bio-jet fuel delivery and combustion in aircraft, heating value maize, heating value fuel allocation and electricity substitution", "MJ fuel")

HH_fueltMJ_LCI.addExchanges({ 
    
    HH_fuelpMJ_LCI:1,
    T_jetfuel_lorry: 1/H_biojet*D_lorry_jetfuel/1000,
    T_jetfuel_train: 1/H_biojet*D_train_jetfuel/1000,
    Elec_delivery: Elec_fuel_delivery/H_biojet,
    CH4_fuele:CH4_fuelem,
    N2O_fuele:N2O_fuelem,
    CO2_fuele:CO2_fuelem,
    VOC_fuele:VOC_fuelem,
    CO_fuele:CO_fuelem,
    NOx_fuele:NOx_fuelem,
    PM10_fuele:PM10_fuelem,
    })
#printAct(M_fuelpMJ_LCI)

#### heating value maize - market value fuel 

In [ ]:
Allocate_RE_fuelp_V = m_biojet_tb*V_biojet_tb/(m_biojet_tb*V_biojet_tb+m_gasoline_tb*V_gasoline_tb+m_diesel_tb*V_diesel_tb+m_paraffin_tb*V_paraffin_tb)
   
HV_fuelpMJ_LCI = newActivity(USER_DB, "bio-jet fuel production from maize straw gasification and Fischer-Tropsch, heating value maize, marketing value fuel allocation and electricity substitution", "MJ fuel")

HV_fuelpMJ_LCI.addExchanges({ 
    
    H_fuelpMJ_LCI: Allocate_RE_fuelp_V,
    Elec_subst: -(4221-2644)/1000*Allocate_RE_fuelp_V/H_biojet,
    Water_gen: m_elecsteam_tb*Allocate_RE_fuelp_V/H_biojet
    })
#printAct(M_fuelpMJ_LCI)

In [ ]:
#Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
   
HV_fueltMJ_LCI = newActivity(USER_DB, "bio-jet fuel delivery and combustion in aircraft, heating value maize, marketing value fuel allocation and electricity substitution", "MJ fuel")

HV_fueltMJ_LCI.addExchanges({ 
    
    HV_fuelpMJ_LCI:1,
    T_jetfuel_lorry: 1/H_biojet*D_lorry_jetfuel/1000,
    T_jetfuel_train: 1/H_biojet*D_train_jetfuel/1000,
    Elec_delivery: Elec_fuel_delivery/H_biojet,
    CH4_fuele:CH4_fuelem,
    N2O_fuele:N2O_fuelem,
    CO2_fuele:CO2_fuelem,
    VOC_fuele:VOC_fuelem,
    CO_fuele:CO_fuelem,
    NOx_fuele:NOx_fuelem,
    PM10_fuele:PM10_fuelem,
    })
#printAct(M_fuelpMJ_LCI)

#### marketing value maize - mass fuel 

In [ ]:
V_strawp_LCI = newActivity(USER_DB, "maize straw production, harvest,transported from field to farm, market value allocation", "kg straw")

V_strawp_LCI.addExchanges({ 
    

    maize_all_CN:R_straw*V_straw/(1*V_grain+R_straw*V_straw+R_cob*V_cob)/R_straw,
    
})

In [ ]:
V_strawT_LCI = newActivity(USER_DB, "maize straw production, harvest,transport to gasification plant, marketing value allocation", "kg straw")

V_strawT_LCI.addExchanges({ 
    
    V_strawp_LCI : 1,
    
    Elec_crush_pack : Elec_C/1000,
    Diesel_crush_pack : Diesel_C/1000*(1/0.0222), # diesel density 1/0.0222 
    
    T_straw_lorry : TD_straw*1/1000
    
})

In [ ]:
V_fuelp_LCI = newActivity(USER_DB, "maize straw gasification and Fischer-Tropsch bio-jet fuel production, marketing value based straw allocation", "kg straw")

V_fuelp_LCI.addExchanges({ 
    
    Steel_building: P_steel*1000/strawc_total,
    Iron_building: P_iron*1000/strawc_total,
    Al_building: P_Al*1000/strawc_total,
    cement_building: P_cement*1000/strawc_total,
    Steel_EoL: -(P_steel+P_iron)*1000/strawc_total,
    Al_EoL: -P_Al*1000/strawc_total,
    cement_EoL: -P_cement*1000/strawc_total,
    
    V_strawT_LCI: 1,
    Elec_millp: Elec_milling/1000,
    
    Si_sand: 500/((7200/8760)*24)*5*1000/strawc_total,
    Water_p: (5292.23/11760)*1,          
    
    Co_catalyst: Q_catalyst_Co,
    Ru_catalyst: Q_catalyst_Ru,
    ZrO2_catalyst: Q_catalyst_ZrO2,
    Al2O3_catalyst: Q_catalyst_Al2O3,
    
    NiO_catalyst: Q_catalyst_NiO, # represent NiO, same reaseon as stated for Ru catalyst
    W_catalyst: Q_catalyst_W, # use ammonium paratungstate in ecoinvent to represent WO3 based on W atomic mass conservation, as same reaseon as stated for Ru catalyst 
    HY_catalyst: Q_catalyst_HY,
    
    EoL_catalyst: Q_catalyst_Co+Q_catalyst_Ru+Q_catalyst_ZrO2+Q_catalyst_Al2O3+Q_catalyst_NiO+Q_catalyst_W+Q_catalyst_HY,
    
    CO2_e: CO2_pe,
    CH4_e: CH4_pe,
    N2O_e: N2O_pe,
    NOx_e: NOx_pe,
    SO2_e: SO2_pe,
    CO_e : CO_pe,
    HC_e : HC_pe,
    PM10_e: PM10_pe,
    COD_e: COD_pe,
    solidwaste_EoL: -soildw_pe,
    TN_e: TN_pe,
    NH4_e: NH3N_pe/14.0067*118.04,
    
    
})

In [ ]:
V_fuelpMJ_LCI = newActivity(USER_DB, "bio-jet fuel production from maize straw gasification and Fischer-Tropsch, marketing vale straw, unallocated", "MJ fuel")

V_fuelpMJ_LCI.addExchanges({ 
    
    V_fuelp_LCI: R_biojet_straw/H_biojet,
    
    })
# calcualted based on 1 kg bio-jet produced from corn straw
#printAct(M_fuelpMJ_LCI)

In [ ]:
Allocate_RE_fuelp_M = m_biojet_tb/(m_biojet_tb+m_gasoline_tb+m_diesel_tb+m_paraffin_tb)
   
VM_fuelpMJ_LCI = newActivity(USER_DB, "bio-jet fuel production from maize straw gasification and Fischer-Tropsch, marketing value maize, fuel mass allocation and electricity substitution", "MJ fuel")

VM_fuelpMJ_LCI.addExchanges({ 
    
    V_fuelpMJ_LCI: Allocate_RE_fuelp_M,
    Elec_subst: -(4221-2644)/1000*Allocate_RE_fuelp_M/H_biojet,
    Water_gen: m_elecsteam_tb*Allocate_RE_fuelp_M/H_biojet
    })
#printAct(M_fuelpMJ_LCI)

In [ ]:
#Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
   
VM_fueltMJ_LCI = newActivity(USER_DB, "bio-jet fuel delivery and combustion in aircraft, marketing value maize, mass fuel and electricity substitution", "MJ fuel")

VM_fueltMJ_LCI.addExchanges({ 
    
    VM_fuelpMJ_LCI:1,
    T_jetfuel_lorry: 1/H_biojet*D_lorry_jetfuel/1000,
    T_jetfuel_train: 1/H_biojet*D_train_jetfuel/1000,
    Elec_delivery: Elec_fuel_delivery/H_biojet,
    CH4_fuele:CH4_fuelem,
    N2O_fuele:N2O_fuelem,
    CO2_fuele:CO2_fuelem,
    VOC_fuele:VOC_fuelem,
    CO_fuele:CO_fuelem,
    NOx_fuele:NOx_fuelem,
    PM10_fuele:PM10_fuelem,
    })
#printAct(M_fuelpMJ_LCI)

#### marketing value maize - energy fuel 

In [ ]:
Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
   
VH_fuelpMJ_LCI = newActivity(USER_DB, "bio-jet fuel production from maize straw gasification and Fischer-Tropsch, marketing value maize, heating value fuel allocation and electricity substitution", "MJ fuel")

VH_fuelpMJ_LCI.addExchanges({ 
    
    V_fuelpMJ_LCI: Allocate_RE_fuelp_B,
    Elec_subst: -(4221-2644)/1000*Allocate_RE_fuelp_B/H_biojet,
    Water_gen: m_elecsteam_tb*Allocate_RE_fuelp_B/H_biojet
    })
#printAct(M_fuelpMJ_LCI)

In [ ]:
#Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
   
VH_fueltMJ_LCI = newActivity(USER_DB, "bio-jet fuel delivery and combustion in aircraft, marketing value maize, heating value fuel allocation and electricity substitution", "MJ fuel")

VH_fueltMJ_LCI.addExchanges({ 
    
    VH_fuelpMJ_LCI:1,
    T_jetfuel_lorry: 1/H_biojet*D_lorry_jetfuel/1000,
    T_jetfuel_train: 1/H_biojet*D_train_jetfuel/1000,
    Elec_delivery: Elec_fuel_delivery/H_biojet,
    CH4_fuele:CH4_fuelem,
    N2O_fuele:N2O_fuelem,
    CO2_fuele:CO2_fuelem,
    VOC_fuele:VOC_fuelem,
    CO_fuele:CO_fuelem,
    NOx_fuele:NOx_fuelem,
    PM10_fuele:PM10_fuelem,
    })
#printAct(M_fuelpMJ_LCI)

#### marketing value maize - marketing value fuel 

In [ ]:
Allocate_RE_fuelp_V = m_biojet_tb*V_biojet_tb/(m_biojet_tb*V_biojet_tb+m_gasoline_tb*V_gasoline_tb+m_diesel_tb*V_diesel_tb+m_paraffin_tb*V_paraffin_tb)
   
VV_fuelpMJ_LCI = newActivity(USER_DB, "bio-jet fuel production from maize straw gasification and Fischer-Tropsch, marketing value maize, marketing value fuel allocation and electricity substitution", "MJ fuel")

VV_fuelpMJ_LCI.addExchanges({ 
    
    V_fuelpMJ_LCI: Allocate_RE_fuelp_V,
    Elec_subst: -(4221-2644)/1000*Allocate_RE_fuelp_V/H_biojet,
    Water_gen: m_elecsteam_tb*Allocate_RE_fuelp_V/H_biojet
    })
#printAct(M_fuelpMJ_LCI)

In [ ]:
#Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
   
VV_fueltMJ_LCI = newActivity(USER_DB, "bio-jet fuel delivery and combustion in aircraft, marketing value maize, marketing value fuel allocation and electricity substitution", "MJ fuel")

VV_fueltMJ_LCI.addExchanges({ 
    
    VV_fuelpMJ_LCI:1,
    T_jetfuel_lorry: 1/H_biojet*D_lorry_jetfuel/1000,
    T_jetfuel_train: 1/H_biojet*D_train_jetfuel/1000,
    Elec_delivery: Elec_fuel_delivery/H_biojet,
    CH4_fuele:CH4_fuelem,
    N2O_fuele:N2O_fuelem,
    CO2_fuele:CO2_fuelem,
    VOC_fuele:VOC_fuelem,
    CO_fuele:CO_fuelem,
    NOx_fuele:NOx_fuelem,
    PM10_fuele:PM10_fuelem,
    })
#printAct(M_fuelpMJ_LCI)

#### maize environmental burdern free - baseline scenario

In [ ]:
Ze_strawp_LCI = newActivity(USER_DB, "maize straw production, harvest,transported from field to farm, environmental bourden free cultivation", "kg straw")

Ze_strawp_LCI.addExchanges({ 
    

    maize_all_CN: 0, #R_straw*H_straw/(1*H_grain+R_straw*H_straw+R_cob*H_cob)/R_straw,
    
})

In [ ]:
Ze_strawT_LCI = newActivity(USER_DB, "maize straw production, harvest,transport to gasification plant, environmental bourden free cultivation", "kg straw")

Ze_strawT_LCI.addExchanges({ 
    
    Ze_strawp_LCI : 1,
    
    Elec_crush_pack : Elec_C/1000,
    Diesel_crush_pack : Diesel_C/1000*(1/0.0222), # diesel density 1/0.0222 
    
    T_straw_lorry : TD_straw*1/1000
    
})

In [ ]:
Ze_fuelp_LCI = newActivity(USER_DB, "maize straw gasification and Fischer-Tropsch bio-jet fuel production, environmental burden free cultivation", "kg straw")

Ze_fuelp_LCI.addExchanges({ 
    
    Steel_building: P_steel*1000/strawc_total,
    Iron_building: P_iron*1000/strawc_total,
    Al_building: P_Al*1000/strawc_total,
    cement_building: P_cement*1000/strawc_total,
    Steel_EoL: -(P_steel+P_iron)*1000/strawc_total,
    Al_EoL: -P_Al*1000/strawc_total,
    cement_EoL: -P_cement*1000/strawc_total,
    
    Ze_strawT_LCI: 1,
    Elec_millp: Elec_milling/1000,
    
    Si_sand: 500/((7200/8760)*24)*5*1000/strawc_total,
    Water_p: (5292.23/11760)*1,          
    
    Co_catalyst: Q_catalyst_Co,
    Ru_catalyst: Q_catalyst_Ru,
    ZrO2_catalyst: Q_catalyst_ZrO2,
    Al2O3_catalyst: Q_catalyst_Al2O3,
    
    NiO_catalyst: Q_catalyst_NiO, # represent NiO, same reaseon as stated for Ru catalyst
    W_catalyst: Q_catalyst_W, # use ammonium paratungstate in ecoinvent to represent WO3 based on W atomic mass conservation, as same reaseon as stated for Ru catalyst 
    HY_catalyst: Q_catalyst_HY,
    
    EoL_catalyst: Q_catalyst_Co+Q_catalyst_Ru+Q_catalyst_ZrO2+Q_catalyst_Al2O3+Q_catalyst_NiO+Q_catalyst_W+Q_catalyst_HY,
    
    CO2_e: CO2_pe,
    CH4_e: CH4_pe,
    N2O_e: N2O_pe,
    NOx_e: NOx_pe,
    SO2_e: SO2_pe,
    CO_e : CO_pe,
    HC_e : HC_pe,
    PM10_e: PM10_pe,
    COD_e: COD_pe,
    solidwaste_EoL: -soildw_pe,
    TN_e: TN_pe,
    NH4_e: NH3N_pe/14.0067*118.04,
    
    
})

In [ ]:
Ze_fuelpMJ_LCI = newActivity(USER_DB, "bio-jet fuel production from maize straw gasification and Fischer-Tropsch, straw environmental burden free cultivation, unallocated", "MJ fuel")

Ze_fuelpMJ_LCI.addExchanges({ 
    
    Ze_fuelp_LCI: R_biojet_straw/H_biojet,
    
    })
# calcualted based on 1 kg bio-jet produced from corn straw
#printAct(M_fuelpMJ_LCI)

In [ ]:
Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
   
ZeE_fuelpMJ_LCI = newActivity(USER_DB, "bio-jet fuel production from maize straw gasification and Fischer-Tropsch, straw environmental burden free cultivation, energy allocation and electricity substitution", "MJ fuel")

ZeE_fuelpMJ_LCI.addExchanges({ 
    
    Ze_fuelpMJ_LCI: Allocate_RE_fuelp_B,
    Elec_subst: -(4221-2644)/1000*Allocate_RE_fuelp_B/H_biojet,
    Water_gen: m_elecsteam_tb*Allocate_RE_fuelp_B/H_biojet
    })
#printAct(M_fuelpMJ_LCI)

In [ ]:
#Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
   
ZeE_fueltMJ_LCI = newActivity(USER_DB, "bio-jet fuel delivery and combustion in aircraft, straw environmental burden free cultivation, energy allocation and electricity substitution", "MJ fuel")

ZeE_fueltMJ_LCI.addExchanges({ 
    
    ZeE_fuelpMJ_LCI:1,
    T_jetfuel_lorry: 1/H_biojet*D_lorry_jetfuel/1000,
    T_jetfuel_train: 1/H_biojet*D_train_jetfuel/1000,
    Elec_delivery: Elec_fuel_delivery/H_biojet,
    CH4_fuele:CH4_fuelem,
    N2O_fuele:N2O_fuelem,
    CO2_fuele:CO2_fuelem,
    VOC_fuele:VOC_fuelem,
    CO_fuele:CO_fuelem,
    NOx_fuele:NOx_fuelem,
    PM10_fuele:PM10_fuelem,
    })
#printAct(M_fuelpMJ_LCI)

In [ ]:
Contribution_Analysis_one_impact_category_noact(ZeE_fueltMJ_LCI, NIF_ON, ratio = 0.9, length_max = 9)

In [ ]:
plot_sankey(act = ZeE_fueltMJ_LCI, method = NIF_IN, cutoff = 0.1)

#### maize environmental burdern all taken - baseline scenario

In [ ]:
ALL_strawp_LCI = newActivity(USER_DB, "maize straw production, harvest,transported from field to farm, cultivation environmental bourden all taken", "kg straw")

ALL_strawp_LCI.addExchanges({ 
    

    maize_all_CN: 1/R_straw,
    
})

In [ ]:
ALL_strawT_LCI = newActivity(USER_DB, "maize straw production, harvest,transport to gasification plant, cultivation environmental bourden all taken", "kg straw")

ALL_strawT_LCI.addExchanges({ 
    
    ALL_strawp_LCI : 1,
    
    Elec_crush_pack : Elec_C/1000,
    Diesel_crush_pack : Diesel_C/1000*(1/0.0222), # diesel density 1/0.0222 
    
    T_straw_lorry : TD_straw*1/1000
    
})

In [ ]:
ALL_fuelp_LCI = newActivity(USER_DB, "maize straw gasification and Fischer-Tropsch bio-jet production, cultivation environmental burden all taken", "kg straw")

ALL_fuelp_LCI.addExchanges({ 
    
    Steel_building: P_steel*1000/strawc_total,
    Iron_building: P_iron*1000/strawc_total,
    Al_building: P_Al*1000/strawc_total,
    cement_building: P_cement*1000/strawc_total,
    Steel_EoL: -(P_steel+P_iron)*1000/strawc_total,
    Al_EoL: -P_Al*1000/strawc_total,
    cement_EoL: -P_cement*1000/strawc_total,
    
    ALL_strawT_LCI: 1,
    Elec_millp: Elec_milling/1000,
    
    Si_sand: 500/((7200/8760)*24)*5*1000/strawc_total,
    Water_p: (5292.23/11760)*1,          
    
    Co_catalyst: Q_catalyst_Co,
    Ru_catalyst: Q_catalyst_Ru,
    ZrO2_catalyst: Q_catalyst_ZrO2,
    Al2O3_catalyst: Q_catalyst_Al2O3,
    
    NiO_catalyst: Q_catalyst_NiO, # represent NiO, same reaseon as stated for Ru catalyst
    W_catalyst: Q_catalyst_W, # use ammonium paratungstate in ecoinvent to represent WO3 based on W atomic mass conservation, as same reaseon as stated for Ru catalyst 
    HY_catalyst: Q_catalyst_HY,
    
    EoL_catalyst: Q_catalyst_Co+Q_catalyst_Ru+Q_catalyst_ZrO2+Q_catalyst_Al2O3+Q_catalyst_NiO+Q_catalyst_W+Q_catalyst_HY,
    
    CO2_e: CO2_pe,
    CH4_e: CH4_pe,
    N2O_e: N2O_pe,
    NOx_e: NOx_pe,
    SO2_e: SO2_pe,
    CO_e : CO_pe,
    HC_e : HC_pe,
    PM10_e: PM10_pe,
    COD_e: COD_pe,
    solidwaste_EoL: -soildw_pe,
    TN_e: TN_pe,
    NH4_e: NH3N_pe/14.0067*118.04,
    
    
})

In [ ]:
ALL_fuelpMJ_LCI = newActivity(USER_DB, "bio-jet fuel production from maize straw gasification and Fischer-Tropsch, straw cultivation environmental burden all taken, unallocated", "MJ fuel")

ALL_fuelpMJ_LCI.addExchanges({ 
    
    ALL_fuelp_LCI: R_biojet_straw/H_biojet,
    
    })
# calcualted based on 1 kg bio-jet produced from corn straw
#printAct(M_fuelpMJ_LCI)

In [ ]:
Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
   
ALLE_fuelpMJ_LCI = newActivity(USER_DB, "bio-jet fuel production from maize straw gasification and Fischer-Tropsch, straw cultivation environmental burden all taken, energy allocation and electricity substitution", "MJ fuel")

ALLE_fuelpMJ_LCI.addExchanges({ 
    
    ALL_fuelpMJ_LCI: Allocate_RE_fuelp_B,
    Elec_subst: -(4221-2644)/1000*Allocate_RE_fuelp_B/H_biojet,
    Water_gen: m_elecsteam_tb*Allocate_RE_fuelp_B/H_biojet
    })
#printAct(M_fuelpMJ_LCI)

In [ ]:
#Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
   
ALLE_fueltMJ_LCI = newActivity(USER_DB, "bio-jet fuel delivery and combustion in aircraft, straw cultivation environmental burden all taken , energy allocation and electricity substitution", "MJ fuel")

ALLE_fueltMJ_LCI.addExchanges({ 
    
    ALLE_fuelpMJ_LCI:1,
    T_jetfuel_lorry: 1/H_biojet*D_lorry_jetfuel/1000,
    T_jetfuel_train: 1/H_biojet*D_train_jetfuel/1000,
    Elec_delivery: Elec_fuel_delivery/H_biojet,
    CH4_fuele:CH4_fuelem,
    N2O_fuele:N2O_fuelem,
    CO2_fuele:CO2_fuelem,
    VOC_fuele:VOC_fuelem,
    CO_fuele:CO_fuelem,
    NOx_fuele:NOx_fuelem,
    PM10_fuele:PM10_fuelem,
    })
#printAct(M_fuelpMJ_LCI)

### scenario variation - agricultural practices and field management strategies for maize plant cultivation

#### scenario of rainfed maize cultivation 

##### update the inventory to Chinese context

In [ ]:
water_air = [act for act in bio if 'Water, in air' in act['name'] 
                                            #and 'fluoride' not in act['name']
                                            #and 'oxides' not in act['name']
                                            and 'air' in str(act['categories'])
                                            and 'natural resource' in str(act['categories'])
                                            #and 'ocean' not in str(act['categories'])
                                            #and 'surface' not in str(act['categories'])
         ][0]
water_air

In [ ]:
water_e_unspecified = [act for act in bio if 'Water' in act['name'] 
                                            #and 'fluoride' not in act['name']
                                            #and 'oxides' not in act['name']
                                            and 'air' in str(act['categories'])
                                            and 'natural resource' not in str(act['categories'])
                                            and 'urban' not in str(act['categories'])
                                            and 'low' not in str(act['categories'])
                                            #and 'ocean' not in str(act['categories'])
                                            #and 'surface' not in str(act['categories'])
         ][0]
water_e_unspecified

In [ ]:
water_e_surface = [act for act in bio if 'Water' in act['name'] 
                                            #and 'fluoride' not in act['name']
                                            #and 'oxides' not in act['name']
                                            #and 'air' in str(act['categories'])
                                            and 'natural resource' not in str(act['categories'])
                                            #and 'urban' not in str(act['categories'])
                                            #and 'low' not in str(act['categories'])
                                            and 'surface water' in str(act['categories'])
                                            #and 'surface' not in str(act['categories'])
         ][0]
water_e_surface

In [ ]:
water_e_ground = [act for act in bio if 'Water' in act['name'] 
                                            #and 'fluoride' not in act['name']
                                            #and 'oxides' not in act['name']
                                            and 'ground' in str(act['categories'])
                                            and 'natural resource' not in str(act['categories'])
                                            and 'urban' not in str(act['categories'])
                                            and 'lo' not in str(act['categories'])
                                            #and 'ocean' not in str(act['categories'])
                                            #and 'surface' not in str(act['categories'])
         ][0]
water_e_ground

In [ ]:
r_maize_all = copyActivity(
    USER_DB, # base utilisateur
    eidb_cgrain, # activité initiale 
    'rainfed maize production in china, unallocated whole plant') # Nouveau nom
#printAct(MonoSi_ingot_CN)

#MonoSi_ingot_CN.updateExchanges({
#   'market group for electricity, medium voltage' : Elec_MonoSi_ingot})
#printAct(MonoSi_ingot_CN, MonoSi_ingot_init_CN)

r_maize_all.addExchanges({ 
    
    CO2_air : CO2_maizep_total,
    N_recycling : (R_SM*FRN_WWSM_eidbgrain +(1-R_SM)*FRN_OM_eidbgrain),
    N2_air : 10, # demo value 反硝化作用
    #NO3 : (R_SM*N_KGcgrain_eidb*26.9/100 + (1-R_SM)*N_KGcgrain_eidb*(11.8/(1+(ER_NO3-1)*0.3)*(1+(ER_NO3-1)*R_Straw_return))/100)/(14.0067/62.004),
    #P2O5_RoW_LCI : 100
    #iron : Q_iron_gear
    water_e_unspecified: -0.170072,
    water_e_surface: -0.014934,
    water_e_ground: -0.059737,
    water_air:irr_water,
})

N2O_KGcgrain_eidb = (R_SM*N_KGcgrain_eidb*0.9/100 + (1-R_SM)*N_KGcgrain_eidb*(0.9/(1+(ER_N2O-1)*0.3)*(1+(ER_N2O-1)*R_Straw_return))/100)/(14.0067/44.013)

r_maize_all.updateExchanges({ 
    
    'transport, tractor and trailer, agricultural' : T_infield,
    'potassium chloride': KCl_RoW,
    'lime': lime_RoW,
    
    'inorganic phosphorus fertiliser, as P2O5''#CN':P2O5_CN_m,
    'inorganic phosphorus fertiliser, as P2O5''#AU':0,
    'inorganic phosphorus fertiliser, as P2O5''#BD':0,
    'inorganic phosphorus fertiliser, as P2O5''#IL':0,
    'inorganic phosphorus fertiliser, as P2O5''#KE':0,
    'inorganic phosphorus fertiliser, as P2O5''#LK':0,
    'inorganic phosphorus fertiliser, as P2O5''#MX':0,
    'inorganic phosphorus fertiliser, as P2O5''#MY':0,
    'inorganic phosphorus fertiliser, as P2O5''#NZ':0,
    'inorganic phosphorus fertiliser, as P2O5''#PE':0,
    'inorganic phosphorus fertiliser, as P2O5''#PH':0,
    'inorganic phosphorus fertiliser, as P2O5''#RU':0,
    'inorganic phosphorus fertiliser, as P2O5''#TH':0,
    'inorganic phosphorus fertiliser, as P2O5''#CI':0,
    'inorganic phosphorus fertiliser, as P2O5''#TR':0,
    'inorganic phosphorus fertiliser, as P2O5''#VN':0,
    'inorganic phosphorus fertiliser, as P2O5''#RER':0,
    'inorganic phosphorus fertiliser, as P2O5''#RoW':0,
    'inorganic phosphorus fertiliser, as P2O5''#CL':0,
    'inorganic phosphorus fertiliser, as P2O5''#CO':0,
    'inorganic phosphorus fertiliser, as P2O5''#CR':0,
    'inorganic phosphorus fertiliser, as P2O5''#CY':0,
    'inorganic phosphorus fertiliser, as P2O5''#EC':0,
    'inorganic phosphorus fertiliser, as P2O5''#ID':0,
    
    'ammonium nitrate''#CN' : ammonium_CN_m,
    'ammonium nitrate''#RoW' : 0, # for China, the demostic ammonium nitrate comspumption is self-satified, the ammonium nitrate import is less than 1% on comsumption side, which is ignored. 
    'ammonium nitrate''#RER' : 0,
    
    'urea''#RER':0,
    'urea''#CN':urea_CN_m,
    
    #'irrigation''#!CN':0,
    
    #'ammonium nitrate''#RoW' : 0.01175，
    'ammonia, anhydrous, liquid''#CN' : ammonia_CN_m,
    'ammonia, anhydrous, liquid''#RoW' : 0, 
    'ammonia, anhydrous, liquid''#UN-SEASIA' :0,
    'ammonia, anhydrous, liquid''#RER' :0,
    
    'drying of maize straw and whole-plant' : 0.409840*(1+R_straw+R_cob), # to onclude the drying of maize straw and maize cob
    'Ammonia' : (R_SM*N_KGcgrain_eidb*7.4/100 + (1-R_SM)*N_KGcgrain_eidb*(5.2/(1-(1-ER_NH3)*0.3)*(1-(1-ER_NH3)*R_Straw_return))/100)/(14.0067/17.0305),
    'Dinitrogen monoxide' : N2O_KGcgrain_eidb,
    'Nitrate': (R_SM*N_KGcgrain_eidb*26.9/100 + (1-R_SM)*N_KGcgrain_eidb*(11.8/(1+(ER_NO3-1)*0.3)*(1+(ER_NO3-1)*R_Straw_return))/100)/(14.0067/62.004),
    'Nitrogen' : (1-ER_NO2_N2)/ER_NO2_N2*N2O_KGcgrain_eidb,
    #iron : Q_iron_gear
    
    'irrigation''#CH':0,
    'irrigation''#CN':0,
    'irrigation''#TN':0,
    'irrigation''#CO':0,
    'irrigation''#DE':0,
    'irrigation''#ES':0,
    'irrigation''#FR':0,
    'irrigation''#MA':0,
    'irrigation''#MY':0,
    'irrigation''#PE':0,
    'irrigation''#PH':0,
    
})

printAct(eidb_cgrain, r_maize_all)


In [ ]:
M_strawp_LCI_rainf = newActivity(USER_DB, "rainfed maize straw production, harvested, at farm, mass allocation", "kg straw")

M_strawp_LCI_rainf.addExchanges({ 
    

    r_maize_all:R_straw/(1+R_straw+R_cob)/R_straw,
    
})

In [ ]:
M_strawT_LCI_rainf = newActivity(USER_DB, "rainfed maize straw production, harvested and packaged, transported to gasification plant, mass allocation", "kg straw")

M_strawT_LCI_rainf.addExchanges({ 
    
    M_strawp_LCI_rainf : 1,
    
    Elec_crush_pack : Elec_C/1000,
    Diesel_crush_pack : Diesel_C/1000*(1/0.0222), # diesel density 1/0.0222 
    
    T_straw_lorry : TD_straw*1/1000
    
})

In [ ]:
M_fuelp_LCI_rainf = newActivity(USER_DB, "rainfed maize straw gasification and Fischer-Tropsch bio-jet fuel production", "kg straw")

M_fuelp_LCI_rainf.addExchanges({ 
    
    Steel_building: P_steel*1000/strawc_total,
    Iron_building: P_iron*1000/strawc_total,
    Al_building: P_Al*1000/strawc_total,
    cement_building: P_cement*1000/strawc_total,
    Steel_EoL: -(P_steel+P_iron)*1000/strawc_total,
    Al_EoL: -P_Al*1000/strawc_total,
    cement_EoL: -P_cement*1000/strawc_total,
    
    M_strawT_LCI_rainf: 1,
    Elec_millp: Elec_milling/1000,
    
    Si_sand: 500/((7200/8760)*24)*5*1000/strawc_total,
    Water_p: (5292.23/11760)*1,          
    
    Co_catalyst: Q_catalyst_Co,
    Ru_catalyst: Q_catalyst_Ru,
    ZrO2_catalyst: Q_catalyst_ZrO2,
    Al2O3_catalyst: Q_catalyst_Al2O3,
    
    NiO_catalyst: Q_catalyst_NiO, # represent NiO, same reaseon as stated for Ru catalyst
    W_catalyst: Q_catalyst_W, # use ammonium paratungstate in ecoinvent to represent WO3 based on W atomic mass conservation, as same reaseon as stated for Ru catalyst 
    HY_catalyst: Q_catalyst_HY,
    
    EoL_catalyst: Q_catalyst_Co+Q_catalyst_Ru+Q_catalyst_ZrO2+Q_catalyst_Al2O3+Q_catalyst_NiO+Q_catalyst_W+Q_catalyst_HY,
    
    CO2_e: CO2_pe,
    CH4_e: CH4_pe,
    N2O_e: N2O_pe,
    NOx_e: NOx_pe,
    SO2_e: SO2_pe,
    CO_e : CO_pe,
    HC_e : HC_pe,
    PM10_e: PM10_pe,
    COD_e: COD_pe,
    solidwaste_EoL: -soildw_pe,
    TN_e: TN_pe,
    NH4_e: NH3N_pe/14.0067*118.04,
    
    
})

In [ ]:
M_fuelpMJ_LCI_rainfed = newActivity(USER_DB, "bio-jet fuel production from rainfed maize straw gasification and Fischer-Tropsch, unallocated", "MJ fuel")

M_fuelpMJ_LCI_rainfed.addExchanges({ 
    
    M_fuelp_LCI_rainf: R_biojet_straw/H_biojet,
    
    })
# calcualted based on 1 kg bio-jet produced from corn straw
#printAct(M_fuelpMJ_LCI)

In [ ]:
Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
   
ME_fuelpMJ_LCI_rainf = newActivity(USER_DB, "bio-jet fuel produced from rainfed maize straw gasification and Fischer-Tropsch, energy allocation and electricity substitution, at plant", "MJ fuel")

ME_fuelpMJ_LCI_rainf.addExchanges({ 
    
    M_fuelpMJ_LCI_rainfed: Allocate_RE_fuelp_B,
    Elec_subst: -(4221-2644)/1000*Allocate_RE_fuelp_B/H_biojet,
    Water_gen: m_elecsteam_tb*Allocate_RE_fuelp_B/H_biojet
    })
#printAct(M_fuelpMJ_LCI)

In [ ]:
#Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
   
ME_fueltMJ_LCI_rainf = newActivity(USER_DB, "bio-jet fuel delivery and combustion in aircraft, rainfed maize", "MJ fuel")

ME_fueltMJ_LCI_rainf.addExchanges({ 
    
    ME_fuelpMJ_LCI_rainf:1,
    T_jetfuel_lorry: 1/H_biojet*D_lorry_jetfuel/1000,
    T_jetfuel_train: 1/H_biojet*D_train_jetfuel/1000,
    Elec_delivery: Elec_fuel_delivery/H_biojet,
    CH4_fuele:CH4_fuelem,
    N2O_fuele:N2O_fuelem,
    CO2_fuele:CO2_fuelem,
    VOC_fuele:VOC_fuelem,
    CO_fuele:CO_fuelem,
    NOx_fuele:NOx_fuelem,
    PM10_fuele:PM10_fuelem,
    })
#printAct(M_fuelpMJ_LCI)

##### C_N_W footprint contribution

In [ ]:
FixedParamMode='mean'

In [ ]:
freezeParams(USER_DB)

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI_rainf, NIF_IN, ratio = 0.9, length_max = 10)

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI_rainf, NIF_ON, ratio = 0.9, length_max = 10)

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI_rainf, NOF_IN, ratio = 0.9, length_max = 10)

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI_rainf, NOF_ON, ratio = 0.9, length_max = 10)

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI_rainf, CC_EF3, ratio = 0.9, length_max = 10)

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI_rainf, water_EF, ratio = 0.9, length_max = 9)

In [ ]:
#water_EF
#NOF_IN
#water_ReCiPe_v
impact = NIF_IN
act = maize_all_CN
# turbine_1_3MW_onshore
# glass_fibre_reinforced
# concrete_40MPa

df = exploreImpacts(impact, act, R_SM=62.7 / (62.7+160.2),R_Straw_return=0.30,ER_NH3=0.870,CO2_maizep_total=0.63812,
                    ER_N2O=1.26,ER_NO3=1.78,ER_NO2_N2=0.375,N_Straw=0.0061,R_straw=0.84,N_strawFR_SM_100=22.2973,
                    R_cob=0.2105, CO2_fuelem=0.0703,CO_fuelem=8.968e-05,N2O_fuelem=2.088e-07,CH4_fuelem=1.064e-07,
                   NOx_fuelem=0.0003295,PM10_fuelem=3.411e-07,VOC_fuelem=1.299e-05,D_train_jetfuel=749,H_biojet=43.07,
                   D_lorry_jetfuel=50,Elec_fuel_delivery=0.0056)

df.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\maize production_v2.xlsx', sheet_name='maize production', index = True)
df2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\maize production_v2.xlsx', sheet_name='maize production', skiprows=[0])
df2
#df['impact']=df['impact'].astype(float)
#df=df.astype(float)
#df['blocks']=df['blocks'].astype(float)

In [ ]:
#df.to_excel(r'F:\学也无止境\intership\OIE internship\Internship on LCA of hydrogen production\d0se00222d1\CH4-to-H2 lca_algebraic\H2LCIA_result\H2LCIA_result_SMR_HTLT_CCS98_EU_elec.xlsx', sheet_name='H2_CIA_result', index = True)
A = df2.columns[3]
name = df2.columns[0]
graph = df2.plot.bar(figsize=(100,50), y = A,x=name, fontsize = 20)


graph.legend(loc='center left', bbox_to_anchor=(0.04, 1.1)) 


In [ ]:
Contribution_Analysis_one_impact_category_noact_longlegend(ME_fueltMJ_LCI, NIF_ON, ratio = 0.9, length_max = 9)

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI, NOF_IN, ratio = 0.9, length_max = 10)

In [ ]:
plot_sankey(act = ME_fueltMJ_LCI, method = NOF_IN, cutoff = 0.1) #ME_fueltMJ_LCI, NIF_ON #Elec_delivery
#water_EF

In [ ]:
#water_EF
#NOF_IN
#water_ReCiPe_v
impact = NOF_IN
act = maize_all_CN
# turbine_1_3MW_onshore
# glass_fibre_reinforced
# concrete_40MPa

df = exploreImpacts(impact, act, R_SM=62.7 / (62.7+160.2),R_Straw_return=0.30,ER_NH3=0.870,CO2_maizep_total=0.63812,
                    ER_N2O=1.26,ER_NO3=1.78,ER_NO2_N2=0.375,N_Straw=0.0061,R_straw=0.84,N_strawFR_SM_100=22.2973,
                    R_cob=0.2105, CO2_fuelem=0.0703,CO_fuelem=8.968e-05,N2O_fuelem=2.088e-07,CH4_fuelem=1.064e-07,
                   NOx_fuelem=0.0003295,PM10_fuelem=3.411e-07,VOC_fuelem=1.299e-05,D_train_jetfuel=749,H_biojet=43.07,
                   D_lorry_jetfuel=50,Elec_fuel_delivery=0.0056)

df.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\maize production_v2.xlsx', sheet_name='maize production', index = True)
df2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\maize production_v2.xlsx', sheet_name='maize production', skiprows=[0])
df2
#df['impact']=df['impact'].astype(float)
#df=df.astype(float)
#df['blocks']=df['blocks'].astype(float)

In [ ]:
#df.to_excel(r'F:\学也无止境\intership\OIE internship\Internship on LCA of hydrogen production\d0se00222d1\CH4-to-H2 lca_algebraic\H2LCIA_result\H2LCIA_result_SMR_HTLT_CCS98_EU_elec.xlsx', sheet_name='H2_CIA_result', index = True)
A = df2.columns[3]
name = df2.columns[0]
graph = df2.plot.bar(figsize=(100,50), y = A,x=name, fontsize = 20)


graph.legend(loc='center left', bbox_to_anchor=(0.04, 1.1)) 


In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI, NOF_ON, ratio = 0.9, length_max = 9)

In [ ]:
plot_sankey(act = ME_fueltMJ_LCI, method = NOF_ON, cutoff = 0.1) #ME_fueltMJ_LCI, NIF_ON #Elec_delivery
#water_EF

In [ ]:
#water_EF
#NOF_IN
#water_ReCiPe_v
impact = NOF_ON
act = ME_fueltMJ_LCI
# turbine_1_3MW_onshore
# glass_fibre_reinforced
# concrete_40MPa

df = exploreImpacts(impact, act, R_SM=62.7 / (62.7+160.2),R_Straw_return=0.30,ER_NH3=0.870,CO2_maizep_total=0.63812,
                    ER_N2O=1.26,ER_NO3=1.78,ER_NO2_N2=0.375,N_Straw=0.0061,R_straw=0.84,N_strawFR_SM_100=22.2973,
                    R_cob=0.2105, CO2_fuelem=0.0703,CO_fuelem=8.968e-05,N2O_fuelem=2.088e-07,CH4_fuelem=1.064e-07,
                   NOx_fuelem=0.0003295,PM10_fuelem=3.411e-07,VOC_fuelem=1.299e-05,D_train_jetfuel=749,H_biojet=43.07,
                   D_lorry_jetfuel=50,Elec_fuel_delivery=0.0056)

df.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\maize production_v2.xlsx', sheet_name='maize production', index = True)
df2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\maize production_v2.xlsx', sheet_name='maize production', skiprows=[0])
df2
#df['impact']=df['impact'].astype(float)
#df=df.astype(float)
#df['blocks']=df['blocks'].astype(float)

In [ ]:
#df.to_excel(r'F:\学也无止境\intership\OIE internship\Internship on LCA of hydrogen production\d0se00222d1\CH4-to-H2 lca_algebraic\H2LCIA_result\H2LCIA_result_SMR_HTLT_CCS98_EU_elec.xlsx', sheet_name='H2_CIA_result', index = True)
A = df2.columns[3]
name = df2.columns[0]
graph = df2.plot.bar(figsize=(100,50), y = A,x=name, fontsize = 20)


graph.legend(loc='center left', bbox_to_anchor=(0.04, 1.1)) 


In [ ]:
#water_EF
#NOF_IN
#water_ReCiPe_v
impact = NOF_ON
act = maize_all_CN
# turbine_1_3MW_onshore
# glass_fibre_reinforced
# concrete_40MPa

df = exploreImpacts(impact, act, R_SM=62.7 / (62.7+160.2),R_Straw_return=0.30,ER_NH3=0.870,CO2_maizep_total=0.63812,ER_N2O=1.26,ER_NO3=1.78,ER_NO2_N2=0.375,N_Straw=0.0061,R_straw=0.84,N_strawFR_SM_100=22.2973,R_cob=0.2105)

df.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\maize production_v2.xlsx', sheet_name='maize production', index = True)
df2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\maize production_v2.xlsx', sheet_name='maize production', skiprows=[0])
df2
#df['impact']=df['impact'].astype(float)
#df=df.astype(float)
#df['blocks']=df['blocks'].astype(float)

In [ ]:
#df.to_excel(r'F:\学也无止境\intership\OIE internship\Internship on LCA of hydrogen production\d0se00222d1\CH4-to-H2 lca_algebraic\H2LCIA_result\H2LCIA_result_SMR_HTLT_CCS98_EU_elec.xlsx', sheet_name='H2_CIA_result', index = True)
A = df2.columns[3]
name = df2.columns[0]
graph = df2.plot.bar(figsize=(100,50), y = A,x=name, fontsize = 20)


graph.legend(loc='center left', bbox_to_anchor=(0.04, 1.1)) 


In [ ]:
sb.distplot(NOF_ON_scores)

In [ ]:
list_functional_units = [{ME_fueltMJ_LCI_rainf.key:1}]
list_methods = [CC_EF3,water_EF,NIF_IN,NIF_ON,NOF_IN,NOF_ON]
bw.calculation_setups['panneau_production'] = {'inv':list_functional_units, 'ia':list_methods}
bw.calculation_setups['panneau_production']
PVMultiLCA = bw.LeastSquaresLCA('panneau_production')
PVMultiLCA.results.shape
dfPVLCIA=pd.DataFrame(index=list_methods, columns=[ME_fueltMJ_LCI_rainf['name']], data=PVMultiLCA.results.T)
dfPVLCIA

#### scenario of organic maize cultivation 

In [ ]:
org_irr_m = 0.000852+0.000036+0.000258+0.001427+0.000056+0.003381+0.000198+0.027484+0.007619+0.000125+0.000006+0.000354+0.000058+0.014641+0.000281+0.000096

In [ ]:
Elec_org_m = 0.000052+0.000010+0.000211+0.000759+0.000155+0.002349+0.000379+0.001100

In [ ]:
eidb_cgrain_org = findActivity(name='maize grain production, organic', loc='RoW', db_name='ecoinvent 3.7.1 apos')
eidb_cgrain_org

In [ ]:
Elec_Trichogramma = findActivity(name='market group for electricity, low voltage', loc='CN', db_name='ecoinvent 3.7.1 apos')
Elec_Trichogramma

In [ ]:
organic_maize_all = copyActivity(
    USER_DB, # base utilisateur
    eidb_cgrain_org, # activité initiale 
    'organic maize production in china, unallocated whole plant with grain, straw, and cob, flow localizated') # Nouveau nom
#printAct(MonoSi_ingot_CN)

#MonoSi_ingot_CN.updateExchanges({
#   'market group for electricity, medium voltage' : Elec_MonoSi_ingot})
#printAct(MonoSi_ingot_CN, MonoSi_ingot_init_CN)

organic_maize_all.addExchanges({ 
    
    CO2_air : CO2_maizep_total,
    N_recycling : (R_SM*FRN_WWSM_eidbgrain +(1-R_SM)*FRN_OM_eidbgrain),
    N2_air : 10, # demo value 反硝化作用
    Elec_Trichogramma: Elec_org_m,  
    #NO3 : (R_SM*N_KGcgrain_eidb*26.9/100 + (1-R_SM)*N_KGcgrain_eidb*(11.8/(1+(ER_NO3-1)*0.3)*(1+(ER_NO3-1)*R_Straw_return))/100)/(14.0067/62.004),
    #P2O5_RoW_LCI : 100
    #iron : Q_iron_gear
    #water_e_unspecified: -0.170072,
    #water_e_surface: -0.014934,
    #water_e_ground: -0.059737,
})

In [ ]:
organic_maize_all_CN = copyActivity(
    USER_DB, # base utilisateur
    organic_maize_all, # activité initiale 
    'organic maize production in china, unallocated whole plant')

N2O_KGcgrain_eidb = (R_SM*N_KGcgrain_eidb*0.9/100 + (1-R_SM)*N_KGcgrain_eidb*(0.9/(1+(ER_N2O-1)*0.3)*(1+(ER_N2O-1)*R_Straw_return))/100)/(14.0067/44.013)

organic_maize_all_CN.updateExchanges({ 
    
    #'transport, tractor and trailer, agricultural' : T_infield,
    #'potassium chloride': KCl_RoW,
    #'lime': lime_RoW,
    
    'drying of maize straw and whole-plant' : 0.409840*(1+R_straw+R_cob), # to onclude the drying of maize straw and maize cob
    'Ammonia' : (R_SM*N_KGcgrain_eidb*7.4/100 + (1-R_SM)*N_KGcgrain_eidb*(5.2/(1-(1-ER_NH3)*0.3)*(1-(1-ER_NH3)*R_Straw_return))/100)/(14.0067/17.0305),
    'Dinitrogen monoxide' : N2O_KGcgrain_eidb,
    'Nitrate': (R_SM*N_KGcgrain_eidb*26.9/100 + (1-R_SM)*N_KGcgrain_eidb*(11.8/(1+(ER_NO3-1)*0.3)*(1+(ER_NO3-1)*R_Straw_return))/100)/(14.0067/62.004),
    'Nitrogen' : (1-ER_NO2_N2)/ER_NO2_N2*N2O_KGcgrain_eidb,
    #iron : Q_iron_gear
    
    'irrigation''#CN':org_irr_m,
    'irrigation''#ES':0,
    'irrigation''#IN':0,
    'irrigation''#BR':0,
    'irrigation''#CA-QC':0,
    'irrigation''#CO':0,
    'irrigation''#DE':0,
    'irrigation''#FR':0,
    'irrigation''#PE':0,
    'irrigation''#MY':0,
    'irrigation''#PH':0,
    'irrigation''#US':0,
    'irrigation''#TN':0,
    'irrigation''#ZA':0,
    'irrigation''#RoW':0,
    'irrigation''#MA':0,
    
    'electricity, low voltage''#RLA':0,
    'electricity, low voltage''#RAS':0,
    'electricity, low voltage''#RAF':0,
    'electricity, low voltage''#RNA':0,
    'electricity, low voltage''#RU':0,
    'electricity, low voltage''#NZ':0,
    'electricity, low voltage''#AU':0,
    'electricity, low voltage''#Europe without Switzerland':0,
    
})

printAct(organic_maize_all, organic_maize_all_CN)

In [ ]:
M_strawp_LCI_org = newActivity(USER_DB, "organic maize straw production, harvested, at farm, mass allocation", "kg straw")

M_strawp_LCI_org.addExchanges({ 
    

    organic_maize_all_CN:R_straw/(1+R_straw+R_cob)/R_straw,
    
})

In [ ]:
M_strawT_LCI_org = newActivity(USER_DB, "organic maize straw production, harvested and packaged, transported to gasification plant, mass allocation", "kg straw")

M_strawT_LCI_org.addExchanges({ 
    
    M_strawp_LCI_org : 1,
    
    Elec_crush_pack : Elec_C/1000,
    Diesel_crush_pack : Diesel_C/1000*(1/0.0222), # diesel density 1/0.0222 
    
    T_straw_lorry : TD_straw*1/1000
    
})

In [ ]:
M_fuelp_LCI_org = newActivity(USER_DB, "organic maize straw gasification and Fischer-Tropsch bio-jet fuel production", "kg straw")

M_fuelp_LCI_org.addExchanges({ 
    
    Steel_building: P_steel*1000/strawc_total,
    Iron_building: P_iron*1000/strawc_total,
    Al_building: P_Al*1000/strawc_total,
    cement_building: P_cement*1000/strawc_total,
    Steel_EoL: -(P_steel+P_iron)*1000/strawc_total,
    Al_EoL: -P_Al*1000/strawc_total,
    cement_EoL: -P_cement*1000/strawc_total,
    
    M_strawT_LCI_org: 1,
    Elec_millp: Elec_milling/1000,
    
    Si_sand: 500/((7200/8760)*24)*5*1000/strawc_total,
    Water_p: (5292.23/11760)*1,          
    
    Co_catalyst: Q_catalyst_Co,
    Ru_catalyst: Q_catalyst_Ru,
    ZrO2_catalyst: Q_catalyst_ZrO2,
    Al2O3_catalyst: Q_catalyst_Al2O3,
    
    NiO_catalyst: Q_catalyst_NiO, # represent NiO, same reaseon as stated for Ru catalyst
    W_catalyst: Q_catalyst_W, # use ammonium paratungstate in ecoinvent to represent WO3 based on W atomic mass conservation, as same reaseon as stated for Ru catalyst 
    HY_catalyst: Q_catalyst_HY,
    
    EoL_catalyst: Q_catalyst_Co+Q_catalyst_Ru+Q_catalyst_ZrO2+Q_catalyst_Al2O3+Q_catalyst_NiO+Q_catalyst_W+Q_catalyst_HY,
    
    CO2_e: CO2_pe,
    CH4_e: CH4_pe,
    N2O_e: N2O_pe,
    NOx_e: NOx_pe,
    SO2_e: SO2_pe,
    CO_e : CO_pe,
    HC_e : HC_pe,
    PM10_e: PM10_pe,
    COD_e: COD_pe,
    solidwaste_EoL: -soildw_pe,
    TN_e: TN_pe,
    NH4_e: NH3N_pe/14.0067*118.04,
    
    
})

In [ ]:
M_fuelpMJ_LCI_org = newActivity(USER_DB, "bio-jet fuel production from organic maize straw gasification and Fischer-Tropsch, unallocated", "MJ fuel")

M_fuelpMJ_LCI_org.addExchanges({ 
    
    M_fuelp_LCI_org: R_biojet_straw/H_biojet,
    
    })
# calcualted based on 1 kg bio-jet produced from corn straw
#printAct(M_fuelpMJ_LCI)

In [ ]:
Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
   
ME_fuelpMJ_LCI_org = newActivity(USER_DB, "bio-jet fuel produced from organic maize straw gasification and Fischer-Tropsch, energy allocation and electricity substitution, at plant", "MJ fuel")

ME_fuelpMJ_LCI_org.addExchanges({ 
    
    M_fuelpMJ_LCI_org: Allocate_RE_fuelp_B,
    Elec_subst: -(4221-2644)/1000*Allocate_RE_fuelp_B/H_biojet,
    Water_gen: m_elecsteam_tb*Allocate_RE_fuelp_B/H_biojet
    })
#printAct(M_fuelpMJ_LCI)

In [ ]:
#Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
   
ME_fueltMJ_LCI_org = newActivity(USER_DB, "bio-jet fuel delivery and combustion in aircraft, organic maize", "MJ fuel")

ME_fueltMJ_LCI_org.addExchanges({ 
    
    ME_fuelpMJ_LCI_org:1,
    T_jetfuel_lorry: 1/H_biojet*D_lorry_jetfuel/1000,
    T_jetfuel_train: 1/H_biojet*D_train_jetfuel/1000,
    Elec_delivery: Elec_fuel_delivery/H_biojet,
    CH4_fuele:CH4_fuelem,
    N2O_fuele:N2O_fuelem,
    CO2_fuele:CO2_fuelem,
    VOC_fuele:VOC_fuelem,
    CO_fuele:CO_fuelem,
    NOx_fuele:NOx_fuelem,
    PM10_fuele:PM10_fuelem,
    })
#printAct(M_fuelpMJ_LCI)

##### C_N_W footprint contribution

In [ ]:
FixedParamMode='mean'

In [ ]:
freezeParams(USER_DB)

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI_org, NIF_IN, ratio = 0.9, length_max = 10)

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI_org, NIF_ON, ratio = 0.9, length_max = 10)

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI_org, NOF_IN, ratio = 0.9, length_max = 10)

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI_org, NOF_ON, ratio = 0.9, length_max = 10)

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI_org, CC_EF3, ratio = 0.9, length_max = 10)

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI_org, water_EF, ratio = 0.9, length_max = 9)

## nitrogen flow sustainability index, carbon, water footprint calculation

### sum of paramaters

In [ ]:
list_parameters()

In [ ]:
FixedParamMode='mean'

In [ ]:
freezeParams(USER_DB)

### Carbon footprint calculation and its contribution on biosphere elementary flow and technosphere activities

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI, CC_EF3, ratio = 0.91, length_max = 9)

In [ ]:
plot_sankey(act = ME_fueltMJ_LCI, method = CC_EF3, cutoff = 0.05)

In [ ]:
#water_EF
#NOF_IN
#water_ReCiPe_v
# heat_CA_sankey
impact = CC_EF3
act = heat_CA_sankey
# turbine_1_3MW_onshore
# glass_fibre_reinforced
# concrete_40MPa

df = exploreImpacts(impact, act)
#, R_SM=62.7 / (62.7+160.2),R_Straw_return=0.30,ER_NH3=0.870,CO2_maizep_total=0.63812,ER_N2O=1.26,ER_NO3=1.78,ER_NO2_N2=0.375,N_Straw=0.0061,R_straw=0.84,N_strawFR_SM_100=22.2973,R_cob=0.2105)

df.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\heatp10MW_v2.xlsx', sheet_name='maize production', index = True)
df2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\heatp10MW_v2.xlsx', sheet_name='maize production', skiprows=[0])
df2
#df['impact']=df['impact'].astype(float)
#df=df.astype(float)
#df['blocks']=df['blocks'].astype(float)

In [ ]:
#df.to_excel(r'F:\学也无止境\intership\OIE internship\Internship on LCA of hydrogen production\d0se00222d1\CH4-to-H2 lca_algebraic\H2LCIA_result\H2LCIA_result_SMR_HTLT_CCS98_EU_elec.xlsx', sheet_name='H2_CIA_result', index = True)
A = df2.columns[3]
name = df2.columns[0]
graph = df2.plot.bar(figsize=(20,5), y = A,x=name, fontsize = 10)


graph.legend(loc='center left', bbox_to_anchor=(0.04, 1.1)) 


In [ ]:
plot_sankey(act = ammonium_nitrate_CNS, method = CC_EF3, cutoff = 0.05)

In [ ]:
plot_sankey(act = T_jetfuel_lorryCA, method = CC_EF3, cutoff = 0.05)

#### other scenarios

##### S-10

In [ ]:
#water_EF
#NOF_IN
#water_ReCiPe_v
#maize_all_CN
#ME_fueltMJ_LCI
#Diesel_sankey
#Elec_deliveryCA
#irrigation_sankey
# ME_fuelpMJ_LCI
#M_fuelp_LCI

impact = NIF_ON
act = ZeE_fuelpMJ_LCI

# turbine_1_3MW_onshore
# glass_fibre_reinforced
# concrete_40MPa

df = exploreImpacts(impact, act, R_SM=62.7 / (62.7+160.2),R_Straw_return=0.30,ER_NH3=0.870,CO2_maizep_total=0.63812,
                    ER_N2O=1.26,ER_NO3=1.78,ER_NO2_N2=0.375,N_Straw=0.0061,R_straw=0.84,N_strawFR_SM_100=22.2973,
                    R_cob=0.2105, CO2_fuelem=0.0703,CO_fuelem=8.968e-05,N2O_fuelem=2.088e-07,CH4_fuelem=1.064e-07,
                   NOx_fuelem=0.0003295,PM10_fuelem=3.411e-07,VOC_fuelem=1.299e-05,D_train_jetfuel=749,H_biojet=43.07,
                   D_lorry_jetfuel=50,Elec_fuel_delivery=0.0056, CoRu_catalyst=3, R_biojet_straw= 11.76, NiW_catalyst= 3,
                   P_Al=6, DR_feed= 500, Life_FTplant=15, P_iron=3,P_cement=1523,P_steel=484,Elec_milling=42,Diesel_C=0.67,Elec_C=18.2,
                   m_biojet_tb=85.03,H_diesel_tb=46,m_diesel_tb=19.83,H_gasoline_tb=43.1, m_gasoline_tb=43.79,H_paraffin_tb=44.857,
                   m_paraffin_tb=7.99, m_elecsteam_tb=5.292
                   
)




df.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\Nacc_check_v2.xlsx', sheet_name='maize production', index = True)
df2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\Nacc_check_v2.xlsx', sheet_name='maize production', skiprows=[0])
df2
#df['impact']=df['impact'].astype(float)
#df=df.astype(float)
#df['blocks']=df['blocks'].astype(float)

In [ ]:
#list_parameters()

In [ ]:
#df.to_excel(r'F:\学也无止境\intership\OIE internship\Internship on LCA of hydrogen production\d0se00222d1\CH4-to-H2 lca_algebraic\H2LCIA_result\H2LCIA_result_SMR_HTLT_CCS98_EU_elec.xlsx', sheet_name='H2_CIA_result', index = True)
A = df2.columns[3]
name = df2.columns[0]
graph = df2.plot.bar(figsize=(10,5), y = A,x=name, fontsize = 10)


graph.legend(loc='center left', bbox_to_anchor=(0.04, 1.1)) 


In [ ]:
Contribution_Analysis_one_impact_category_noact(ZeE_fueltMJ_LCI, NIF_IN, ratio = 0.9, length_max = 9)

In [ ]:
plot_sankey(act = ZeE_fueltMJ_LCI, method = NIF_IN, cutoff = 0.1)

In [ ]:
plot_sankey(act = ZeE_fueltMJ_LCI, method = NIF_ON, cutoff = 0.1)

##### S-12

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI_org, NIF_ON, ratio = 0.9, length_max = 9)

In [ ]:
manure_p= [act for act in eidb if 'green manure growing, organic, until April' in act['name'] 
                                            and 'RoW' in act['location']
                                            #and 'non' not in act['name']
                                            #and 'urban air close to ground' in str(act['categories'])
      ][0]
manure_p

In [ ]:
manure_ps = copyActivity(
    USER_DB, # base utilisateur
    manure_p, # activité initiale 
    'manure_ps') # Nouveau nom
manure_ps

In [ ]:
#water_EF
#NOF_IN
#water_ReCiPe_v
#maize_all_CN
#ME_fueltMJ_LCI
#Diesel_sankey
#Elec_deliveryCA
#irrigation_sankey
# ME_fuelpMJ_LCI
#M_fuelp_LCI
#manure_ps
impact = NIF_ON
act = organic_maize_all_CN

# turbine_1_3MW_onshore
# glass_fibre_reinforced
# concrete_40MPa

df = exploreImpacts(impact, act, R_SM=62.7 / (62.7+160.2),R_Straw_return=0.30,ER_NH3=0.870,CO2_maizep_total=0.63812,
                    ER_N2O=1.26,ER_NO3=1.78,ER_NO2_N2=0.375,N_Straw=0.0061,R_straw=0.84,N_strawFR_SM_100=22.2973,
                    R_cob=0.2105, CO2_fuelem=0.0703,CO_fuelem=8.968e-05,N2O_fuelem=2.088e-07,CH4_fuelem=1.064e-07,
                   NOx_fuelem=0.0003295,PM10_fuelem=3.411e-07,VOC_fuelem=1.299e-05,D_train_jetfuel=749,H_biojet=43.07,
                   D_lorry_jetfuel=50,Elec_fuel_delivery=0.0056, CoRu_catalyst=3, R_biojet_straw= 11.76, NiW_catalyst= 3,
                   P_Al=6, DR_feed= 500, Life_FTplant=15, P_iron=3,P_cement=1523,P_steel=484,Elec_milling=42,Diesel_C=0.67,Elec_C=18.2,
                   m_biojet_tb=85.03,H_diesel_tb=46,m_diesel_tb=19.83,H_gasoline_tb=43.1, m_gasoline_tb=43.79,H_paraffin_tb=44.857,
                   m_paraffin_tb=7.99, m_elecsteam_tb=5.292
                   
)




df.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\Nacc_check_v2.xlsx', sheet_name='maize production', index = True)
df2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\Nacc_check_v2.xlsx', sheet_name='maize production', skiprows=[0])
df2
#df['impact']=df['impact'].astype(float)
#df=df.astype(float)
#df['blocks']=df['blocks'].astype(float)

In [ ]:
#list_parameters()

In [ ]:
#df.to_excel(r'F:\学也无止境\intership\OIE internship\Internship on LCA of hydrogen production\d0se00222d1\CH4-to-H2 lca_algebraic\H2LCIA_result\H2LCIA_result_SMR_HTLT_CCS98_EU_elec.xlsx', sheet_name='H2_CIA_result', index = True)
A = df2.columns[3]
name = df2.columns[0]
graph = df2.plot.bar(figsize=(10,5), y = A,x=name, fontsize = 10)


graph.legend(loc='center left', bbox_to_anchor=(0.04, 1.1)) 


In [ ]:
#water_EF
#NOF_IN
#water_ReCiPe_v
#maize_all_CN
#ME_fueltMJ_LCI
#Diesel_sankey
#Elec_deliveryCA
#irrigation_sankey
# ME_fuelpMJ_LCI
#M_fuelp_LCI
#manure_ps
impact = NIF_ON
act = manure_ps

# turbine_1_3MW_onshore
# glass_fibre_reinforced
# concrete_40MPa

df = exploreImpacts(impact, act      
)




df.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\Manurep_v2.xlsx', sheet_name='maize production', index = True)
df2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\Manurep_v2.xlsx', sheet_name='maize production', skiprows=[0])
df2
#df['impact']=df['impact'].astype(float)
#df=df.astype(float)
#df['blocks']=df['blocks'].astype(float)

In [ ]:
#list_parameters()

In [ ]:
#df.to_excel(r'F:\学也无止境\intership\OIE internship\Internship on LCA of hydrogen production\d0se00222d1\CH4-to-H2 lca_algebraic\H2LCIA_result\H2LCIA_result_SMR_HTLT_CCS98_EU_elec.xlsx', sheet_name='H2_CIA_result', index = True)
A = df2.columns[3]
name = df2.columns[0]
graph = df2.plot.bar(figsize=(10,5), y = A,x=name, fontsize = 10)


graph.legend(loc='center left', bbox_to_anchor=(0.04, 1.1)) 


In [ ]:
plot_sankey(act = manure_ps, method = NIF_ON, cutoff = 0.1) #ME_fueltMJ_LCI_org

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI_org, NIF_IN, ratio = 0.9, length_max = 9)

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI_org, NIF_ON, ratio = 0.9, length_max = 9)

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI_org, NOF_IN, ratio = 0.9, length_max = 9)

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI_org, NOF_ON, ratio = 0.9, length_max = 9)

In [ ]:
#water_EF
#NOF_IN
#water_ReCiPe_v
#maize_all_CN
#ME_fueltMJ_LCI
#Diesel_sankey
#Elec_deliveryCA
#irrigation_sankey
# ME_fuelpMJ_LCI
#M_fuelp_LCI
#manure_ps
impact = NOF_ON
act = organic_maize_all_CN

# turbine_1_3MW_onshore
# glass_fibre_reinforced
# concrete_40MPa

df = exploreImpacts(impact, act, R_SM=62.7 / (62.7+160.2),R_Straw_return=0.30,ER_NH3=0.870,CO2_maizep_total=0.63812,
                    ER_N2O=1.26,ER_NO3=1.78,ER_NO2_N2=0.375,N_Straw=0.0061,R_straw=0.84,N_strawFR_SM_100=22.2973,
                    R_cob=0.2105, CO2_fuelem=0.0703,CO_fuelem=8.968e-05,N2O_fuelem=2.088e-07,CH4_fuelem=1.064e-07,
                   NOx_fuelem=0.0003295,PM10_fuelem=3.411e-07,VOC_fuelem=1.299e-05,D_train_jetfuel=749,H_biojet=43.07,
                   D_lorry_jetfuel=50,Elec_fuel_delivery=0.0056, CoRu_catalyst=3, R_biojet_straw= 11.76, NiW_catalyst= 3,
                   P_Al=6, DR_feed= 500, Life_FTplant=15, P_iron=3,P_cement=1523,P_steel=484,Elec_milling=42,Diesel_C=0.67,Elec_C=18.2,
                   m_biojet_tb=85.03,H_diesel_tb=46,m_diesel_tb=19.83,H_gasoline_tb=43.1, m_gasoline_tb=43.79,H_paraffin_tb=44.857,
                   m_paraffin_tb=7.99, m_elecsteam_tb=5.292
                   
)




df.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\Organic_maizep_NOF_ON.xlsx', sheet_name='maize production', index = True)
df2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\Organic_maizep_NOF_ON.xlsx', sheet_name='maize production', skiprows=[0])
df2
#df['impact']=df['impact'].astype(float)
#df=df.astype(float)
#df['blocks']=df['blocks'].astype(float)

In [ ]:
#list_parameters()

In [ ]:
#df.to_excel(r'F:\学也无止境\intership\OIE internship\Internship on LCA of hydrogen production\d0se00222d1\CH4-to-H2 lca_algebraic\H2LCIA_result\H2LCIA_result_SMR_HTLT_CCS98_EU_elec.xlsx', sheet_name='H2_CIA_result', index = True)
A = df2.columns[3]
name = df2.columns[0]
graph = df2.plot.bar(figsize=(10,5), y = A,x=name, fontsize = 10)


graph.legend(loc='center left', bbox_to_anchor=(0.04, 1.1)) 


In [ ]:
#water_EF
#NOF_IN
#water_ReCiPe_v
#maize_all_CN
#ME_fueltMJ_LCI
#Diesel_sankey
#Elec_deliveryCA
#irrigation_sankey
# ME_fuelpMJ_LCI
#M_fuelp_LCI
#manure_ps
impact = NOF_ON
act = manure_ps

# turbine_1_3MW_onshore
# glass_fibre_reinforced
# concrete_40MPa

df = exploreImpacts(impact, act      
)




df.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\Manurep_v2.xlsx', sheet_name='maize production', index = True)
df2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\Manurep_v2.xlsx', sheet_name='maize production', skiprows=[0])
df2
#df['impact']=df['impact'].astype(float)
#df=df.astype(float)
#df['blocks']=df['blocks'].astype(float)

In [ ]:
#list_parameters()

In [ ]:
#df.to_excel(r'F:\学也无止境\intership\OIE internship\Internship on LCA of hydrogen production\d0se00222d1\CH4-to-H2 lca_algebraic\H2LCIA_result\H2LCIA_result_SMR_HTLT_CCS98_EU_elec.xlsx', sheet_name='H2_CIA_result', index = True)
A = df2.columns[3]
name = df2.columns[0]
graph = df2.plot.bar(figsize=(10,5), y = A,x=name, fontsize = 10)


graph.legend(loc='center left', bbox_to_anchor=(0.04, 1.1)) 


In [ ]:
plot_sankey(act = ME_fueltMJ_LCI_org, method = NOF_ON, cutoff = 0.01) #ME_fueltMJ_LCI_org

In [ ]:
plot_sankey(act = ME_fueltMJ_LCI_org, method = CC_EF3, cutoff = 0.1) #ME_fueltMJ_LCI_org

In [ ]:
#water_EF
#NOF_IN
#water_ReCiPe_v
#maize_all_CN
#ME_fueltMJ_LCI
#Diesel_sankey
#Elec_deliveryCA
#irrigation_sankey
# ME_fuelpMJ_LCI
#M_fuelp_LCI
#manure_ps
impact = CC_EF3
act = organic_maize_all_CN

# turbine_1_3MW_onshore
# glass_fibre_reinforced
# concrete_40MPa

df = exploreImpacts(impact, act,R_SM=62.7 / (62.7+160.2),R_Straw_return=0.30,ER_NH3=0.870,CO2_maizep_total=0.63812,
                    ER_N2O=1.26,ER_NO3=1.78,ER_NO2_N2=0.375,N_Straw=0.0061,R_straw=0.84,N_strawFR_SM_100=22.2973,
                    R_cob=0.2105, CO2_fuelem=0.0703,CO_fuelem=8.968e-05,N2O_fuelem=2.088e-07,CH4_fuelem=1.064e-07,
                   NOx_fuelem=0.0003295,PM10_fuelem=3.411e-07,VOC_fuelem=1.299e-05,D_train_jetfuel=749,H_biojet=43.07,
                   D_lorry_jetfuel=50,Elec_fuel_delivery=0.0056, CoRu_catalyst=3, R_biojet_straw= 11.76, NiW_catalyst= 3,
                   P_Al=6, DR_feed= 500, Life_FTplant=15, P_iron=3,P_cement=1523,P_steel=484,Elec_milling=42,Diesel_C=0.67,Elec_C=18.2,
                   m_biojet_tb=85.03,H_diesel_tb=46,m_diesel_tb=19.83,H_gasoline_tb=43.1, m_gasoline_tb=43.79,H_paraffin_tb=44.857,
                   m_paraffin_tb=7.99, m_elecsteam_tb=5.292     
)




df.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\Maizep_org_CF.xlsx', sheet_name='maize production', index = True)
df2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\Maizep_org_CF.xlsx', sheet_name='maize production', skiprows=[0])
df2
#df['impact']=df['impact'].astype(float)
#df=df.astype(float)
#df['blocks']=df['blocks'].astype(float)

In [ ]:
#df.to_excel(r'F:\学也无止境\intership\OIE internship\Internship on LCA of hydrogen production\d0se00222d1\CH4-to-H2 lca_algebraic\H2LCIA_result\H2LCIA_result_SMR_HTLT_CCS98_EU_elec.xlsx', sheet_name='H2_CIA_result', index = True)
A = df2.columns[3]
name = df2.columns[0]
graph = df2.plot.bar(figsize=(10,5), y = A,x=name, fontsize = 10)


graph.legend(loc='center left', bbox_to_anchor=(0.04, 1.1)) 


### Water footprint calculation and its contribution on biosphere elementary flow and technosphere activities

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI, water_EF, ratio = 0.9, length_max = 9)

In [ ]:
#Contribution_Analysis_one_impact_category_noact(maize_all_CN, water_ReCiPe_v, ratio = 0.9, length_max = 9)

In [ ]:
#water_EF
#NOF_IN
#water_ReCiPe_v
impact = water_EF
act = ME_fueltMJ_LCI
# turbine_1_3MW_onshore
# glass_fibre_reinforced
# concrete_40MPa

df = exploreImpacts(impact, act, R_SM=62.7 / (62.7+160.2),R_Straw_return=0.30,ER_NH3=0.870,CO2_maizep_total=0.63812,
                    ER_N2O=1.26,ER_NO3=1.78,ER_NO2_N2=0.375,N_Straw=0.0061,R_straw=0.84,N_strawFR_SM_100=22.2973,
                    R_cob=0.2105, CO2_fuelem=0.0703,CO_fuelem=8.968e-05,N2O_fuelem=2.088e-07,CH4_fuelem=1.064e-07,
                   NOx_fuelem=0.0003295,PM10_fuelem=3.411e-07,VOC_fuelem=1.299e-05,D_train_jetfuel=749,H_biojet=43.07,
                   D_lorry_jetfuel=50,Elec_fuel_delivery=0.0056, CoRu_catalyst=3, R_biojet_straw= 11.76, NiW_catalyst= 3,
                   P_Al=6, DR_feed= 500, Life_FTplant=15, P_iron=3,P_cement=1523,P_steel=484,Elec_milling=42,Diesel_C=0.67,Elec_C=18.2
                   )
df.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\water_v3.xlsx', sheet_name='maize production', index = True)
df2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\water_v3.xlsx', sheet_name='maize production', skiprows=[0])
df2
#df['impact']=df['impact'].astype(float)
#df=df.astype(float)
#df['blocks']=df['blocks'].astype(float)

In [ ]:
#df.to_excel(r'F:\学也无止境\intership\OIE internship\Internship on LCA of hydrogen production\d0se00222d1\CH4-to-H2 lca_algebraic\H2LCIA_result\H2LCIA_result_SMR_HTLT_CCS98_EU_elec.xlsx', sheet_name='H2_CIA_result', index = True)
A = df2.columns[3]
name = df2.columns[0]
graph = df2.plot.bar(figsize=(20,5), y = A,x=name, fontsize = 10)


graph.legend(loc='center left', bbox_to_anchor=(0.04, 1.1)) 


In [ ]:
#T_train_diesel = findActivity(name='transport, freight train, diesel', loc='CN', db_name='ecoinvent 3.7.1 apos')

In [ ]:
plot_sankey(act = ME_fueltMJ_LCI, method = water_EF, cutoff = 0.1)

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI, water_ILCD, ratio = 0.9, length_max = 9)

In [ ]:
Contribution_Analysis_one_impact_category_noact(eidb_cgrain, water_EF, ratio = 0.9, length_max = 9)

In [ ]:
plot_sankey(act = maize_all_CN, method = water_EF, cutoff = 0.05)

In [ ]:
#water_EF
#NOF_IN
#water_ReCiPe_v
impact = water_EF
act = maize_all_CN
# turbine_1_3MW_onshore
# glass_fibre_reinforced
# concrete_40MPa

df = exploreImpacts(impact, act, R_SM=62.7 / (62.7+160.2),R_Straw_return=0.30,ER_NH3=0.870,CO2_maizep_total=0.63812,
                    ER_N2O=1.26,ER_NO3=1.78,ER_NO2_N2=0.375,N_Straw=0.0061,R_straw=0.84,N_strawFR_SM_100=22.2973,
                    R_cob=0.2105, CO2_fuelem=0.0703,CO_fuelem=8.968e-05,N2O_fuelem=2.088e-07,CH4_fuelem=1.064e-07,
                   NOx_fuelem=0.0003295,PM10_fuelem=3.411e-07,VOC_fuelem=1.299e-05,D_train_jetfuel=749,H_biojet=43.07,
                   D_lorry_jetfuel=50,Elec_fuel_delivery=0.0056, CoRu_catalyst=3, R_biojet_straw= 11.76, NiW_catalyst= 3,
                   P_Al=6, DR_feed= 500, Life_FTplant=15, P_iron=3,P_cement=1523,P_steel=484,Elec_milling=42,Diesel_C=0.67,Elec_C=18.2
                   )
df.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\water_v3.xlsx', sheet_name='maize production', index = True)
df2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\water_v3.xlsx', sheet_name='maize production', skiprows=[0])
df2
#df['impact']=df['impact'].astype(float)
#df=df.astype(float)
#df['blocks']=df['blocks'].astype(float)

In [ ]:
#df.to_excel(r'F:\学也无止境\intership\OIE internship\Internship on LCA of hydrogen production\d0se00222d1\CH4-to-H2 lca_algebraic\H2LCIA_result\H2LCIA_result_SMR_HTLT_CCS98_EU_elec.xlsx', sheet_name='H2_CIA_result', index = True)
A = df2.columns[3]
name = df2.columns[0]
graph = df2.plot.bar(figsize=(20,5), y = A,x=name, fontsize = 10)


graph.legend(loc='center left', bbox_to_anchor=(0.04, 1.1)) 


#### water footprint check

In [ ]:
water_surface_irr = findActivity(name='irrigation, surface', loc='CN', db_name='ecoinvent 3.7.1 apos')

In [ ]:
Contribution_Analysis_one_impact_category_noact(water_surface_irr, water_EF, ratio = 0.9, length_max = 9)

In [ ]:
Contribution_Analysis_one_impact_category_noact(Elec_delivery, water_EF, ratio = 0.9, length_max = 9)

In [ ]:
plot_sankey(act = ME_fueltMJ_LCI, method = NOF_ON, cutoff = 0.1) #ME_fueltMJ_LCI, NIF_ON #Elec_delivery
#water_EF

### Backbone N flow footprint and their contributions on biosphere elementary flow and technosphere activities based on the GNFA theory

In [ ]:
list_functional_units = [{ME_fueltMJ_LCI.key:1}, {panneau2.key:1}]
list_methods = [NIF_ON]
bw.calculation_setups['panneau_production'] = {'inv':list_functional_units, 'ia':list_methods}
bw.calculation_setups['panneau_production']
PVMultiLCA = bw.MultiLCA('panneau_production')
PVMultiLCA.results.shape
dfPVLCIA=pd.DataFrame(index=list_methods, columns=[ME_fueltMJ_LCI['name'], panneau2['name']], data=PVMultiLCA.results.T)
dfPVLCIA

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI, NIF_IN, ratio = 0.9, length_max = 10)

In [ ]:
plot_sankey(act = ME_fueltMJ_LCI, method = NIF_IN, cutoff = 0.05) #ME_fueltMJ_LCI, NIF_ON #Elec_delivery
#water_EF

In [ ]:
# drying of maize straw and whole-plant
# heat production, at hard coal industrial furnace 1-10MW
# market for hard coal 
# hard coal mine operation and hard coal preparation
# market for irrigation
# M_fuelp_LCI
# T_jetfuel_lorryCA
# M_strawT_LCI
# Elec_deliveryCA
# Co_catalyst_sankey
test= [act for act in eidb if 'market for irrigation' in act['name'] 
                                            and 'CN' in act['location']
                                            #and 'CN-SGCC' in act['location']
                                            and 'run-of-mine' not in act['name']
                                            and 'briquettes' not in act['name']
                                            and 'ash' not in act['name']
                                            #and 'urban air close to ground' in str(act['categories'])
      ][0]
test

In [ ]:
plot_sankey(act = Diesel_sankey, method = NIF_IN, cutoff = 0.05) #ME_fueltMJ_LCI, NIF_ON #Elec_delivery
#water_EF

In [ ]:
# heat production, at hard coal industrial furnace 1-10MW
# market for hard coal 
# hard coal mine operation and hard coal preparation
test= [act for act in eidb if 'heat production, at hard coal industrial furnace 1-10MW' in act['name'] 
                                            and 'Europe without Switzerland' in act['location']
                                            and 'run-of-mine' not in act['name']
                                            and 'briquettes' not in act['name']
                                            and 'ash' not in act['name']
                                            #and 'urban air close to ground' in str(act['categories'])
      ][0]
test

In [ ]:
plot_sankey(act = Elec_deliveryCA, method = NIF_IN, cutoff = 0.05) #ME_fueltMJ_LCI, NIF_ON #Elec_delivery
#water_EF

In [ ]:
Diesel_sankey = copyActivity(
    USER_DB, # base utilisateur
    Diesel_crush_pack, # activité initiale 
    'Diesel_sankey') # Nouveau nom
Diesel_sankey

Elec_lowV_sankey = copyActivity(
    USER_DB, # base utilisateur
    Elec_lowV, # activité initiale 
    'Elec_lowV_sankey') # Nouveau nom
Elec_lowV_sankey

T_jetfuel_train_sankey = copyActivity(
    USER_DB, # base utilisateur
    T_jetfuel_train, # activité initiale 
    'T_jetfuel_train_sankey') # Nouveau nom

Co_catalyst_sankey = copyActivity(
    USER_DB, # base utilisateur
    Co_catalyst, # activité initiale 
    'Co_catalyst_sankey') # Nouveau nom


Elec_lowV
Elec_lowV_sankey
T_jetfuel_train_sankey
Co_catalyst_sankey

In [ ]:
#water_EF
#NOF_IN
#water_ReCiPe_v
#maize_all_CN
#ME_fueltMJ_LCI
#Diesel_sankey
#Elec_deliveryCA
impact = NIF_ON
act = maize_all_CN
# turbine_1_3MW_onshore
# glass_fibre_reinforced
# concrete_40MPa

df = exploreImpacts(impact, act, R_SM=62.7 / (62.7+160.2),R_Straw_return=0.30,ER_NH3=0.870,CO2_maizep_total=0.63812,
                    ER_N2O=1.26,ER_NO3=1.78,ER_NO2_N2=0.375,N_Straw=0.0061,R_straw=0.84,N_strawFR_SM_100=22.2973,
                    R_cob=0.2105, CO2_fuelem=0.0703,CO_fuelem=8.968e-05,N2O_fuelem=2.088e-07,CH4_fuelem=1.064e-07,
                   NOx_fuelem=0.0003295,PM10_fuelem=3.411e-07,VOC_fuelem=1.299e-05,D_train_jetfuel=749,H_biojet=43.07,
                   D_lorry_jetfuel=50,Elec_fuel_delivery=0.0056, CoRu_catalyst=3, R_biojet_straw= 11.76, NiW_catalyst= 3,
                   P_Al=6, DR_feed= 500, Life_FTplant=15, P_iron=3,P_cement=1523,P_steel=484,Elec_milling=42,Diesel_C=0.67,Elec_C=18.2
)


df.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\maize production_v3_NR.xlsx', sheet_name='maize production', index = True)
df2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\maize production_v3_NR.xlsx', sheet_name='maize production', skiprows=[0])
df2
#df['impact']=df['impact'].astype(float)
#df=df.astype(float)
#df['blocks']=df['blocks'].astype(float)

In [ ]:
#list_parameters()

In [ ]:
#df.to_excel(r'F:\学也无止境\intership\OIE internship\Internship on LCA of hydrogen production\d0se00222d1\CH4-to-H2 lca_algebraic\H2LCIA_result\H2LCIA_result_SMR_HTLT_CCS98_EU_elec.xlsx', sheet_name='H2_CIA_result', index = True)
# see amount: A = df2.columns[2]

A = df2.columns[3]
name = df2.columns[0]
graph = df2.plot.bar(figsize=(10,5), y = A,x=name, fontsize = 4)


graph.legend(loc='center left', bbox_to_anchor=(0.04, 1.1)) 


In [ ]:
#water_EF
#NOF_IN
#water_ReCiPe_v
#maize_all_CN
#ME_fueltMJ_LCI
#Diesel_sankey
#Elec_deliveryCA
impact = NIF_IN
act = M_fuelp_LCI
# turbine_1_3MW_onshore
# glass_fibre_reinforced
# concrete_40MPa

df = exploreImpacts(impact, act, R_SM=62.7 / (62.7+160.2),R_Straw_return=0.30,ER_NH3=0.870,CO2_maizep_total=0.63812,
                    ER_N2O=1.26,ER_NO3=1.78,ER_NO2_N2=0.375,N_Straw=0.0061,R_straw=0.84,N_strawFR_SM_100=22.2973,
                    R_cob=0.2105, CO2_fuelem=0.0703,CO_fuelem=8.968e-05,N2O_fuelem=2.088e-07,CH4_fuelem=1.064e-07,
                   NOx_fuelem=0.0003295,PM10_fuelem=3.411e-07,VOC_fuelem=1.299e-05,D_train_jetfuel=749,H_biojet=43.07,
                   D_lorry_jetfuel=50,Elec_fuel_delivery=0.0056, CoRu_catalyst=3, R_biojet_straw= 11.76, NiW_catalyst= 3,
                   P_Al=6, DR_feed= 500, Life_FTplant=15, P_iron=3,P_cement=1523,P_steel=484,Elec_milling=42,Diesel_C=0.67,Elec_C=18.2
)


df.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\maize production_v2.xlsx', sheet_name='maize production', index = True)
df2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\maize production_v2.xlsx', sheet_name='maize production', skiprows=[0])
df2
#df['impact']=df['impact'].astype(float)
#df=df.astype(float)
#df['blocks']=df['blocks'].astype(float)

In [ ]:
#list_parameters()

In [ ]:
#df.to_excel(r'F:\学也无止境\intership\OIE internship\Internship on LCA of hydrogen production\d0se00222d1\CH4-to-H2 lca_algebraic\H2LCIA_result\H2LCIA_result_SMR_HTLT_CCS98_EU_elec.xlsx', sheet_name='H2_CIA_result', index = True)
A = df2.columns[3]
name = df2.columns[0]
graph = df2.plot.bar(figsize=(10,5), y = A,x=name, fontsize = 4)


graph.legend(loc='center left', bbox_to_anchor=(0.04, 1.1)) 


In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI, NIF_ON, ratio = 0.9, length_max = 9)

In [ ]:
plot_sankey(act = ME_fueltMJ_LCI, method = NIF_ON, cutoff = 0.1) #ME_fueltMJ_LCI, NIF_ON #Elec_delivery
#water_EF

In [ ]:
#water_EF
#NOF_IN
#water_ReCiPe_v
#maize_all_CN
#ME_fueltMJ_LCI
#Diesel_sankey
#Elec_deliveryCA
#irrigation_sankey
impact = NIF_ON
act = mine_infrastucture_S

# turbine_1_3MW_onshore
# glass_fibre_reinforced
# concrete_40MPa

df = exploreImpacts(impact, act, )

#R_SM=62.7 / (62.7+160.2),R_Straw_return=0.30,ER_NH3=0.870,CO2_maizep_total=0.63812,
#                    ER_N2O=1.26,ER_NO3=1.78,ER_NO2_N2=0.375,N_Straw=0.0061,R_straw=0.84,N_strawFR_SM_100=22.2973,
#                    R_cob=0.2105, CO2_fuelem=0.0703,CO_fuelem=8.968e-05,N2O_fuelem=2.088e-07,CH4_fuelem=1.064e-07,
#                   NOx_fuelem=0.0003295,PM10_fuelem=3.411e-07,VOC_fuelem=1.299e-05,D_train_jetfuel=749,H_biojet=43.07,
#                   D_lorry_jetfuel=50,Elec_fuel_delivery=0.0056, CoRu_catalyst=3, R_biojet_straw= 11.76, NiW_catalyst= 3,
#                   P_Al=6, DR_feed= 500, Life_FTplant=15, P_iron=3,P_cement=1523,P_steel=484,Elec_milling=42,Diesel_C=0.67,Elec_C=18.2



df.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\mine_infrastucture_S.xlsx', sheet_name='maize production', index = True)
df2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\mine_infrastucture_S.xlsx', sheet_name='maize production', skiprows=[0])
df2
#df['impact']=df['impact'].astype(float)
#df=df.astype(float)
#df['blocks']=df['blocks'].astype(float)

In [ ]:
#list_parameters()

In [ ]:
#df.to_excel(r'F:\学也无止境\intership\OIE internship\Internship on LCA of hydrogen production\d0se00222d1\CH4-to-H2 lca_algebraic\H2LCIA_result\H2LCIA_result_SMR_HTLT_CCS98_EU_elec.xlsx', sheet_name='H2_CIA_result', index = True)
A = df2.columns[3]
name = df2.columns[0]
graph = df2.plot.bar(figsize=(10,5), y = A,x=name, fontsize = 5)


graph.legend(loc='center left', bbox_to_anchor=(0.04, 1.1)) 


In [ ]:
plot_sankey(act = Elec_deliveryCA, method = NIF_ON, cutoff = 0.1) #ME_fueltMJ_LCI, NIF_ON #Elec_delivery
#water_EF #highV_elec_CNSGCC_S

In [ ]:
highV_elec_CNSGCC_S = copyActivity(
    USER_DB, # base utilisateur
    highV_elec_CNSGCC, # activité initiale 
    'highV_elec_CNSGCC_S') # Nouveau nom

#highV_elec_CNSGCC

In [ ]:
hardcoal_mine_S = copyActivity(
    USER_DB, # base utilisateur
    hardcoal_mine, # activité initiale 
    'hardcoal_mine_S') # Nouveau nom
hardcoal_mine_S
#highV_elec_CNSGCC

In [ ]:
mine_infrastucture_S = copyActivity(
    USER_DB, # base utilisateur
    mine_infrastucture, # activité initiale 
    'mine_infrastucture_S') # Nouveau nom
mine_infrastucture_S
#highV_elec_CNSGCC

In [ ]:
maize_seed_sankey = copyActivity(
    USER_DB, # base utilisateur
    seed, # activité initiale 
    'maize_seed_sankey') # Nouveau nom

irrigation_sankey = copyActivity(
    USER_DB, # base utilisateur
    irrigation_CA, # activité initiale 
    'irrigation_sankey')

drying_maize_CA_sankey = copyActivity(
    USER_DB, # base utilisateur
    drying_maize_CA, # activité initiale 
    'drying_maize_CA_sankey')

heat_CA_sankey = copyActivity(
    USER_DB, # base utilisateur
    heat_CA, # activité initiale 
    'heat_CA_sankey')

Elec_coal_sankey = copyActivity(
    USER_DB, # base utilisateur
    Elec_coal, # activité initiale 
    'Elec_coal_sankey')

TrainT_diesel_sankey = copyActivity(
    USER_DB, # base utilisateur
    TrainT_diesel, # activité initiale 
    'TrainT_diesel_sankey')


NH3_RoW_S = copyActivity(
    USER_DB, # base utilisateur
    NH3_RoW, # activité initiale 
    'NH3_RoW_S')

sulfuric_acid_S = copyActivity(
    USER_DB, # base utilisateur
    sulfuric_acid, # activité initiale 
    'sulfuric_acid_S')

waster_water_tS = copyActivity(
    USER_DB, # base utilisateur
    waster_water_t, # activité initiale 
    'waster_water_tS')

# waster_water_t
# sulfuric_acid
#NH3_RoW
#ammonium_nitrate_CN
#TrainT_diesel
#Elec_coal
#heat_CA
#drying_maize_CA
#Elec_lowV
#Elec_lowV_sankey
#T_jetfuel_train_sankey
#Co_catalyst_sankey
#maize_seed_sankey
#irrigation_sankey
#heat_CA

In [ ]:
# drying of maize straw and whole-plant
# heat production, at hard coal industrial furnace 1-10MW
# market for hard coal 
# hard coal mine operation and hard coal preparation
# market for irrigation
# M_fuelp_LCI
# T_jetfuel_lorryCA
# M_strawT_LCI
# Elec_deliveryCA
# Co_catalyst_sankey
# market for maize seed, for sowing GLO
# 
seed= [act for act in eidb if 'maize seed production, at farm' in act['name'] 
                                            and 'GLO' in act['location']
                                            #and 'CN-SGCC' in act['location']
                                            and 'run-of-mine' not in act['name']
                                            and 'briquettes' not in act['name']
                                            and 'ash' not in act['name']
                                            #and 'urban air close to ground' in str(act['categories'])
      ][0]
seed

In [ ]:
highV_elec_CNSGCC= [act for act in eidb if 'market for electricity, high voltage' in act['name'] 
                                            #and 'CN-SGC' in act['location']
                                            and 'CN-SGCC' in act['location']
                                            and 'run-of-mine' not in act['name']
                                            and 'briquettes' not in act['name']
                                            and 'ash' not in act['name']
                                            #and 'urban air close to ground' in str(act['categories'])
      ][0]
highV_elec_CNSGCC

In [ ]:
mine_infrastucture= [act for act in eidb if 'mine construction, underground, hard coal' in act['name'] 
                                            #and 'CN-SGC' in act['location']
                                            and 'CN' in act['location']
                                            and 'run-of-mine' not in act['name']
                                            and 'briquettes' not in act['name']
                                            and 'ash' not in act['name']
                                            #and 'urban air close to ground' in str(act['categories'])
      ][0]
mine_infrastucture

In [ ]:
hardcoal_mine= [act for act in eidb if 'hard coal mine operation and hard coal preparation' in act['name'] 
                                            #and 'CN-SGC' in act['location']
                                            and 'CN' in act['location']
                                            and 'run-of-mine' not in act['name']
                                            and 'briquettes' not in act['name']
                                            and 'ash' not in act['name']
                                            #and 'urban air close to ground' in str(act['categories'])
      ][0]
hardcoal_mine

In [ ]:
irrigation_CA= [act for act in eidb if 'maize seed production, at farm' in act['name'] 
                                            and 'GLO' in act['location']
                                            #and 'CN-SGCC' in act['location']
                                            and 'run-of-mine' not in act['name']
                                            and 'briquettes' not in act['name']
                                            and 'ash' not in act['name']
                                            #and 'urban air close to ground' in str(act['categories'])
      ][0]
irrigation_CA

In [ ]:
drying_maize_CA= [act for act in eidb if 'drying of maize straw and whole-plant' in act['name'] 
                                            and 'GLO' in act['location']
                                            #and 'CN-SGCC' in act['location']
                                            and 'run-of-mine' not in act['name']
                                            and 'briquettes' not in act['name']
                                            and 'ash' not in act['name']
                                            #and 'urban air close to ground' in str(act['categories'])
      ][0]
drying_maize_CA

In [ ]:
heat_CA= [act for act in eidb if 'heat production, at hard coal industrial furnace 1-10MW' in act['name'] 
                                            and 'RoW' in act['location']
                                            #and 'CN-SGCC' in act['location']
                                            and 'run-of-mine' not in act['name']
                                            and 'briquettes' not in act['name']
                                            and 'ash' not in act['name']
                                            #and 'urban air close to ground' in str(act['categories'])
      ][0]
heat_CA

In [ ]:
Elec_coal= [act for act in eidb if 'electricity production, hard coal' in act['name'] 
                                            and 'CN-JS' in act['location']
                                            #and 'CN-SGCC' in act['location']
                                            and 'run-of-mine' not in act['name']
                                            and 'briquettes' not in act['name']
                                            and 'ash' not in act['name']
                                            #and 'urban air close to ground' in str(act['categories'])
      ][0]
Elec_coal

In [ ]:
TrainT_diesel= [act for act in eidb if 'transport, freight train, diesel' in act['name'] 
                                            and 'CN' in act['location']
                                            #and 'CN-SGCC' in act['location']
                                            and 'run-of-mine' not in act['name']
                                            and 'briquettes' not in act['name']
                                            and 'ash' not in act['name']
                                            #and 'urban air close to ground' in str(act['categories'])
      ][0]
TrainT_diesel

In [ ]:
sulfuric_acid= [act for act in eidb if 'market for sulfuric acid' in act['name'] 
                                            and 'RoW' in act['location']
                                            #and 'CN-SGCC' in act['location']
                                            and 'run-of-mine' not in act['name']
                                            and 'briquettes' not in act['name']
                                            and 'ash' not in act['name']
                                            #and 'urban air close to ground' in str(act['categories'])
      ][0]
sulfuric_acid

In [ ]:
waster_water_t= [act for act in eidb if 'treatment of wastewater, unpolluted, capacity 5E9l/year' in act['name'] 
                                            and 'RoW' in act['location']
                                            #and 'CN-SGCC' in act['location']
                                            and 'run-of-mine' not in act['name']
                                            and 'briquettes' not in act['name']
                                            and 'ash' not in act['name']
                                            #and 'urban air close to ground' in str(act['categories'])
      ][0]
waster_water_t

In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI, NOF_IN, ratio = 0.9, length_max = 10)

In [ ]:
plot_sankey(act = ME_fueltMJ_LCI, method = NOF_IN, cutoff = 0.1) #ME_fueltMJ_LCI, NIF_ON #Elec_delivery
#water_EF

In [ ]:
#water_EF
#NOF_IN
#water_ReCiPe_v
impact = NOF_IN
act = maize_all_CN
# turbine_1_3MW_onshore
# glass_fibre_reinforced
# concrete_40MPa

df = exploreImpacts(impact, act, R_SM=62.7 / (62.7+160.2),R_Straw_return=0.30,ER_NH3=0.870,CO2_maizep_total=0.63812,
                    ER_N2O=1.26,ER_NO3=1.78,ER_NO2_N2=0.375,N_Straw=0.0061,R_straw=0.84,N_strawFR_SM_100=22.2973,
                    R_cob=0.2105, CO2_fuelem=0.0703,CO_fuelem=8.968e-05,N2O_fuelem=2.088e-07,CH4_fuelem=1.064e-07,
                   NOx_fuelem=0.0003295,PM10_fuelem=3.411e-07,VOC_fuelem=1.299e-05,D_train_jetfuel=749,H_biojet=43.07,
                   D_lorry_jetfuel=50,Elec_fuel_delivery=0.0056)

df.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\maize production_v2.xlsx', sheet_name='maize production', index = True)
df2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\maize production_v2.xlsx', sheet_name='maize production', skiprows=[0])
df2
#df['impact']=df['impact'].astype(float)
#df=df.astype(float)
#df['blocks']=df['blocks'].astype(float)

In [ ]:
#df.to_excel(r'F:\学也无止境\intership\OIE internship\Internship on LCA of hydrogen production\d0se00222d1\CH4-to-H2 lca_algebraic\H2LCIA_result\H2LCIA_result_SMR_HTLT_CCS98_EU_elec.xlsx', sheet_name='H2_CIA_result', index = True)
A = df2.columns[3]
name = df2.columns[0]
graph = df2.plot.bar(figsize=(100,50), y = A,x=name, fontsize = 20)


graph.legend(loc='center left', bbox_to_anchor=(0.04, 1.1)) 


In [ ]:
Contribution_Analysis_one_impact_category_noact(ME_fueltMJ_LCI, NOF_ON, ratio = 0.9, length_max = 9)

In [ ]:
plot_sankey(act = ME_fueltMJ_LCI, method = NOF_ON, cutoff = 0.1) #ME_fueltMJ_LCI, NIF_ON #Elec_delivery
#water_EF

In [ ]:
#water_EF
#NOF_IN
#water_ReCiPe_v
#maize_all_CN
#ME_fueltMJ_LCI
#Diesel_sankey
#Elec_deliveryCA
#irrigation_sankey
# ME_fuelpMJ_LCI
#M_fuelp_LCI

impact = NOF_ON
act = Water_genCA

# turbine_1_3MW_onshore
# glass_fibre_reinforced
# concrete_40MPa

df = exploreImpacts(impact, act, R_SM=62.7 / (62.7+160.2),R_Straw_return=0.30,ER_NH3=0.870,CO2_maizep_total=0.63812,
                    ER_N2O=1.26,ER_NO3=1.78,ER_NO2_N2=0.375,N_Straw=0.0061,R_straw=0.84,N_strawFR_SM_100=22.2973,
                    R_cob=0.2105, CO2_fuelem=0.0703,CO_fuelem=8.968e-05,N2O_fuelem=2.088e-07,CH4_fuelem=1.064e-07,
                   NOx_fuelem=0.0003295,PM10_fuelem=3.411e-07,VOC_fuelem=1.299e-05,D_train_jetfuel=749,H_biojet=43.07,
                   D_lorry_jetfuel=50,Elec_fuel_delivery=0.0056, CoRu_catalyst=3, R_biojet_straw= 11.76, NiW_catalyst= 3,
                   P_Al=6, DR_feed= 500, Life_FTplant=15, P_iron=3,P_cement=1523,P_steel=484,Elec_milling=42,Diesel_C=0.67,Elec_C=18.2,
                   m_biojet_tb=85.03,H_diesel_tb=46,m_diesel_tb=19.83,H_gasoline_tb=43.1, m_gasoline_tb=43.79,H_paraffin_tb=44.857,
                   m_paraffin_tb=7.99, m_elecsteam_tb=5.292
                   
)




df.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\maize production_v2.xlsx', sheet_name='maize production', index = True)
df2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\maize production_v2.xlsx', sheet_name='maize production', skiprows=[0])
df2
#df['impact']=df['impact'].astype(float)
#df=df.astype(float)
#df['blocks']=df['blocks'].astype(float)

In [ ]:
#list_parameters()

In [ ]:
#df.to_excel(r'F:\学也无止境\intership\OIE internship\Internship on LCA of hydrogen production\d0se00222d1\CH4-to-H2 lca_algebraic\H2LCIA_result\H2LCIA_result_SMR_HTLT_CCS98_EU_elec.xlsx', sheet_name='H2_CIA_result', index = True)
A = df2.columns[3]
name = df2.columns[0]
graph = df2.plot.bar(figsize=(10,5), y = A,x=name, fontsize = 10)


graph.legend(loc='center left', bbox_to_anchor=(0.04, 1.1)) 


In [ ]:
#water_EF
#NOF_IN
#water_ReCiPe_v
impact = NOF_ON
act = ME_fuelpMJ_LCI
# turbine_1_3MW_onshore
# glass_fibre_reinforced
# concrete_40MPa

df = exploreImpacts(impact, act, R_SM=62.7 / (62.7+160.2),R_Straw_return=0.30,ER_NH3=0.870,CO2_maizep_total=0.63812,
                    ER_N2O=1.26,ER_NO3=1.78,ER_NO2_N2=0.375,N_Straw=0.0061,R_straw=0.84,N_strawFR_SM_100=22.2973,
                    R_cob=0.2105, CO2_fuelem=0.0703,CO_fuelem=8.968e-05,N2O_fuelem=2.088e-07,CH4_fuelem=1.064e-07,
                   NOx_fuelem=0.0003295,PM10_fuelem=3.411e-07,VOC_fuelem=1.299e-05,D_train_jetfuel=749,H_biojet=43.07,
                   D_lorry_jetfuel=50,Elec_fuel_delivery=0.0056)

df.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\maize production_v2.xlsx', sheet_name='maize production', index = True)
df2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\maize production_v2.xlsx', sheet_name='maize production', skiprows=[0])
df2
#df['impact']=df['impact'].astype(float)
#df=df.astype(float)
#df['blocks']=df['blocks'].astype(float)

In [ ]:
plot_sankey(act = Water_genCA, method = NOF_ON, cutoff = 0.05) #ME_fueltMJ_LCI, NIF_ON #Elec_delivery
#water_EF

In [ ]:
plot_sankey(act = sulfuric_acid_S, method = NOF_ON, cutoff = 0.05) #ME_fueltMJ_LCI, NIF_ON #Elec_delivery
#water_EF

In [ ]:
Contribution_Analysis_one_impact_category_noact(sulfuric_acid_S, NOF_ON, ratio = 0.9, length_max = 10)

In [ ]:
#water_EF
#NOF_IN
#water_ReCiPe_v
# drying_maize_CA
# heat_CA_sankey
impact = NOF_ON
act = TrainT_diesel_sankey
# turbine_1_3MW_onshore
# glass_fibre_reinforced
# concrete_40MPa

df = exploreImpacts(impact, act, )
df.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\electricity production coal.xlsx', sheet_name='maize production', index = True)
df2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\electricity production coal.xlsx', sheet_name='maize production', skiprows=[0])
df2
#df['impact']=df['impact'].astype(float)
#df=df.astype(float)
#df['blocks']=df['blocks'].astype(float)

In [ ]:
#df.to_excel(r'F:\学也无止境\intership\OIE internship\Internship on LCA of hydrogen production\d0se00222d1\CH4-to-H2 lca_algebraic\H2LCIA_result\H2LCIA_result_SMR_HTLT_CCS98_EU_elec.xlsx', sheet_name='H2_CIA_result', index = True)
A = df2.columns[3]
name = df2.columns[0]
graph = df2.plot.bar(figsize=(20,5), y = A,x=name, fontsize = 10)


graph.legend(loc='center left', bbox_to_anchor=(0.04, 1.1)) 


In [ ]:
sb.distplot(NOF_ON_scores)

### Backbone N flow footprint contribution on the level of system life cycle stage

#### National benckmark maize straw cultivation

In [ ]:
#Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
ME_fueltMJ_CA = newActivity(USER_DB, "bio-jet fuel delivery and combustion in aircraft CA", "MJ fuel")


ME_fueltMJ_CA.addExchanges({ 
    
    maize_all_CN:R_straw/(1+R_straw+R_cob)/R_straw*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    Elec_crush_pack : Elec_C/1000*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Diesel_crush_pack : Diesel_C/1000*(1/0.0222)*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B, # diesel density 1/0.0222 
    
    T_straw_lorry : TD_straw*1/1000*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    
    Elec_millpCA: Elec_milling/1000*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    Steel_building: P_steel*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Iron_building: P_iron*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Al_building: P_Al*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    cement_building: P_cement*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    Steel_EoL: -(P_steel+P_iron)*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Al_EoL: -P_Al*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    cement_EoL: -P_cement*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
       
    Si_sand: 500/((7200/8760)*24)*5*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Water_p: (5292.23/11760)*1*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,          
    
    Co_catalyst: Q_catalyst_Co*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Ru_catalyst: Q_catalyst_Ru*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    ZrO2_catalyst: Q_catalyst_ZrO2*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Al2O3_catalyst: Q_catalyst_Al2O3*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    NiO_catalyst: Q_catalyst_NiO*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B, # represent NiO, same reaseon as stated for Ru catalyst
    W_catalyst: Q_catalyst_W*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B, # use ammonium paratungstate in ecoinvent to represent WO3 based on W atomic mass conservation, as same reaseon as stated for Ru catalyst 
    HY_catalyst: Q_catalyst_HY*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    EoL_catalyst: (Q_catalyst_Co+Q_catalyst_Ru+Q_catalyst_ZrO2+Q_catalyst_Al2O3+Q_catalyst_NiO+Q_catalyst_W+Q_catalyst_HY)*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    CO2_e: CO2_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    CH4_e: CH4_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    N2O_e: N2O_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    NOx_e: NOx_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    SO2_e: SO2_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    CO_e : CO_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    HC_e : HC_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    PM10_e: PM10_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    COD_e: COD_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    solidwaste_EoL: -soildw_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    TN_e: TN_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    NH4_e: NH3N_pe/14.0067*118.04*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,   
    
    
    Elec_substCA: -(4221-2644)/1000*Allocate_RE_fuelp_B/H_biojet,
    Water_genCA: m_elecsteam_tb*Allocate_RE_fuelp_B/H_biojet,
    
    T_jetfuel_lorryCA: 1/H_biojet*D_lorry_jetfuel/1000,
    T_jetfuel_train: 1/H_biojet*D_train_jetfuel/1000,
    Elec_deliveryCA: Elec_fuel_delivery/H_biojet,
    CH4_fueleCA:1,
    N2O_fueleCA:1,
    CO2_fueleCA:1,
    VOC_fueleCA:1,
    CO_fueleCA:1,
    NOx_fueleCA:1,
    PM10_fueleCA:1,
    })

#exploreImpacts(NIF_IN, ME_fueltMJ_CA, R_SM=62.7 / (62.7+160.2),R_Straw_return=0.30,ER_NH3=0.870,CO2_maizep_total=0.63812,ER_N2O=1.26,ER_NO3=1.78,ER_NO2_N2=0.375,N_Straw=0.0061,R_straw=0.84,N_strawFR_SM_100=22.2973,R_cob=0.2105)


In [ ]:
freezeParams(USER_DB)

The code for Unit process contribution to LCIA result is mainly modefied from the work of Antonini et al. 2020 DOI: 10.1039/d0se00222d

In [ ]:
activities = [ME_fueltMJ_CA]

In [ ]:
len(activities)

In [ ]:
# Make sure both ATR and SMR databases are loaded
methods=[NIF_IN,NIF_ON,NOF_IN,NOF_ON,CC_EF3,water_EF]
lca = bw.LCA({activities[0]: 1, activities[-1]: 1}, method=methods[5])
lca.lci()
lca.lcia()
lca.score

In [ ]:
result_array = [[[] for _ in activities] for _ in methods]

In [ ]:
for i, method in enumerate(pyprind.prog_bar(methods)):
    lca.switch_method(method)
    for j, activity in enumerate(activities):
        lca.redo_lcia({activity: 1})
        # Add total to check afterwards that all exchanges add up to total
        result_array[i][j].append(("total", lca.score))
        for exc in activity.exchanges():
            if exc['type'] == 'technosphere':
                lca.redo_lcia({exc.input: exc['amount']})
                result_array[i][j].append((exc, lca.score))
            elif exc['type'] == 'biosphere':
                # Need to multiple the amount times its CF
                cf = lca.characterization_matrix[lca.biosphere_dict[exc.input], :].sum()
                result_array[i][j].append((exc, cf * exc['amount']))

In [ ]:
result_array

Check to make sure our sums add up

In [ ]:
for method in result_array:
    for activity in method:
        if not np.allclose(activity[0][1], sum([o[1] for o in activity[1:]])):
            print("Mismatch")
            break

We will create separate worksheets for each method, and group exchanges into certain categories. This will leave us with two dimensions per worksheet (LCIA impact category).

In [ ]:
group_mapping = {
    'maize production in china, unallocated whole plant': "Straw feedstock cultivation",
    
    'market group for electricity, low voltage': "Feedstock pre-treatment",
    'market for diesel, burned in agricultural machinery': "Feedstock pre-treatment",

    "market for transport, freight, lorry 7.5-16 metric ton, EURO5": "Feedstock transportation",
    
    'market for steel, low-alloyed, hot rolled': "Convesion plant construction",
    'market for cast iron': "Convesion plant construction",
    'market for aluminium, wrought alloy': "Convesion plant construction",
    'market for cement, Portland': "Convesion plant construction",
    'market for scrap steel': "Convesion plant construction",
    'market for scrap aluminium': "Convesion plant construction",
    'treatment of inert waste, inert material landfill': "Convesion plant construction",
    
    "market for silica sand": "Chemicals and catalyst",
    'water production, deionised': "Chemicals and catalyst",
    'market for cobalt': "Chemicals and catalyst",
    "Ruthenium, in ground": "Chemicals and catalyst",
    "zirconium oxide production": "Chemicals and catalyst",
    "market for aluminium oxide, metallurgical": "Chemicals and catalyst",
    "market for nickel, class 1": "Chemicals and catalyst", 
    "market for ammonium paratungstate": "Chemicals and catalyst",  
    "market for zeolite, powder": "Chemicals and catalyst",
    "market for municipal solid waste": "Chemicals and catalyst",
   
    
    'market group for electricity, low voltage CA': "Gasification and FT synthesis operation",
    "Carbon dioxide, from soil or biomass stock": "Gasification and FT synthesis operation",
    "Methane, from soil or biomass stock": "Gasification and FT synthesis operation",
    "Dinitrogen monoxide": "Gasification and FT synthesis operation",
    "Nitrogen oxides": "Gasification and FT synthesis operation",
    "Sulfur dioxide": "Gasification and FT synthesis operation",
    "Carbon monoxide, from soil or biomass stock" : "Gasification and FT synthesis operation",
    "Hydrocarbons, unspecified" : "Gasification and FT synthesis operation",
    "Particulates, > 2.5 um, and < 10um": "Gasification and FT synthesis operation",
    "COD, Chemical Oxygen Demand": "Gasification and FT synthesis operation",
    "treatment of inert waste, inert material landfill": "Gasification and FT synthesis operation",
    "Nitrogen, organic bound": "Gasification and FT synthesis operation",
    "Ammonium, ion": "Gasification and FT synthesis operation",
    
    "market group for electricity, low voltage substCA": "Gasification and FT synthesis operation",
    "water production, deionised CA": "Gasification and FT synthesis operation",
    
    "market for transport, freight, lorry 7.5-16 metric ton, EURO5, jetfuel CA": "Bio-jet transportation and delivery",
    "market for transport, freight train": "Bio-jet transportation and delivery",
    "market group for electricity, low voltage deliveryCA": "Bio-jet transportation and delivery",
        
    'Methane, non-fossil, jetfuel CA':"Bio-jet fuel combustion",
    "Dinitrogen monoxide, jetfuel CA":"Bio-jet fuel combustion",
    "Carbon dioxide, non-fossil, jetfuel CA":"Bio-jet fuel combustion",
    'VOC, volatile organic compounds, unspecified origin, jetfuel CA':"Bio-jet fuel combustion",
    'Carbon monoxide, non-fossil, jetfuel CA':"Bio-jet fuel combustion",
    'Nitrogen oxides, jetfuel CA':"Bio-jet fuel combustion",
    'Particulates, > 2.5 um, and < 10um, jetfuel CA':"Bio-jet fuel combustion"
       
}

In [ ]:
def group_exchange_scores(lst):
    # We will store results in a dict, with keys for group label and values of LCIA score.
    grouped_results = defaultdict(int)

    assert lst[0][0] == 'total'
    
    for exc, score in lst[1:]:
        grouped_results[group_mapping[exc.input['name']]] += score
    return grouped_results

Test to make sure our function works as we think

In [ ]:
group_exchange_scores(result_array[5][0])

Create new array with grouped scores

In [ ]:
grouped_array = [[group_exchange_scores(j) for j in i] for i in result_array]

In [ ]:
grouped_array[2][0]

From here it should be relatively simple to create worksheets in the format you want, with however you want to abbreviate things. You have `methods` and `activities`, which are in the same order as `result_array` and `grouped_array`; you can also create `sorted(set(group_mapping.values()))` to get those in a consistent order.

In [ ]:
result_array
result_array[5][0] #first impact, #second activities alalyzed

In [ ]:
df = pd.DataFrame(grouped_array)
df#.to_excel(r'C:\Users\DELL\学术\N sustainability\demo_LCAstage_contrib_check.xlsx', sheet_name='contrib', index = True)

In [ ]:
list_contrib = ['Straw feedstock cultivation', 'Feedstock pre-treatment', 'Feedstock transportation',
                'Convesion plant construction', 'Chemicals and catalyst', 'Gasification and FT synthesis operation',
                'Bio-jet transportation and delivery', 'Bio-jet fuel combustion']


def cell_func(row):
    m = methods[row.name]

    a = 0
    list_output=[]
    for cell in row:
        act = activities[a]
        
        for k in cell:
            
            list_output.append([m, act, k, cell[k]])
        a+=1

    return list_output

list_output = df.apply(cell_func, axis=1)


In [ ]:
# only need to run once!
import csv

with open("bio-jet_fuel_contribution_deafult_check4.csv", "w", newline="") as f:
    writer = csv.writer(f, delimiter=",")
    for a in list_output:
        writer.writerows(a)

In [ ]:
# load dataset
data_P = pd.read_excel("C:/Users/DELL/学术/N sustainability/bio-jet fuel production/bio-jet_fuel_contribution_deafult.xlsx", 'bio-jet_fuel_stack_PCT')
  
# view dataset
print(data_P)
  
# plot a Stacked Bar Chart using matplotlib
data_P.plot(
    x = 'categories',
    kind = 'barh',
    stacked = True,
    #title = 'Life cycle stage contribution of nitrogen, carbon, water footprint',
    colormap='tab10',
    mark_right = True,
    figsize=(16, 7),
    alpha = 0.6,
    width=0.75,
    rot = 30)

plt.title(r'Life cycle stage contributions of system carbon, water footprint, and N flow', fontsize=22)

# 'ILCD 2.0 2018 midpoint, climate change, climate change total CO2_-eq/MJ H2 produced'

#plt.rcParams["figure.figsize"] = (8, 6)

plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

plt.legend(ncol=1,fontsize=20,bbox_to_anchor=(1.46, 1), loc='upper right', frameon=False)

plt.xlabel("PCT(%)",fontsize=20)
plt.ylabel("Impact categories",fontsize=20)
plt.show()

#### Organic maize straw cultivation

In [ ]:
#Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
ME_fueltMJ_CA_org = newActivity(USER_DB, "bio-jet fuel delivery and combustion in aircraft CA organic maize", "MJ fuel")


ME_fueltMJ_CA_org.addExchanges({ 
    
    organic_maize_all_CN:R_straw/(1+R_straw+R_cob)/R_straw*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    Elec_crush_pack : Elec_C/1000*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Diesel_crush_pack : Diesel_C/1000*(1/0.0222)*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B, # diesel density 1/0.0222 
    
    T_straw_lorry : TD_straw*1/1000*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    
    Elec_millpCA: Elec_milling/1000*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    Steel_building: P_steel*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Iron_building: P_iron*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Al_building: P_Al*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    cement_building: P_cement*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    Steel_EoL: -(P_steel+P_iron)*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Al_EoL: -P_Al*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    cement_EoL: -P_cement*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
       
    Si_sand: 500/((7200/8760)*24)*5*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Water_p: (5292.23/11760)*1*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,          
    
    Co_catalyst: Q_catalyst_Co*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Ru_catalyst: Q_catalyst_Ru*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    ZrO2_catalyst: Q_catalyst_ZrO2*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Al2O3_catalyst: Q_catalyst_Al2O3*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    NiO_catalyst: Q_catalyst_NiO*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B, # represent NiO, same reaseon as stated for Ru catalyst
    W_catalyst: Q_catalyst_W*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B, # use ammonium paratungstate in ecoinvent to represent WO3 based on W atomic mass conservation, as same reaseon as stated for Ru catalyst 
    HY_catalyst: Q_catalyst_HY*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    EoL_catalyst: (Q_catalyst_Co+Q_catalyst_Ru+Q_catalyst_ZrO2+Q_catalyst_Al2O3+Q_catalyst_NiO+Q_catalyst_W+Q_catalyst_HY)*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    CO2_e: CO2_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    CH4_e: CH4_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    N2O_e: N2O_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    NOx_e: NOx_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    SO2_e: SO2_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    CO_e : CO_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    HC_e : HC_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    PM10_e: PM10_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    COD_e: COD_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    solidwaste_EoL: -soildw_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    TN_e: TN_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    NH4_e: NH3N_pe/14.0067*118.04*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,   
    
    
    Elec_substCA: -(4221-2644)/1000*Allocate_RE_fuelp_B/H_biojet,
    Water_genCA: m_elecsteam_tb*Allocate_RE_fuelp_B/H_biojet,
    
    T_jetfuel_lorryCA: 1/H_biojet*D_lorry_jetfuel/1000,
    T_jetfuel_train: 1/H_biojet*D_train_jetfuel/1000,
    Elec_deliveryCA: Elec_fuel_delivery/H_biojet,
    CH4_fueleCA:1,
    N2O_fueleCA:1,
    CO2_fueleCA:1,
    VOC_fueleCA:1,
    CO_fueleCA:1,
    NOx_fueleCA:1,
    PM10_fueleCA:1,
    })

#exploreImpacts(NIF_IN, ME_fueltMJ_CA, R_SM=62.7 / (62.7+160.2),R_Straw_return=0.30,ER_NH3=0.870,CO2_maizep_total=0.63812,ER_N2O=1.26,ER_NO3=1.78,ER_NO2_N2=0.375,N_Straw=0.0061,R_straw=0.84,N_strawFR_SM_100=22.2973,R_cob=0.2105)


In [ ]:
freezeParams(USER_DB)

The code for Unit process contribution to LCIA result is mainly modefied from the work of Antonini et al. 2020 DOI: 10.1039/d0se00222d

In [ ]:
activities = [ME_fueltMJ_CA_org]

In [ ]:
len(activities)

In [ ]:
# Make sure both ATR and SMR databases are loaded
methods=[NIF_IN,NIF_ON,NOF_IN,NOF_ON,CC_EF3,water_EF]
lca = bw.LCA({activities[0]: 1, activities[-1]: 1}, method=methods[5])
lca.lci()
lca.lcia()
lca.score

In [ ]:
result_array = [[[] for _ in activities] for _ in methods]

In [ ]:
for i, method in enumerate(pyprind.prog_bar(methods)):
    lca.switch_method(method)
    for j, activity in enumerate(activities):
        lca.redo_lcia({activity: 1})
        # Add total to check afterwards that all exchanges add up to total
        result_array[i][j].append(("total", lca.score))
        for exc in activity.exchanges():
            if exc['type'] == 'technosphere':
                lca.redo_lcia({exc.input: exc['amount']})
                result_array[i][j].append((exc, lca.score))
            elif exc['type'] == 'biosphere':
                # Need to multiple the amount times its CF
                cf = lca.characterization_matrix[lca.biosphere_dict[exc.input], :].sum()
                result_array[i][j].append((exc, cf * exc['amount']))

In [ ]:
result_array

Check to make sure our sums add up

In [ ]:
for method in result_array:
    for activity in method:
        if not np.allclose(activity[0][1], sum([o[1] for o in activity[1:]])):
            print("Mismatch")
            break

We will create separate worksheets for each method, and group exchanges into certain categories. This will leave us with two dimensions per worksheet (LCIA impact category).

In [ ]:
group_mapping = {
    'organic maize production in china, unallocated whole plant': "Straw feedstock cultivation",
    
    'market group for electricity, low voltage': "Feedstock pre-treatment",
    'market for diesel, burned in agricultural machinery': "Feedstock pre-treatment",

    "market for transport, freight, lorry 7.5-16 metric ton, EURO5": "Feedstock transportation",
    
    'market for steel, low-alloyed, hot rolled': "Convesion plant construction",
    'market for cast iron': "Convesion plant construction",
    'market for aluminium, wrought alloy': "Convesion plant construction",
    'market for cement, Portland': "Convesion plant construction",
    'market for scrap steel': "Convesion plant construction",
    'market for scrap aluminium': "Convesion plant construction",
    'treatment of inert waste, inert material landfill': "Convesion plant construction",
    
    "market for silica sand": "Chemicals and catalyst",
    'water production, deionised': "Chemicals and catalyst",
    'market for cobalt': "Chemicals and catalyst",
    "Ruthenium, in ground": "Chemicals and catalyst",
    "zirconium oxide production": "Chemicals and catalyst",
    "market for aluminium oxide, metallurgical": "Chemicals and catalyst",
    "market for nickel, class 1": "Chemicals and catalyst", 
    "market for ammonium paratungstate": "Chemicals and catalyst",  
    "market for zeolite, powder": "Chemicals and catalyst",
    "market for municipal solid waste": "Chemicals and catalyst",
   
    
    'market group for electricity, low voltage CA': "Gasification and FT synthesis operation",
    "Carbon dioxide, from soil or biomass stock": "Gasification and FT synthesis operation",
    "Methane, from soil or biomass stock": "Gasification and FT synthesis operation",
    "Dinitrogen monoxide": "Gasification and FT synthesis operation",
    "Nitrogen oxides": "Gasification and FT synthesis operation",
    "Sulfur dioxide": "Gasification and FT synthesis operation",
    "Carbon monoxide, from soil or biomass stock" : "Gasification and FT synthesis operation",
    "Hydrocarbons, unspecified" : "Gasification and FT synthesis operation",
    "Particulates, > 2.5 um, and < 10um": "Gasification and FT synthesis operation",
    "COD, Chemical Oxygen Demand": "Gasification and FT synthesis operation",
    "treatment of inert waste, inert material landfill": "Gasification and FT synthesis operation",
    "Nitrogen, organic bound": "Gasification and FT synthesis operation",
    "Ammonium, ion": "Gasification and FT synthesis operation",
    
    "market group for electricity, low voltage substCA": "Gasification and FT synthesis operation",
    "water production, deionised CA": "Gasification and FT synthesis operation",
    
    "market for transport, freight, lorry 7.5-16 metric ton, EURO5, jetfuel CA": "Bio-jet transportation and delivery",
    "market for transport, freight train": "Bio-jet transportation and delivery",
    "market group for electricity, low voltage deliveryCA": "Bio-jet transportation and delivery",
        
    'Methane, non-fossil, jetfuel CA':"Bio-jet fuel combustion",
    "Dinitrogen monoxide, jetfuel CA":"Bio-jet fuel combustion",
    "Carbon dioxide, non-fossil, jetfuel CA":"Bio-jet fuel combustion",
    'VOC, volatile organic compounds, unspecified origin, jetfuel CA':"Bio-jet fuel combustion",
    'Carbon monoxide, non-fossil, jetfuel CA':"Bio-jet fuel combustion",
    'Nitrogen oxides, jetfuel CA':"Bio-jet fuel combustion",
    'Particulates, > 2.5 um, and < 10um, jetfuel CA':"Bio-jet fuel combustion"
       
}

In [ ]:
def group_exchange_scores(lst):
    # We will store results in a dict, with keys for group label and values of LCIA score.
    grouped_results = defaultdict(int)

    assert lst[0][0] == 'total'
    
    for exc, score in lst[1:]:
        grouped_results[group_mapping[exc.input['name']]] += score
    return grouped_results

Test to make sure our function works as we think

In [ ]:
group_exchange_scores(result_array[5][0])

Create new array with grouped scores

In [ ]:
grouped_array = [[group_exchange_scores(j) for j in i] for i in result_array]

In [ ]:
grouped_array[2][0]

From here it should be relatively simple to create worksheets in the format you want, with however you want to abbreviate things. You have `methods` and `activities`, which are in the same order as `result_array` and `grouped_array`; you can also create `sorted(set(group_mapping.values()))` to get those in a consistent order.

In [ ]:
result_array
result_array[5][0] #first impact, #second activities alalyzed

In [ ]:
df = pd.DataFrame(grouped_array)
df#.to_excel(r'C:\Users\DELL\学术\N sustainability\demo_LCAstage_contrib_check.xlsx', sheet_name='contrib', index = True)

In [ ]:
list_contrib = ['Straw feedstock cultivation', 'Feedstock pre-treatment', 'Feedstock transportation',
                'Convesion plant construction', 'Chemicals and catalyst', 'Gasification and FT synthesis operation',
                'Bio-jet transportation and delivery', 'Bio-jet fuel combustion']


def cell_func(row):
    m = methods[row.name]

    a = 0
    list_output=[]
    for cell in row:
        act = activities[a]
        
        for k in cell:
            
            list_output.append([m, act, k, cell[k]])
        a+=1

    return list_output

list_output = df.apply(cell_func, axis=1)


In [ ]:
# only need to run once!
import csv

with open("bio-jet_fuel_contribution_deafult_org.csv", "w", newline="") as f:
    writer = csv.writer(f, delimiter=",")
    for a in list_output:
        writer.writerows(a)

In [ ]:
# load dataset
data_P = pd.read_excel("C:/Users/DELL/学术/N sustainability/bio-jet fuel production/bio-jet_fuel_contribution_deafult_org.xlsx", 'bio-jet_fuel_stack_PCT')
  
# view dataset
print(data_P)
  
# plot a Stacked Bar Chart using matplotlib
data_P.plot(
    x = 'categories',
    kind = 'barh',
    stacked = True,
    #title = 'Life cycle stage contribution of nitrogen, carbon, water footprint',
    colormap='tab10',
    mark_right = True,
    figsize=(16, 7),
    alpha = 0.6,
    width=0.75,
    rot = 30)

plt.title(r'Life cycle stage contributions of system carbon, water footprint, and N flow', fontsize=22)

# 'ILCD 2.0 2018 midpoint, climate change, climate change total CO2_-eq/MJ H2 produced'

#plt.rcParams["figure.figsize"] = (8, 6)

plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

plt.legend(ncol=1,fontsize=20,bbox_to_anchor=(1.46, 1), loc='upper right', frameon=False)

plt.xlabel("PCT(%)",fontsize=20)
plt.ylabel("Impact categories",fontsize=20)
plt.show()

#### rainfed maize straw cultivation

In [ ]:
#Allocate_RE_fuelp_B = m_biojet_tb*H_biojet/(m_biojet_tb*H_biojet+m_gasoline_tb*H_gasoline_tb+m_diesel_tb*H_diesel_tb+m_paraffin_tb*H_paraffin_tb)
ME_fueltMJ_CA_rainfed = newActivity(USER_DB, "bio-jet fuel delivery and combustion in aircraft CA rainfed maize", "MJ fuel")


ME_fueltMJ_CA_rainfed.addExchanges({ 
    
    r_maize_all:R_straw/(1+R_straw+R_cob)/R_straw*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    Elec_crush_pack : Elec_C/1000*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Diesel_crush_pack : Diesel_C/1000*(1/0.0222)*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B, # diesel density 1/0.0222 
    
    T_straw_lorry : TD_straw*1/1000*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    
    Elec_millpCA: Elec_milling/1000*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    Steel_building: P_steel*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Iron_building: P_iron*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Al_building: P_Al*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    cement_building: P_cement*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    Steel_EoL: -(P_steel+P_iron)*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Al_EoL: -P_Al*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    cement_EoL: -P_cement*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
       
    Si_sand: 500/((7200/8760)*24)*5*1000/strawc_total*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Water_p: (5292.23/11760)*1*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,          
    
    Co_catalyst: Q_catalyst_Co*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Ru_catalyst: Q_catalyst_Ru*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    ZrO2_catalyst: Q_catalyst_ZrO2*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    Al2O3_catalyst: Q_catalyst_Al2O3*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    NiO_catalyst: Q_catalyst_NiO*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B, # represent NiO, same reaseon as stated for Ru catalyst
    W_catalyst: Q_catalyst_W*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B, # use ammonium paratungstate in ecoinvent to represent WO3 based on W atomic mass conservation, as same reaseon as stated for Ru catalyst 
    HY_catalyst: Q_catalyst_HY*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    EoL_catalyst: (Q_catalyst_Co+Q_catalyst_Ru+Q_catalyst_ZrO2+Q_catalyst_Al2O3+Q_catalyst_NiO+Q_catalyst_W+Q_catalyst_HY)*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    
    CO2_e: CO2_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    CH4_e: CH4_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    N2O_e: N2O_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    NOx_e: NOx_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    SO2_e: SO2_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    CO_e : CO_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    HC_e : HC_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    PM10_e: PM10_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    COD_e: COD_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    solidwaste_EoL: -soildw_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    TN_e: TN_pe*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,
    NH4_e: NH3N_pe/14.0067*118.04*R_biojet_straw/H_biojet* Allocate_RE_fuelp_B,   
    
    
    Elec_substCA: -(4221-2644)/1000*Allocate_RE_fuelp_B/H_biojet,
    Water_genCA: m_elecsteam_tb*Allocate_RE_fuelp_B/H_biojet,
    
    T_jetfuel_lorryCA: 1/H_biojet*D_lorry_jetfuel/1000,
    T_jetfuel_train: 1/H_biojet*D_train_jetfuel/1000,
    Elec_deliveryCA: Elec_fuel_delivery/H_biojet,
    CH4_fueleCA:1,
    N2O_fueleCA:1,
    CO2_fueleCA:1,
    VOC_fueleCA:1,
    CO_fueleCA:1,
    NOx_fueleCA:1,
    PM10_fueleCA:1,
    })

#exploreImpacts(NIF_IN, ME_fueltMJ_CA, R_SM=62.7 / (62.7+160.2),R_Straw_return=0.30,ER_NH3=0.870,CO2_maizep_total=0.63812,ER_N2O=1.26,ER_NO3=1.78,ER_NO2_N2=0.375,N_Straw=0.0061,R_straw=0.84,N_strawFR_SM_100=22.2973,R_cob=0.2105)


In [ ]:
freezeParams(USER_DB)

The code for Unit process contribution to LCIA result is mainly modefied from the work of Antonini et al. 2020 DOI: 10.1039/d0se00222d

In [ ]:
activities = [ME_fueltMJ_CA_rainfed]

In [ ]:
len(activities)

In [ ]:
# Make sure both ATR and SMR databases are loaded
methods=[NIF_IN,NIF_ON,NOF_IN,NOF_ON,CC_EF3,water_EF]
lca = bw.LCA({activities[0]: 1, activities[-1]: 1}, method=methods[5])
lca.lci()
lca.lcia()
lca.score

In [ ]:
result_array = [[[] for _ in activities] for _ in methods]

In [ ]:
for i, method in enumerate(pyprind.prog_bar(methods)):
    lca.switch_method(method)
    for j, activity in enumerate(activities):
        lca.redo_lcia({activity: 1})
        # Add total to check afterwards that all exchanges add up to total
        result_array[i][j].append(("total", lca.score))
        for exc in activity.exchanges():
            if exc['type'] == 'technosphere':
                lca.redo_lcia({exc.input: exc['amount']})
                result_array[i][j].append((exc, lca.score))
            elif exc['type'] == 'biosphere':
                # Need to multiple the amount times its CF
                cf = lca.characterization_matrix[lca.biosphere_dict[exc.input], :].sum()
                result_array[i][j].append((exc, cf * exc['amount']))

In [ ]:
result_array

Check to make sure our sums add up

In [ ]:
for method in result_array:
    for activity in method:
        if not np.allclose(activity[0][1], sum([o[1] for o in activity[1:]])):
            print("Mismatch")
            break

We will create separate worksheets for each method, and group exchanges into certain categories. This will leave us with two dimensions per worksheet (LCIA impact category).

In [ ]:
group_mapping = {
    'rainfed maize production in china, unallocated whole plant': "Straw feedstock cultivation",
    
    'market group for electricity, low voltage': "Feedstock pre-treatment",
    'market for diesel, burned in agricultural machinery': "Feedstock pre-treatment",

    "market for transport, freight, lorry 7.5-16 metric ton, EURO5": "Feedstock transportation",
    
    'market for steel, low-alloyed, hot rolled': "Convesion plant construction",
    'market for cast iron': "Convesion plant construction",
    'market for aluminium, wrought alloy': "Convesion plant construction",
    'market for cement, Portland': "Convesion plant construction",
    'market for scrap steel': "Convesion plant construction",
    'market for scrap aluminium': "Convesion plant construction",
    'treatment of inert waste, inert material landfill': "Convesion plant construction",
    
    "market for silica sand": "Chemicals and catalyst",
    'water production, deionised': "Chemicals and catalyst",
    'market for cobalt': "Chemicals and catalyst",
    "Ruthenium, in ground": "Chemicals and catalyst",
    "zirconium oxide production": "Chemicals and catalyst",
    "market for aluminium oxide, metallurgical": "Chemicals and catalyst",
    "market for nickel, class 1": "Chemicals and catalyst", 
    "market for ammonium paratungstate": "Chemicals and catalyst",  
    "market for zeolite, powder": "Chemicals and catalyst",
    "market for municipal solid waste": "Chemicals and catalyst",
   
    
    'market group for electricity, low voltage CA': "Gasification and FT synthesis operation",
    "Carbon dioxide, from soil or biomass stock": "Gasification and FT synthesis operation",
    "Methane, from soil or biomass stock": "Gasification and FT synthesis operation",
    "Dinitrogen monoxide": "Gasification and FT synthesis operation",
    "Nitrogen oxides": "Gasification and FT synthesis operation",
    "Sulfur dioxide": "Gasification and FT synthesis operation",
    "Carbon monoxide, from soil or biomass stock" : "Gasification and FT synthesis operation",
    "Hydrocarbons, unspecified" : "Gasification and FT synthesis operation",
    "Particulates, > 2.5 um, and < 10um": "Gasification and FT synthesis operation",
    "COD, Chemical Oxygen Demand": "Gasification and FT synthesis operation",
    "treatment of inert waste, inert material landfill": "Gasification and FT synthesis operation",
    "Nitrogen, organic bound": "Gasification and FT synthesis operation",
    "Ammonium, ion": "Gasification and FT synthesis operation",
    
    "market group for electricity, low voltage substCA": "Gasification and FT synthesis operation",
    "water production, deionised CA": "Gasification and FT synthesis operation",
    
    "market for transport, freight, lorry 7.5-16 metric ton, EURO5, jetfuel CA": "Bio-jet transportation and delivery",
    "market for transport, freight train": "Bio-jet transportation and delivery",
    "market group for electricity, low voltage deliveryCA": "Bio-jet transportation and delivery",
        
    'Methane, non-fossil, jetfuel CA':"Bio-jet fuel combustion",
    "Dinitrogen monoxide, jetfuel CA":"Bio-jet fuel combustion",
    "Carbon dioxide, non-fossil, jetfuel CA":"Bio-jet fuel combustion",
    'VOC, volatile organic compounds, unspecified origin, jetfuel CA':"Bio-jet fuel combustion",
    'Carbon monoxide, non-fossil, jetfuel CA':"Bio-jet fuel combustion",
    'Nitrogen oxides, jetfuel CA':"Bio-jet fuel combustion",
    'Particulates, > 2.5 um, and < 10um, jetfuel CA':"Bio-jet fuel combustion"
       
}

In [ ]:
def group_exchange_scores(lst):
    # We will store results in a dict, with keys for group label and values of LCIA score.
    grouped_results = defaultdict(int)

    assert lst[0][0] == 'total'
    
    for exc, score in lst[1:]:
        grouped_results[group_mapping[exc.input['name']]] += score
    return grouped_results

Test to make sure our function works as we think

In [ ]:
group_exchange_scores(result_array[5][0])

Create new array with grouped scores

In [ ]:
grouped_array = [[group_exchange_scores(j) for j in i] for i in result_array]

In [ ]:
grouped_array[2][0]

From here it should be relatively simple to create worksheets in the format you want, with however you want to abbreviate things. You have `methods` and `activities`, which are in the same order as `result_array` and `grouped_array`; you can also create `sorted(set(group_mapping.values()))` to get those in a consistent order.

In [ ]:
result_array
result_array[5][0] #first impact, #second activities alalyzed

In [ ]:
df = pd.DataFrame(grouped_array)
df#.to_excel(r'C:\Users\DELL\学术\N sustainability\demo_LCAstage_contrib_check.xlsx', sheet_name='contrib', index = True)

In [ ]:
list_contrib = ['Straw feedstock cultivation', 'Feedstock pre-treatment', 'Feedstock transportation',
                'Convesion plant construction', 'Chemicals and catalyst', 'Gasification and FT synthesis operation',
                'Bio-jet transportation and delivery', 'Bio-jet fuel combustion']


def cell_func(row):
    m = methods[row.name]

    a = 0
    list_output=[]
    for cell in row:
        act = activities[a]
        
        for k in cell:
            
            list_output.append([m, act, k, cell[k]])
        a+=1

    return list_output

list_output = df.apply(cell_func, axis=1)


In [ ]:
# only need to run once!
import csv

with open("bio-jet_fuel_contribution_deafult_rainfed.csv", "w", newline="") as f:
    writer = csv.writer(f, delimiter=",")
    for a in list_output:
        writer.writerows(a)

In [ ]:
# load dataset
data_P = pd.read_excel("C:/Users/DELL/学术/N sustainability/bio-jet fuel production/bio-jet_fuel_contribution_deafult_rainfed.xlsx", 'bio-jet_fuel_stack_PCT')
  
# view dataset
print(data_P)
  
# plot a Stacked Bar Chart using matplotlib
data_P.plot(
    x = 'categories',
    kind = 'barh',
    stacked = True,
    #title = 'Life cycle stage contribution of nitrogen, carbon, water footprint',
    colormap='tab10',
    mark_right = True,
    figsize=(16, 7),
    alpha = 0.6,
    width=0.75,
    rot = 30)

plt.title(r'Life cycle stage contributions of system carbon, water footprint, and N flow', fontsize=22)

# 'ILCD 2.0 2018 midpoint, climate change, climate change total CO2_-eq/MJ H2 produced'

#plt.rcParams["figure.figsize"] = (8, 6)

plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

plt.legend(ncol=1,fontsize=20,bbox_to_anchor=(1.46, 1), loc='upper right', frameon=False)

plt.xlabel("PCT(%)",fontsize=20)
plt.ylabel("Impact categories",fontsize=20)
plt.show()

### N flow sustainability index calculation and carbon, water, N flow nexus among all life cycle scenarios analyzed

In [ ]:
biojet_method = [CC_EF3,water_EF,NIF_IN,NIF_ON,NOF_IN,NOF_ON]

In [ ]:
list_functional_units = [{MM_fueltMJ_LCI.key:1}, {ME_fueltMJ_LCI.key:1}, {MV_fueltMJ_LCI.key:1},{HM_fueltMJ_LCI.key:1}, {HH_fueltMJ_LCI.key:1}, {HV_fueltMJ_LCI.key:1},{VM_fueltMJ_LCI.key:1}, {VH_fueltMJ_LCI.key:1}, {VV_fueltMJ_LCI.key:1}, {ZeE_fueltMJ_LCI.key:1}, {ALLE_fueltMJ_LCI.key:1}, {ME_fueltMJ_LCI_org.key:1},{ME_fueltMJ_LCI_rainf.key:1}] #ALLE_fueltMJ_LCI
list_methods = [CC_EF3,water_EF,NIF_IN,NIF_ON,NOF_IN,NOF_ON]
bw.calculation_setups['bio_jet_fuel_chain'] = {'inv':list_functional_units, 'ia':list_methods}
bw.calculation_setups['bio_jet_fuel_chain']
BJFMultiLCA = bw.MultiLCA('bio_jet_fuel_chain')
BJFMultiLCA.results.shape
dfBJFLCA=pd.DataFrame(index=list_methods, columns=[MM_fueltMJ_LCI['name'], ME_fueltMJ_LCI['name'], MV_fueltMJ_LCI['name'],HM_fueltMJ_LCI['name'], HH_fueltMJ_LCI['name'], HV_fueltMJ_LCI['name'],VM_fueltMJ_LCI['name'], VH_fueltMJ_LCI['name'], VV_fueltMJ_LCI['name'], ZeE_fueltMJ_LCI['name'], ALLE_fueltMJ_LCI['name'], ME_fueltMJ_LCI_org['name'], ME_fueltMJ_LCI_rainf['name']], data=BJFMultiLCA.results.T)
dfBJFLCA

####  N flow sustainability index calculation

In [ ]:
test = dfBJFLCA.values.tolist()
#print(test)
GWP_SV=test[0]
WF_SV =test[1]
NIF_IN_SV=test[2]
NIF_ON_SV=test[3]
NOF_IN_SV=test[4]
NOF_ON_SV=test[5]
NIF_ON_SV

In [ ]:
NIF_SV=list_add(NIF_IN_SV,NIF_ON_SV)
NIF_SV

In [ ]:
NOF_SV=list_add(NOF_IN_SV,NOF_ON_SV)
NOF_SV

In [ ]:
Nacc_SV=list_minus(NIF_SV,NOF_SV)
Nacc_SV

##### SV_inert N flow index

###### x indices

In [ ]:
NFI_IN_x_N_SV=list_minus(NOF_IN_SV,NIF_IN_SV)
NFI_IN_x_N_SV

In [ ]:
NFI_IN_x_SV=list_divide(NFI_IN_x_N_SV,NIF_SV)
NFI_IN_x_SV

In [ ]:
percentage_list = [100]*(len(NFI_IN_x_SV))
#print(cf_key_list)
len(percentage_list)
NFI_IN_x_per_SV=list_plus(NFI_IN_x_SV,percentage_list)
NFI_IN_x_per_SV

###### y indices

In [ ]:
NFI_IN_y_N_SV=list_minus(NOF_IN_SV,NIF_IN_SV)
NFI_IN_y_N_SV

In [ ]:
Nhuman_list = [N_human]*(len(NFI_IN_y_N_SV))
#print(cf_key_list)
len(Nhuman_list)
NFI_IN_y_SV=list_divide(NFI_IN_y_N_SV,Nhuman_list)
NFI_IN_y_SV

In [ ]:
percentage_list = [100]*(len(NFI_IN_y_SV))
#print(cf_key_list)
len(percentage_list)
NFI_IN_y_per_SV=list_plus(NFI_IN_y_SV,percentage_list)
NFI_IN_y_per_SV

##### reacive N flow index

###### x indices

In [ ]:
NFI_ON_x_N_SV=list_minus(NOF_ON_SV,NIF_ON_SV)
NFI_ON_x_N_SV

In [ ]:
NFI_ON_x_SV=list_divide(NFI_ON_x_N_SV,NIF_SV)
NFI_ON_x_SV

In [ ]:
percentage_list = [100]*(len(NFI_ON_x_SV))
#print(cf_key_list)
len(percentage_list)
NFI_ON_x_per_SV=list_plus(NFI_ON_x_SV,percentage_list)
NFI_ON_x_per_SV

###### y indices

In [ ]:
NFI_ON_y_N_SV=list_minus(NOF_ON_SV,NIF_ON_SV)
NFI_ON_y_N_SV

In [ ]:
Nhuman_list = [N_human]*(len(NFI_ON_y_N_SV))
#print(cf_key_list)
len(Nhuman_list)
NFI_ON_y_SV=list_divide(NFI_ON_y_N_SV,Nhuman_list)
NFI_ON_y_SV

In [ ]:
percentage_list = [100]*(len(NFI_ON_y_SV))
#print(cf_key_list)
len(percentage_list)
NFI_ON_y_per_SV=list_plus(NFI_ON_y_SV,percentage_list)
NFI_ON_y_per_SV

##### N accumulation

###### x indices

In [ ]:
NFI_Nacc_x_N_SV=Nacc_SV
NFI_Nacc_x_N_SV

In [ ]:
NFI_Nacc_x_SV=list_divide(NFI_Nacc_x_N_SV,NIF_SV)
NFI_Nacc_x_SV

In [ ]:
percentage_list = [100]*(len(NFI_Nacc_x_SV))
#print(cf_key_list)
len(percentage_list)
NFI_Nacc_x_per_SV=list_plus(NFI_Nacc_x_SV,percentage_list)
NFI_Nacc_x_per_SV

###### y indices

In [ ]:
NFI_Nacc_y_N_SV=Nacc_SV
NFI_Nacc_y_N_SV

In [ ]:
Nhuman_list = [N_human]*(len(NFI_Nacc_y_N_SV))
#print(cf_key_list)
len(Nhuman_list)
NFI_Nacc_y_SV=list_divide(NFI_Nacc_y_N_SV,Nhuman_list)
NFI_Nacc_y_SV

In [ ]:
percentage_list = [100]*(len(NFI_Nacc_y_SV))
#print(cf_key_list)
len(percentage_list)
NFI_Nacc_y_per_SV=list_plus(NFI_Nacc_y_SV,percentage_list)
NFI_Nacc_y_per_SV

In [ ]:
df_SV = pd.DataFrame()

df_SV['GWP_SV'] = GWP_SV
df_SV['WF_SV'] = WF_SV

df_SV['NFI_IN_x_per_SV'] = NFI_IN_x_per_SV
df_SV['NFI_IN_y_per_SV'] = NFI_IN_y_per_SV

df_SV['NFI_ON_x_per_SV'] = NFI_ON_x_per_SV
df_SV['NFI_ON_y_per_SV'] = NFI_ON_y_per_SV

df_SV['NFI_Nacc_x_per_SV'] = NFI_Nacc_x_per_SV
df_SV['NFI_Nacc_y_per_SV'] = NFI_Nacc_y_per_SV

df_SV

In [ ]:
#df_SV.to_excel(r'C:\Users\DELL\学术\N sustainability\NFI_SV.xlsx', sheet_name='NFI', index = True)

In [ ]:
import plotly.express as px
import pandas as pd

In [ ]:
import scipy

In [ ]:
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
import matplotlib as mpl 
from scipy.stats import gmean

In [ ]:
def eigsorted(cov):
    vals, vecs = np.linalg.eigh(cov)
    order = vals.argsort()[::-1]
    return vals[order], vecs[:,order]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def mscatter(x,y,ax=None, m=None, **kw):
    import matplotlib.markers as mmarkers
    if not ax: ax=plt.gca()
    sc = ax.scatter(x,y,**kw)
    if (m is not None) and (len(m)==len(x)):
        paths = []
        for marker in m:
            if isinstance(marker, mmarkers.MarkerStyle):
                marker_obj = marker
            else:
                marker_obj = mmarkers.MarkerStyle(marker)
            path = marker_obj.get_path().transformed(
                        marker_obj.get_transform())
            paths.append(path)
        sc.set_paths(paths)
    return sc

In [ ]:
randomlist = random.sample(range(10, 30), 5)
randomlist

In [ ]:
import numpy
plt.figure(figsize=(9,7))

import matplotlib as mpl

#myfont1 = mpl.font_manager.FontProperties(fname='/System/Library/Fonts/Supplemental/Arial Italic.ttf')     
myfont2 = mpl.font_manager.FontProperties(fname='C:/Users/DELL/AppData/Local/Microsoft/Windows/Fonts/TimesNewRomanPSBoldMT.ttf')
#test_x=random.sample(range(0, 1000), 12)
#test_y=random.sample(range(0, 1000), 12)

plt.rcParams['text.usetex'] =  False #     True
plt.rcParams['axes.unicode_minus'] = False

plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['mathtext.it'] = 'STIXGeneral:italic'
plt.rcParams['mathtext.bf'] = 'STIXGeneral:italic:bold'



#inert nitrogen flow index perpare
NFI_IN_x_plotMM = NFI_IN_x_per_SV[0]
NFI_IN_y_plotMM = NFI_IN_y_per_SV[0]

NFI_IN_x_plotME = NFI_IN_x_per_SV[1]
NFI_IN_y_plotME = NFI_IN_y_per_SV[1]

NFI_IN_x_plotMV = NFI_IN_x_per_SV[2]
NFI_IN_y_plotMV = NFI_IN_y_per_SV[2]

NFI_IN_x_plotHM = NFI_IN_x_per_SV[3]
NFI_IN_y_plotHM = NFI_IN_y_per_SV[3]

NFI_IN_x_plotHH = NFI_IN_x_per_SV[4]
NFI_IN_y_plotHH = NFI_IN_y_per_SV[4]

NFI_IN_x_plotHV = NFI_IN_x_per_SV[5]
NFI_IN_y_plotHV = NFI_IN_y_per_SV[5]

NFI_IN_x_plotVM = NFI_IN_x_per_SV[6]
NFI_IN_y_plotVM = NFI_IN_y_per_SV[6]

NFI_IN_x_plotVH = NFI_IN_x_per_SV[7]
NFI_IN_y_plotVH = NFI_IN_y_per_SV[7]

NFI_IN_x_plotVV = NFI_IN_x_per_SV[8]
NFI_IN_y_plotVV = NFI_IN_y_per_SV[8]


NFI_IN_x_plotZe = NFI_IN_x_per_SV[9]
NFI_IN_y_plotZe = NFI_IN_y_per_SV[9]

NFI_IN_x_plotAll = NFI_IN_x_per_SV[10]
NFI_IN_y_plotAll = NFI_IN_y_per_SV[10]

NFI_IN_x_plotOrg = NFI_IN_x_per_SV[11]
NFI_IN_y_plotOrg = NFI_IN_y_per_SV[11]

NFI_IN_x_plotRainf = NFI_IN_x_per_SV[12]
NFI_IN_y_plotRainf = NFI_IN_y_per_SV[12]

#reactive nitrogen flow index perpare
NFI_ON_x_plotMM = NFI_ON_x_per_SV[0]
NFI_ON_y_plotMM = NFI_ON_y_per_SV[0]

NFI_ON_x_plotME = NFI_ON_x_per_SV[1]
NFI_ON_y_plotME = NFI_ON_y_per_SV[1]

NFI_ON_x_plotMV = NFI_ON_x_per_SV[2]
NFI_ON_y_plotMV = NFI_ON_y_per_SV[2]

NFI_ON_x_plotHM = NFI_ON_x_per_SV[3]
NFI_ON_y_plotHM = NFI_ON_y_per_SV[3]

NFI_ON_x_plotHH = NFI_ON_x_per_SV[4]
NFI_ON_y_plotHH = NFI_ON_y_per_SV[4]

NFI_ON_x_plotHV = NFI_ON_x_per_SV[5]
NFI_ON_y_plotHV = NFI_ON_y_per_SV[5]

NFI_ON_x_plotVM = NFI_ON_x_per_SV[6]
NFI_ON_y_plotVM = NFI_ON_y_per_SV[6]

NFI_ON_x_plotVH = NFI_ON_x_per_SV[7]
NFI_ON_y_plotVH = NFI_ON_y_per_SV[7]

NFI_ON_x_plotVV = NFI_ON_x_per_SV[8]
NFI_ON_y_plotVV = NFI_ON_y_per_SV[8]

NFI_ON_x_plotZe = NFI_ON_x_per_SV[9]
NFI_ON_y_plotZe = NFI_ON_y_per_SV[9]

NFI_ON_x_plotAll = NFI_ON_x_per_SV[10]
NFI_ON_y_plotAll = NFI_ON_y_per_SV[10]

NFI_ON_x_plotOrg = NFI_ON_x_per_SV[11]
NFI_ON_y_plotOrg = NFI_ON_y_per_SV[11]

NFI_ON_x_plotRainf = NFI_ON_x_per_SV[12]
NFI_ON_y_plotRainf = NFI_ON_y_per_SV[12]

# nitrogen accumulation index perpare
NFI_Nacc_x_plotMM = NFI_Nacc_x_per_SV[0]
NFI_Nacc_y_plotMM = NFI_Nacc_y_per_SV[0]

NFI_Nacc_x_plotME = NFI_Nacc_x_per_SV[1]
NFI_Nacc_y_plotME = NFI_Nacc_y_per_SV[1]

NFI_Nacc_x_plotMV = NFI_Nacc_x_per_SV[2]
NFI_Nacc_y_plotMV = NFI_Nacc_y_per_SV[2]

NFI_Nacc_x_plotHM = NFI_Nacc_x_per_SV[3]
NFI_Nacc_y_plotHM = NFI_Nacc_y_per_SV[3]

NFI_Nacc_x_plotHH = NFI_Nacc_x_per_SV[4]
NFI_Nacc_y_plotHH = NFI_Nacc_y_per_SV[4]

NFI_Nacc_x_plotHV = NFI_Nacc_x_per_SV[5]
NFI_Nacc_y_plotHV = NFI_Nacc_y_per_SV[5]

NFI_Nacc_x_plotVM = NFI_Nacc_x_per_SV[6]
NFI_Nacc_y_plotVM = NFI_Nacc_y_per_SV[6]

NFI_Nacc_x_plotVH = NFI_Nacc_x_per_SV[7]
NFI_Nacc_y_plotVH = NFI_Nacc_y_per_SV[7]

NFI_Nacc_x_plotVV = NFI_Nacc_x_per_SV[8]
NFI_Nacc_y_plotVV = NFI_Nacc_y_per_SV[8]


NFI_Nacc_x_plotZe = NFI_Nacc_x_per_SV[9]
NFI_Nacc_y_plotZe = NFI_Nacc_y_per_SV[9]

NFI_Nacc_x_plotAll = NFI_Nacc_x_per_SV[10]
NFI_Nacc_y_plotAll = NFI_Nacc_y_per_SV[10]

NFI_Nacc_x_plotOrg = NFI_Nacc_x_per_SV[11]
NFI_Nacc_y_plotOrg = NFI_Nacc_y_per_SV[11]

NFI_Nacc_x_plotRainf = NFI_Nacc_x_per_SV[12]
NFI_Nacc_y_plotRainf = NFI_Nacc_y_per_SV[12]

#carbon footprint and waterfootprint perpare
GWP_SV_x_plotMM = GWP_SV[0]
WF_SV_y_plotMM  =  WF_SV[0]

GWP_SV_x_plotME = GWP_SV[1]
WF_SV_y_plotME  =  WF_SV[1]

GWP_SV_x_plotMV = GWP_SV[2]
WF_SV_y_plotMV  =  WF_SV[2]

GWP_SV_x_plotHM = GWP_SV[3]
WF_SV_y_plotHM  =  WF_SV[3]

GWP_SV_x_plotHH = GWP_SV[4]
WF_SV_y_plotHH  =  WF_SV[4]

GWP_SV_x_plotHV = GWP_SV[5]
WF_SV_y_plotHV  =  WF_SV[5]

GWP_SV_x_plotVM = GWP_SV[6]
WF_SV_y_plotVM  =  WF_SV[6]

GWP_SV_x_plotVH = GWP_SV[7]
WF_SV_y_plotVH  =  WF_SV[7]

GWP_SV_x_plotVV = GWP_SV[8]
WF_SV_y_plotVV  =  WF_SV[8]

GWP_SV_x_plotZe = GWP_SV[9]
WF_SV_y_plotZe  =  WF_SV[9]

GWP_SV_x_plotAll = GWP_SV[10]
WF_SV_y_plotAll  =  WF_SV[10]

GWP_SV_x_plotOrg = GWP_SV[11]
WF_SV_y_plotOrg  =  WF_SV[11]

GWP_SV_x_plotRainf = GWP_SV[12]
WF_SV_y_plotRainf  =  WF_SV[12]

Size_legend1=100
# plotting point shape legend 
plt.scatter(x=NFI_IN_x_plotMM, y=NFI_IN_y_plotMM, c = 'black', marker = '1', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotME, y=NFI_IN_y_plotME, c = 'black', marker = 'x', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotMV, y=NFI_IN_y_plotMV, c = 'black', marker = 'v', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotHM, y=NFI_IN_y_plotHM, c = 'black', marker = '2', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotHH, y=NFI_IN_y_plotHH, c = 'black', marker = '^', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotHV, y=NFI_IN_y_plotHV, c = 'black', marker = 'p', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotVM, y=NFI_IN_y_plotVM, c = 'black', marker = '3', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotVH, y=NFI_IN_y_plotVH, c = 'black', marker = '<', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotVV, y=NFI_IN_y_plotVV, c = 'black', marker = '*', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotZe, y=NFI_IN_y_plotZe, c = 'black', marker = '+', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotAll, y=NFI_IN_y_plotAll, c = 'black', marker = '4', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotOrg, y=NFI_IN_y_plotOrg, c = 'black', marker = 'h', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotRainf, y=NFI_IN_y_plotRainf, c = 'black', marker = '$☉$', s = Size_legend1)

#plottingg colour legend
p1=plt.scatter(x=NFI_IN_x_plotHV, y=NFI_IN_y_plotHV, c = 'green', marker = 'o', s = 10)
p2=plt.scatter(x=NFI_IN_x_plotHV, y=NFI_IN_y_plotHV, c = 'blue', marker = 'o', s = 10)
p3=plt.scatter(x=NFI_IN_x_plotHV, y=NFI_IN_y_plotHV, c = 'red', marker = 'o', s = 10)
p4=plt.scatter(x=NFI_IN_x_plotHV, y=NFI_IN_y_plotHV, c = 'fuchsia', marker = 'o', s = 10)

#plotting inert nitrogen flow index
plt.scatter(x=NFI_IN_x_plotMM, y=NFI_IN_y_plotMM, c = 'green', marker = '1', s = Size_legend1+90*2)
plt.scatter(x=NFI_IN_x_plotME, y=NFI_IN_y_plotME, c = 'green', marker = 'x', s = Size_legend1+60*2)
plt.scatter(x=NFI_IN_x_plotMV, y=NFI_IN_y_plotMV, c = 'green', marker = 'v', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotHM, y=NFI_IN_y_plotHM, c = 'green', marker = '2', s = Size_legend1+90*2)
plt.scatter(x=NFI_IN_x_plotHH, y=NFI_IN_y_plotHH, c = 'green', marker = '^', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotHV, y=NFI_IN_y_plotHV, c = 'green', marker = 'p', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotVM, y=NFI_IN_y_plotVM, c = 'green', marker = '3', s = Size_legend1+50*2)
plt.scatter(x=NFI_IN_x_plotVH, y=NFI_IN_y_plotVH, c = 'green', marker = '<', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotVV, y=NFI_IN_y_plotVV, c = 'green', marker = '*', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotZe, y=NFI_IN_y_plotZe, c = 'green', marker = '+', s = Size_legend1+60*2)
plt.scatter(x=NFI_IN_x_plotAll, y=NFI_IN_y_plotAll, c = 'green', marker = '4', s = Size_legend1+90*2)
plt.scatter(x=NFI_IN_x_plotOrg, y=NFI_IN_y_plotOrg, c = 'green', marker = 'h', s = Size_legend1)
plt.scatter(x=NFI_IN_x_plotRainf, y=NFI_IN_y_plotRainf, c = 'green', marker = '$☉$', s = Size_legend1) # $☉$


#plotting reactive nitrogen flow index
plt.scatter(x=NFI_ON_x_plotMM, y=NFI_ON_y_plotMM, c = 'blue', marker = '1', s = Size_legend1+90*2)
plt.scatter(x=NFI_ON_x_plotME, y=NFI_ON_y_plotME, c = 'blue', marker = 'x', s = Size_legend1+60*2)
plt.scatter(x=NFI_ON_x_plotMV, y=NFI_ON_y_plotMV, c = 'blue', marker = 'v', s = Size_legend1)
plt.scatter(x=NFI_ON_x_plotHM, y=NFI_ON_y_plotHM, c = 'blue', marker = '2', s = Size_legend1+90*2)
plt.scatter(x=NFI_ON_x_plotHH, y=NFI_ON_y_plotHH, c = 'blue', marker = '^', s = Size_legend1)
plt.scatter(x=NFI_ON_x_plotHV, y=NFI_ON_y_plotHV, c = 'blue', marker = 'p', s = Size_legend1)
plt.scatter(x=NFI_ON_x_plotVM, y=NFI_ON_y_plotVM, c = 'blue', marker = '3', s = Size_legend1+50*2)
plt.scatter(x=NFI_ON_x_plotVH, y=NFI_ON_y_plotVH, c = 'blue', marker = '<', s = Size_legend1)
plt.scatter(x=NFI_ON_x_plotVV, y=NFI_ON_y_plotVV, c = 'blue', marker = '*', s = Size_legend1)
plt.scatter(x=NFI_ON_x_plotZe, y=NFI_ON_y_plotZe, c = 'blue', marker = '+', s = Size_legend1+60*2)
plt.scatter(x=NFI_ON_x_plotAll, y=NFI_ON_y_plotAll, c = 'blue', marker = '4', s = Size_legend1+90*2)
plt.scatter(x=NFI_ON_x_plotOrg, y=NFI_ON_y_plotOrg, c = 'blue', marker = 'h', s = Size_legend1)
plt.scatter(x=NFI_ON_x_plotRainf, y=NFI_ON_y_plotRainf, c = 'blue', marker = '$☉$', s = Size_legend1)


#plotting nitrogen accumulation flow index
plt.scatter(x=NFI_Nacc_x_plotMM, y=NFI_Nacc_y_plotMM, c = 'red', marker = '1', s = Size_legend1+90*2)
plt.scatter(x=NFI_Nacc_x_plotME, y=NFI_Nacc_y_plotME, c = 'red', marker = 'x', s = Size_legend1+60*2)
plt.scatter(x=NFI_Nacc_x_plotMV, y=NFI_Nacc_y_plotMV, c = 'red', marker = 'v', s = Size_legend1)
plt.scatter(x=NFI_Nacc_x_plotHM, y=NFI_Nacc_y_plotHM, c = 'red', marker = '2', s = Size_legend1+90*2)
plt.scatter(x=NFI_Nacc_x_plotHH, y=NFI_Nacc_y_plotHH, c = 'red', marker = '^', s = Size_legend1)
plt.scatter(x=NFI_Nacc_x_plotHV, y=NFI_Nacc_y_plotHV, c = 'red', marker = 'p', s = Size_legend1)
plt.scatter(x=NFI_Nacc_x_plotVM, y=NFI_Nacc_y_plotVM, c = 'red', marker = '3', s = Size_legend1+50*2)
plt.scatter(x=NFI_Nacc_x_plotVH, y=NFI_Nacc_y_plotVH, c = 'red', marker = '<', s = Size_legend1)
plt.scatter(x=NFI_Nacc_x_plotVV, y=NFI_Nacc_y_plotVV, c = 'red', marker = '*', s = Size_legend1)
plt.scatter(x=NFI_Nacc_x_plotZe, y=NFI_Nacc_y_plotZe, c = 'red', marker = '+', s = Size_legend1+60*2)
plt.scatter(x=NFI_Nacc_x_plotAll, y=NFI_Nacc_y_plotAll, c = 'red', marker = '4', s = Size_legend1+90*2)
plt.scatter(x=NFI_Nacc_x_plotOrg, y=NFI_Nacc_y_plotOrg, c = 'red', marker = 'h', s = Size_legend1)
plt.scatter(x=NFI_Nacc_x_plotRainf, y=NFI_Nacc_y_plotRainf, c = 'red', marker = '$☉$', s = Size_legend1)

#plotting carbon footprint and waterfootprint
plt.scatter(x=GWP_SV_x_plotMM, y=WF_SV_y_plotMM, c = 'fuchsia', marker = '1', s = Size_legend1+90*2)
plt.scatter(x=GWP_SV_x_plotME, y=WF_SV_y_plotME, c = 'fuchsia', marker = 'x', s = Size_legend1+60*2)
plt.scatter(x=GWP_SV_x_plotMV, y=WF_SV_y_plotMV, c = 'fuchsia', marker = 'v', s = Size_legend1)
plt.scatter(x=GWP_SV_x_plotHM, y=WF_SV_y_plotHM, c = 'fuchsia', marker = '2', s = Size_legend1+90*2)
plt.scatter(x=GWP_SV_x_plotHH, y=WF_SV_y_plotHH, c = 'fuchsia', marker = '^', s = Size_legend1)
plt.scatter(x=GWP_SV_x_plotHV, y=WF_SV_y_plotHV, c = 'fuchsia', marker = 'p', s = Size_legend1)
plt.scatter(x=GWP_SV_x_plotVM, y=WF_SV_y_plotVM, c = 'fuchsia', marker = '3', s = Size_legend1+50*2)
plt.scatter(x=GWP_SV_x_plotVH, y=WF_SV_y_plotVH, c = 'fuchsia', marker = '<', s = Size_legend1)
plt.scatter(x=GWP_SV_x_plotVV, y=WF_SV_y_plotVV, c = 'fuchsia', marker = '*', s = Size_legend1)
plt.scatter(x=GWP_SV_x_plotZe, y=WF_SV_y_plotZe, c = 'fuchsia', marker = '+', s = Size_legend1+60*2)
plt.scatter(x=GWP_SV_x_plotAll, y=WF_SV_y_plotAll, c = 'fuchsia', marker = '4', s = Size_legend1+90*2)
plt.scatter(x=GWP_SV_x_plotOrg, y=WF_SV_y_plotOrg, c = 'fuchsia', marker = 'h', s = Size_legend1)
plt.scatter(x=GWP_SV_x_plotRainf, y=WF_SV_y_plotRainf, c = 'fuchsia', marker = '$☉$', s = Size_legend1)


#col_g=["green","green","green","green","green","green","green","green","green","green","green","green"]
markers = ["1","x","v","2","^","p","3","<","*","+","4","h"]

# scale X,Y aix length
#plt.scatter(x=0, y=10, c = 'fuchsia', marker = 'None', s = Size_legend1)
#plt.scatter(x=0, y=-10, c = 'fuchsia', marker = 'None', s = Size_legend1)
#plt.scatter(x=10000, y=0, c = 'fuchsia', marker = 'None', s = Size_legend1)
#plt.scatter(x=-10000, y=0, c = 'fuchsia', marker = 'None', s = Size_legend1)


#size = [10,15,20,30,40,50,60,70,80,85,90,95]  
#for m in enumerate(np.unique(markers)):  #c=col_g[col_g==c],
    #plt.scatter(NFI_IN_x_per_SV[:],NFI_IN_y_per_SV[:],c=col_g[col_g==c], marker=markers[i])
#    plt.scatter(test_x[:],test_y[:],marker=markers[markers==m],color='green')


#sns.scatterplot(data=df_SV, x='NFI_IN_x_per_SV', y='NFI_IN_y_per_SV',color='green',s=25)
#sns.scatterplot(data=df_SV, x='GWP_SV', y='WF_SV',color='white',s=1, marker=',')
#sns.scatterplot(data=df_SV, x='NFI_ON_x_per_SV', y='NFI_ON_y_per_SV',color='black',s=25, marker='>')
#sns.scatterplot(data=df_SV, x='NFI_Nacc_x_per_SV', y='NFI_Nacc_y_per_SV',color='red',s=25, marker=',')
#sns.scatterplot(data=df_SV, x='GWP_SV', y='WF_SV',color='fuchsia',s=25, marker=',')

mpl.rc('font',family='Times New Roman')

legend1=plt.legend(["S-1: Stage 1 mass, Stage 3 mass", "S-2: Stage 1 mass, Stage 3 energy (Basline Scenario)", "S-3: Stage 1 mass, Stage 3  marketing value",
            "S-4: Stage 1 energy, Stage 3 mass", "S-5: Stage 1 energy, Stage 3 energy", "S-6: Stage 1 energy, Stage 3 marketing value",
            "S-7: Stage 1 marketing value, Stage 3 mass", "S-8: Stage 1 marketing value, Stage 3 energy", "S-9: Stage 1 marketing value, Stage 3 marketing value",
            "S-10: Maize straw environmental burden free in Stage 1","S-11: Maize straw taken all environmental impact of Stage 1",
            "S-12: Organic cultivation of maize straw in Stage 1","S-13: Rainfed cultivation of maize straw in Stage 1"], 
                   fontsize=18, bbox_to_anchor=(1.02, 1.17),title="Life cycle scenario",title_fontsize=18, frameon=False)
# bbox_to_anchor=(1.035, 1.17)
#legend1.set_title('Shape',prop=myfont2) title="$\\bf{Shape}$"
#"S-7: Stage 1 marketing value, Stage 3 mass", "S-8: Stage 1 marketing value, Stage 3 energy", "S-9: Stage 1 marketing value, Stage 3 marketing value",
#"S-10: Maize straw environmental burden free in Stage 1, Stage 3 energy","S-11: Maize straw taken all environmental impact of Stage 1, Stage 3 energy",
#"S-12: Organic cultivation of maize straw in Stage 1 and mass based allocation, Stage 3 energy","S-13: Rainfed cultivation of maize straw in Stage 1 and mass based allocation, Stage 3 energy"], 



plt.legend([p1, p2, p3, p4], ['Inert nitrogen flow index', 'Reactive nitrogen flow index', 'Nitrogen accumulation index', 'X axis: EF v3.0 climate change total \nY aixs: EF v3.0 deprivation-weighted water consumption'], loc='lower right',
           scatterpoints=1, bbox_to_anchor=(2.04, -0.23),fontsize=18, title="Color",title_fontsize=18, frameon=False)
plt.gca().add_artist(legend1)

#'Analyse de contribution de l\'activité "%s" \n pour la catégorie : %s\n Impact total = %0.2g %s/MJ H2' # $\it{NFPC}$,
plt.title('N flow sustainability index, Carbon footprint, Water footprint', fontsize=22,fontname="Times New Roman")
plt.xlabel('Relative percentage of N flow change or N accumulation (%)\n Carbon footprint (kg CO$_{2-eq}$)', fontsize=20,fontname="Times New Roman")
plt.ylabel('Magnitude of N flow change or N accumulation (%)\n Water footprint (m$^3$ world eq. water deprived)', fontsize=20,fontname="Times New Roman")

# rotation=45,horizontalalignment="right"
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

mpl.rc('font',family='Times New Roman')

# Benchmark Mean values          
          
plt.axvline(x=0, color='k',linestyle='--', linewidth=1) 
plt.axhline(y=0, color='k', linestyle='--', linewidth=1) 


plt.yscale('log')
plt.yscale('symlog')
plt.xscale('log')
plt.xscale('symlog')

mpl.rcParams["figure.dpi"] = 900
#plt.ylim(ymin = 0)



#panel.ellipse(NFI_IN_x_per, NFI_IN_y_per, lwd = 2, col="green", robust=FALSE,  level=0.65)

#nstd = 1
#ax = plt.subplot(111)

#x1=NFI_IN_x_per
#y1=NFI_IN_y_per

#cov = np.cov(x1, y1)
#vals, vecs = eigsorted(cov)
#theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
#w, h = 2 * nstd * np.sqrt(vals)
#ell = Ellipse(xy=(np.mean(x1), np.mean(y1)),
#              width=w, height=h,
#              angle=theta, color='grey')
#ell.set_facecolor('none')
#ax.add_artist(ell)

#x2=NFI_ON_x_per
#y2=NFI_ON_y_per

#cov = np.cov(x2, y2)
#vals, vecs = eigsorted(cov)
#theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
#w, h = 2 * nstd * np.sqrt(vals)
#ell = Ellipse(xy=(np.mean(x2), np.mean(y2)),
#              width=w, height=h,
#              angle=theta, color='blue')
#ell.set_facecolor('none')
#ax.add_artist(ell)

#sns.scatterplot(x='-7.81376479400426', y='-987.4604094144358',color='yellow',s=100)


#x3=NFI_Nacc_x_per
#y3=NFI_Nacc_y_per

#cov = np.cov(x3, y3)
#vals, vecs = eigsorted(cov)
#theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
#w, h = 2 * nstd * np.sqrt(vals)
#ell = Ellipse(xy=(np.mean(x3), np.mean(y3)),
#              width=w, height=h,
#              angle=theta, color='yellow',lw=1.5)
#ell.set_facecolor('none')
#ax.add_artist(ell)

#ax = sns.regplot(x=x1, y=y1, ci=95,scatter=False,color='lime')
#ax = sns.regplot(x=x2, y=y2, ci=95,scatter=False,color='blue')
#ax = sns.regplot(x=x3, y=y3, ci=95,scatter=False,color='red')

#x1_ave = np.average(x1)
#y1_ave = np.average(y1)
#plt.scatter(x1_ave, y1_ave, c = 'lime', marker = '*', s = 300)

#x2_ave = np.average(x2)
#y2_ave = np.average(y2)
#plt.scatter(x2_ave, y2_ave, c = 'cyan', marker = '*', s = 300)

#x3_ave = np.average(x3)
#y3_ave = np.average(y3)
#plt.scatter(x3_ave, y3_ave, c = 'yellow', marker = '*', s = 300)


plt.show()

### Carbon, water footprint, and system N flow uncertainty analysis

#### Monte Carlo LCA

##### Build the monte carlo sampling functions correlating multi impact categories

In [ ]:
import bw2analyzer as bwa

In [ ]:
import seaborn as sb
import stats_arrays

In [ ]:
def multiImpactMonteCarloLCA(functional_unit, list_methods, iterations):
    # Step 1
    MC_lca = bw.MonteCarloLCA(functional_unit)
    MC_lca.lci()
    # Step 2
    C_matrices = {}
    # Step 3
    for method in list_methods:
        MC_lca.switch_method(method)
        C_matrices[method] = MC_lca.characterization_matrix
    # Step 4
    results = np.empty((len(list_methods), iterations))
    # Step 5
    for iteration in range(iterations):
        next(MC_lca)
        for method_index, method in enumerate(list_methods):
            results[method_index, iteration] = (C_matrices[method]*MC_lca.inventory).sum()
    return results

In [ ]:
test_results = multiImpactMonteCarloLCA({ME_fueltMJ_LCI:1}, [NIF_IN,NIF_ON], 1000)

In [ ]:
test_results

##### Correlating multi sustainability indicators Monte Carlo sampling

In [ ]:
correlateMCLCA_results = multiImpactMonteCarloLCA({ME_fueltMJ_LCI:1}, [NIF_IN,NIF_ON,NOF_IN,NOF_ON,CC_EF3,water_EF], 100000)

In [ ]:
correlateMCLCA_results #.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\footprint_correlated_MC_40000.xlsx', sheet_name='correlated_MC', index = True)

In [ ]:
correlateMCLCAdf = pd.DataFrame(correlateMCLCA_results)
#correlateMCLCAdf.T.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\footprint_correlated_MC_40000.xlsx', sheet_name='correlated_MC', index = True)

##### Monte Carlo sampling data import

In [ ]:
df_CNWnexus= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\footprint_correlated_MC_40000_orig.xlsx', sheet_name='correlated_MC') # , skiprows=[0]
df_CNWnexus

In [ ]:
NIF_IN_scores = df_CNWnexus.iloc[:,1].values.tolist()
NIF_ON_scores = df_CNWnexus.iloc[:,2].values.tolist()

NOF_IN_scores = df_CNWnexus.iloc[:,3].values.tolist()
NOF_ON_scores = df_CNWnexus.iloc[:,4].values.tolist()

NIF = list_add(NIF_IN_scores,NIF_ON_scores)
NOF = list_add(NOF_IN_scores,NOF_ON_scores)

N_acc=list_minus(NIF,NOF)


MC_GWP_scores = df_CNWnexus.iloc[:,5].values.tolist()
MC_water_scores = df_CNWnexus.iloc[:,6].values.tolist()

NIF_IN_scores

#### N flow sustainability index building from Monte Carlo smapling outcome

##### inert N flow index

###### x indices

In [ ]:
NFI_IN_x_N=list_minus(NOF_IN_scores,NIF_IN_scores)
NFI_IN_x_N

In [ ]:
NFI_IN_x=list_divide(NFI_IN_x_N,NIF)
NFI_IN_x

In [ ]:
percentage_list = [100]*(len(NFI_IN_x))
#print(cf_key_list)
len(percentage_list)
NFI_IN_x_per=list_plus(NFI_IN_x,percentage_list)
NFI_IN_x_per

###### y indices

In [ ]:
NFI_IN_y_N=list_minus(NOF_IN_scores,NIF_IN_scores)
NFI_IN_y_N

In [ ]:
Nhuman_list = [N_human]*(len(NFI_IN_y_N))
#print(cf_key_list)
len(Nhuman_list)
NFI_IN_y=list_divide(NFI_IN_y_N,Nhuman_list)
NFI_IN_y

In [ ]:
percentage_list = [100]*(len(NFI_IN_y))
#print(cf_key_list)
len(percentage_list)
NFI_IN_y_per=list_plus(NFI_IN_y,percentage_list)
NFI_IN_y_per

##### reacive N flow index

###### x indices

In [ ]:
NFI_ON_x_N=list_minus(NOF_ON_scores,NIF_ON_scores)
NFI_ON_x_N

In [ ]:
NFI_ON_x=list_divide(NFI_ON_x_N,NIF)
NFI_ON_x

In [ ]:
percentage_list = [100]*(len(NFI_ON_x))
#print(cf_key_list)
len(percentage_list)
NFI_ON_x_per=list_plus(NFI_ON_x,percentage_list)
NFI_ON_x_per

###### y indices

In [ ]:
NFI_ON_y_N=list_minus(NOF_ON_scores,NIF_ON_scores)
NFI_ON_y_N

In [ ]:
Nhuman_list = [N_human]*(len(NFI_ON_y_N))
#print(cf_key_list)
len(Nhuman_list)
NFI_ON_y=list_divide(NFI_ON_y_N,Nhuman_list)
NFI_ON_y

In [ ]:
percentage_list = [100]*(len(NFI_ON_y))
#print(cf_key_list)
len(percentage_list)
NFI_ON_y_per=list_plus(NFI_ON_y,percentage_list)
NFI_ON_y_per

In [ ]:
#test=list_xy(NFI_IN_x_per,NFI_IN_y_per)
#test

##### N accumulation

###### x indices

In [ ]:
NFI_Nacc_x_N=N_acc
NFI_Nacc_x_N

In [ ]:
NFI_Nacc_x=list_divide(NFI_Nacc_x_N,NIF)
NFI_Nacc_x

In [ ]:
percentage_list = [100]*(len(NFI_Nacc_x))
#print(cf_key_list)
len(percentage_list)
NFI_Nacc_x_per=list_plus(NFI_Nacc_x,percentage_list)
NFI_Nacc_x_per

###### y indices

In [ ]:
NFI_Nacc_y_N=N_acc
NFI_Nacc_y_N

In [ ]:
Nhuman_list = [N_human]*(len(NFI_Nacc_y_N))
#print(cf_key_list)
len(Nhuman_list)
NFI_Nacc_y=list_divide(NFI_Nacc_y_N,Nhuman_list)
NFI_Nacc_y

In [ ]:
percentage_list = [100]*(len(NFI_Nacc_y))
#print(cf_key_list)
len(percentage_list)
NFI_Nacc_y_per=list_plus(NFI_Nacc_y,percentage_list)
NFI_Nacc_y_per

In [ ]:
df_NFI = pd.DataFrame()

df_NFI['NFI_IN_x(per)'] = NFI_IN_x_per
df_NFI['NFI_IN_y(per)'] = NFI_IN_y_per

df_NFI['NFI_ON_x(per)'] = NFI_ON_x_per
df_NFI['NFI_ON_y(per)'] = NFI_ON_y_per

df_NFI['NFI_Nacc_x(per)'] = NFI_Nacc_x_per
df_NFI['NFI_Nacc_y(per)'] = NFI_Nacc_y_per

df_NFI['GWP_x_m'] = MC_GWP_scores
df_NFI['WF_y_m'] = MC_water_scores

df_NFI

In [ ]:
import scipy

In [ ]:
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
import matplotlib as mpl 
from scipy.stats import gmean

In [ ]:
def eigsorted(cov):
    vals, vecs = np.linalg.eigh(cov)
    order = vals.argsort()[::-1]
    return vals[order], vecs[:,order]

In [ ]:
df_NFI_PD= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\NFI_correlated_MCLCA_40000_orig.xlsx', sheet_name='MC_uncertainty_40000') # , skiprows=[0]
df_NFI_PD

##### Functions for ploting the probability distributions of Monte Carlo sampling outcome

In [ ]:
def connectpoints_IN_x(x,y,z,p1,p2):
    x1, x2 = x[p1], z[p2]
    y1, y2 = y[p1], z[p2]
    plt.plot([x1,y1],[x2,y2],'green',lw=1)

In [ ]:
def connectpoints_IN_y(x,y,z,p1,p2):
    x1, x2 = z[p2], x[p1]
    y1, y2 = z[p2], y[p1]
    plt.plot([x1,y1],[x2,y2],'green',lw=1)

In [ ]:
def connectpoints_ON_x(x,y,z,p1,p2):
    x1, x2 = x[p1], z[p2]
    y1, y2 = y[p1], z[p2]
    plt.plot([x1,y1],[x2,y2],'lightgreen',lw=1)

In [ ]:
def connectpoints_ON_y(x,y,z,p1,p2):
    x1, x2 = z[p2], x[p1]
    y1, y2 = z[p2], y[p1]
    plt.plot([x1,y1],[x2,y2],'lightgreen',lw=1)

In [ ]:
def connectpoints_Nacc_x(x,y,z,p1,p2):
    x1, x2 = x[p1], z[p2]
    y1, y2 = y[p1], z[p2]
    plt.plot([x1,y1],[x2,y2],'orange',lw=1)

In [ ]:
def connectpoints_Nacc_y(x,y,z,p1,p2):
    x1, x2 = z[p2], x[p1]
    y1, y2 = z[p2], y[p1]
    plt.plot([x1,y1],[x2,y2],'orange',lw=1)

In [ ]:
def connectpoints_CF_x(x,y,z,p1,p2):
    x1, x2 = x[p1], z[p2]
    y1, y2 = y[p1], z[p2]
    plt.plot([x1,y1],[x2,y2],'blueviolet',lw=1)

In [ ]:
def connectpoints_WF_y(x,y,z,p1,p2):
    x1, x2 = z[p2], x[p1]
    y1, y2 = z[p2], y[p1]
    plt.plot([x1,y1],[x2,y2],'blueviolet',lw=1)

In [ ]:
df_CNWPC= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\NFI_correlated_MCLCA_40000.xlsx', sheet_name='ave and P') # , skiprows=[0]
df_CNWPC

In [ ]:
df_CNWPC.drop('PC', axis=1, inplace=True)
df_CNWPC

In [ ]:
CNW_PC_ave = df_CNWPC.iloc[0,:].values.tolist()
CNW_PC_P10 = df_CNWPC.iloc[1,:].values.tolist()

CNW_PC_P25 = df_CNWPC.iloc[2,:].values.tolist()
CNW_PC_P50 = df_CNWPC.iloc[3,:].values.tolist()

CNW_PC_P75 = df_CNWPC.iloc[4,:].values.tolist()
CNW_PC_P90 = df_CNWPC.iloc[5,:].values.tolist()

CNW_PC_P5 = df_CNWPC.iloc[6,:].values.tolist()
CNW_PC_P95 = df_CNWPC.iloc[7,:].values.tolist()

CNW_PC_LAV = df_CNWPC.iloc[8,:].values.tolist()
CNW_PC_HAV = df_CNWPC.iloc[9,:].values.tolist()

[CNW_PC_P10[0],CNW_PC_ave[1]],[CNW_PC_P90[0],CNW_PC_ave[1]]

In [ ]:
import matplotlib.pyplot as plt
import numpy
import os

plt.figure(figsize=(12,8))

plt.rcParams['text.usetex'] =  False #     True
plt.rcParams['axes.unicode_minus'] = False

plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['mathtext.it'] = 'STIXGeneral:italic'
plt.rcParams['mathtext.bf'] = 'STIXGeneral:italic:bold'


# plotting box plot legend 
box0=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'red', marker = 'None', s = 100)
box1=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'yellowgreen', marker = '$▬$', s = 100)
box2=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'deepskyblue', marker = '$▬$', s = 100)
box3=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'orange', marker = '$▬$', s = 100)
box4=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'blueviolet', marker = '$▬$', s = 100)

# plotting standard deviation curve
SDc0=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'red', marker='None', s = 500)
SDc1=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'lime', marker = '_', s = 500)
SDc2=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'blue', marker = '_', s = 500)
SDc3=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'yellow', marker = '_', s = 500)
SDc4=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'lightpink', marker = '_', s = 500)

sns.scatterplot(data=df_NFI_PD, x='NFI_IN_x(per)', y='NFI_IN_y(per)',color='green',s=25)
sns.scatterplot(data=df_NFI_PD, x='NFI_ON_x(per)', y='NFI_ON_y(per)',color='blue',s=25, marker='>')
sns.scatterplot(data=df_NFI_PD, x='NFI_Nacc_x(per)', y='NFI_Nacc_y(per)',color='red',s=25, marker=',')
sns.scatterplot(data=df_NFI_PD, x='GWP_x_m', y='WF_y_m',color='fuchsia',s=25, marker='s')

# plotting point shape legend 
ps0=plt.scatter(x=NFI_IN_x_per[0], y=NFI_IN_y_per[0], c = 'red',marker='None', s = 21)
ps1=plt.scatter(x=NFI_IN_x_per[0], y=NFI_IN_y_per[0], c = 'green', s = 21)
ps2=plt.scatter(x=NFI_ON_x_per[0], y=NFI_IN_y_per[0], c = 'blue', marker = '>', s = 21)
ps3=plt.scatter(x=NFI_Nacc_x_per[0], y=NFI_Nacc_y_per[0], c = 'red', marker = ',', s = 21)
ps4=plt.scatter(x=MC_GWP_scores[0], y=MC_water_scores[0], c = 'fuchsia', marker = 's', s = 21)



#legend1=plt.legend(['Inert nitrogen flow index', 'Reactive nitrogen flow index', 'Nitrogen accumulation index', 'X axis: EF v3.0 climate change total \nY aixs EF v3.0 deprivation-weighted water consumption'],
#           loc='best', fontsize=16, frameon=False, title="Indicators",title_fontsize=17, bbox_to_anchor=(0.62, -0.17))

#legend1=plt.legend([ps1,SDc1,Mean1],['Monte Carlo sampling outcome distribution', 'Standard deviation curve of sampling result','Arithmetic mean of sampling outcome'],
#           loc='best', fontsize=16, frameon=False, title="Inert nitrogen flow index",title_fontsize=17, bbox_to_anchor=(0.62, -0.17),
#                      )

legend1_4_y =-0.277 # -0.263 medium
legend1_4_x = 1.2 # 1.06 medium


# ave star
x1_ave = np.average(x1)
y1_ave = np.average(y1)
Mean1=plt.scatter(x1_ave, y1_ave, c = 'lime', marker = '$☉$', s = 200)

Mean0=plt.scatter(x1_ave, y1_ave, c = 'white', marker='None', s = 200)

x2_ave = np.average(x2)
y2_ave = np.average(y2)
Mean2=plt.scatter(x2_ave, y2_ave, c = 'cyan', marker = '$☉$', s = 200)

x3_ave = np.average(x3)
y3_ave = np.average(y3)
Mean3=plt.scatter(x3_ave, y3_ave, c = 'yellow', marker = '$☉$', s = 200)

x4_ave = np.average(x4)
y4_ave = np.average(y4)
Mean4=plt.scatter(x4_ave, y4_ave, c = 'purple', marker = '$☉$', s = 200)



# legend 4
legend4=plt.legend([ps4,SDc4,Mean4,box4],['Monte Carlo sampling outcome distribution', 'Standard deviation curve of sampling result','Arithmetic mean of sampling outcome','Representative color for boxplot'],
           loc='best', fontsize=16, frameon=False, bbox_to_anchor=(legend1_4_x-0.4835, legend1_4_y),
                      )
for text in legend4.get_texts():
    text.set_color("white")

# legend 3
legend3=plt.legend([ps3,SDc3,Mean3,box3],['Monte Carlo sampling outcome distribution', 'Standard deviation curve of sampling result','Arithmetic mean of sampling outcome','Representative color for boxplot'],
           loc='best', fontsize=16, frameon=False, bbox_to_anchor=(legend1_4_x-0.325, legend1_4_y),
                      )
for text in legend3.get_texts():
    text.set_color("white")
    
plt.gca().add_artist(legend4)


# legend 2
legend2=plt.legend([ps2,SDc2,Mean2,box2],['Monte Carlo sampling outcome distribution', 'Standard deviation curve of sampling result','Arithmetic mean of sampling outcome','Representative color for boxplot'],
           loc='best', fontsize=16, frameon=False, bbox_to_anchor=(legend1_4_x-0.188, legend1_4_y),
                      )
for text in legend2.get_texts():
    text.set_color("white")
    
plt.gca().add_artist(legend3)


# legend 1
legend1=plt.legend([ps1,SDc1,Mean1,box1],['Monte Carlo sampling outcome distribution', 'Standard deviation curve of sampling result','Arithmetic mean of sampling outcome','Representative color for boxplot'],
           loc='best', fontsize=16, frameon=False, bbox_to_anchor=(legend1_4_x-0.07, legend1_4_y),
                      )
for text in legend1.get_texts():
    text.set_color("white")
plt.gca().add_artist(legend2)

legend1_1=plt.legend([ps0],['    Inert N \n flow index'],
           loc='best', fontsize=16, frameon=False, bbox_to_anchor=(legend1_4_x-0.48, legend1_4_y+0.08),
                      )
plt.gca().add_artist(legend1)

legend2_1=plt.legend([ps0],[' Reactive N \n flow index'],
           loc='best', fontsize=16, frameon=False, bbox_to_anchor=(legend1_4_x-0.587, legend1_4_y+0.08),
                      )
plt.gca().add_artist(legend1_1)

legend3_1=plt.legend([ps0],['N accumulation \n         index'],
           loc='best', fontsize=16, frameon=False, bbox_to_anchor=(legend1_4_x-0.711, legend1_4_y+0.08),
                      )
plt.gca().add_artist(legend2_1)

legend4_1=plt.legend([ps0],['Carbon & water \n        footprint'],
           loc='best', fontsize=16, frameon=False, bbox_to_anchor=(legend1_4_x-0.875, legend1_4_y+0.08),
                      )
plt.gca().add_artist(legend3_1)

legend1_2=plt.legend([ps0,SDc0,Mean0,box0],['Monte Carlo sampling outcome distribution', 'Standard deviation curve of sampling result', 'Arithmetic mean of sampling outcome', 'Representative color for boxplot'],
           loc='best', fontsize=16, frameon=False, bbox_to_anchor=(legend1_4_x-0.044, legend1_4_y),
                      )
plt.gca().add_artist(legend4_1)


    
#plt.legend([ps2, SDc2, Mean2], ['Inert nitrogen flow index', 'Reactive nitrogen flow index', 'Nitrogen accumulation index', 'X axis: EF v3.0 climate change total \nY aixs: EF v3.0 deprivation-weighted water consumption'], loc='lower right',
#           scatterpoints=1, bbox_to_anchor=(2.04, -0.23),fontsize=18, title="Reactive nitrogen flow index",title_fontsize=18, frameon=False)
#plt.gca().add_artist(legend1)
# handles = [plt.plot([],ls="-")[0]],


plt.title('N flow sustainability index, Carbon footprint, Water footprint uncertainty', fontsize=22,fontname="Times New Roman")
plt.xlabel('Relative percentage of N flow change or N accumulation (%)\n Carbon footprint (kg CO$_{2-eq}$)', fontsize=20,fontname="Times New Roman")
plt.ylabel('Magnitude of N flow change or N accumulation (%)\n Water footprint (m$^3$ world eq. water deprived)', fontsize=20,fontname="Times New Roman")

# rotation=45,horizontalalignment="right"
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
mpl.rc('font',family='Times New Roman')

#plt.legend(loc='best', fontsize=16) 
#for i in range(hdi_df.shape[0]):
#          plt.text(hdi_df.gni_pc[i], y=hdi_df.life_ex[i], s=hdi_df.Country[i], alpha=0.8)


# Benchmark Mean values 
plt.axvline(x=0, color='k',linestyle='--', linewidth=1) 
plt.axhline(y=0, color='k', linestyle='--', linewidth=1) 

plt.yscale('log')
plt.yscale('symlog')
plt.xscale('log')
plt.xscale('symlog')


#plt.ylim(ymin = 0)
#panel.ellipse(NFI_IN_x_per, NFI_IN_y_per, lwd = 2, col="green", robust=FALSE,  level=0.65)

# draw standard deviation curve
nstd = 1
ax = plt.subplot(111)

x1=NFI_IN_x_per
y1=NFI_IN_y_per

cov = np.cov(x1, y1)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w1, h1 = 2 * nstd * np.sqrt(vals)
ell1 = Ellipse(xy=(np.mean(x1), np.mean(y1)),
              width=w1, height=h1,
              angle=theta, color='lime', linewidth = 2)
ell1.set_facecolor('none')
ax.add_artist(ell1)

x2=NFI_ON_x_per
y2=NFI_ON_y_per

cov = np.cov(x2, y2)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w2, h2 = 2 * nstd * np.sqrt(vals)
ell2 = Ellipse(xy=(np.mean(x2), np.mean(y2)),
              width=w2, height=h2,
              angle=theta, color='blue')
ell2.set_facecolor('none')
ax.add_artist(ell2)

#sns.scatterplot(x='-7.81376479400426', y='-987.4604094144358',color='yellow',s=100)


x3=NFI_Nacc_x_per
y3=NFI_Nacc_y_per

cov = np.cov(x3, y3)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w3, h3 = 2 * nstd * np.sqrt(vals)
ell3 = Ellipse(xy=(np.mean(x3), np.mean(y3)),
              width=w3, height=h3,
              angle=theta, color='yellow') #,lw=1.5
ell3.set_facecolor('none')
ax.add_artist(ell3)

#ax = sns.regplot(x=x1, y=y1, ci=95,scatter=False,color='lime')
#ax = sns.regplot(x=x2, y=y2, ci=95,scatter=False,color='blue')
#ax = sns.regplot(x=x3, y=y3, ci=95,scatter=False,color='red')



# Percentile and PD scale


#plt.plot([100,0],[0,0],'green')

# NFI_ON_x
connectpoints_ON_x(CNW_PC_LAV,CNW_PC_HAV,CNW_PC_ave,2,3)
plt.scatter(CNW_PC_LAV[2], CNW_PC_ave[3], c = 'deepskyblue', marker = '|', s = 75)
plt.scatter(CNW_PC_P5[2], CNW_PC_ave[3], c = 'deepskyblue', marker = '|', s = 75*2)
#plt.scatter(CNW_PC_P10[0], CNW_PC_ave[1], c = 'green', marker = '|', s = 200)
plt.scatter(CNW_PC_P25[2], CNW_PC_ave[3], c = 'deepskyblue', marker = '|', s = 75*3)
plt.scatter(CNW_PC_P75[2], CNW_PC_ave[3], c = 'deepskyblue', marker = '|', s = 75*3)
#plt.scatter(CNW_PC_P90[0], CNW_PC_ave[1], c = 'green', marker = '|', s = 200)
plt.scatter(CNW_PC_P95[2], CNW_PC_ave[3], c = 'deepskyblue', marker = '|', s = 75*2)
plt.scatter(CNW_PC_HAV[2], CNW_PC_ave[3], c = 'deepskyblue', marker = '|', s = 75)

# NFI_ON_y
connectpoints_ON_y(CNW_PC_LAV,CNW_PC_HAV,CNW_PC_ave,3,2)
plt.scatter(CNW_PC_ave[2],CNW_PC_LAV[3], c = 'deepskyblue', marker = '_', s = 75)
plt.scatter(CNW_PC_ave[2],CNW_PC_P5[3], c = 'deepskyblue', marker = '_', s = 75*2)
#plt.scatter(CNW_PC_P10[0], CNW_PC_ave[1], c = 'green', marker = '|', s = 200)
plt.scatter(CNW_PC_ave[2],CNW_PC_P25[3],  c = 'deepskyblue', marker = '_', s = 75*3)
plt.scatter(CNW_PC_ave[2],CNW_PC_P75[3],  c = 'deepskyblue', marker = '_', s = 75*3)
#plt.scatter(CNW_PC_P90[0], CNW_PC_ave[1], c = 'green', marker = '|', s = 200)
plt.scatter(CNW_PC_ave[2],CNW_PC_P95[3],  c = 'deepskyblue', marker = '_', s = 75*2)
plt.scatter(CNW_PC_ave[2],CNW_PC_HAV[3],  c = 'deepskyblue', marker = '_', s = 75)

# avoid overlayer
# NFI_IN_x
connectpoints_IN_x(CNW_PC_LAV,CNW_PC_HAV,CNW_PC_ave,0,1)
plt.scatter(CNW_PC_LAV[0], CNW_PC_ave[1], c = 'yellowgreen', marker = '|', s = 75)
plt.scatter(CNW_PC_P5[0], CNW_PC_ave[1], c = 'yellowgreen', marker = '|', s = 75*2)
#plt.scatter(CNW_PC_P10[0], CNW_PC_ave[1], c = 'green', marker = '|', s = 200)
plt.scatter(CNW_PC_P25[0], CNW_PC_ave[1], c = 'yellowgreen', marker = '|', s = 75*3)
plt.scatter(CNW_PC_P75[0], CNW_PC_ave[1], c = 'yellowgreen', marker = '|', s = 75*3)
#plt.scatter(CNW_PC_P90[0], CNW_PC_ave[1], c = 'green', marker = '|', s = 200)
plt.scatter(CNW_PC_P95[0], CNW_PC_ave[1], c = 'yellowgreen', marker = '|', s = 75*2)
plt.scatter(CNW_PC_HAV[0], CNW_PC_ave[1], c = 'yellowgreen', marker = '|', s = 75)

# NFI_IN_y
connectpoints_IN_y(CNW_PC_LAV,CNW_PC_HAV,CNW_PC_ave,1,0)
plt.scatter(CNW_PC_ave[0],CNW_PC_LAV[1], c = 'yellowgreen', marker = '_', s = 75)
plt.scatter(CNW_PC_ave[0],CNW_PC_P5[1], c = 'yellowgreen', marker = '_', s = 75*2)
#plt.scatter(CNW_PC_P10[0], CNW_PC_ave[1], c = 'green', marker = '|', s = 200)
plt.scatter(CNW_PC_ave[0],CNW_PC_P25[1],  c = 'yellowgreen', marker = '_', s = 75*3)
plt.scatter(CNW_PC_ave[0],CNW_PC_P75[1],  c = 'yellowgreen', marker = '_', s = 75*3)
#plt.scatter(CNW_PC_P90[0], CNW_PC_ave[1], c = 'green', marker = '|', s = 200)
plt.scatter(CNW_PC_ave[0],CNW_PC_P95[1],  c = 'yellowgreen', marker = '_', s = 75*2)
plt.scatter(CNW_PC_ave[0],CNW_PC_HAV[1],  c = 'yellowgreen', marker = '_', s = 75)

# NFI_Nacc_x
connectpoints_Nacc_x(CNW_PC_LAV,CNW_PC_HAV,CNW_PC_ave,4,5)
plt.scatter(CNW_PC_LAV[4], CNW_PC_ave[5], c = 'orange', marker = '|', s = 75)
plt.scatter(CNW_PC_P5[4], CNW_PC_ave[5], c = 'orange', marker = '|', s = 75*2)
#plt.scatter(CNW_PC_P10[0], CNW_PC_ave[1], c = 'green', marker = '|', s = 200)
plt.scatter(CNW_PC_P25[4], CNW_PC_ave[5], c = 'orange', marker = '|', s = 75*3)
plt.scatter(CNW_PC_P75[4], CNW_PC_ave[5], c = 'orange', marker = '|', s = 75*3)
#plt.scatter(CNW_PC_P90[0], CNW_PC_ave[1], c = 'green', marker = '|', s = 200)
plt.scatter(CNW_PC_P95[4], CNW_PC_ave[5], c = 'orange', marker = '|', s = 75*2)
plt.scatter(CNW_PC_HAV[4], CNW_PC_ave[5], c = 'orange', marker = '|', s = 75)

# NFI_Nacc_y
connectpoints_Nacc_y(CNW_PC_LAV,CNW_PC_HAV,CNW_PC_ave,5,4)
plt.scatter(CNW_PC_ave[4],CNW_PC_LAV[5], c = 'orange', marker = '_', s = 75)
plt.scatter(CNW_PC_ave[4],CNW_PC_P5[5], c = 'orange', marker = '_', s = 75*2)
#plt.scatter(CNW_PC_P10[0], CNW_PC_ave[1], c = 'green', marker = '|', s = 200)
plt.scatter(CNW_PC_ave[4],CNW_PC_P25[5],  c = 'orange', marker = '_', s = 75*3)
plt.scatter(CNW_PC_ave[4],CNW_PC_P75[5],  c = 'orange', marker = '_', s = 75*3)
#plt.scatter(CNW_PC_P90[0], CNW_PC_ave[1], c = 'green', marker = '|', s = 200)
plt.scatter(CNW_PC_ave[4],CNW_PC_P95[5],  c = 'orange', marker = '_', s = 75*2)
plt.scatter(CNW_PC_ave[4],CNW_PC_HAV[5],  c = 'orange', marker = '_', s = 75)


# Carbon_water_y
connectpoints_WF_y(CNW_PC_LAV,CNW_PC_HAV,CNW_PC_ave,7,6)
plt.scatter(CNW_PC_ave[6],CNW_PC_LAV[7], c = 'blueviolet', marker = '_', s = 75)
plt.scatter(CNW_PC_ave[6],CNW_PC_P5[7], c = 'blueviolet', marker = '_', s = 75*2)
#plt.scatter(CNW_PC_P10[0], CNW_PC_ave[1], c = 'green', marker = '|', s = 200)
plt.scatter(CNW_PC_ave[6],CNW_PC_P25[7],  c = 'blueviolet', marker = '_', s = 75*3)
plt.scatter(CNW_PC_ave[6],CNW_PC_P75[7],  c = 'blueviolet', marker = '_', s = 75*3)
#plt.scatter(CNW_PC_P90[0], CNW_PC_ave[1], c = 'green', marker = '|', s = 200)
plt.scatter(CNW_PC_ave[6],CNW_PC_P95[7],  c = 'blueviolet', marker = '_', s = 75*2)
plt.scatter(CNW_PC_ave[6],CNW_PC_HAV[7],  c = 'blueviolet', marker = '_', s = 75)

x4=MC_GWP_scores
y4=MC_water_scores

cov = np.cov(x4, y4)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w4, h4 = 2 * nstd * np.sqrt(vals)
ell4 = Ellipse(xy=(np.mean(x4), np.mean(y4)),
              width=w4, height=h4,
              angle=theta, color='lightpink',lw=2)
ell4.set_facecolor('none')
ax.add_artist(ell4)

# Carbon_water_x
connectpoints_CF_x(CNW_PC_LAV,CNW_PC_HAV,CNW_PC_ave,6,7)
plt.scatter(CNW_PC_LAV[6], CNW_PC_ave[7], c = 'blueviolet', marker = '|', s = 75)
plt.scatter(CNW_PC_P5[6], CNW_PC_ave[7], c = 'blueviolet', marker = '|', s = 75*2)
#plt.scatter(CNW_PC_P10[0], CNW_PC_ave[1], c = 'green', marker = '|', s = 200)
plt.scatter(CNW_PC_P25[6], CNW_PC_ave[7], c = 'blueviolet', marker = '|', s = 75*3)
plt.scatter(CNW_PC_P75[6], CNW_PC_ave[7], c = 'blueviolet', marker = '|', s = 75*3)
#plt.scatter(CNW_PC_P90[0], CNW_PC_ave[1], c = 'green', marker = '|', s = 200)
plt.scatter(CNW_PC_P95[6], CNW_PC_ave[7], c = 'blueviolet', marker = '|', s = 75*2)
plt.scatter(CNW_PC_HAV[6], CNW_PC_ave[7], c = 'blueviolet', marker = '|', s = 75)


# SD curve legend
#plt.legend([ell1, ell2, ell3, ell4], ['Inert nitrogen flow index', 'Reactive nitrogen flow index', 'Nitrogen accumulation index', 'X axis: EF v3.0 climate change total \nY aixs: EF v3.0 deprivation-weighted water consumption'], loc='lower right',
#           scatterpoints=1, bbox_to_anchor=(2.04, -0.23),fontsize=18, title="Color",title_fontsize=18, frameon=False)
#plt.gca().add_artist(legend1)


mpl.rcParams["figure.dpi"] = 900

plt.show()


#output_path = os.path.join('/Users/DELL/学术/N sustainability/bio-jet fuel production',str(NFI_MC_nexus)+'.png')
#plt.savefig()


#### Probability distributions for system NIF and NOF

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import ticker as mticker
import seaborn as sns
import numpy as np
#dataframe.import latex

plt.rcParams['text.usetex'] =  False #     True
plt.rcParams['axes.unicode_minus'] = False

plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['mathtext.it'] = 'STIXGeneral:italic'
plt.rcParams['mathtext.bf'] = 'STIXGeneral:italic:bold'



data = [NIF_ON_scores,NOF_ON_scores]

log_data = [[np.log10(d) for d in row] for row in data]

fig, ax = plt.subplots(ncols=3, figsize=(7, 5), sharey=True)
#plt.rcParams['text.usetex'] = False
sns.violinplot(data=log_data, ax=ax[0]) #, x=['Reactive N input footprint','Reactive N output footprint']
sns.boxplot(data=log_data,   ax=ax[1]) #s=3, 
sns.stripplot(data=log_data, ax=ax[2])
ax[0].yaxis.set_major_formatter(mticker.StrMethodFormatter("$10^{{{x:.0f}}}$"))
ymin, ymax = ax[0].get_ylim()
tick_range = np.arange(np.floor(ymin), ymax)
ax[0].yaxis.set_ticks(tick_range)
ax[0].yaxis.set_ticks([np.log10(x) for p in tick_range for x in np.linspace(10 ** p, 10 ** (p + 1), 10)], minor=True)

ax[0].set_xticklabels(["Nr-N NIF","Nr-N NOF"], fontsize=12.5)
ax[1].set_xticklabels(["Nr-N NIF","Nr-N NOF"], fontsize=12.5)
ax[2].set_xticklabels(["Nr-N NIF","Nr-N NOF"], fontsize=12.5)

ax[0].set_title('Violin plot', fontsize=13.7)
ax[1].set_title('Box plot', fontsize=13.7)
ax[2].set_title('Strip plot', fontsize=13.7)

for a in [ax[0]] : #[ax0, ax1, ax2]:
    for label in (a.get_yticklabels()):
        label.set_fontsize(12)
        label.set_fontfamily('Times New Roman')
        

#plt.yticks(fontsize = 15)
plt.ylabel("System reactive nitrogen input or output footprint (kg N /MJ fuel)", fontsize=12)

fontname = 'Times New Roman'

for ax in ax.flatten():
    labels = ax.get_xticklabels() + ax.get_yticklabels()
    [label.set_fontname(fontname) for label in labels]

mpl.rcParams["figure.dpi"] = 700
    
plt.tight_layout()
plt.show()

### System sustainability indicators sensitivity to life cycle paramaters

In [ ]:
list_parameters()

In [ ]:
para_FTbiojet_list=[TM_corngrain,R_straw,R_cob,N_Straw,R_Straw_return,ER_NO3,ER_NO2_N2,ER_NH3,ER_N2O,CO2_maizep_total,
                     Elec_C,Diesel_C,DR_feed,  ##CR_straw,
                     Elec_milling,R_biojet_straw,P_steel,P_iron,P_cement,P_Al,NiW_catalyst,CoRu_catalyst,Life_FTplant,
                     D_lorry_jetfuel,D_train_jetfuel,CO2_fuelem,CH4_fuelem,CO_fuelem,NOx_fuelem,N2O_fuelem,PM10_fuelem,VOC_fuelem

    #TM_corngrain,R_straw,R_cob,ER_NO2_N2,R_Straw_return，CO2_maizep_total
                    #R_straw
                    #,N_Straw,,ER_NO3,,ER_NH3,ER_N2O,CO2_maizep_total,
                    # Elec_C,Diesel_C,DR_feed,  ##CR_straw,
                    # Elec_milling,R_biojet_straw,P_steel,P_iron,P_cement,P_Al,NiW_catalyst,CoRu_catalyst,Life_FTplant,
                    # D_lorry_jetfuel,D_train_jetfuel,CO2_fuelem,CH4_fuelem,CO_fuelem,NOx_fuelem,N2O_fuelem,PM10_fuelem,VOC_fuelem
                    ]

#TM_corngrain,R_straw,R_cob,N_Straw,R_Straw_return,ER_NO3,ER_NO2_N2,ER_NH3,ER_N2O,CO2_maizep_total,
#                     Elec_C,Diesel_C,DR_feed,  ##CR_straw,
#                     Elec_milling,R_biojet_straw,P_steel,P_iron,P_cement,P_Al,NiW_catalyst,CoRu_catalyst,Life_FTplant,
#                     D_lorry_jetfuel,D_train_jetfuel,CO2_fuelem,CH4_fuelem,CO_fuelem,NOx_fuelem,N2O_fuelem,PM10_fuelem,VOC_fuelem

# incer_stochastic_dashboard(ME_fueltMJ_LCI, biojet_method, var_params=para_FTbiojet_list, n=100000,figsize = (10, 15)) 

In [ ]:
incer_stochastic_dashboard(ME_fueltMJ_LCI, biojet_method, var_params=para_FTbiojet_list, n=40000,figsize = (10, 15)) 

In [ ]:
sobol_simplify_model(ME_fueltMJ_LCI, biojet_method, min_ratio=1,var_params=para_FTbiojet_list, n=40000)

#### Life cycle paramaters sensitivity correlation formula construction

##### All by default

In [ ]:
GWP__R_straw = 0.0125*R_biojet_straw*(0.000114*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 0.00571*Diesel_C + 0.0011*Elec_C + 0.0011*Elec_milling + (CO2_maizep_total + 0.151*R_cob + 0.151*R_straw + 0.419 + 0.0932*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 0.0215
GWP_R_straw_corref = GWP__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
GWP_R_straw_corref

In [ ]:
WF__R_straw = 0.0925*R_biojet_straw/(R_cob + R_straw + 1)

WF_R_straw_corref = WF__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
WF_R_straw_corref

In [ ]:
NIF_IN__R_straw = 8.79e-9*D_train_jetfuel + 0.0125*R_biojet_straw*(0.000354*CoRu_catalyst/R_biojet_straw + 4.4e-7*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 4.08e-5*Diesel_C + 1.38e-6*Elec_C + 1.38e-6*Elec_milling + 4.94e-5*NiW_catalyst/R_biojet_straw + (0.000305*R_cob + 0.000305*R_straw + 0.00161)/(R_cob + R_straw + 1) + 0.00028*P_steel/(DR_feed*Life_FTplant)) - 2.66e-5
NIF_IN__R_straw
NIF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])



In [ ]:
NIF_ON__R_straw = (0.0125*R_biojet_straw*(0.0105*R_cob + 0.0105*R_straw + 0.188) - 0.000276*R_cob - 0.000276*R_straw - 0.000276)/(R_cob + R_straw + 1)

NIF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
#NIF_ON__R_straw

In [ ]:
NOF_IN__R_straw = 0.0125*R_biojet_straw*(2.01e-5*ER_NO2_N2*(0.3*ER_N2O + 0.7) - (ER_NO2_N2 - 1)*(3.67e-5*ER_N2O + 0.000313*R_Straw_return*(ER_N2O - 1) + 0.000398))/(ER_NO2_N2*(0.3*ER_N2O + 0.7)*(R_cob + R_straw + 1))
NOF_IN__R_straw
NOF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_ON__R_straw = 0.452*NOx_fuelem + 0.0125*R_biojet_straw*(2.12e-5*Diesel_C + 1.44e-6*Elec_C + 1.44e-6*Elec_milling + 6.54e-5*NiW_catalyst/R_biojet_straw + 0.000201 + (0.000185*R_cob + 0.000185*R_straw + 0.0023 + 0.0013*(R_Straw_return*(ER_NO3 - 1) + 1)/(0.3*ER_NO3 + 0.7) + 0.000575*(R_Straw_return*(ER_NH3 - 1) + 1)/(0.3*ER_NH3 + 0.7) + 0.000199*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 2.81e-5
NOF_ON__R_straw
NOF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NIF_R_straw = NIF_IN__R_straw + NIF_ON__R_straw
NIF_R_straw
NIF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_R_straw = NOF_IN__R_straw + NOF_ON__R_straw
NOF_R_straw
NOF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
Nacc_R_straw = NIF_R_straw - NOF_R_straw
Nacc_R_straw
Nacc_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NFI_N2m_R_straw = NOF_IN__R_straw - NIF_IN__R_straw

NFI_N2m_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_N2m_R_straw

In [ ]:
NFI_Nrm_R_straw = NOF_ON__R_straw - NIF_ON__R_straw

NFI_Nrm_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_Nrm_R_straw_corref

In [ ]:
NFI_N2_x_MCf_R_straw = NFI_N2m_R_straw/NIF_R_straw

NFI_N2_x_MCf_R_straw_corref=NFI_N2_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_N2_x_MCf_R_straw_corref

In [ ]:
NFI_N2_y_MCf_R_straw = NFI_N2m_R_straw/N_human

NFI_N2_y_MCf_R_straw_corref= NFI_N2_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_N2_y_MCf_R_straw_corref

In [ ]:
NFI_Nr_x_MCf_R_straw = NFI_Nrm_R_straw/NIF_R_straw

NFI_Nr_x_MCf_R_straw_corref = NFI_Nr_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_Nr_x_MCf_R_straw_corref

In [ ]:
NFI_Nr_y_MCf_R_straw = NFI_Nrm_R_straw/N_human

NFI_Nr_y_MCf_R_straw_corref = NFI_Nr_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nr_y_MCf_R_straw_corref

In [ ]:
NFI_Nacc_x_MCf_R_straw = Nacc_R_straw/NIF_R_straw

NFI_Nacc_x_MCf_R_straw_corref = NFI_Nacc_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_x_MCf_R_straw_corref

In [ ]:
NFI_Nacc_y_MCf_R_straw = Nacc_R_straw/N_human

NFI_Nacc_y_MCf_R_straw_corref = NFI_Nacc_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_y_MCf_R_straw_corref

In [ ]:
R_straw_list=[[GWP_R_straw_corref,WF_R_straw_corref],
              [NFI_N2_x_MCf_R_straw_corref,NFI_N2_y_MCf_R_straw_corref],
              [NFI_Nr_x_MCf_R_straw_corref,NFI_Nr_y_MCf_R_straw_corref],
              [NFI_Nacc_x_MCf_R_straw_corref,NFI_Nacc_y_MCf_R_straw_corref],
             ]
R_straw_list

##### R_straw

In [ ]:
GWP__R_straw = 0.0125*R_biojet_straw*(0.000114*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 0.00571*Diesel_C + 0.0011*Elec_C + 0.0011*Elec_milling + (CO2_maizep_total + 0.151*R_cob + 0.151*R_straw + 0.419 + 0.0932*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 0.0215
GWP_R_straw_corref = GWP__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             #(R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
GWP_R_straw_corref

In [ ]:
WF__R_straw = 0.0925*R_biojet_straw/(R_cob + R_straw + 1)

WF_R_straw_corref = WF__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             #(R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
WF_R_straw_corref

In [ ]:
NIF_IN__R_straw = 8.79e-9*D_train_jetfuel + 0.0125*R_biojet_straw*(0.000354*CoRu_catalyst/R_biojet_straw + 4.4e-7*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 4.08e-5*Diesel_C + 1.38e-6*Elec_C + 1.38e-6*Elec_milling + 4.94e-5*NiW_catalyst/R_biojet_straw + (0.000305*R_cob + 0.000305*R_straw + 0.00161)/(R_cob + R_straw + 1) + 0.00028*P_steel/(DR_feed*Life_FTplant)) - 2.66e-5
NIF_IN__R_straw
NIF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             #(R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])



In [ ]:
NIF_ON__R_straw = (0.0125*R_biojet_straw*(0.0105*R_cob + 0.0105*R_straw + 0.188) - 0.000276*R_cob - 0.000276*R_straw - 0.000276)/(R_cob + R_straw + 1)

NIF_ON__R_straw_corref=NIF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             #(R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NIF_ON__R_straw_corref

In [ ]:
NOF_IN__R_straw = 0.0125*R_biojet_straw*(2.01e-5*ER_NO2_N2*(0.3*ER_N2O + 0.7) - (ER_NO2_N2 - 1)*(3.67e-5*ER_N2O + 0.000313*R_Straw_return*(ER_N2O - 1) + 0.000398))/(ER_NO2_N2*(0.3*ER_N2O + 0.7)*(R_cob + R_straw + 1))
NOF_IN__R_straw
NOF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             #(R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_ON__R_straw = 0.452*NOx_fuelem + 0.0125*R_biojet_straw*(2.12e-5*Diesel_C + 1.44e-6*Elec_C + 1.44e-6*Elec_milling + 6.54e-5*NiW_catalyst/R_biojet_straw + 0.000201 + (0.000185*R_cob + 0.000185*R_straw + 0.0023 + 0.0013*(R_Straw_return*(ER_NO3 - 1) + 1)/(0.3*ER_NO3 + 0.7) + 0.000575*(R_Straw_return*(ER_NH3 - 1) + 1)/(0.3*ER_NH3 + 0.7) + 0.000199*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 2.81e-5
NOF_ON__R_straw
NOF_ON__R_straw_corref=NOF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             #(R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NOF_ON__R_straw_corref

In [ ]:
NIF_R_straw = NIF_IN__R_straw + NIF_ON__R_straw
NIF_R_straw
NIF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             #(R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_R_straw = NOF_IN__R_straw + NOF_ON__R_straw
NOF_R_straw
NOF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             #(R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
Nacc_R_straw = NIF_R_straw - NOF_R_straw
Nacc_R_straw
Nacc_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             #(R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NFI_N2m_R_straw = NOF_IN__R_straw - NIF_IN__R_straw

NFI_N2m_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             #(R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_N2m_R_straw

In [ ]:
NFI_Nrm_R_straw = NOF_ON__R_straw - NIF_ON__R_straw

NFI_Nrm_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             #(R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_Nrm_R_straw_corref

In [ ]:
NFI_N2_x_MCf_R_straw = NFI_N2m_R_straw/NIF_R_straw

NFI_N2_x_MCf_R_straw_corref=NFI_N2_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             #(R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_N2_x_MCf_R_straw_corref

In [ ]:
NFI_N2_y_MCf_R_straw = NFI_N2m_R_straw/N_human

NFI_N2_y_MCf_R_straw_corref= NFI_N2_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             #(R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_N2_y_MCf_R_straw_corref

In [ ]:
NFI_Nr_x_MCf_R_straw = NFI_Nrm_R_straw/NIF_R_straw

NFI_Nr_x_MCf_R_straw_corref = NFI_Nr_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             #(R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_Nr_x_MCf_R_straw_corref

In [ ]:
NFI_Nr_y_MCf_R_straw = NFI_Nrm_R_straw/N_human

NFI_Nr_y_MCf_R_straw_corref = NFI_Nr_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             #(R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nr_y_MCf_R_straw_corref

In [ ]:
NFI_Nacc_x_MCf_R_straw = Nacc_R_straw/NIF_R_straw

NFI_Nacc_x_MCf_R_straw_corref = NFI_Nacc_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             #(R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_x_MCf_R_straw_corref

In [ ]:
NFI_Nacc_y_MCf_R_straw = Nacc_R_straw/N_human

NFI_Nacc_y_MCf_R_straw_corref = NFI_Nacc_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             #(R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_y_MCf_R_straw_corref

In [ ]:
R_straw_list=[[GWP_R_straw_corref,WF_R_straw_corref],
              [NFI_N2_x_MCf_R_straw_corref,NFI_N2_y_MCf_R_straw_corref],
              [NFI_Nr_x_MCf_R_straw_corref,NFI_Nr_y_MCf_R_straw_corref],
              [NFI_Nacc_x_MCf_R_straw_corref,NFI_Nacc_y_MCf_R_straw_corref],
             ]
R_straw_list

In [ ]:
R_straw_ON_list=[NIF_ON__R_straw_corref,NOF_ON__R_straw_corref]
R_straw_ON_list

##### R_cob

In [ ]:
GWP__R_straw = 0.0125*R_biojet_straw*(0.000114*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 0.00571*Diesel_C + 0.0011*Elec_C + 0.0011*Elec_milling + (CO2_maizep_total + 0.151*R_cob + 0.151*R_straw + 0.419 + 0.0932*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 0.0215
GWP_R_straw_corref = GWP__R_straw.subs([
             #(R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
GWP_R_straw_corref

In [ ]:
WF__R_straw = 0.0925*R_biojet_straw/(R_cob + R_straw + 1)

WF_R_straw_corref = WF__R_straw.subs([
             #(R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
WF_R_straw_corref

In [ ]:
NIF_IN__R_straw = 8.79e-9*D_train_jetfuel + 0.0125*R_biojet_straw*(0.000354*CoRu_catalyst/R_biojet_straw + 4.4e-7*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 4.08e-5*Diesel_C + 1.38e-6*Elec_C + 1.38e-6*Elec_milling + 4.94e-5*NiW_catalyst/R_biojet_straw + (0.000305*R_cob + 0.000305*R_straw + 0.00161)/(R_cob + R_straw + 1) + 0.00028*P_steel/(DR_feed*Life_FTplant)) - 2.66e-5
NIF_IN__R_straw
NIF_IN__R_straw.subs([
             #(R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])



In [ ]:
NIF_ON__R_straw = (0.0125*R_biojet_straw*(0.0105*R_cob + 0.0105*R_straw + 0.188) - 0.000276*R_cob - 0.000276*R_straw - 0.000276)/(R_cob + R_straw + 1)

NIF_ON__R_straw_corref=NIF_ON__R_straw.subs([
             #(R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NIF_ON__R_straw_corref

In [ ]:
NOF_IN__R_straw = 0.0125*R_biojet_straw*(2.01e-5*ER_NO2_N2*(0.3*ER_N2O + 0.7) - (ER_NO2_N2 - 1)*(3.67e-5*ER_N2O + 0.000313*R_Straw_return*(ER_N2O - 1) + 0.000398))/(ER_NO2_N2*(0.3*ER_N2O + 0.7)*(R_cob + R_straw + 1))
NOF_IN__R_straw
NOF_IN__R_straw.subs([
             #(R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_ON__R_straw = 0.452*NOx_fuelem + 0.0125*R_biojet_straw*(2.12e-5*Diesel_C + 1.44e-6*Elec_C + 1.44e-6*Elec_milling + 6.54e-5*NiW_catalyst/R_biojet_straw + 0.000201 + (0.000185*R_cob + 0.000185*R_straw + 0.0023 + 0.0013*(R_Straw_return*(ER_NO3 - 1) + 1)/(0.3*ER_NO3 + 0.7) + 0.000575*(R_Straw_return*(ER_NH3 - 1) + 1)/(0.3*ER_NH3 + 0.7) + 0.000199*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 2.81e-5
NOF_ON__R_straw
NOF_ON__R_straw_corref=NOF_ON__R_straw.subs([
             #(R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NOF_ON__R_straw_corref

In [ ]:
NIF_R_straw = NIF_IN__R_straw + NIF_ON__R_straw
NIF_R_straw
NIF_R_straw.subs([
             #(R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_R_straw = NOF_IN__R_straw + NOF_ON__R_straw
NOF_R_straw
NOF_R_straw.subs([
             #(R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
Nacc_R_straw = NIF_R_straw - NOF_R_straw
Nacc_R_straw
Nacc_R_straw.subs([
             #(R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NFI_N2m_R_straw = NOF_IN__R_straw - NIF_IN__R_straw

NFI_N2m_R_straw.subs([
             #(R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_N2m_R_straw

In [ ]:
NFI_Nrm_R_straw = NOF_ON__R_straw - NIF_ON__R_straw

NFI_Nrm_R_straw.subs([
             #(R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_Nrm_R_straw_corref

In [ ]:
NFI_N2_x_MCf_R_straw = NFI_N2m_R_straw/NIF_R_straw

NFI_N2_x_MCf_R_straw_corref=NFI_N2_x_MCf_R_straw.subs([
             #(R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_N2_x_MCf_R_straw_corref

In [ ]:
NFI_N2_y_MCf_R_straw = NFI_N2m_R_straw/N_human

NFI_N2_y_MCf_R_straw_corref= NFI_N2_y_MCf_R_straw.subs([
             #(R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_N2_y_MCf_R_straw_corref

In [ ]:
NFI_Nr_x_MCf_R_straw = NFI_Nrm_R_straw/NIF_R_straw

NFI_Nr_x_MCf_R_straw_corref = NFI_Nr_x_MCf_R_straw.subs([
             #(R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_Nr_x_MCf_R_straw_corref

In [ ]:
NFI_Nr_y_MCf_R_straw = NFI_Nrm_R_straw/N_human

NFI_Nr_y_MCf_R_straw_corref = NFI_Nr_y_MCf_R_straw.subs([
             #(R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nr_y_MCf_R_straw_corref

In [ ]:
NFI_Nacc_x_MCf_R_straw = Nacc_R_straw/NIF_R_straw

NFI_Nacc_x_MCf_R_straw_corref = NFI_Nacc_x_MCf_R_straw.subs([
             #(R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_x_MCf_R_straw_corref

In [ ]:
NFI_Nacc_y_MCf_R_straw = Nacc_R_straw/N_human

NFI_Nacc_y_MCf_R_straw_corref = NFI_Nacc_y_MCf_R_straw.subs([
             #(R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_y_MCf_R_straw_corref

In [ ]:
R_cob_list=[[GWP_R_straw_corref,WF_R_straw_corref],
              [NFI_N2_x_MCf_R_straw_corref,NFI_N2_y_MCf_R_straw_corref],
              [NFI_Nr_x_MCf_R_straw_corref,NFI_Nr_y_MCf_R_straw_corref],
              [NFI_Nacc_x_MCf_R_straw_corref,NFI_Nacc_y_MCf_R_straw_corref],
             ]
R_cob_list

In [ ]:
R_cob_ON_list=[NIF_ON__R_straw_corref,NOF_ON__R_straw_corref]
R_cob_ON_list

##### ER_NO2_N2

In [ ]:
GWP__R_straw = 0.0125*R_biojet_straw*(0.000114*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 0.00571*Diesel_C + 0.0011*Elec_C + 0.0011*Elec_milling + (CO2_maizep_total + 0.151*R_cob + 0.151*R_straw + 0.419 + 0.0932*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 0.0215
GWP_R_straw_corref = GWP__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             #(ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
GWP_R_straw_corref

In [ ]:
WF__R_straw = 0.0925*R_biojet_straw/(R_cob + R_straw + 1)

WF_R_straw_corref = WF__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             #(ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
WF_R_straw_corref

In [ ]:
NIF_IN__R_straw = 8.79e-9*D_train_jetfuel + 0.0125*R_biojet_straw*(0.000354*CoRu_catalyst/R_biojet_straw + 4.4e-7*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 4.08e-5*Diesel_C + 1.38e-6*Elec_C + 1.38e-6*Elec_milling + 4.94e-5*NiW_catalyst/R_biojet_straw + (0.000305*R_cob + 0.000305*R_straw + 0.00161)/(R_cob + R_straw + 1) + 0.00028*P_steel/(DR_feed*Life_FTplant)) - 2.66e-5
NIF_IN__R_straw
NIF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             #(ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])



In [ ]:
NIF_ON__R_straw = (0.0125*R_biojet_straw*(0.0105*R_cob + 0.0105*R_straw + 0.188) - 0.000276*R_cob - 0.000276*R_straw - 0.000276)/(R_cob + R_straw + 1)

NIF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             #(ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
#NIF_ON__R_straw

In [ ]:
NOF_IN__R_straw = 0.0125*R_biojet_straw*(2.01e-5*ER_NO2_N2*(0.3*ER_N2O + 0.7) - (ER_NO2_N2 - 1)*(3.67e-5*ER_N2O + 0.000313*R_Straw_return*(ER_N2O - 1) + 0.000398))/(ER_NO2_N2*(0.3*ER_N2O + 0.7)*(R_cob + R_straw + 1))
NOF_IN__R_straw
NOF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             #(ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_ON__R_straw = 0.452*NOx_fuelem + 0.0125*R_biojet_straw*(2.12e-5*Diesel_C + 1.44e-6*Elec_C + 1.44e-6*Elec_milling + 6.54e-5*NiW_catalyst/R_biojet_straw + 0.000201 + (0.000185*R_cob + 0.000185*R_straw + 0.0023 + 0.0013*(R_Straw_return*(ER_NO3 - 1) + 1)/(0.3*ER_NO3 + 0.7) + 0.000575*(R_Straw_return*(ER_NH3 - 1) + 1)/(0.3*ER_NH3 + 0.7) + 0.000199*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 2.81e-5
NOF_ON__R_straw
NOF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             #(ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NIF_R_straw = NIF_IN__R_straw + NIF_ON__R_straw
NIF_R_straw
NIF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             #(ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_R_straw = NOF_IN__R_straw + NOF_ON__R_straw
NOF_R_straw
NOF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             #(ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
Nacc_R_straw = NIF_R_straw - NOF_R_straw
Nacc_R_straw
Nacc_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             #(ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NFI_N2m_R_straw = NOF_IN__R_straw - NIF_IN__R_straw

NFI_N2m_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             #(ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_N2m_R_straw

In [ ]:
NFI_Nrm_R_straw = NOF_ON__R_straw - NIF_ON__R_straw

NFI_Nrm_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             #(ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_Nrm_R_straw_corref

In [ ]:
NFI_N2_x_MCf_R_straw = NFI_N2m_R_straw/NIF_R_straw

NFI_N2_x_MCf_R_straw_corref=NFI_N2_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             #(ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_N2_x_MCf_R_straw_corref

In [ ]:
NFI_N2_y_MCf_R_straw = NFI_N2m_R_straw/N_human

NFI_N2_y_MCf_R_straw_corref= NFI_N2_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             #(ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_N2_y_MCf_R_straw_corref

In [ ]:
NFI_Nr_x_MCf_R_straw = NFI_Nrm_R_straw/NIF_R_straw

NFI_Nr_x_MCf_R_straw_corref = NFI_Nr_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             #(ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_Nr_x_MCf_R_straw_corref

In [ ]:
NFI_Nr_y_MCf_R_straw = NFI_Nrm_R_straw/N_human

NFI_Nr_y_MCf_R_straw_corref = NFI_Nr_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             #(ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nr_y_MCf_R_straw_corref

In [ ]:
NFI_Nacc_x_MCf_R_straw = Nacc_R_straw/NIF_R_straw

NFI_Nacc_x_MCf_R_straw_corref = NFI_Nacc_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             #(ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_x_MCf_R_straw_corref

In [ ]:
NFI_Nacc_y_MCf_R_straw = Nacc_R_straw/N_human

NFI_Nacc_y_MCf_R_straw_corref = NFI_Nacc_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             #(ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_y_MCf_R_straw_corref

In [ ]:
ER_NO2_N2_list=[[GWP_R_straw_corref,WF_R_straw_corref],
              [NFI_N2_x_MCf_R_straw_corref,NFI_N2_y_MCf_R_straw_corref],
              [NFI_Nr_x_MCf_R_straw_corref,NFI_Nr_y_MCf_R_straw_corref],
              [NFI_Nacc_x_MCf_R_straw_corref,NFI_Nacc_y_MCf_R_straw_corref],
             ]
ER_NO2_N2_list

##### R_Straw_return

In [ ]:
GWP__R_straw = 0.0125*R_biojet_straw*(0.000114*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 0.00571*Diesel_C + 0.0011*Elec_C + 0.0011*Elec_milling + (CO2_maizep_total + 0.151*R_cob + 0.151*R_straw + 0.419 + 0.0932*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 0.0215
GWP_R_straw_corref = GWP__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
GWP_R_straw_corref

In [ ]:
WF__R_straw = 0.0925*R_biojet_straw/(R_cob + R_straw + 1)

WF_R_straw_corref = WF__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
WF_R_straw_corref

In [ ]:
NIF_IN__R_straw = 8.79e-9*D_train_jetfuel + 0.0125*R_biojet_straw*(0.000354*CoRu_catalyst/R_biojet_straw + 4.4e-7*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 4.08e-5*Diesel_C + 1.38e-6*Elec_C + 1.38e-6*Elec_milling + 4.94e-5*NiW_catalyst/R_biojet_straw + (0.000305*R_cob + 0.000305*R_straw + 0.00161)/(R_cob + R_straw + 1) + 0.00028*P_steel/(DR_feed*Life_FTplant)) - 2.66e-5
NIF_IN__R_straw
NIF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])



In [ ]:
NIF_ON__R_straw = (0.0125*R_biojet_straw*(0.0105*R_cob + 0.0105*R_straw + 0.188) - 0.000276*R_cob - 0.000276*R_straw - 0.000276)/(R_cob + R_straw + 1)

NIF_ON__R_straw_corref=NIF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NIF_ON__R_straw_corref

In [ ]:
NOF_IN__R_straw = 0.0125*R_biojet_straw*(2.01e-5*ER_NO2_N2*(0.3*ER_N2O + 0.7) - (ER_NO2_N2 - 1)*(3.67e-5*ER_N2O + 0.000313*R_Straw_return*(ER_N2O - 1) + 0.000398))/(ER_NO2_N2*(0.3*ER_N2O + 0.7)*(R_cob + R_straw + 1))
NOF_IN__R_straw
NOF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_ON__R_straw = 0.452*NOx_fuelem + 0.0125*R_biojet_straw*(2.12e-5*Diesel_C + 1.44e-6*Elec_C + 1.44e-6*Elec_milling + 6.54e-5*NiW_catalyst/R_biojet_straw + 0.000201 + (0.000185*R_cob + 0.000185*R_straw + 0.0023 + 0.0013*(R_Straw_return*(ER_NO3 - 1) + 1)/(0.3*ER_NO3 + 0.7) + 0.000575*(R_Straw_return*(ER_NH3 - 1) + 1)/(0.3*ER_NH3 + 0.7) + 0.000199*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 2.81e-5
NOF_ON__R_straw
NOF_ON__R_straw_corref=NOF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NOF_ON__R_straw_corref

In [ ]:
NIF_R_straw = NIF_IN__R_straw + NIF_ON__R_straw
NIF_R_straw
NIF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_R_straw = NOF_IN__R_straw + NOF_ON__R_straw
NOF_R_straw
NOF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
Nacc_R_straw = NIF_R_straw - NOF_R_straw
Nacc_R_straw
Nacc_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NFI_N2m_R_straw = NOF_IN__R_straw - NIF_IN__R_straw

NFI_N2m_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_N2m_R_straw

In [ ]:
NFI_Nrm_R_straw = NOF_ON__R_straw - NIF_ON__R_straw

NFI_Nrm_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_Nrm_R_straw_corref

In [ ]:
NFI_N2_x_MCf_R_straw = NFI_N2m_R_straw/NIF_R_straw

NFI_N2_x_MCf_R_straw_corref=NFI_N2_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_N2_x_MCf_R_straw_corref

In [ ]:
NFI_N2_y_MCf_R_straw = NFI_N2m_R_straw/N_human

NFI_N2_y_MCf_R_straw_corref= NFI_N2_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_N2_y_MCf_R_straw_corref

In [ ]:
NFI_Nr_x_MCf_R_straw = NFI_Nrm_R_straw/NIF_R_straw

NFI_Nr_x_MCf_R_straw_corref = NFI_Nr_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_Nr_x_MCf_R_straw_corref

In [ ]:
NFI_Nr_y_MCf_R_straw = NFI_Nrm_R_straw/N_human

NFI_Nr_y_MCf_R_straw_corref = NFI_Nr_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nr_y_MCf_R_straw_corref

In [ ]:
NFI_Nacc_x_MCf_R_straw = Nacc_R_straw/NIF_R_straw

NFI_Nacc_x_MCf_R_straw_corref = NFI_Nacc_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_x_MCf_R_straw_corref

In [ ]:
NFI_Nacc_y_MCf_R_straw = Nacc_R_straw/N_human

NFI_Nacc_y_MCf_R_straw_corref = NFI_Nacc_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_y_MCf_R_straw_corref

In [ ]:
R_Straw_return_list=[[GWP_R_straw_corref,WF_R_straw_corref],
              [NFI_N2_x_MCf_R_straw_corref,NFI_N2_y_MCf_R_straw_corref],
              [NFI_Nr_x_MCf_R_straw_corref,NFI_Nr_y_MCf_R_straw_corref],
              [NFI_Nacc_x_MCf_R_straw_corref,NFI_Nacc_y_MCf_R_straw_corref],
             ]
R_Straw_return_list

In [ ]:
R_Straw_return_ON_list=[NIF_ON__R_straw,NOF_ON__R_straw]
R_Straw_return_ON_list

In [ ]:
R_Straw_return_ON_list=[NIF_ON__R_straw_corref,NOF_ON__R_straw_corref]
R_Straw_return_ON_list

##### CO2_maizep_total

In [ ]:
GWP__R_straw = 0.0125*R_biojet_straw*(0.000114*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 0.00571*Diesel_C + 0.0011*Elec_C + 0.0011*Elec_milling + (CO2_maizep_total + 0.151*R_cob + 0.151*R_straw + 0.419 + 0.0932*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 0.0215
GWP_R_straw_corref = GWP__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             #(CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
GWP_R_straw_corref

In [ ]:
WF__R_straw = 0.0925*R_biojet_straw/(R_cob + R_straw + 1)

WF_R_straw_corref = WF__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
WF_R_straw_corref

In [ ]:
NIF_IN__R_straw = 8.79e-9*D_train_jetfuel + 0.0125*R_biojet_straw*(0.000354*CoRu_catalyst/R_biojet_straw + 4.4e-7*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 4.08e-5*Diesel_C + 1.38e-6*Elec_C + 1.38e-6*Elec_milling + 4.94e-5*NiW_catalyst/R_biojet_straw + (0.000305*R_cob + 0.000305*R_straw + 0.00161)/(R_cob + R_straw + 1) + 0.00028*P_steel/(DR_feed*Life_FTplant)) - 2.66e-5
NIF_IN__R_straw
NIF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])



In [ ]:
NIF_ON__R_straw = (0.0125*R_biojet_straw*(0.0105*R_cob + 0.0105*R_straw + 0.188) - 0.000276*R_cob - 0.000276*R_straw - 0.000276)/(R_cob + R_straw + 1)

NIF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
#NIF_ON__R_straw

In [ ]:
NOF_IN__R_straw = 0.0125*R_biojet_straw*(2.01e-5*ER_NO2_N2*(0.3*ER_N2O + 0.7) - (ER_NO2_N2 - 1)*(3.67e-5*ER_N2O + 0.000313*R_Straw_return*(ER_N2O - 1) + 0.000398))/(ER_NO2_N2*(0.3*ER_N2O + 0.7)*(R_cob + R_straw + 1))
NOF_IN__R_straw
NOF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_ON__R_straw = 0.452*NOx_fuelem + 0.0125*R_biojet_straw*(2.12e-5*Diesel_C + 1.44e-6*Elec_C + 1.44e-6*Elec_milling + 6.54e-5*NiW_catalyst/R_biojet_straw + 0.000201 + (0.000185*R_cob + 0.000185*R_straw + 0.0023 + 0.0013*(R_Straw_return*(ER_NO3 - 1) + 1)/(0.3*ER_NO3 + 0.7) + 0.000575*(R_Straw_return*(ER_NH3 - 1) + 1)/(0.3*ER_NH3 + 0.7) + 0.000199*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 2.81e-5
NOF_ON__R_straw
NOF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NIF_R_straw = NIF_IN__R_straw + NIF_ON__R_straw
NIF_R_straw
NIF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_R_straw = NOF_IN__R_straw + NOF_ON__R_straw
NOF_R_straw
NOF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
Nacc_R_straw = NIF_R_straw - NOF_R_straw
Nacc_R_straw
Nacc_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NFI_N2m_R_straw = NOF_IN__R_straw - NIF_IN__R_straw

NFI_N2m_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_N2m_R_straw

In [ ]:
NFI_Nrm_R_straw = NOF_ON__R_straw - NIF_ON__R_straw

NFI_Nrm_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_Nrm_R_straw_corref

In [ ]:
NFI_N2_x_MCf_R_straw = NFI_N2m_R_straw/NIF_R_straw

NFI_N2_x_MCf_R_straw_corref=NFI_N2_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_N2_x_MCf_R_straw_corref

In [ ]:
NFI_N2_y_MCf_R_straw = NFI_N2m_R_straw/N_human

NFI_N2_y_MCf_R_straw_corref= NFI_N2_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_N2_y_MCf_R_straw_corref

In [ ]:
NFI_Nr_x_MCf_R_straw = NFI_Nrm_R_straw/NIF_R_straw

NFI_Nr_x_MCf_R_straw_corref = NFI_Nr_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_Nr_x_MCf_R_straw_corref

In [ ]:
NFI_Nr_y_MCf_R_straw = NFI_Nrm_R_straw/N_human

NFI_Nr_y_MCf_R_straw_corref = NFI_Nr_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nr_y_MCf_R_straw_corref

In [ ]:
NFI_Nacc_x_MCf_R_straw = Nacc_R_straw/NIF_R_straw

NFI_Nacc_x_MCf_R_straw_corref = NFI_Nacc_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_x_MCf_R_straw_corref

In [ ]:
NFI_Nacc_y_MCf_R_straw = Nacc_R_straw/N_human

NFI_Nacc_y_MCf_R_straw_corref = NFI_Nacc_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             #(R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_y_MCf_R_straw_corref

In [ ]:
CO2_maizep_total_list=[[GWP_R_straw_corref,0],
              [0,0],
              [0,0],
              [0,0],
             ]
CO2_maizep_total_list

##### Elec_C

In [ ]:
GWP__R_straw = 0.0125*R_biojet_straw*(0.000114*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 0.00571*Diesel_C + 0.0011*Elec_C + 0.0011*Elec_milling + (CO2_maizep_total + 0.151*R_cob + 0.151*R_straw + 0.419 + 0.0932*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 0.0215
GWP_R_straw_corref = GWP__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             #(Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
GWP_R_straw_corref

In [ ]:
WF__R_straw = 0.0925*R_biojet_straw/(R_cob + R_straw + 1)

WF_R_straw_corref = WF__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             #(Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
WF_R_straw_corref

In [ ]:
NIF_IN__R_straw = 8.79e-9*D_train_jetfuel + 0.0125*R_biojet_straw*(0.000354*CoRu_catalyst/R_biojet_straw + 4.4e-7*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 4.08e-5*Diesel_C + 1.38e-6*Elec_C + 1.38e-6*Elec_milling + 4.94e-5*NiW_catalyst/R_biojet_straw + (0.000305*R_cob + 0.000305*R_straw + 0.00161)/(R_cob + R_straw + 1) + 0.00028*P_steel/(DR_feed*Life_FTplant)) - 2.66e-5
NIF_IN__R_straw
NIF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             #(Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])



In [ ]:
NIF_ON__R_straw = (0.0125*R_biojet_straw*(0.0105*R_cob + 0.0105*R_straw + 0.188) - 0.000276*R_cob - 0.000276*R_straw - 0.000276)/(R_cob + R_straw + 1)

NIF_ON__R_straw_corref=NIF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             #(Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NIF_ON__R_straw_corref

In [ ]:
NOF_IN__R_straw = 0.0125*R_biojet_straw*(2.01e-5*ER_NO2_N2*(0.3*ER_N2O + 0.7) - (ER_NO2_N2 - 1)*(3.67e-5*ER_N2O + 0.000313*R_Straw_return*(ER_N2O - 1) + 0.000398))/(ER_NO2_N2*(0.3*ER_N2O + 0.7)*(R_cob + R_straw + 1))
NOF_IN__R_straw
NOF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             #(Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_ON__R_straw = 0.452*NOx_fuelem + 0.0125*R_biojet_straw*(2.12e-5*Diesel_C + 1.44e-6*Elec_C + 1.44e-6*Elec_milling + 6.54e-5*NiW_catalyst/R_biojet_straw + 0.000201 + (0.000185*R_cob + 0.000185*R_straw + 0.0023 + 0.0013*(R_Straw_return*(ER_NO3 - 1) + 1)/(0.3*ER_NO3 + 0.7) + 0.000575*(R_Straw_return*(ER_NH3 - 1) + 1)/(0.3*ER_NH3 + 0.7) + 0.000199*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 2.81e-5
NOF_ON__R_straw
NOF_ON__R_straw_corref=NOF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             #(Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NOF_ON__R_straw_corref

In [ ]:
NIF_R_straw = NIF_IN__R_straw + NIF_ON__R_straw
NIF_R_straw
NIF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             #(Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_R_straw = NOF_IN__R_straw + NOF_ON__R_straw
NOF_R_straw
NOF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             #(Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
Nacc_R_straw = NIF_R_straw - NOF_R_straw
Nacc_R_straw
Nacc_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             #(Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NFI_N2m_R_straw = NOF_IN__R_straw - NIF_IN__R_straw

NFI_N2m_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             #(Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_N2m_R_straw

In [ ]:
NFI_Nrm_R_straw = NOF_ON__R_straw - NIF_ON__R_straw

NFI_Nrm_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             #(Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_Nrm_R_straw_corref

In [ ]:
NFI_N2_x_MCf_R_straw = NFI_N2m_R_straw/NIF_R_straw

NFI_N2_x_MCf_R_straw_corref=NFI_N2_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             #(Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_N2_x_MCf_R_straw_corref

In [ ]:
NFI_N2_y_MCf_R_straw = NFI_N2m_R_straw/N_human

NFI_N2_y_MCf_R_straw_corref= NFI_N2_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             #(Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_N2_y_MCf_R_straw_corref

In [ ]:
NFI_Nr_x_MCf_R_straw = NFI_Nrm_R_straw/NIF_R_straw

NFI_Nr_x_MCf_R_straw_corref = NFI_Nr_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             #(Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_Nr_x_MCf_R_straw_corref

In [ ]:
NFI_Nr_y_MCf_R_straw = NFI_Nrm_R_straw/N_human

NFI_Nr_y_MCf_R_straw_corref = NFI_Nr_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             #(Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nr_y_MCf_R_straw_corref

In [ ]:
NFI_Nacc_x_MCf_R_straw = Nacc_R_straw/NIF_R_straw

NFI_Nacc_x_MCf_R_straw_corref = NFI_Nacc_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             #(Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_x_MCf_R_straw_corref

In [ ]:
NFI_Nacc_y_MCf_R_straw = Nacc_R_straw/N_human

NFI_Nacc_y_MCf_R_straw_corref = NFI_Nacc_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             #(Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_y_MCf_R_straw_corref

In [ ]:
Elec_C_list=[[GWP_R_straw_corref,WF_R_straw_corref],
              [NFI_N2_x_MCf_R_straw_corref,NFI_N2_y_MCf_R_straw_corref],
              [NFI_Nr_x_MCf_R_straw_corref,NFI_Nr_y_MCf_R_straw_corref],
              [NFI_Nacc_x_MCf_R_straw_corref,NFI_Nacc_y_MCf_R_straw_corref],
             ]
Elec_C_list

In [ ]:
Elec_C_ON_list=[NIF_ON__R_straw_corref,NOF_ON__R_straw_corref]
Elec_C_ON_list

##### Diesel_C

In [ ]:
GWP__R_straw = 0.0125*R_biojet_straw*(0.000114*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 0.00571*Diesel_C + 0.0011*Elec_C + 0.0011*Elec_milling + (CO2_maizep_total + 0.151*R_cob + 0.151*R_straw + 0.419 + 0.0932*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 0.0215
GWP_R_straw_corref = GWP__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             #(Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
GWP_R_straw_corref

In [ ]:
WF__R_straw = 0.0925*R_biojet_straw/(R_cob + R_straw + 1)

WF_R_straw_corref = WF__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             #(Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
WF_R_straw_corref

In [ ]:
NIF_IN__R_straw = 8.79e-9*D_train_jetfuel + 0.0125*R_biojet_straw*(0.000354*CoRu_catalyst/R_biojet_straw + 4.4e-7*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 4.08e-5*Diesel_C + 1.38e-6*Elec_C + 1.38e-6*Elec_milling + 4.94e-5*NiW_catalyst/R_biojet_straw + (0.000305*R_cob + 0.000305*R_straw + 0.00161)/(R_cob + R_straw + 1) + 0.00028*P_steel/(DR_feed*Life_FTplant)) - 2.66e-5
NIF_IN__R_straw
NIF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             #(Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])



In [ ]:
NIF_ON__R_straw = (0.0125*R_biojet_straw*(0.0105*R_cob + 0.0105*R_straw + 0.188) - 0.000276*R_cob - 0.000276*R_straw - 0.000276)/(R_cob + R_straw + 1)

NIF_ON__R_straw_corref=NIF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             #(Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
#NIF_ON__R_straw
NIF_ON__R_straw_corref

In [ ]:
NOF_IN__R_straw = 0.0125*R_biojet_straw*(2.01e-5*ER_NO2_N2*(0.3*ER_N2O + 0.7) - (ER_NO2_N2 - 1)*(3.67e-5*ER_N2O + 0.000313*R_Straw_return*(ER_N2O - 1) + 0.000398))/(ER_NO2_N2*(0.3*ER_N2O + 0.7)*(R_cob + R_straw + 1))
NOF_IN__R_straw
NOF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             #(Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_ON__R_straw = 0.452*NOx_fuelem + 0.0125*R_biojet_straw*(2.12e-5*Diesel_C + 1.44e-6*Elec_C + 1.44e-6*Elec_milling + 6.54e-5*NiW_catalyst/R_biojet_straw + 0.000201 + (0.000185*R_cob + 0.000185*R_straw + 0.0023 + 0.0013*(R_Straw_return*(ER_NO3 - 1) + 1)/(0.3*ER_NO3 + 0.7) + 0.000575*(R_Straw_return*(ER_NH3 - 1) + 1)/(0.3*ER_NH3 + 0.7) + 0.000199*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 2.81e-5
NOF_ON__R_straw
NOF_ON__R_straw_corref=NOF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             #(Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NOF_ON__R_straw_corref

In [ ]:
NIF_R_straw = NIF_IN__R_straw + NIF_ON__R_straw
NIF_R_straw
NIF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             #(Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_R_straw = NOF_IN__R_straw + NOF_ON__R_straw
NOF_R_straw
NOF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             #(Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
Nacc_R_straw = NIF_R_straw - NOF_R_straw
Nacc_R_straw
Nacc_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             #(Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NFI_N2m_R_straw = NOF_IN__R_straw - NIF_IN__R_straw

NFI_N2m_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             #(Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_N2m_R_straw

In [ ]:
NFI_Nrm_R_straw = NOF_ON__R_straw - NIF_ON__R_straw

NFI_Nrm_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             #(Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_Nrm_R_straw_corref

In [ ]:
NFI_N2_x_MCf_R_straw = NFI_N2m_R_straw/NIF_R_straw

NFI_N2_x_MCf_R_straw_corref=NFI_N2_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             #(Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_N2_x_MCf_R_straw_corref

In [ ]:
NFI_N2_y_MCf_R_straw = NFI_N2m_R_straw/N_human

NFI_N2_y_MCf_R_straw_corref= NFI_N2_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             #(Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_N2_y_MCf_R_straw_corref

In [ ]:
NFI_Nr_x_MCf_R_straw = NFI_Nrm_R_straw/NIF_R_straw

NFI_Nr_x_MCf_R_straw_corref = NFI_Nr_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             #(Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_Nr_x_MCf_R_straw_corref

In [ ]:
NFI_Nr_y_MCf_R_straw = NFI_Nrm_R_straw/N_human

NFI_Nr_y_MCf_R_straw_corref = NFI_Nr_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             #(Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nr_y_MCf_R_straw_corref

In [ ]:
NFI_Nacc_x_MCf_R_straw = Nacc_R_straw/NIF_R_straw

NFI_Nacc_x_MCf_R_straw_corref = NFI_Nacc_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             #(Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_x_MCf_R_straw_corref

In [ ]:
NFI_Nacc_y_MCf_R_straw = Nacc_R_straw/N_human

NFI_Nacc_y_MCf_R_straw_corref = NFI_Nacc_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             #(Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_y_MCf_R_straw_corref

In [ ]:
Diesel_C_list=[[GWP_R_straw_corref,WF_R_straw_corref],
              [NFI_N2_x_MCf_R_straw_corref,NFI_N2_y_MCf_R_straw_corref],
              [NFI_Nr_x_MCf_R_straw_corref,NFI_Nr_y_MCf_R_straw_corref],
              [NFI_Nacc_x_MCf_R_straw_corref,NFI_Nacc_y_MCf_R_straw_corref],
             ]
Diesel_C_list

In [ ]:
Diesel_C_ON_list=[NIF_ON__R_straw_corref,NOF_ON__R_straw_corref]
Diesel_C_ON_list

##### Elec_milling

In [ ]:
GWP__R_straw = 0.0125*R_biojet_straw*(0.000114*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 0.00571*Diesel_C + 0.0011*Elec_C + 0.0011*Elec_milling + (CO2_maizep_total + 0.151*R_cob + 0.151*R_straw + 0.419 + 0.0932*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 0.0215
GWP_R_straw_corref = GWP__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             #(Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
GWP_R_straw_corref

In [ ]:
WF__R_straw = 0.0925*R_biojet_straw/(R_cob + R_straw + 1)

WF_R_straw_corref = WF__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             #(Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
WF_R_straw_corref

In [ ]:
NIF_IN__R_straw = 8.79e-9*D_train_jetfuel + 0.0125*R_biojet_straw*(0.000354*CoRu_catalyst/R_biojet_straw + 4.4e-7*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 4.08e-5*Diesel_C + 1.38e-6*Elec_C + 1.38e-6*Elec_milling + 4.94e-5*NiW_catalyst/R_biojet_straw + (0.000305*R_cob + 0.000305*R_straw + 0.00161)/(R_cob + R_straw + 1) + 0.00028*P_steel/(DR_feed*Life_FTplant)) - 2.66e-5
NIF_IN__R_straw
NIF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             #(Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])



In [ ]:
NIF_ON__R_straw = (0.0125*R_biojet_straw*(0.0105*R_cob + 0.0105*R_straw + 0.188) - 0.000276*R_cob - 0.000276*R_straw - 0.000276)/(R_cob + R_straw + 1)

NIF_ON__R_straw_corref=NIF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             #(Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
#NIF_ON__R_straw
NIF_ON__R_straw_corref

In [ ]:
NOF_IN__R_straw = 0.0125*R_biojet_straw*(2.01e-5*ER_NO2_N2*(0.3*ER_N2O + 0.7) - (ER_NO2_N2 - 1)*(3.67e-5*ER_N2O + 0.000313*R_Straw_return*(ER_N2O - 1) + 0.000398))/(ER_NO2_N2*(0.3*ER_N2O + 0.7)*(R_cob + R_straw + 1))
NOF_IN__R_straw
NOF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             #(Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_ON__R_straw = 0.452*NOx_fuelem + 0.0125*R_biojet_straw*(2.12e-5*Diesel_C + 1.44e-6*Elec_C + 1.44e-6*Elec_milling + 6.54e-5*NiW_catalyst/R_biojet_straw + 0.000201 + (0.000185*R_cob + 0.000185*R_straw + 0.0023 + 0.0013*(R_Straw_return*(ER_NO3 - 1) + 1)/(0.3*ER_NO3 + 0.7) + 0.000575*(R_Straw_return*(ER_NH3 - 1) + 1)/(0.3*ER_NH3 + 0.7) + 0.000199*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 2.81e-5
NOF_ON__R_straw
NOF_ON__R_straw_corref=NOF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             #(Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NOF_ON__R_straw_corref

In [ ]:
NIF_R_straw = NIF_IN__R_straw + NIF_ON__R_straw
NIF_R_straw
NIF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             #(Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_R_straw = NOF_IN__R_straw + NOF_ON__R_straw
NOF_R_straw
NOF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             #(Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
Nacc_R_straw = NIF_R_straw - NOF_R_straw
Nacc_R_straw
Nacc_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             #(Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NFI_N2m_R_straw = NOF_IN__R_straw - NIF_IN__R_straw

NFI_N2m_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             #(Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_N2m_R_straw

In [ ]:
NFI_Nrm_R_straw = NOF_ON__R_straw - NIF_ON__R_straw

NFI_Nrm_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             #(Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_Nrm_R_straw_corref

In [ ]:
NFI_N2_x_MCf_R_straw = NFI_N2m_R_straw/NIF_R_straw

NFI_N2_x_MCf_R_straw_corref=NFI_N2_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             #(Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_N2_x_MCf_R_straw_corref

In [ ]:
NFI_N2_y_MCf_R_straw = NFI_N2m_R_straw/N_human

NFI_N2_y_MCf_R_straw_corref= NFI_N2_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             #(Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_N2_y_MCf_R_straw_corref

In [ ]:
NFI_Nr_x_MCf_R_straw = NFI_Nrm_R_straw/NIF_R_straw

NFI_Nr_x_MCf_R_straw_corref = NFI_Nr_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             #(Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_Nr_x_MCf_R_straw_corref

In [ ]:
NFI_Nr_y_MCf_R_straw = NFI_Nrm_R_straw/N_human

NFI_Nr_y_MCf_R_straw_corref = NFI_Nr_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             #(Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nr_y_MCf_R_straw_corref

In [ ]:
NFI_Nacc_x_MCf_R_straw = Nacc_R_straw/NIF_R_straw

NFI_Nacc_x_MCf_R_straw_corref = NFI_Nacc_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             #(Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_x_MCf_R_straw_corref

In [ ]:
NFI_Nacc_y_MCf_R_straw = Nacc_R_straw/N_human

NFI_Nacc_y_MCf_R_straw_corref = NFI_Nacc_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             #(Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_y_MCf_R_straw_corref

In [ ]:
Elec_milling_list=[[GWP_R_straw_corref,WF_R_straw_corref],
              [NFI_N2_x_MCf_R_straw_corref,NFI_N2_y_MCf_R_straw_corref],
              [NFI_Nr_x_MCf_R_straw_corref,NFI_Nr_y_MCf_R_straw_corref],
              [NFI_Nacc_x_MCf_R_straw_corref,NFI_Nacc_y_MCf_R_straw_corref],
             ]
Elec_milling_list

In [ ]:
Elec_milling_ON_list=[NIF_ON__R_straw_corref,NOF_ON__R_straw_corref]
Elec_milling_ON_list

##### R_biojet_straw

In [ ]:
GWP__R_straw = 0.0125*R_biojet_straw*(0.000114*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 0.00571*Diesel_C + 0.0011*Elec_C + 0.0011*Elec_milling + (CO2_maizep_total + 0.151*R_cob + 0.151*R_straw + 0.419 + 0.0932*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 0.0215
GWP_R_straw_corref = GWP__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             #(R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
GWP_R_straw_corref

In [ ]:
WF__R_straw = 0.0925*R_biojet_straw/(R_cob + R_straw + 1)

WF_R_straw_corref = WF__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             #(R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
WF_R_straw_corref

In [ ]:
NIF_IN__R_straw = 8.79e-9*D_train_jetfuel + 0.0125*R_biojet_straw*(0.000354*CoRu_catalyst/R_biojet_straw + 4.4e-7*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 4.08e-5*Diesel_C + 1.38e-6*Elec_C + 1.38e-6*Elec_milling + 4.94e-5*NiW_catalyst/R_biojet_straw + (0.000305*R_cob + 0.000305*R_straw + 0.00161)/(R_cob + R_straw + 1) + 0.00028*P_steel/(DR_feed*Life_FTplant)) - 2.66e-5
NIF_IN__R_straw
NIF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             #(R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])



In [ ]:
NIF_ON__R_straw = (0.0125*R_biojet_straw*(0.0105*R_cob + 0.0105*R_straw + 0.188) - 0.000276*R_cob - 0.000276*R_straw - 0.000276)/(R_cob + R_straw + 1)

NIF_ON__R_straw_corref=NIF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             #(R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NIF_ON__R_straw_corref

In [ ]:
NOF_IN__R_straw = 0.0125*R_biojet_straw*(2.01e-5*ER_NO2_N2*(0.3*ER_N2O + 0.7) - (ER_NO2_N2 - 1)*(3.67e-5*ER_N2O + 0.000313*R_Straw_return*(ER_N2O - 1) + 0.000398))/(ER_NO2_N2*(0.3*ER_N2O + 0.7)*(R_cob + R_straw + 1))
NOF_IN__R_straw
NOF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             #(R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_ON__R_straw = 0.452*NOx_fuelem + 0.0125*R_biojet_straw*(2.12e-5*Diesel_C + 1.44e-6*Elec_C + 1.44e-6*Elec_milling + 6.54e-5*NiW_catalyst/R_biojet_straw + 0.000201 + (0.000185*R_cob + 0.000185*R_straw + 0.0023 + 0.0013*(R_Straw_return*(ER_NO3 - 1) + 1)/(0.3*ER_NO3 + 0.7) + 0.000575*(R_Straw_return*(ER_NH3 - 1) + 1)/(0.3*ER_NH3 + 0.7) + 0.000199*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 2.81e-5
NOF_ON__R_straw
NOF_ON__R_straw_corref=NOF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             #(R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NOF_ON__R_straw_corref

In [ ]:
NIF_R_straw = NIF_IN__R_straw + NIF_ON__R_straw
NIF_R_straw
NIF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             #(R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_R_straw = NOF_IN__R_straw + NOF_ON__R_straw
NOF_R_straw
NOF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             #(R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
Nacc_R_straw = NIF_R_straw - NOF_R_straw
Nacc_R_straw
Nacc_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             #(R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NFI_N2m_R_straw = NOF_IN__R_straw - NIF_IN__R_straw

NFI_N2m_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             #(R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_N2m_R_straw

In [ ]:
NFI_Nrm_R_straw = NOF_ON__R_straw - NIF_ON__R_straw

NFI_Nrm_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             #(R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_Nrm_R_straw_corref

In [ ]:
NFI_N2_x_MCf_R_straw = NFI_N2m_R_straw/NIF_R_straw

NFI_N2_x_MCf_R_straw_corref=NFI_N2_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             #(R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_N2_x_MCf_R_straw_corref

In [ ]:
NFI_N2_y_MCf_R_straw = NFI_N2m_R_straw/N_human

NFI_N2_y_MCf_R_straw_corref= NFI_N2_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             #(R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_N2_y_MCf_R_straw_corref

In [ ]:
NFI_Nr_x_MCf_R_straw = NFI_Nrm_R_straw/NIF_R_straw

NFI_Nr_x_MCf_R_straw_corref = NFI_Nr_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             #(R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_Nr_x_MCf_R_straw_corref

In [ ]:
NFI_Nr_y_MCf_R_straw = NFI_Nrm_R_straw/N_human

NFI_Nr_y_MCf_R_straw_corref = NFI_Nr_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             #(R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nr_y_MCf_R_straw_corref

In [ ]:
NFI_Nacc_x_MCf_R_straw = Nacc_R_straw/NIF_R_straw

NFI_Nacc_x_MCf_R_straw_corref = NFI_Nacc_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             #(R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_x_MCf_R_straw_corref

In [ ]:
NFI_Nacc_y_MCf_R_straw = Nacc_R_straw/N_human

NFI_Nacc_y_MCf_R_straw_corref = NFI_Nacc_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             #(R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_y_MCf_R_straw_corref

In [ ]:
R_biojet_straw_list=[[GWP_R_straw_corref,WF_R_straw_corref],
              [NFI_N2_x_MCf_R_straw_corref,NFI_N2_y_MCf_R_straw_corref],
              [NFI_Nr_x_MCf_R_straw_corref,NFI_Nr_y_MCf_R_straw_corref],
              [NFI_Nacc_x_MCf_R_straw_corref,NFI_Nacc_y_MCf_R_straw_corref],
             ]
R_biojet_straw_list

In [ ]:
R_biojet_straw_ON_list=[NIF_ON__R_straw_corref,NOF_ON__R_straw_corref]
R_biojet_straw_ON_list

##### CoRu_catalyst

In [ ]:
GWP__R_straw = 0.0125*R_biojet_straw*(0.000114*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 0.00571*Diesel_C + 0.0011*Elec_C + 0.0011*Elec_milling + (CO2_maizep_total + 0.151*R_cob + 0.151*R_straw + 0.419 + 0.0932*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 0.0215
GWP_R_straw_corref = GWP__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             #(CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
GWP_R_straw_corref

In [ ]:
WF__R_straw = 0.0925*R_biojet_straw/(R_cob + R_straw + 1)

WF_R_straw_corref = WF__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             #(CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
WF_R_straw_corref

In [ ]:
NIF_IN__R_straw = 8.79e-9*D_train_jetfuel + 0.0125*R_biojet_straw*(0.000354*CoRu_catalyst/R_biojet_straw + 4.4e-7*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 4.08e-5*Diesel_C + 1.38e-6*Elec_C + 1.38e-6*Elec_milling + 4.94e-5*NiW_catalyst/R_biojet_straw + (0.000305*R_cob + 0.000305*R_straw + 0.00161)/(R_cob + R_straw + 1) + 0.00028*P_steel/(DR_feed*Life_FTplant)) - 2.66e-5
NIF_IN__R_straw
NIF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             #(CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])



In [ ]:
NIF_ON__R_straw = (0.0125*R_biojet_straw*(0.0105*R_cob + 0.0105*R_straw + 0.188) - 0.000276*R_cob - 0.000276*R_straw - 0.000276)/(R_cob + R_straw + 1)

NIF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             #(CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
#NIF_ON__R_straw

In [ ]:
NOF_IN__R_straw = 0.0125*R_biojet_straw*(2.01e-5*ER_NO2_N2*(0.3*ER_N2O + 0.7) - (ER_NO2_N2 - 1)*(3.67e-5*ER_N2O + 0.000313*R_Straw_return*(ER_N2O - 1) + 0.000398))/(ER_NO2_N2*(0.3*ER_N2O + 0.7)*(R_cob + R_straw + 1))
NOF_IN__R_straw
NOF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             #(CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_ON__R_straw = 0.452*NOx_fuelem + 0.0125*R_biojet_straw*(2.12e-5*Diesel_C + 1.44e-6*Elec_C + 1.44e-6*Elec_milling + 6.54e-5*NiW_catalyst/R_biojet_straw + 0.000201 + (0.000185*R_cob + 0.000185*R_straw + 0.0023 + 0.0013*(R_Straw_return*(ER_NO3 - 1) + 1)/(0.3*ER_NO3 + 0.7) + 0.000575*(R_Straw_return*(ER_NH3 - 1) + 1)/(0.3*ER_NH3 + 0.7) + 0.000199*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 2.81e-5
NOF_ON__R_straw
NOF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             #(CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NIF_R_straw = NIF_IN__R_straw + NIF_ON__R_straw
NIF_R_straw
NIF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             #(CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_R_straw = NOF_IN__R_straw + NOF_ON__R_straw
NOF_R_straw
NOF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             #(CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
Nacc_R_straw = NIF_R_straw - NOF_R_straw
Nacc_R_straw
Nacc_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             #(CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NFI_N2m_R_straw = NOF_IN__R_straw - NIF_IN__R_straw

NFI_N2m_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             #(CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_N2m_R_straw

In [ ]:
NFI_Nrm_R_straw = NOF_ON__R_straw - NIF_ON__R_straw

NFI_Nrm_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             #(CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_Nrm_R_straw_corref

In [ ]:
NFI_N2_x_MCf_R_straw = NFI_N2m_R_straw/NIF_R_straw

NFI_N2_x_MCf_R_straw_corref=NFI_N2_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             #(CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_N2_x_MCf_R_straw_corref

In [ ]:
NFI_N2_y_MCf_R_straw = NFI_N2m_R_straw/N_human

NFI_N2_y_MCf_R_straw_corref= NFI_N2_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
            # (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_N2_y_MCf_R_straw_corref

In [ ]:
NFI_Nr_x_MCf_R_straw = NFI_Nrm_R_straw/NIF_R_straw

NFI_Nr_x_MCf_R_straw_corref = NFI_Nr_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             #(CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_Nr_x_MCf_R_straw_corref

In [ ]:
NFI_Nr_y_MCf_R_straw = NFI_Nrm_R_straw/N_human

NFI_Nr_y_MCf_R_straw_corref = NFI_Nr_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             #(CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nr_y_MCf_R_straw_corref

In [ ]:
NFI_Nacc_x_MCf_R_straw = Nacc_R_straw/NIF_R_straw

NFI_Nacc_x_MCf_R_straw_corref = NFI_Nacc_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             #(CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_x_MCf_R_straw_corref

In [ ]:
NFI_Nacc_y_MCf_R_straw = Nacc_R_straw/N_human

NFI_Nacc_y_MCf_R_straw_corref = NFI_Nacc_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             #(CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_y_MCf_R_straw_corref

In [ ]:
CoRu_catalyst_list=[[GWP_R_straw_corref,WF_R_straw_corref],
              [NFI_N2_x_MCf_R_straw_corref,NFI_N2_y_MCf_R_straw_corref],
              [NFI_Nr_x_MCf_R_straw_corref,NFI_Nr_y_MCf_R_straw_corref],
              [NFI_Nacc_x_MCf_R_straw_corref,NFI_Nacc_y_MCf_R_straw_corref],
             ]
CoRu_catalyst_list

##### NOx_fuelem

In [ ]:
GWP__R_straw = 0.0125*R_biojet_straw*(0.000114*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 0.00571*Diesel_C + 0.0011*Elec_C + 0.0011*Elec_milling + (CO2_maizep_total + 0.151*R_cob + 0.151*R_straw + 0.419 + 0.0932*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 0.0215
GWP_R_straw_corref = GWP__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             #(NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
GWP_R_straw_corref

In [ ]:
WF__R_straw = 0.0925*R_biojet_straw/(R_cob + R_straw + 1)

WF_R_straw_corref = WF__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             #(NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
WF_R_straw_corref

In [ ]:
NIF_IN__R_straw = 8.79e-9*D_train_jetfuel + 0.0125*R_biojet_straw*(0.000354*CoRu_catalyst/R_biojet_straw + 4.4e-7*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 4.08e-5*Diesel_C + 1.38e-6*Elec_C + 1.38e-6*Elec_milling + 4.94e-5*NiW_catalyst/R_biojet_straw + (0.000305*R_cob + 0.000305*R_straw + 0.00161)/(R_cob + R_straw + 1) + 0.00028*P_steel/(DR_feed*Life_FTplant)) - 2.66e-5
NIF_IN__R_straw
NIF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             #(NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])



In [ ]:
NIF_ON__R_straw = (0.0125*R_biojet_straw*(0.0105*R_cob + 0.0105*R_straw + 0.188) - 0.000276*R_cob - 0.000276*R_straw - 0.000276)/(R_cob + R_straw + 1)

NIF_ON__R_straw_corref=NIF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             #(NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NIF_ON__R_straw_corref

In [ ]:
NOF_IN__R_straw = 0.0125*R_biojet_straw*(2.01e-5*ER_NO2_N2*(0.3*ER_N2O + 0.7) - (ER_NO2_N2 - 1)*(3.67e-5*ER_N2O + 0.000313*R_Straw_return*(ER_N2O - 1) + 0.000398))/(ER_NO2_N2*(0.3*ER_N2O + 0.7)*(R_cob + R_straw + 1))
NOF_IN__R_straw
NOF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             #(NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_ON__R_straw = 0.452*NOx_fuelem + 0.0125*R_biojet_straw*(2.12e-5*Diesel_C + 1.44e-6*Elec_C + 1.44e-6*Elec_milling + 6.54e-5*NiW_catalyst/R_biojet_straw + 0.000201 + (0.000185*R_cob + 0.000185*R_straw + 0.0023 + 0.0013*(R_Straw_return*(ER_NO3 - 1) + 1)/(0.3*ER_NO3 + 0.7) + 0.000575*(R_Straw_return*(ER_NH3 - 1) + 1)/(0.3*ER_NH3 + 0.7) + 0.000199*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 2.81e-5
NOF_ON__R_straw
NOF_ON__R_straw_corref=NOF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             #(NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NOF_ON__R_straw_corref

In [ ]:
NIF_R_straw = NIF_IN__R_straw + NIF_ON__R_straw
NIF_R_straw
NIF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             #(NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_R_straw = NOF_IN__R_straw + NOF_ON__R_straw
NOF_R_straw
NOF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             #(NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
Nacc_R_straw = NIF_R_straw - NOF_R_straw
Nacc_R_straw
Nacc_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             #(NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NFI_N2m_R_straw = NOF_IN__R_straw - NIF_IN__R_straw

NFI_N2m_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             #(NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_N2m_R_straw

In [ ]:
NFI_Nrm_R_straw = NOF_ON__R_straw - NIF_ON__R_straw

NFI_Nrm_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             #(NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_Nrm_R_straw_corref

In [ ]:
NFI_N2_x_MCf_R_straw = NFI_N2m_R_straw/NIF_R_straw

NFI_N2_x_MCf_R_straw_corref=NFI_N2_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             #(NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_N2_x_MCf_R_straw_corref

In [ ]:
NFI_N2_y_MCf_R_straw = NFI_N2m_R_straw/N_human

NFI_N2_y_MCf_R_straw_corref= NFI_N2_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             #(NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_N2_y_MCf_R_straw_corref

In [ ]:
NFI_Nr_x_MCf_R_straw = NFI_Nrm_R_straw/NIF_R_straw

NFI_Nr_x_MCf_R_straw_corref = NFI_Nr_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             #(NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_Nr_x_MCf_R_straw_corref

In [ ]:
NFI_Nr_y_MCf_R_straw = NFI_Nrm_R_straw/N_human

NFI_Nr_y_MCf_R_straw_corref = NFI_Nr_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             #(NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nr_y_MCf_R_straw_corref

In [ ]:
NFI_Nacc_x_MCf_R_straw = Nacc_R_straw/NIF_R_straw

NFI_Nacc_x_MCf_R_straw_corref = NFI_Nacc_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             #(NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_x_MCf_R_straw_corref

In [ ]:
NFI_Nacc_y_MCf_R_straw = Nacc_R_straw/N_human

NFI_Nacc_y_MCf_R_straw_corref = NFI_Nacc_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             #(NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_y_MCf_R_straw_corref

In [ ]:
NOx_fuelem_list=[[GWP_R_straw_corref,WF_R_straw_corref],
              [NFI_N2_x_MCf_R_straw_corref,NFI_N2_y_MCf_R_straw_corref],
              [NFI_Nr_x_MCf_R_straw_corref,NFI_Nr_y_MCf_R_straw_corref],
              [NFI_Nacc_x_MCf_R_straw_corref,NFI_Nacc_y_MCf_R_straw_corref],
             ]
NOx_fuelem_list

In [ ]:
NOx_fuelem_ON_list=[NIF_ON__R_straw_corref,NOF_ON__R_straw_corref]
NOx_fuelem_ON_list

##### DR_feed

In [ ]:
GWP__R_straw = 0.0125*R_biojet_straw*(0.000114*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 0.00571*Diesel_C + 0.0011*Elec_C + 0.0011*Elec_milling + (CO2_maizep_total + 0.151*R_cob + 0.151*R_straw + 0.419 + 0.0932*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 0.0215
GWP_R_straw_corref = GWP__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             #(DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
GWP_R_straw_corref

In [ ]:
WF__R_straw = 0.0925*R_biojet_straw/(R_cob + R_straw + 1)

WF_R_straw_corref = WF__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             #(DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
WF_R_straw_corref

In [ ]:
NIF_IN__R_straw = 8.79e-9*D_train_jetfuel + 0.0125*R_biojet_straw*(0.000354*CoRu_catalyst/R_biojet_straw + 4.4e-7*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 4.08e-5*Diesel_C + 1.38e-6*Elec_C + 1.38e-6*Elec_milling + 4.94e-5*NiW_catalyst/R_biojet_straw + (0.000305*R_cob + 0.000305*R_straw + 0.00161)/(R_cob + R_straw + 1) + 0.00028*P_steel/(DR_feed*Life_FTplant)) - 2.66e-5
NIF_IN__R_straw
NIF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             #(DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])



In [ ]:
NIF_ON__R_straw = (0.0125*R_biojet_straw*(0.0105*R_cob + 0.0105*R_straw + 0.188) - 0.000276*R_cob - 0.000276*R_straw - 0.000276)/(R_cob + R_straw + 1)

NIF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             #(DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
#NIF_ON__R_straw

In [ ]:
NOF_IN__R_straw = 0.0125*R_biojet_straw*(2.01e-5*ER_NO2_N2*(0.3*ER_N2O + 0.7) - (ER_NO2_N2 - 1)*(3.67e-5*ER_N2O + 0.000313*R_Straw_return*(ER_N2O - 1) + 0.000398))/(ER_NO2_N2*(0.3*ER_N2O + 0.7)*(R_cob + R_straw + 1))
NOF_IN__R_straw
NOF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             #(DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_ON__R_straw = 0.452*NOx_fuelem + 0.0125*R_biojet_straw*(2.12e-5*Diesel_C + 1.44e-6*Elec_C + 1.44e-6*Elec_milling + 6.54e-5*NiW_catalyst/R_biojet_straw + 0.000201 + (0.000185*R_cob + 0.000185*R_straw + 0.0023 + 0.0013*(R_Straw_return*(ER_NO3 - 1) + 1)/(0.3*ER_NO3 + 0.7) + 0.000575*(R_Straw_return*(ER_NH3 - 1) + 1)/(0.3*ER_NH3 + 0.7) + 0.000199*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 2.81e-5
NOF_ON__R_straw
NOF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             #(DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NIF_R_straw = NIF_IN__R_straw + NIF_ON__R_straw
NIF_R_straw
NIF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             #(DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_R_straw = NOF_IN__R_straw + NOF_ON__R_straw
NOF_R_straw
NOF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             #(DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
Nacc_R_straw = NIF_R_straw - NOF_R_straw
Nacc_R_straw
Nacc_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             #(DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NFI_N2m_R_straw = NOF_IN__R_straw - NIF_IN__R_straw

NFI_N2m_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             #(DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_N2m_R_straw

In [ ]:
NFI_Nrm_R_straw = NOF_ON__R_straw - NIF_ON__R_straw

NFI_Nrm_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             #(DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_Nrm_R_straw_corref

In [ ]:
NFI_N2_x_MCf_R_straw = NFI_N2m_R_straw/NIF_R_straw

NFI_N2_x_MCf_R_straw_corref=NFI_N2_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             #(DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_N2_x_MCf_R_straw_corref

In [ ]:
NFI_N2_y_MCf_R_straw = NFI_N2m_R_straw/N_human

NFI_N2_y_MCf_R_straw_corref= NFI_N2_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             #(DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_N2_y_MCf_R_straw_corref

In [ ]:
NFI_Nr_x_MCf_R_straw = NFI_Nrm_R_straw/NIF_R_straw

NFI_Nr_x_MCf_R_straw_corref = NFI_Nr_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             #(DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_Nr_x_MCf_R_straw_corref

In [ ]:
NFI_Nr_y_MCf_R_straw = NFI_Nrm_R_straw/N_human

NFI_Nr_y_MCf_R_straw_corref = NFI_Nr_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             #(DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nr_y_MCf_R_straw_corref

In [ ]:
NFI_Nacc_x_MCf_R_straw = Nacc_R_straw/NIF_R_straw

NFI_Nacc_x_MCf_R_straw_corref = NFI_Nacc_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             #(DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_x_MCf_R_straw_corref

In [ ]:
NFI_Nacc_y_MCf_R_straw = Nacc_R_straw/N_human

NFI_Nacc_y_MCf_R_straw_corref = NFI_Nacc_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             #(DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_y_MCf_R_straw_corref

In [ ]:
DR_feed_list=[[GWP_R_straw_corref,WF_R_straw_corref],
              [NFI_N2_x_MCf_R_straw_corref,NFI_N2_y_MCf_R_straw_corref],
              [NFI_Nr_x_MCf_R_straw_corref,NFI_Nr_y_MCf_R_straw_corref],
              [NFI_Nacc_x_MCf_R_straw_corref,NFI_Nacc_y_MCf_R_straw_corref],
             ]
DR_feed_list

##### ER_N2O

In [ ]:
GWP__R_straw = 0.0125*R_biojet_straw*(0.000114*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 0.00571*Diesel_C + 0.0011*Elec_C + 0.0011*Elec_milling + (CO2_maizep_total + 0.151*R_cob + 0.151*R_straw + 0.419 + 0.0932*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 0.0215
GWP_R_straw_corref = GWP__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             #(ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
GWP_R_straw_corref

In [ ]:
WF__R_straw = 0.0925*R_biojet_straw/(R_cob + R_straw + 1)

WF_R_straw_corref = WF__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             #(ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
WF_R_straw_corref

In [ ]:
NIF_IN__R_straw = 8.79e-9*D_train_jetfuel + 0.0125*R_biojet_straw*(0.000354*CoRu_catalyst/R_biojet_straw + 4.4e-7*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 4.08e-5*Diesel_C + 1.38e-6*Elec_C + 1.38e-6*Elec_milling + 4.94e-5*NiW_catalyst/R_biojet_straw + (0.000305*R_cob + 0.000305*R_straw + 0.00161)/(R_cob + R_straw + 1) + 0.00028*P_steel/(DR_feed*Life_FTplant)) - 2.66e-5
NIF_IN__R_straw
NIF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             #(ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])



In [ ]:
NIF_ON__R_straw = (0.0125*R_biojet_straw*(0.0105*R_cob + 0.0105*R_straw + 0.188) - 0.000276*R_cob - 0.000276*R_straw - 0.000276)/(R_cob + R_straw + 1)

NIF_ON__R_straw_corref=NIF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             #(ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NIF_ON__R_straw_corref

In [ ]:
NOF_IN__R_straw = 0.0125*R_biojet_straw*(2.01e-5*ER_NO2_N2*(0.3*ER_N2O + 0.7) - (ER_NO2_N2 - 1)*(3.67e-5*ER_N2O + 0.000313*R_Straw_return*(ER_N2O - 1) + 0.000398))/(ER_NO2_N2*(0.3*ER_N2O + 0.7)*(R_cob + R_straw + 1))
NOF_IN__R_straw
NOF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             #(ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_ON__R_straw = 0.452*NOx_fuelem + 0.0125*R_biojet_straw*(2.12e-5*Diesel_C + 1.44e-6*Elec_C + 1.44e-6*Elec_milling + 6.54e-5*NiW_catalyst/R_biojet_straw + 0.000201 + (0.000185*R_cob + 0.000185*R_straw + 0.0023 + 0.0013*(R_Straw_return*(ER_NO3 - 1) + 1)/(0.3*ER_NO3 + 0.7) + 0.000575*(R_Straw_return*(ER_NH3 - 1) + 1)/(0.3*ER_NH3 + 0.7) + 0.000199*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 2.81e-5
NOF_ON__R_straw
NOF_ON__R_straw_corref=NOF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             #(ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NOF_ON__R_straw_corref

In [ ]:
NIF_R_straw = NIF_IN__R_straw + NIF_ON__R_straw
NIF_R_straw
NIF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             #(ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_R_straw = NOF_IN__R_straw + NOF_ON__R_straw
NOF_R_straw
NOF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             #(ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
Nacc_R_straw = NIF_R_straw - NOF_R_straw
Nacc_R_straw
Nacc_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             #(ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NFI_N2m_R_straw = NOF_IN__R_straw - NIF_IN__R_straw

NFI_N2m_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             #(ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_N2m_R_straw

In [ ]:
NFI_Nrm_R_straw = NOF_ON__R_straw - NIF_ON__R_straw

NFI_Nrm_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             #(ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_Nrm_R_straw_corref

In [ ]:
NFI_N2_x_MCf_R_straw = NFI_N2m_R_straw/NIF_R_straw

NFI_N2_x_MCf_R_straw_corref=NFI_N2_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             #(ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_N2_x_MCf_R_straw_corref

In [ ]:
NFI_N2_y_MCf_R_straw = NFI_N2m_R_straw/N_human

NFI_N2_y_MCf_R_straw_corref= NFI_N2_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             #(ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_N2_y_MCf_R_straw_corref

In [ ]:
NFI_Nr_x_MCf_R_straw = NFI_Nrm_R_straw/NIF_R_straw

NFI_Nr_x_MCf_R_straw_corref = NFI_Nr_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             #(ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_Nr_x_MCf_R_straw_corref

In [ ]:
NFI_Nr_y_MCf_R_straw = NFI_Nrm_R_straw/N_human

NFI_Nr_y_MCf_R_straw_corref = NFI_Nr_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             #(ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nr_y_MCf_R_straw_corref

In [ ]:
NFI_Nacc_x_MCf_R_straw = Nacc_R_straw/NIF_R_straw

NFI_Nacc_x_MCf_R_straw_corref = NFI_Nacc_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             #(ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_x_MCf_R_straw_corref

In [ ]:
NFI_Nacc_y_MCf_R_straw = Nacc_R_straw/N_human

NFI_Nacc_y_MCf_R_straw_corref = NFI_Nacc_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             #(ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_y_MCf_R_straw_corref

In [ ]:
ER_N2O_list=[[GWP_R_straw_corref,WF_R_straw_corref],
              [NFI_N2_x_MCf_R_straw_corref,NFI_N2_y_MCf_R_straw_corref],
              [NFI_Nr_x_MCf_R_straw_corref,NFI_Nr_y_MCf_R_straw_corref],
              [NFI_Nacc_x_MCf_R_straw_corref,NFI_Nacc_y_MCf_R_straw_corref],
             ]
ER_N2O_list

In [ ]:
ER_N2O_ON_list=[NIF_ON__R_straw_corref,NOF_ON__R_straw_corref]
ER_N2O_ON_list

##### D_train_jetfuel

In [ ]:
GWP__R_straw = 0.0125*R_biojet_straw*(0.000114*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 0.00571*Diesel_C + 0.0011*Elec_C + 0.0011*Elec_milling + (CO2_maizep_total + 0.151*R_cob + 0.151*R_straw + 0.419 + 0.0932*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 0.0215
GWP_R_straw_corref = GWP__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             #(D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
GWP_R_straw_corref

In [ ]:
WF__R_straw = 0.0925*R_biojet_straw/(R_cob + R_straw + 1)

WF_R_straw_corref = WF__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             #(D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
WF_R_straw_corref

In [ ]:
NIF_IN__R_straw = 8.79e-9*D_train_jetfuel + 0.0125*R_biojet_straw*(0.000354*CoRu_catalyst/R_biojet_straw + 4.4e-7*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 4.08e-5*Diesel_C + 1.38e-6*Elec_C + 1.38e-6*Elec_milling + 4.94e-5*NiW_catalyst/R_biojet_straw + (0.000305*R_cob + 0.000305*R_straw + 0.00161)/(R_cob + R_straw + 1) + 0.00028*P_steel/(DR_feed*Life_FTplant)) - 2.66e-5
NIF_IN__R_straw
NIF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             #(D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])



In [ ]:
NIF_ON__R_straw = (0.0125*R_biojet_straw*(0.0105*R_cob + 0.0105*R_straw + 0.188) - 0.000276*R_cob - 0.000276*R_straw - 0.000276)/(R_cob + R_straw + 1)

NIF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             #(D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
#NIF_ON__R_straw

In [ ]:
NOF_IN__R_straw = 0.0125*R_biojet_straw*(2.01e-5*ER_NO2_N2*(0.3*ER_N2O + 0.7) - (ER_NO2_N2 - 1)*(3.67e-5*ER_N2O + 0.000313*R_Straw_return*(ER_N2O - 1) + 0.000398))/(ER_NO2_N2*(0.3*ER_N2O + 0.7)*(R_cob + R_straw + 1))
NOF_IN__R_straw
NOF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             #(D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_ON__R_straw = 0.452*NOx_fuelem + 0.0125*R_biojet_straw*(2.12e-5*Diesel_C + 1.44e-6*Elec_C + 1.44e-6*Elec_milling + 6.54e-5*NiW_catalyst/R_biojet_straw + 0.000201 + (0.000185*R_cob + 0.000185*R_straw + 0.0023 + 0.0013*(R_Straw_return*(ER_NO3 - 1) + 1)/(0.3*ER_NO3 + 0.7) + 0.000575*(R_Straw_return*(ER_NH3 - 1) + 1)/(0.3*ER_NH3 + 0.7) + 0.000199*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 2.81e-5
NOF_ON__R_straw
NOF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             #(D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NIF_R_straw = NIF_IN__R_straw + NIF_ON__R_straw
NIF_R_straw
NIF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             #(D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_R_straw = NOF_IN__R_straw + NOF_ON__R_straw
NOF_R_straw
NOF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             #(D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
Nacc_R_straw = NIF_R_straw - NOF_R_straw
Nacc_R_straw
Nacc_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             #(D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NFI_N2m_R_straw = NOF_IN__R_straw - NIF_IN__R_straw

NFI_N2m_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             #(D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_N2m_R_straw

In [ ]:
NFI_Nrm_R_straw = NOF_ON__R_straw - NIF_ON__R_straw

NFI_Nrm_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             #(D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_Nrm_R_straw_corref

In [ ]:
NFI_N2_x_MCf_R_straw = NFI_N2m_R_straw/NIF_R_straw

NFI_N2_x_MCf_R_straw_corref=NFI_N2_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             #(D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_N2_x_MCf_R_straw_corref

In [ ]:
NFI_N2_y_MCf_R_straw = NFI_N2m_R_straw/N_human

NFI_N2_y_MCf_R_straw_corref= NFI_N2_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             #(D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_N2_y_MCf_R_straw_corref

In [ ]:
NFI_Nr_x_MCf_R_straw = NFI_Nrm_R_straw/NIF_R_straw

NFI_Nr_x_MCf_R_straw_corref = NFI_Nr_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             #(D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_Nr_x_MCf_R_straw_corref

In [ ]:
NFI_Nr_y_MCf_R_straw = NFI_Nrm_R_straw/N_human

NFI_Nr_y_MCf_R_straw_corref = NFI_Nr_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             #(D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nr_y_MCf_R_straw_corref

In [ ]:
NFI_Nacc_x_MCf_R_straw = Nacc_R_straw/NIF_R_straw

NFI_Nacc_x_MCf_R_straw_corref = NFI_Nacc_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             #(D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_x_MCf_R_straw_corref

In [ ]:
NFI_Nacc_y_MCf_R_straw = Nacc_R_straw/N_human

NFI_Nacc_y_MCf_R_straw_corref = NFI_Nacc_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             #(D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_y_MCf_R_straw_corref

In [ ]:
D_train_jetfuel_list=[[GWP_R_straw_corref,WF_R_straw_corref],
              [NFI_N2_x_MCf_R_straw_corref,NFI_N2_y_MCf_R_straw_corref],
              [NFI_Nr_x_MCf_R_straw_corref,NFI_Nr_y_MCf_R_straw_corref],
              [NFI_Nacc_x_MCf_R_straw_corref,NFI_Nacc_y_MCf_R_straw_corref],
             ]
D_train_jetfuel_list

##### NiW_catalyst

In [ ]:
GWP__R_straw = 0.0125*R_biojet_straw*(0.000114*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 0.00571*Diesel_C + 0.0011*Elec_C + 0.0011*Elec_milling + (CO2_maizep_total + 0.151*R_cob + 0.151*R_straw + 0.419 + 0.0932*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 0.0215
GWP_R_straw_corref = GWP__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             #(NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
GWP_R_straw_corref

In [ ]:
WF__R_straw = 0.0925*R_biojet_straw/(R_cob + R_straw + 1)

WF_R_straw_corref = WF__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             #(NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
WF_R_straw_corref

In [ ]:
NIF_IN__R_straw = 8.79e-9*D_train_jetfuel + 0.0125*R_biojet_straw*(0.000354*CoRu_catalyst/R_biojet_straw + 4.4e-7*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 4.08e-5*Diesel_C + 1.38e-6*Elec_C + 1.38e-6*Elec_milling + 4.94e-5*NiW_catalyst/R_biojet_straw + (0.000305*R_cob + 0.000305*R_straw + 0.00161)/(R_cob + R_straw + 1) + 0.00028*P_steel/(DR_feed*Life_FTplant)) - 2.66e-5
NIF_IN__R_straw
NIF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             #(NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])



In [ ]:
NIF_ON__R_straw = (0.0125*R_biojet_straw*(0.0105*R_cob + 0.0105*R_straw + 0.188) - 0.000276*R_cob - 0.000276*R_straw - 0.000276)/(R_cob + R_straw + 1)

NIF_ON__R_straw_corref=NIF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             #(NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NIF_ON__R_straw_corref

In [ ]:
NOF_IN__R_straw = 0.0125*R_biojet_straw*(2.01e-5*ER_NO2_N2*(0.3*ER_N2O + 0.7) - (ER_NO2_N2 - 1)*(3.67e-5*ER_N2O + 0.000313*R_Straw_return*(ER_N2O - 1) + 0.000398))/(ER_NO2_N2*(0.3*ER_N2O + 0.7)*(R_cob + R_straw + 1))
NOF_IN__R_straw
NOF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             #(NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_ON__R_straw = 0.452*NOx_fuelem + 0.0125*R_biojet_straw*(2.12e-5*Diesel_C + 1.44e-6*Elec_C + 1.44e-6*Elec_milling + 6.54e-5*NiW_catalyst/R_biojet_straw + 0.000201 + (0.000185*R_cob + 0.000185*R_straw + 0.0023 + 0.0013*(R_Straw_return*(ER_NO3 - 1) + 1)/(0.3*ER_NO3 + 0.7) + 0.000575*(R_Straw_return*(ER_NH3 - 1) + 1)/(0.3*ER_NH3 + 0.7) + 0.000199*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 2.81e-5
NOF_ON__R_straw
NOF_ON__R_straw_corref=NOF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             #(NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NOF_ON__R_straw_corref

In [ ]:
NIF_R_straw = NIF_IN__R_straw + NIF_ON__R_straw
NIF_R_straw
NIF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             #(NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_R_straw = NOF_IN__R_straw + NOF_ON__R_straw
NOF_R_straw
NOF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             #(NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
Nacc_R_straw = NIF_R_straw - NOF_R_straw
Nacc_R_straw
Nacc_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             #(NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NFI_N2m_R_straw = NOF_IN__R_straw - NIF_IN__R_straw

NFI_N2m_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             #(NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_N2m_R_straw

In [ ]:
NFI_Nrm_R_straw = NOF_ON__R_straw - NIF_ON__R_straw

NFI_Nrm_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             #(NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_Nrm_R_straw_corref

In [ ]:
NFI_N2_x_MCf_R_straw = NFI_N2m_R_straw/NIF_R_straw

NFI_N2_x_MCf_R_straw_corref=NFI_N2_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             #(NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_N2_x_MCf_R_straw_corref

In [ ]:
NFI_N2_y_MCf_R_straw = NFI_N2m_R_straw/N_human

NFI_N2_y_MCf_R_straw_corref= NFI_N2_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             #(NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_N2_y_MCf_R_straw_corref

In [ ]:
NFI_Nr_x_MCf_R_straw = NFI_Nrm_R_straw/NIF_R_straw

NFI_Nr_x_MCf_R_straw_corref = NFI_Nr_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             #(NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_Nr_x_MCf_R_straw_corref

In [ ]:
NFI_Nr_y_MCf_R_straw = NFI_Nrm_R_straw/N_human

NFI_Nr_y_MCf_R_straw_corref = NFI_Nr_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             #(NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nr_y_MCf_R_straw_corref

In [ ]:
NFI_Nacc_x_MCf_R_straw = Nacc_R_straw/NIF_R_straw

NFI_Nacc_x_MCf_R_straw_corref = NFI_Nacc_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             #(NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_x_MCf_R_straw_corref

In [ ]:
NFI_Nacc_y_MCf_R_straw = Nacc_R_straw/N_human

NFI_Nacc_y_MCf_R_straw_corref = NFI_Nacc_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             #(NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_y_MCf_R_straw_corref

In [ ]:
NiW_catalyst_list=[[GWP_R_straw_corref,WF_R_straw_corref],
              [NFI_N2_x_MCf_R_straw_corref,NFI_N2_y_MCf_R_straw_corref],
              [NFI_Nr_x_MCf_R_straw_corref,NFI_Nr_y_MCf_R_straw_corref],
              [NFI_Nacc_x_MCf_R_straw_corref,NFI_Nacc_y_MCf_R_straw_corref],
             ]
NiW_catalyst_list

In [ ]:
NiW_catalyst_ON_list=[NIF_ON__R_straw_corref,NOF_ON__R_straw_corref]
NiW_catalyst_ON_list

##### P_steel

In [ ]:
GWP__R_straw = 0.0125*R_biojet_straw*(0.000114*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 0.00571*Diesel_C + 0.0011*Elec_C + 0.0011*Elec_milling + (CO2_maizep_total + 0.151*R_cob + 0.151*R_straw + 0.419 + 0.0932*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 0.0215
GWP_R_straw_corref = GWP__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             #(P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
GWP_R_straw_corref

In [ ]:
WF__R_straw = 0.0925*R_biojet_straw/(R_cob + R_straw + 1)

WF_R_straw_corref = WF__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             #(P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
WF_R_straw_corref

In [ ]:
NIF_IN__R_straw = 8.79e-9*D_train_jetfuel + 0.0125*R_biojet_straw*(0.000354*CoRu_catalyst/R_biojet_straw + 4.4e-7*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 4.08e-5*Diesel_C + 1.38e-6*Elec_C + 1.38e-6*Elec_milling + 4.94e-5*NiW_catalyst/R_biojet_straw + (0.000305*R_cob + 0.000305*R_straw + 0.00161)/(R_cob + R_straw + 1) + 0.00028*P_steel/(DR_feed*Life_FTplant)) - 2.66e-5
NIF_IN__R_straw
NIF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             #(P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])



In [ ]:
NIF_ON__R_straw = (0.0125*R_biojet_straw*(0.0105*R_cob + 0.0105*R_straw + 0.188) - 0.000276*R_cob - 0.000276*R_straw - 0.000276)/(R_cob + R_straw + 1)

NIF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             #(P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
#NIF_ON__R_straw

In [ ]:
NOF_IN__R_straw = 0.0125*R_biojet_straw*(2.01e-5*ER_NO2_N2*(0.3*ER_N2O + 0.7) - (ER_NO2_N2 - 1)*(3.67e-5*ER_N2O + 0.000313*R_Straw_return*(ER_N2O - 1) + 0.000398))/(ER_NO2_N2*(0.3*ER_N2O + 0.7)*(R_cob + R_straw + 1))
NOF_IN__R_straw
NOF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             #(P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_ON__R_straw = 0.452*NOx_fuelem + 0.0125*R_biojet_straw*(2.12e-5*Diesel_C + 1.44e-6*Elec_C + 1.44e-6*Elec_milling + 6.54e-5*NiW_catalyst/R_biojet_straw + 0.000201 + (0.000185*R_cob + 0.000185*R_straw + 0.0023 + 0.0013*(R_Straw_return*(ER_NO3 - 1) + 1)/(0.3*ER_NO3 + 0.7) + 0.000575*(R_Straw_return*(ER_NH3 - 1) + 1)/(0.3*ER_NH3 + 0.7) + 0.000199*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 2.81e-5
NOF_ON__R_straw
NOF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             #(P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NIF_R_straw = NIF_IN__R_straw + NIF_ON__R_straw
NIF_R_straw
NIF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             #(P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_R_straw = NOF_IN__R_straw + NOF_ON__R_straw
NOF_R_straw
NOF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             #(P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
Nacc_R_straw = NIF_R_straw - NOF_R_straw
Nacc_R_straw
Nacc_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             #(P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NFI_N2m_R_straw = NOF_IN__R_straw - NIF_IN__R_straw

NFI_N2m_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             #(P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_N2m_R_straw

In [ ]:
NFI_Nrm_R_straw = NOF_ON__R_straw - NIF_ON__R_straw

NFI_Nrm_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             #(P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_Nrm_R_straw_corref

In [ ]:
NFI_N2_x_MCf_R_straw = NFI_N2m_R_straw/NIF_R_straw

NFI_N2_x_MCf_R_straw_corref=NFI_N2_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             #(P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_N2_x_MCf_R_straw_corref

In [ ]:
NFI_N2_y_MCf_R_straw = NFI_N2m_R_straw/N_human

NFI_N2_y_MCf_R_straw_corref= NFI_N2_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             #(P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_N2_y_MCf_R_straw_corref

In [ ]:
NFI_Nr_x_MCf_R_straw = NFI_Nrm_R_straw/NIF_R_straw

NFI_Nr_x_MCf_R_straw_corref = NFI_Nr_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             #(P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_Nr_x_MCf_R_straw_corref

In [ ]:
NFI_Nr_y_MCf_R_straw = NFI_Nrm_R_straw/N_human

NFI_Nr_y_MCf_R_straw_corref = NFI_Nr_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             #(P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nr_y_MCf_R_straw_corref

In [ ]:
NFI_Nacc_x_MCf_R_straw = Nacc_R_straw/NIF_R_straw

NFI_Nacc_x_MCf_R_straw_corref = NFI_Nacc_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             #(P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_x_MCf_R_straw_corref

In [ ]:
NFI_Nacc_y_MCf_R_straw = Nacc_R_straw/N_human

NFI_Nacc_y_MCf_R_straw_corref = NFI_Nacc_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             #(P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_y_MCf_R_straw_corref

In [ ]:
P_steel_list=[[GWP_R_straw_corref,WF_R_straw_corref],
              [NFI_N2_x_MCf_R_straw_corref,NFI_N2_y_MCf_R_straw_corref],
              [NFI_Nr_x_MCf_R_straw_corref,NFI_Nr_y_MCf_R_straw_corref],
              [NFI_Nacc_x_MCf_R_straw_corref,NFI_Nacc_y_MCf_R_straw_corref],
             ]
P_steel_list

##### Life_FTplant

In [ ]:
GWP__R_straw = 0.0125*R_biojet_straw*(0.000114*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 0.00571*Diesel_C + 0.0011*Elec_C + 0.0011*Elec_milling + (CO2_maizep_total + 0.151*R_cob + 0.151*R_straw + 0.419 + 0.0932*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 0.0215
GWP_R_straw_corref = GWP__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             #(Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
GWP_R_straw_corref

In [ ]:
WF__R_straw = 0.0925*R_biojet_straw/(R_cob + R_straw + 1)

WF_R_straw_corref = WF__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             #(Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
WF_R_straw_corref

In [ ]:
NIF_IN__R_straw = 8.79e-9*D_train_jetfuel + 0.0125*R_biojet_straw*(0.000354*CoRu_catalyst/R_biojet_straw + 4.4e-7*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 4.08e-5*Diesel_C + 1.38e-6*Elec_C + 1.38e-6*Elec_milling + 4.94e-5*NiW_catalyst/R_biojet_straw + (0.000305*R_cob + 0.000305*R_straw + 0.00161)/(R_cob + R_straw + 1) + 0.00028*P_steel/(DR_feed*Life_FTplant)) - 2.66e-5
NIF_IN__R_straw
NIF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             #(Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])



In [ ]:
NIF_ON__R_straw = (0.0125*R_biojet_straw*(0.0105*R_cob + 0.0105*R_straw + 0.188) - 0.000276*R_cob - 0.000276*R_straw - 0.000276)/(R_cob + R_straw + 1)

NIF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             #(Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
#NIF_ON__R_straw

In [ ]:
NOF_IN__R_straw = 0.0125*R_biojet_straw*(2.01e-5*ER_NO2_N2*(0.3*ER_N2O + 0.7) - (ER_NO2_N2 - 1)*(3.67e-5*ER_N2O + 0.000313*R_Straw_return*(ER_N2O - 1) + 0.000398))/(ER_NO2_N2*(0.3*ER_N2O + 0.7)*(R_cob + R_straw + 1))
NOF_IN__R_straw
NOF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             #(Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_ON__R_straw = 0.452*NOx_fuelem + 0.0125*R_biojet_straw*(2.12e-5*Diesel_C + 1.44e-6*Elec_C + 1.44e-6*Elec_milling + 6.54e-5*NiW_catalyst/R_biojet_straw + 0.000201 + (0.000185*R_cob + 0.000185*R_straw + 0.0023 + 0.0013*(R_Straw_return*(ER_NO3 - 1) + 1)/(0.3*ER_NO3 + 0.7) + 0.000575*(R_Straw_return*(ER_NH3 - 1) + 1)/(0.3*ER_NH3 + 0.7) + 0.000199*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 2.81e-5
NOF_ON__R_straw
NOF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             #(Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NIF_R_straw = NIF_IN__R_straw + NIF_ON__R_straw
NIF_R_straw
NIF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             #(Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_R_straw = NOF_IN__R_straw + NOF_ON__R_straw
NOF_R_straw
NOF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             #(Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
Nacc_R_straw = NIF_R_straw - NOF_R_straw
Nacc_R_straw
Nacc_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             #(Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NFI_N2m_R_straw = NOF_IN__R_straw - NIF_IN__R_straw

NFI_N2m_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             #(Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_N2m_R_straw

In [ ]:
NFI_Nrm_R_straw = NOF_ON__R_straw - NIF_ON__R_straw

NFI_Nrm_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             #(Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_Nrm_R_straw_corref

In [ ]:
NFI_N2_x_MCf_R_straw = NFI_N2m_R_straw/NIF_R_straw

NFI_N2_x_MCf_R_straw_corref=NFI_N2_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             #(Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_N2_x_MCf_R_straw_corref

In [ ]:
NFI_N2_y_MCf_R_straw = NFI_N2m_R_straw/N_human

NFI_N2_y_MCf_R_straw_corref= NFI_N2_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             #(Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_N2_y_MCf_R_straw_corref

In [ ]:
NFI_Nr_x_MCf_R_straw = NFI_Nrm_R_straw/NIF_R_straw

NFI_Nr_x_MCf_R_straw_corref = NFI_Nr_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             #(Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_Nr_x_MCf_R_straw_corref

In [ ]:
NFI_Nr_y_MCf_R_straw = NFI_Nrm_R_straw/N_human

NFI_Nr_y_MCf_R_straw_corref = NFI_Nr_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             #(Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nr_y_MCf_R_straw_corref

In [ ]:
NFI_Nacc_x_MCf_R_straw = Nacc_R_straw/NIF_R_straw

NFI_Nacc_x_MCf_R_straw_corref = NFI_Nacc_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             #(Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_x_MCf_R_straw_corref

In [ ]:
NFI_Nacc_y_MCf_R_straw = Nacc_R_straw/N_human

NFI_Nacc_y_MCf_R_straw_corref = NFI_Nacc_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             #(Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_y_MCf_R_straw_corref

In [ ]:
Life_FTplant_list=[[GWP_R_straw_corref,WF_R_straw_corref],
              [NFI_N2_x_MCf_R_straw_corref,NFI_N2_y_MCf_R_straw_corref],
              [NFI_Nr_x_MCf_R_straw_corref,NFI_Nr_y_MCf_R_straw_corref],
              [NFI_Nacc_x_MCf_R_straw_corref,NFI_Nacc_y_MCf_R_straw_corref],
             ]
Life_FTplant_list

##### ER_NO3

In [ ]:
GWP__R_straw = 0.0125*R_biojet_straw*(0.000114*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 0.00571*Diesel_C + 0.0011*Elec_C + 0.0011*Elec_milling + (CO2_maizep_total + 0.151*R_cob + 0.151*R_straw + 0.419 + 0.0932*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 0.0215
GWP_R_straw_corref = GWP__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             #(ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
GWP_R_straw_corref

In [ ]:
WF__R_straw = 0.0925*R_biojet_straw/(R_cob + R_straw + 1)

WF_R_straw_corref = WF__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             #(ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
WF_R_straw_corref

In [ ]:
NIF_IN__R_straw = 8.79e-9*D_train_jetfuel + 0.0125*R_biojet_straw*(0.000354*CoRu_catalyst/R_biojet_straw + 4.4e-7*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 4.08e-5*Diesel_C + 1.38e-6*Elec_C + 1.38e-6*Elec_milling + 4.94e-5*NiW_catalyst/R_biojet_straw + (0.000305*R_cob + 0.000305*R_straw + 0.00161)/(R_cob + R_straw + 1) + 0.00028*P_steel/(DR_feed*Life_FTplant)) - 2.66e-5
NIF_IN__R_straw
NIF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             #(ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])



In [ ]:
NIF_ON__R_straw = (0.0125*R_biojet_straw*(0.0105*R_cob + 0.0105*R_straw + 0.188) - 0.000276*R_cob - 0.000276*R_straw - 0.000276)/(R_cob + R_straw + 1)

NIF_ON__R_straw_corref=NIF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             #(ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NIF_ON__R_straw_corref

In [ ]:
NOF_IN__R_straw = 0.0125*R_biojet_straw*(2.01e-5*ER_NO2_N2*(0.3*ER_N2O + 0.7) - (ER_NO2_N2 - 1)*(3.67e-5*ER_N2O + 0.000313*R_Straw_return*(ER_N2O - 1) + 0.000398))/(ER_NO2_N2*(0.3*ER_N2O + 0.7)*(R_cob + R_straw + 1))
NOF_IN__R_straw
NOF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             #(ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_ON__R_straw = 0.452*NOx_fuelem + 0.0125*R_biojet_straw*(2.12e-5*Diesel_C + 1.44e-6*Elec_C + 1.44e-6*Elec_milling + 6.54e-5*NiW_catalyst/R_biojet_straw + 0.000201 + (0.000185*R_cob + 0.000185*R_straw + 0.0023 + 0.0013*(R_Straw_return*(ER_NO3 - 1) + 1)/(0.3*ER_NO3 + 0.7) + 0.000575*(R_Straw_return*(ER_NH3 - 1) + 1)/(0.3*ER_NH3 + 0.7) + 0.000199*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 2.81e-5
NOF_ON__R_straw
NOF_ON__R_straw_corref=NOF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             #(ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NOF_ON__R_straw_corref

In [ ]:
NIF_R_straw = NIF_IN__R_straw + NIF_ON__R_straw
NIF_R_straw
NIF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             #(ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_R_straw = NOF_IN__R_straw + NOF_ON__R_straw
NOF_R_straw
NOF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             #(ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
Nacc_R_straw = NIF_R_straw - NOF_R_straw
Nacc_R_straw
Nacc_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             #(ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NFI_N2m_R_straw = NOF_IN__R_straw - NIF_IN__R_straw

NFI_N2m_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             #(ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_N2m_R_straw

In [ ]:
NFI_Nrm_R_straw = NOF_ON__R_straw - NIF_ON__R_straw

NFI_Nrm_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             #(ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_Nrm_R_straw_corref

In [ ]:
NFI_N2_x_MCf_R_straw = NFI_N2m_R_straw/NIF_R_straw

NFI_N2_x_MCf_R_straw_corref=NFI_N2_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             #(ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_N2_x_MCf_R_straw_corref

In [ ]:
NFI_N2_y_MCf_R_straw = NFI_N2m_R_straw/N_human

NFI_N2_y_MCf_R_straw_corref= NFI_N2_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             #(ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_N2_y_MCf_R_straw_corref

In [ ]:
NFI_Nr_x_MCf_R_straw = NFI_Nrm_R_straw/NIF_R_straw

NFI_Nr_x_MCf_R_straw_corref = NFI_Nr_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             #(ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_Nr_x_MCf_R_straw_corref

In [ ]:
NFI_Nr_y_MCf_R_straw = NFI_Nrm_R_straw/N_human

NFI_Nr_y_MCf_R_straw_corref = NFI_Nr_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             #(ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nr_y_MCf_R_straw_corref

In [ ]:
NFI_Nacc_x_MCf_R_straw = Nacc_R_straw/NIF_R_straw

NFI_Nacc_x_MCf_R_straw_corref = NFI_Nacc_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             #(ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_x_MCf_R_straw_corref

In [ ]:
NFI_Nacc_y_MCf_R_straw = Nacc_R_straw/N_human

NFI_Nacc_y_MCf_R_straw_corref = NFI_Nacc_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             #(ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             (ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_y_MCf_R_straw_corref

In [ ]:
ER_NO3_list=[[GWP_R_straw_corref,WF_R_straw_corref],
              [NFI_N2_x_MCf_R_straw_corref,NFI_N2_y_MCf_R_straw_corref],
              [NFI_Nr_x_MCf_R_straw_corref,NFI_Nr_y_MCf_R_straw_corref],
              [NFI_Nacc_x_MCf_R_straw_corref,NFI_Nacc_y_MCf_R_straw_corref],
             ]
ER_NO3_list

In [ ]:
ER_NO3_ON_list=[NIF_ON__R_straw_corref,NOF_ON__R_straw_corref]
ER_NO3_ON_list

##### ER_NH3

In [ ]:
GWP__R_straw = 0.0125*R_biojet_straw*(0.000114*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 0.00571*Diesel_C + 0.0011*Elec_C + 0.0011*Elec_milling + (CO2_maizep_total + 0.151*R_cob + 0.151*R_straw + 0.419 + 0.0932*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 0.0215
GWP_R_straw_corref = GWP__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             #(ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
GWP_R_straw_corref

In [ ]:
WF__R_straw = 0.0925*R_biojet_straw/(R_cob + R_straw + 1)

WF_R_straw_corref = WF__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             #(ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
WF_R_straw_corref

In [ ]:
NIF_IN__R_straw = 8.79e-9*D_train_jetfuel + 0.0125*R_biojet_straw*(0.000354*CoRu_catalyst/R_biojet_straw + 4.4e-7*sqrt(5)*sqrt(DR_feed)/(sqrt(R_straw)*sqrt(1 - R_Straw_return)) + 4.08e-5*Diesel_C + 1.38e-6*Elec_C + 1.38e-6*Elec_milling + 4.94e-5*NiW_catalyst/R_biojet_straw + (0.000305*R_cob + 0.000305*R_straw + 0.00161)/(R_cob + R_straw + 1) + 0.00028*P_steel/(DR_feed*Life_FTplant)) - 2.66e-5
NIF_IN__R_straw
NIF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             #(ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])



In [ ]:
NIF_ON__R_straw = (0.0125*R_biojet_straw*(0.0105*R_cob + 0.0105*R_straw + 0.188) - 0.000276*R_cob - 0.000276*R_straw - 0.000276)/(R_cob + R_straw + 1)

NIF_ON__R_straw_corref=NIF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             #(ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NIF_ON__R_straw_corref

In [ ]:
NOF_IN__R_straw = 0.0125*R_biojet_straw*(2.01e-5*ER_NO2_N2*(0.3*ER_N2O + 0.7) - (ER_NO2_N2 - 1)*(3.67e-5*ER_N2O + 0.000313*R_Straw_return*(ER_N2O - 1) + 0.000398))/(ER_NO2_N2*(0.3*ER_N2O + 0.7)*(R_cob + R_straw + 1))
NOF_IN__R_straw
NOF_IN__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             #(ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_ON__R_straw = 0.452*NOx_fuelem + 0.0125*R_biojet_straw*(2.12e-5*Diesel_C + 1.44e-6*Elec_C + 1.44e-6*Elec_milling + 6.54e-5*NiW_catalyst/R_biojet_straw + 0.000201 + (0.000185*R_cob + 0.000185*R_straw + 0.0023 + 0.0013*(R_Straw_return*(ER_NO3 - 1) + 1)/(0.3*ER_NO3 + 0.7) + 0.000575*(R_Straw_return*(ER_NH3 - 1) + 1)/(0.3*ER_NH3 + 0.7) + 0.000199*(R_Straw_return*(ER_N2O - 1) + 1)/(0.3*ER_N2O + 0.7))/(R_cob + R_straw + 1)) - 2.81e-5
NOF_ON__R_straw
NOF_ON__R_straw_corref=NOF_ON__R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             #(ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NOF_ON__R_straw_corref

In [ ]:
NIF_R_straw = NIF_IN__R_straw + NIF_ON__R_straw
NIF_R_straw
NIF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             #(ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NOF_R_straw = NOF_IN__R_straw + NOF_ON__R_straw
NOF_R_straw
NOF_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             #(ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
Nacc_R_straw = NIF_R_straw - NOF_R_straw
Nacc_R_straw
Nacc_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             #(ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

In [ ]:
NFI_N2m_R_straw = NOF_IN__R_straw - NIF_IN__R_straw

NFI_N2m_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             #(ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_N2m_R_straw

In [ ]:
NFI_Nrm_R_straw = NOF_ON__R_straw - NIF_ON__R_straw

NFI_Nrm_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             #(ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

#NFI_Nrm_R_straw_corref

In [ ]:
NFI_N2_x_MCf_R_straw = NFI_N2m_R_straw/NIF_R_straw

NFI_N2_x_MCf_R_straw_corref=NFI_N2_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             #(ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_N2_x_MCf_R_straw_corref

In [ ]:
NFI_N2_y_MCf_R_straw = NFI_N2m_R_straw/N_human

NFI_N2_y_MCf_R_straw_corref= NFI_N2_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             #(ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_N2_y_MCf_R_straw_corref

In [ ]:
NFI_Nr_x_MCf_R_straw = NFI_Nrm_R_straw/NIF_R_straw

NFI_Nr_x_MCf_R_straw_corref = NFI_Nr_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             #(ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])
NFI_Nr_x_MCf_R_straw_corref

In [ ]:
NFI_Nr_y_MCf_R_straw = NFI_Nrm_R_straw/N_human

NFI_Nr_y_MCf_R_straw_corref = NFI_Nr_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             #(ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nr_y_MCf_R_straw_corref

In [ ]:
NFI_Nacc_x_MCf_R_straw = Nacc_R_straw/NIF_R_straw

NFI_Nacc_x_MCf_R_straw_corref = NFI_Nacc_x_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             #(ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_x_MCf_R_straw_corref

In [ ]:
NFI_Nacc_y_MCf_R_straw = Nacc_R_straw/N_human

NFI_Nacc_y_MCf_R_straw_corref = NFI_Nacc_y_MCf_R_straw.subs([
             (R_cob,ParamDef.stat_value(R_cob,mode=FixedParamMode.DEFAULT)),
             (R_straw,ParamDef.stat_value(R_straw,mode=FixedParamMode.DEFAULT)),
             (ER_NO2_N2,ParamDef.stat_value(ER_NO2_N2,mode=FixedParamMode.DEFAULT)),
             (R_Straw_return,ParamDef.stat_value(R_Straw_return,mode=FixedParamMode.DEFAULT)),
             (R_biojet_straw,ParamDef.stat_value(R_biojet_straw,mode=FixedParamMode.DEFAULT)),
             (CO2_maizep_total,ParamDef.stat_value(CO2_maizep_total,mode=FixedParamMode.DEFAULT)),
             (Elec_C,ParamDef.stat_value(Elec_C,mode=FixedParamMode.DEFAULT)),
             (Diesel_C,ParamDef.stat_value(Diesel_C,mode=FixedParamMode.DEFAULT)),              
             (Elec_milling,ParamDef.stat_value(Elec_milling,mode=FixedParamMode.DEFAULT)),
             (CoRu_catalyst,ParamDef.stat_value(CoRu_catalyst,mode=FixedParamMode.DEFAULT)),
             (NOx_fuelem,ParamDef.stat_value(NOx_fuelem,mode=FixedParamMode.DEFAULT)),
             (DR_feed,ParamDef.stat_value(DR_feed,mode=FixedParamMode.DEFAULT)),
             (ER_N2O,ParamDef.stat_value(ER_N2O,mode=FixedParamMode.DEFAULT)),
             (D_train_jetfuel,ParamDef.stat_value(D_train_jetfuel,mode=FixedParamMode.DEFAULT)),
             (NiW_catalyst,ParamDef.stat_value(NiW_catalyst,mode=FixedParamMode.DEFAULT)),    
             (P_steel,ParamDef.stat_value(P_steel,mode=FixedParamMode.DEFAULT)),    
             (Life_FTplant,ParamDef.stat_value(Life_FTplant,mode=FixedParamMode.DEFAULT)),    
             (ER_NO3,ParamDef.stat_value(ER_NO3,mode=FixedParamMode.DEFAULT)),    
             #(ER_NH3,ParamDef.stat_value(ER_NH3,mode=FixedParamMode.DEFAULT)),    
    
             ])

NFI_Nacc_y_MCf_R_straw_corref

In [ ]:
ER_NH3_list=[[GWP_R_straw_corref,WF_R_straw_corref],
              [NFI_N2_x_MCf_R_straw_corref,NFI_N2_y_MCf_R_straw_corref],
              [NFI_Nr_x_MCf_R_straw_corref,NFI_Nr_y_MCf_R_straw_corref],
              [NFI_Nacc_x_MCf_R_straw_corref,NFI_Nacc_y_MCf_R_straw_corref],
             ]
ER_NH3_list

In [ ]:
ER_NH3_ON_list=[NIF_ON__R_straw_corref,NOF_ON__R_straw_corref]
ER_NH3_ON_list

#### Life cycle paramaters sensitivity correlation formula collection

In [ ]:
df_corref = pd.DataFrame()

df_corref['R_straw'] = R_straw_list
df_corref['R_cob'] = R_cob_list
df_corref['ER_NO2_N2'] = ER_NO2_N2_list
df_corref['R_Straw_return'] = R_Straw_return_list
df_corref['CO2_maizep_total'] = CO2_maizep_total_list
df_corref['Elec_C'] = Elec_C_list
df_corref['Diesel_C'] = Diesel_C_list
df_corref['Elec_milling'] = Elec_milling_list
df_corref['R_biojet_straw'] = R_biojet_straw_list
df_corref['CoRu_catalyst'] = CoRu_catalyst_list
df_corref['NOx_fuelem'] = NOx_fuelem_list
df_corref['DR_feed'] = DR_feed_list
df_corref['ER_N2O'] = ER_N2O_list
df_corref['D_train_jetfuel'] = D_train_jetfuel_list
df_corref['NiW_catalyst'] = NiW_catalyst_list
df_corref['P_steel'] = P_steel_list
df_corref['Life_FTplant'] = Life_FTplant_list
df_corref['ER_NO3'] = ER_NO3_list
df_corref['ER_NH3'] = ER_NH3_list
df_corref['test'] = test_R_Straw_GWP_corref



#df_SV_FP['NIF_IN_SV'] = NIF_IN_SV
#df_SV_FP['NIF_ON_SV'] = NIF_ON_SV

#df_SV_FP['NOF_IN_SV'] = NOF_IN_SV
#df_SV_FP['NOF_ON_SV'] = NOF_ON_SV

#df_SV_FP['NIF_SV'] = NIF_SV
#df_SV_FP['NOF_SV'] = NOF_SV

df_corref

In [ ]:
df_corref_ON = pd.DataFrame()

df_corref_ON['R_straw'] = R_straw_ON_list
df_corref_ON['R_cob'] = R_cob_ON_list
#df_corref_ON['ER_NO2_N2'] = ER_NO2_N2_list
df_corref_ON['R_Straw_return'] = R_Straw_return_ON_list
#df_corref_ON['CO2_maizep_total'] = CO2_maizep_total_list
df_corref_ON['Elec_C'] = Elec_C_ON_list
df_corref_ON['Diesel_C'] = Diesel_C_ON_list
df_corref_ON['Elec_milling'] = Elec_milling_ON_list
df_corref_ON['R_biojet_straw'] = R_biojet_straw_ON_list
#df_corref_ON['CoRu_catalyst'] = CoRu_catalyst_list
df_corref_ON['NOx_fuelem'] = NOx_fuelem_ON_list
#df_corref_ON['DR_feed'] = DR_feed_list
df_corref_ON['ER_N2O'] = ER_N2O_ON_list
#df_corref_ON['D_train_jetfuel'] = D_train_jetfuel_list
df_corref_ON['NiW_catalyst'] = NiW_catalyst_ON_list
#df_corref_ON['P_steel'] = P_steel_list
#df_corref_ON['Life_FTplant'] = Life_FTplant_list
df_corref_ON['ER_NO3'] = ER_NO3_ON_list
df_corref_ON['ER_NH3'] = ER_NH3_ON_list
#df_corref_ON['test'] = test_R_Straw_GWP_corref



#df_SV_FP['NIF_IN_SV'] = NIF_IN_SV
#df_SV_FP['NIF_ON_SV'] = NIF_ON_SV

#df_SV_FP['NOF_IN_SV'] = NOF_IN_SV
#df_SV_FP['NOF_ON_SV'] = NOF_ON_SV

#df_SV_FP['NIF_SV'] = NIF_SV
#df_SV_FP['NOF_SV'] = NOF_SV

df_corref_ON

In [ ]:
#df_corref.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\UA and SA_each paramaters\LCI paramater correlation formula.xlsx', sheet_name='corref', index = True)

In [ ]:
#df_corref_ON.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\UA and SA_each paramaters\ON LCI correlation formula_v2.xlsx', sheet_name='ON_corref', index = True)

#### Calculation of sustainability indicators sensitivity to life cycle paramaters 

##### R_straw

In [ ]:
R_straw = np.linspace(0.5,1.55,1001,endpoint=True).tolist()
R_straw

In [ ]:
def test_NIFON(R_straw):
   return (0.0012675*R_straw + 0.02762680875)/(R_straw + 1.2105)

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
R_straw_crroefSAr_NIFON = list(map(test_NIFON, R_straw))
R_straw_crroefSAr_NIFON

In [ ]:
def test_NOFON(R_straw):
   return 0.147*(0.000185*R_straw + 0.0044129425)/(R_straw + 1.2105) + 0.000167664624

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
R_straw_crroefSAr_NOFON = list(map(test_NOFON, R_straw))
R_straw_crroefSAr_NOFON

In [ ]:
def test_GWPCF(R_straw):
   return (0.022197*R_straw + 0.1737695085)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
R_straw_crroefSAr_GWP = list(map(test_GWPCF, R_straw))
R_straw_crroefSAr_GWP

In [ ]:
def test_WFCF(R_straw):
   return 1.0878/(R_straw + 1.2105)
#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
R_straw_crroefSAr_WF = list(map(test_WFCF, R_straw))
R_straw_crroefSAr_WF

In [ ]:
def test_N2xCF(R_straw):
   return (-0.147*(0.000305*R_straw + 0.0016742025)/(R_straw + 1.2105) - 1.3997966e-5 + 0.000109467427272727/(R_straw + 1.2105) - 3.86536932258743e-6/sqrt(R_straw))/(0.147*(0.000305*R_straw + 0.0016742025)/(R_straw + 1.2105) + (0.0012675*R_straw + 0.02762680875)/(R_straw + 1.2105) + 1.3997966e-5 + 3.86536932258743e-6/sqrt(R_straw))
#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
R_straw_crroefSAr_N2x = list(map(test_N2xCF, R_straw))
R_straw_crroefSAr_N2x

In [ ]:
def test_N2yCF(R_straw):
   return -0.0790322580645161*(0.000305*R_straw + 0.0016742025)/(R_straw + 1.2105) - 7.52578817204301e-6 + 5.88534555229717e-5/(R_straw + 1.2105) - 2.07815554977819e-6/sqrt(R_straw)#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
R_straw_crroefSAr_N2y = list(map(test_N2yCF, R_straw))
R_straw_crroefSAr_N2y

In [ ]:
def test_NrxCF(R_straw):
   return (0.147*(0.000185*R_straw + 0.0044129425)/(R_straw + 1.2105) - (0.0012675*R_straw + 0.02762680875)/(R_straw + 1.2105) + 0.000167664624)/(0.147*(0.000305*R_straw + 0.0016742025)/(R_straw + 1.2105) + (0.0012675*R_straw + 0.02762680875)/(R_straw + 1.2105) + 1.3997966e-5 + 3.86536932258743e-6/sqrt(R_straw))
#R_straw = range(1, 6,0.5)
R_straw_crroefSAr_Nrx = list(map(test_NrxCF, R_straw))
R_straw_crroefSAr_Nrx

In [ ]:
def test_NryCF(R_straw):
   return 0.0790322580645161*(0.000185*R_straw + 0.0044129425)/(R_straw + 1.2105) - 0.537634408602151*(0.0012675*R_straw + 0.02762680875)/(R_straw + 1.2105) + 9.01422709677419e-5
#R_straw = range(1, 6,0.5)
R_straw_crroefSAr_Nry = list(map(test_NryCF, R_straw))
R_straw_crroefSAr_Nry

In [ ]:
def test_NaccxCF(R_straw):
   return (-0.147*(0.000185*R_straw + 0.0044129425)/(R_straw + 1.2105) + 0.147*(0.000305*R_straw + 0.0016742025)/(R_straw + 1.2105) + (0.0012675*R_straw + 0.02762680875)/(R_straw + 1.2105) - 0.000153666658 - 0.000109467427272727/(R_straw + 1.2105) + 3.86536932258743e-6/sqrt(R_straw))/(0.147*(0.000305*R_straw + 0.0016742025)/(R_straw + 1.2105) + (0.0012675*R_straw + 0.02762680875)/(R_straw + 1.2105) + 1.3997966e-5 + 3.86536932258743e-6/sqrt(R_straw))
#R_straw = range(1, 6,0.5)
R_straw_crroefSAr_Naccx = list(map(test_NaccxCF, R_straw))
R_straw_crroefSAr_Naccx

In [ ]:
def test_NaccyCF(R_straw):
   return -0.0790322580645161*(0.000185*R_straw + 0.0044129425)/(R_straw + 1.2105) + 0.0790322580645161*(0.000305*R_straw + 0.0016742025)/(R_straw + 1.2105) + 0.537634408602151*(0.0012675*R_straw + 0.02762680875)/(R_straw + 1.2105) - 8.26164827956989e-5 - 5.88534555229717e-5/(R_straw + 1.2105) + 2.07815554977819e-6/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
R_straw_crroefSAr_Naccy = list(map(test_NaccyCF, R_straw))
R_straw_crroefSAr_Naccy

##### R_cob

In [ ]:
R_cob = np.linspace(0.0145,0.4065,1001,endpoint=True).tolist()
R_cob

In [ ]:
def test_NIFON(R_cob):
   return (0.0012675*R_cob + 0.0284247)/(R_cob + 1.84)

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
R_cob_crroefSAr_NIFON = list(map(test_NIFON, R_cob))
R_cob_crroefSAr_NIFON

In [ ]:
def test_NOFON(R_cob):
   return 0.147*(0.000185*R_cob + 0.0045294)/(R_cob + 1.84) + 0.000167664624

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
R_cob_crroefSAr_NOFON = list(map(test_NOFON, R_cob))
R_cob_crroefSAr_NOFON

In [ ]:
def test_GWPCF(R_cob):
   return 0.147*(0.151*R_cob + 1.27716)/(R_cob + 1.84) - 0.0101105755977772

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
R_cob_crroefSAr_GWP = list(map(test_GWPCF, R_cob))
R_cob_crroefSAr_GWP

In [ ]:
def test_WFCF(R_cob):
   return 1.0878/(R_cob + 1.84)
#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
R_cob_crroefSAr_WF = list(map(test_WFCF, R_cob))
R_cob_crroefSAr_WF

In [ ]:
def test_N2xCF(R_cob):
   return (-0.147*(0.000305*R_cob + 0.0018662)/(R_cob + 1.84) - 1.82154296927898e-5 + 0.000109467427272727/(R_cob + 1.84))/(0.147*(0.000305*R_cob + 0.0018662)/(R_cob + 1.84) + (0.0012675*R_cob + 0.0284247)/(R_cob + 1.84) + 1.82154296927898e-5)
#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
R_cob_crroefSAr_N2x = list(map(test_N2xCF, R_cob))
R_cob_crroefSAr_N2x

In [ ]:
def test_N2yCF(R_cob):
   return -0.0790322580645161*(0.000305*R_cob + 0.0018662)/(R_cob + 1.84) - 9.79324177031709e-6 + 5.88534555229717e-5/(R_cob + 1.84)
#R_straw = range(1, 6,0.5)
R_cob_crroefSAr_N2y = list(map(test_N2yCF, R_cob))
R_cob_crroefSAr_N2y

In [ ]:
def test_NrxCF(R_cob):
   return (0.147*(0.000185*R_cob + 0.0045294)/(R_cob + 1.84) - (0.0012675*R_cob + 0.0284247)/(R_cob + 1.84) + 0.000167664624)/(0.147*(0.000305*R_cob + 0.0018662)/(R_cob + 1.84) + (0.0012675*R_cob + 0.0284247)/(R_cob + 1.84) + 1.82154296927898e-5)
#R_straw = range(1, 6,0.5)
R_cob_crroefSAr_Nrx = list(map(test_NrxCF, R_cob))
R_cob_crroefSAr_Nrx

In [ ]:
def test_NryCF(R_cob):
   return 0.0790322580645161*(0.000185*R_cob + 0.0045294)/(R_cob + 1.84) - 0.537634408602151*(0.0012675*R_cob + 0.0284247)/(R_cob + 1.84) + 9.01422709677419e-5
#R_straw = range(1, 6,0.5)
R_cob_crroefSAr_Nry = list(map(test_NryCF, R_cob))
R_cob_crroefSAr_Nry

In [ ]:
def test_NaccxCF(R_cob):
   return (-0.147*(0.000185*R_cob + 0.0045294)/(R_cob + 1.84) + 0.147*(0.000305*R_cob + 0.0018662)/(R_cob + 1.84) + (0.0012675*R_cob + 0.0284247)/(R_cob + 1.84) - 0.00014944919430721 - 0.000109467427272727/(R_cob + 1.84))/(0.147*(0.000305*R_cob + 0.0018662)/(R_cob + 1.84) + (0.0012675*R_cob + 0.0284247)/(R_cob + 1.84) + 1.82154296927898e-5)
#R_straw = range(1, 6,0.5)
R_cob_crroefSAr_Naccx = list(map(test_NaccxCF, R_cob))
R_cob_crroefSAr_Naccx

In [ ]:
def test_NaccyCF(R_cob):
   return -0.0790322580645161*(0.000185*R_cob + 0.0045294)/(R_cob + 1.84) + 0.0790322580645161*(0.000305*R_cob + 0.0018662)/(R_cob + 1.84) + 0.537634408602151*(0.0012675*R_cob + 0.0284247)/(R_cob + 1.84) - 8.03490291974248e-5 - 5.88534555229717e-5/(R_cob + 1.84)
#R_straw = range(1, 6,0.5)
R_cob_crroefSAr_Naccy = list(map(test_NaccyCF, R_cob))
R_cob_crroefSAr_Naccy

##### ER_NO2_N2

In [ ]:
ER_NO2_N2 = np.linspace(0.2525,0.4975,1001,endpoint=True).tolist()
ER_NO2_N2

In [ ]:
def test_N2xCF(ER_NO2_N2):
   return -0.0110682802958537 + 4.70014757987245*(0.000468656 - 0.0004469882*ER_NO2_N2)/ER_NO2_N2
#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
ER_NO2_N2_crroefSAr_N2x = list(map(test_N2xCF, ER_NO2_N2))
ER_NO2_N2_crroefSAr_N2x

In [ ]:
def test_N2yCF(ER_NO2_N2):
   return  -8.41965914647268e-5 + 0.0357541004590112*(0.000468656 - 0.0004469882*ER_NO2_N2)/ER_NO2_N2
#R_straw = range(1, 6,0.5)
ER_NO2_N2_crroefSAr_N2y = list(map(test_N2yCF, ER_NO2_N2))
ER_NO2_N2_crroefSAr_N2y

In [ ]:
def test_NaccxCF(ER_NO2_N2):
   return 0.965003424162359 - 4.70014757987245*(0.000468656 - 0.0004469882*ER_NO2_N2)/ER_NO2_N2
#R_straw = range(1, 6,0.5)
ER_NO2_N2_crroefSAr_Naccx = list(map(test_NaccxCF, ER_NO2_N2))
ER_NO2_N2_crroefSAr_Naccx

In [ ]:
def test_NaccyCF(ER_NO2_N2):
   return 0.00734079702487279 - 0.0357541004590112*(0.000468656 - 0.0004469882*ER_NO2_N2)/ER_NO2_N2
#R_straw = range(1, 6,0.5)
ER_NO2_N2_crroefSAr_Naccy = list(map(test_NaccyCF, ER_NO2_N2))
ER_NO2_N2_crroefSAr_Naccy

##### R_Straw_return

In [ ]:
R_Straw_return = np.linspace(0.176,0.5,1001,endpoint=True).tolist()
R_Straw_return

In [ ]:
def test_NOFON(R_Straw_return):
   return 5.67733801382088e-5*R_Straw_return + 0.000478136315152392

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
R_Straw_return_crroefSAr_NOFON = list(map(test_NOFON, R_Straw_return))
R_Straw_return_crroefSAr_NOFON

In [ ]:
def test_GWPCF(R_Straw_return):
   return 0.00161149165392033*R_Straw_return + 0.0821513530665891 + 0.00091422385114369/sqrt(1 - R_Straw_return)

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
R_Straw_return_crroefSAr_GWP = list(map(test_GWPCF, R_Straw_return))
R_Straw_return_crroefSAr_GWP

In [ ]:
def test_N2xCF(R_Straw_return):
   return (9.01997295559841e-6*R_Straw_return - 0.000101708464215513 - 3.528583285116e-6/sqrt(1 - R_Straw_return))/(0.0141448333317644 + 3.528583285116e-6/sqrt(1 - R_Straw_return))
#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
R_Straw_return_crroefSAr_N2x = list(map(test_N2xCF, R_Straw_return))
R_Straw_return_crroefSAr_N2x

In [ ]:
def test_N2yCF(R_Straw_return):
   return 4.84944782559055e-6*R_Straw_return - 5.46819700083402e-5 - 1.89708778769677e-6/sqrt(1 - R_Straw_return)
#R_straw = range(1, 6,0.5)
R_Straw_return_crroefSAr_N2y = list(map(test_N2yCF, R_Straw_return))
R_Straw_return_crroefSAr_N2y

In [ ]:
def test_NrxCF(R_Straw_return):
   return (5.67733801382088e-5*R_Straw_return - 0.0135143088201805)/(0.0141448333317644 + 3.528583285116e-6/sqrt(1 - R_Straw_return))
#R_straw = range(1, 6,0.5)
R_Straw_return_crroefSAr_Nrx = list(map(test_NrxCF, R_Straw_return))
R_Straw_return_crroefSAr_Nrx

In [ ]:
def test_NryCF(R_Straw_return):
   return 3.0523322654951e-5*R_Straw_return - 0.00726575743020455
#R_straw = range(1, 6,0.5)
R_Straw_return_crroefSAr_Nry = list(map(test_NryCF, R_Straw_return))
R_Straw_return_crroefSAr_Nry

In [ ]:
def test_NaccxCF(R_Straw_return):
   return (-6.57933530938072e-5*R_Straw_return + 0.013616017284396 + 3.528583285116e-6/sqrt(1 - R_Straw_return))/(0.0141448333317644 + 3.528583285116e-6/sqrt(1 - R_Straw_return))
#R_straw = range(1, 6,0.5)
R_Straw_return_crroefSAr_Naccx = list(map(test_NaccxCF, R_Straw_return))
R_Straw_return_crroefSAr_Naccx

In [ ]:
def test_NaccyCF(R_Straw_return):
   return -3.53727704805415e-5*R_Straw_return + 0.00732043940021289 + 1.89708778769677e-6/sqrt(1 - R_Straw_return)
#R_straw = range(1, 6,0.5)
R_Straw_return_crroefSAr_Naccy = list(map(test_NaccyCF, R_Straw_return))
R_Straw_return_crroefSAr_Naccy

##### CO2_maizep_total

In [ ]:
CO2_maizep_total = np.linspace(0.45,0.75,1001,endpoint=True).tolist()
CO2_maizep_total

In [ ]:
def test_GWPCF(CO2_maizep_total):
   return 0.0716898317483541*CO2_maizep_total + 0.0379807916297283
#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
CO2_maizep_total_crroefSAr_GWP = list(map(test_GWPCF, CO2_maizep_total))
CO2_maizep_total_crroefSAr_GWP

##### Elec_C

In [ ]:
Elec_C = np.linspace(7,22,1001,endpoint=True).tolist()
Elec_C

In [ ]:
def test_NOFON(Elec_C):
   return 2.1168e-7*Elec_C + 0.000491315753193855

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
Elec_C_crroefSAr_NOFON = list(map(test_NOFON, Elec_C))
Elec_C_crroefSAr_NOFON

In [ ]:
def test_GWPCF(Elec_C):
   return 0.0001617*Elec_C + 0.080784567064988

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
Elec_C_crroefSAr_GWP = list(map(test_GWPCF, Elec_C))
Elec_C_crroefSAr_GWP

In [ ]:
def test_N2xCF(Elec_C):
   return (-2.0286e-7*Elec_C - 9.95278840216231e-5)/(2.0286e-7*Elec_C + 0.0141453587434572)
#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
Elec_C_crroefSAr_N2x = list(map(test_N2xCF, Elec_C))
Elec_C_crroefSAr_N2x

In [ ]:
def test_N2yCF(Elec_C):
   return -1.09064516129032e-7*Elec_C - 5.35096150653888e-5
#R_straw = range(1, 6,0.5)
Elec_C_crroefSAr_N2y = list(map(test_N2yCF, Elec_C))
Elec_C_crroefSAr_N2y

In [ ]:
def test_NrxCF(Elec_C):
   return (2.1168e-7*Elec_C - 0.013501129382139)/(2.0286e-7*Elec_C + 0.0141453587434572)
#R_straw = range(1, 6,0.5)
Elec_C_crroefSAr_Nrx = list(map(test_NrxCF, Elec_C))
Elec_C_crroefSAr_Nrx

In [ ]:
def test_NryCF(Elec_C):
   return 1.13806451612903e-7*Elec_C - 0.00725867171082741
#R_straw = range(1, 6,0.5)
Elec_C_crroefSAr_Nry = list(map(test_NryCF, Elec_C))
Elec_C_crroefSAr_Nry

In [ ]:
def test_NaccxCF(Elec_C):
   return (0.0136006572661606 - 8.82e-9*Elec_C)/(2.0286e-7*Elec_C + 0.0141453587434572)
#R_straw = range(1, 6,0.5)
Elec_C_crroefSAr_Naccx = list(map(test_NaccxCF, Elec_C))
Elec_C_crroefSAr_Naccx

In [ ]:
def test_NaccyCF(Elec_C):
   return 0.0073121813258928 - 4.74193548387098e-9*Elec_C
#R_straw = range(1, 6,0.5)
Elec_C_crroefSAr_Naccy = list(map(test_NaccyCF, Elec_C))
Elec_C_crroefSAr_Naccy

##### Diesel_C

In [ ]:
Diesel_C = np.linspace(0.5,2.4,1001,endpoint=True).tolist()
Diesel_C

In [ ]:
def test_NOFON(Diesel_C):
   return 3.1164e-6*Diesel_C + 0.000493080341193855

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
Diesel_C_crroefSAr_NOFON = list(map(test_NOFON, Diesel_C))
Diesel_C_crroefSAr_NOFON

In [ ]:
def test_GWPCF(Diesel_C):
   return 0.00083937*Diesel_C + 0.083165129164988

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
Diesel_C_crroefSAr_GWP = list(map(test_GWPCF, Diesel_C))
Diesel_C_crroefSAr_GWP

In [ ]:
def test_N2xCF(Diesel_C):
   return (-5.9976e-6*Diesel_C - 9.92015440216231e-5)/(5.9976e-6*Diesel_C + 0.0141450324034572)
#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
Diesel_C_crroefSAr_N2x = list(map(test_N2xCF, Diesel_C))
Diesel_C_crroefSAr_N2x

In [ ]:
def test_N2yCF(Diesel_C):
   return -3.22451612903226e-6*Diesel_C - 5.33341634524855e-5
#R_straw = range(1, 6,0.5)
Diesel_C_crroefSAr_N2y = list(map(test_N2yCF, Diesel_C))
Diesel_C_crroefSAr_N2y

In [ ]:
def test_NrxCF(Diesel_C):
   return (3.1164e-6*Diesel_C - 0.013499364794139)/(5.9976e-6*Diesel_C + 0.0141450324034572)
#R_straw = range(1, 6,0.5)
Diesel_C_crroefSAr_Nrx = list(map(test_NrxCF, Diesel_C))
Diesel_C_crroefSAr_Nrx

In [ ]:
def test_NryCF(Diesel_C):
   return 1.67548387096774e-6*Diesel_C - 0.00725772300760161
#R_straw = range(1, 6,0.5)
Diesel_C_crroefSAr_Nry = list(map(test_NryCF, Diesel_C))
Diesel_C_crroefSAr_Nry

In [ ]:
def test_NaccxCF(Diesel_C):
   return (2.8812e-6*Diesel_C + 0.0135985663381606)/(5.9976e-6*Diesel_C + 0.0141450324034572)
#R_straw = range(1, 6,0.5)
Diesel_C_crroefSAr_Naccx = list(map(test_NaccxCF, Diesel_C))
Diesel_C_crroefSAr_Naccx

In [ ]:
def test_NaccyCF(Diesel_C):
   return 1.54903225806452e-6*Diesel_C + 0.0073110571710541
#R_straw = range(1, 6,0.5)
Diesel_C_crroefSAr_Naccy = list(map(test_NaccyCF, Diesel_C))
Diesel_C_crroefSAr_Naccy

##### Elec_milling

In [ ]:
Elec_milling = np.linspace(8.87,52.5828,1001,endpoint=True).tolist()
Elec_milling

In [ ]:
def test_NOFON(Elec_milling):
   return 2.1168e-7*Elec_milling + 0.000486277769193855

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
Elec_milling_crroefSAr_NOFON = list(map(test_NOFON, Elec_milling))
Elec_milling_crroefSAr_NOFON

In [ ]:
def test_GWPCF(Elec_milling):
   return 0.0001617*Elec_milling + 0.076936107064988

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
Elec_milling_crroefSAr_GWP = list(map(test_GWPCF, Elec_milling))
Elec_milling_crroefSAr_GWP

In [ ]:
def test_N2xCF(Elec_milling):
   return (-2.0286e-7*Elec_milling - 9.46998160216231e-5)/(2.0286e-7*Elec_milling + 0.0141405306754572)
#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
Elec_milling_crroefSAr_N2x = list(map(test_N2xCF, Elec_milling))
Elec_milling_crroefSAr_N2x

In [ ]:
def test_N2yCF(Elec_milling):
   return -1.09064516129032e-7*Elec_milling - 5.09138795815178e-5
#R_straw = range(1, 6,0.5)
Elec_milling_crroefSAr_N2y = list(map(test_N2yCF, Elec_milling))
Elec_milling_crroefSAr_N2y

In [ ]:
def test_NrxCF(Elec_milling):
   return (2.1168e-7*Elec_milling - 0.013506167366139)/(2.0286e-7*Elec_milling + 0.0141405306754572)
#R_straw = range(1, 6,0.5)
Elec_milling_crroefSAr_Nrx = list(map(test_NrxCF, Elec_milling))
Elec_milling_crroefSAr_Nrx

In [ ]:
def test_NryCF(Elec_milling):
   return 1.13806451612903e-7*Elec_milling - 0.0072613803043758
#R_straw = range(1, 6,0.5)
Elec_milling_crroefSAr_Nry = list(map(test_NryCF, Elec_milling))
Elec_milling_crroefSAr_Nry

In [ ]:
def test_NaccxCF(Elec_milling):
   return (0.0136008671821606 - 8.82e-9*Elec_milling)/(2.0286e-7*Elec_milling + 0.0141405306754572)
#R_straw = range(1, 6,0.5)
Elec_milling_crroefSAr_Naccx = list(map(test_NaccxCF, Elec_milling))
Elec_milling_crroefSAr_Naccx

In [ ]:
def test_NaccyCF(Elec_milling):
   return 0.00731229418395732 - 4.74193548387098e-9*Elec_milling
#R_straw = range(1, 6,0.5)
Elec_milling_crroefSAr_Naccy = list(map(test_NaccyCF, Elec_milling))
Elec_milling_crroefSAr_Naccy

##### R_biojet_straw

In [ ]:
R_biojet_straw = np.linspace(10.8,15,1001,endpoint=True).tolist()
R_biojet_straw

In [ ]:
def test_NIFON(R_biojet_straw):
   return 0.0012133031577664*R_biojet_straw - 0.000276

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
R_biojet_straw_crroefSAr_NIFON = list(map(test_NIFON, R_biojet_straw))
R_biojet_straw_crroefSAr_NIFON

In [ ]:
def test_NOFON(R_biojet_straw):
   return 0.0125*R_biojet_straw*(0.00252980836186296 + 0.0001962/R_biojet_straw) + 0.000120834

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
R_biojet_straw_crroefSAr_NOFON = list(map(test_NOFON, R_biojet_straw))
R_biojet_straw_crroefSAr_NOFON

In [ ]:
def test_GWPCF(R_biojet_straw):
   return 0.00894791726743095*R_biojet_straw - 0.0215

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
R_biojet_straw_crroefSAr_GWP = list(map(test_GWPCF, R_biojet_straw))
R_biojet_straw_crroefSAr_GWP

In [ ]:
def test_WFCF(R_biojet_straw):
   return 0.0451109485491344*R_biojet_straw
#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
R_biojet_straw_crroefSAr_WF = list(map(test_WFCF, R_biojet_straw))
R_biojet_straw_crroefSAr_WF

In [ ]:
def test_N2xCF(R_biojet_straw):
   return (-0.0125*R_biojet_straw*(0.00109860170152647 + 0.0012102/R_biojet_straw) + 4.53960238969122e-6*R_biojet_straw + 2.001629e-5)/(0.0125*R_biojet_straw*(0.00109860170152647 + 0.0012102/R_biojet_straw) + 0.0012133031577664*R_biojet_straw - 0.00029601629)
#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
R_biojet_straw_crroefSAr_N2x = list(map(test_N2xCF, R_biojet_straw))
R_biojet_straw_crroefSAr_N2x

In [ ]:
def test_N2yCF(R_biojet_straw):
   return -0.00672043010752688*R_biojet_straw*(0.00109860170152647 + 0.0012102/R_biojet_straw) + 2.44064644607055e-6*R_biojet_straw + 1.07614462365591e-5
#R_straw = range(1, 6,0.5)
R_biojet_straw_crroefSAr_N2y = list(map(test_N2yCF, R_biojet_straw))
R_biojet_straw_crroefSAr_N2y

In [ ]:
def test_NrxCF(R_biojet_straw):
   return (0.0125*R_biojet_straw*(0.00252980836186296 + 0.0001962/R_biojet_straw) - 0.0012133031577664*R_biojet_straw + 0.000396834)/(0.0125*R_biojet_straw*(0.00109860170152647 + 0.0012102/R_biojet_straw) + 0.0012133031577664*R_biojet_straw - 0.00029601629)
#R_straw = range(1, 6,0.5)
R_biojet_straw_crroefSAr_Nrx = list(map(test_NrxCF, R_biojet_straw))
R_biojet_straw_crroefSAr_Nrx

In [ ]:
def test_NryCF(R_biojet_straw):
   return 0.00672043010752688*R_biojet_straw*(0.00252980836186296 + 0.0001962/R_biojet_straw) - 0.000652313525680859*R_biojet_straw + 0.000213351612903226
#R_straw = range(1, 6,0.5)
R_biojet_straw_crroefSAr_Nry = list(map(test_NryCF, R_biojet_straw))
R_biojet_straw_crroefSAr_Nry

In [ ]:
def test_NaccxCF(R_biojet_straw):
   return (0.0125*R_biojet_straw*(0.00109860170152647 + 0.0012102/R_biojet_straw) - 0.0125*R_biojet_straw*(0.00252980836186296 + 0.0001962/R_biojet_straw) + 0.00120876355537671*R_biojet_straw - 0.00041685029)/(0.0125*R_biojet_straw*(0.00109860170152647 + 0.0012102/R_biojet_straw) + 0.0012133031577664*R_biojet_straw - 0.00029601629)
#R_straw = range(1, 6,0.5)
R_biojet_straw_crroefSAr_Naccx = list(map(test_NaccxCF, R_biojet_straw))
R_biojet_straw_crroefSAr_Naccx

In [ ]:
def test_NaccyCF(R_biojet_straw):
   return 0.00672043010752688*R_biojet_straw*(0.00109860170152647 + 0.0012102/R_biojet_straw) - 0.00672043010752688*R_biojet_straw*(0.00252980836186296 + 0.0001962/R_biojet_straw) + 0.000649872879234789*R_biojet_straw - 0.000224113059139785
#R_straw = range(1, 6,0.5)
R_biojet_straw_crroefSAr_Naccy = list(map(test_NaccyCF, R_biojet_straw))
R_biojet_straw_crroefSAr_Naccy

##### CoRu_catalyst

In [ ]:
CoRu_catalyst = np.linspace(1,5,1001,endpoint=True).tolist()
CoRu_catalyst

In [ ]:
def test_N2xCF(CoRu_catalyst):
   return (-4.425e-6*CoRu_catalyst - 8.99449360216231e-5)/(4.425e-6*CoRu_catalyst + 0.0141357757954572)
#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
CoRu_catalyst_crroefSAr_N2x = list(map(test_N2xCF, CoRu_catalyst))
CoRu_catalyst_crroefSAr_N2x

In [ ]:
def test_N2yCF(CoRu_catalyst):
   return-2.37903225806452e-6*CoRu_catalyst - 4.83574924847436e-5
#R_straw = range(1, 6,0.5)
CoRu_catalyst_crroefSAr_N2y = list(map(test_N2yCF, CoRu_catalyst))
CoRu_catalyst_crroefSAr_N2y

In [ ]:
def test_NrxCF(CoRu_catalyst):
   return -0.013497276806139/(4.425e-6*CoRu_catalyst + 0.0141357757954572)
#R_straw = range(1, 6,0.5)
CoRu_catalyst_crroefSAr_Nrx = list(map(test_NrxCF, CoRu_catalyst))
CoRu_catalyst_crroefSAr_Nrx

In [ ]:
def test_NaccxCF(CoRu_catalyst):
   return (4.425e-6*CoRu_catalyst + 0.0135872217421606)/(4.425e-6*CoRu_catalyst + 0.0141357757954572)
#R_straw = range(1, 6,0.5)
CoRu_catalyst_crroefSAr_Naccx = list(map(test_NaccxCF, CoRu_catalyst))
CoRu_catalyst_crroefSAr_Naccx

In [ ]:
def test_NaccyCF(CoRu_catalyst):
   return 2.37903225806452e-6*CoRu_catalyst + 0.00730495792589281
#R_straw = range(1, 6,0.5)
CoRu_catalyst_crroefSAr_Naccy = list(map(test_NaccyCF, CoRu_catalyst))
CoRu_catalyst_crroefSAr_Naccy

##### NOx_fuelem

In [ ]:
NOx_fuelem = np.linspace(0.0002564,0.0004087,1001,endpoint=True).tolist()
NOx_fuelem

In [ ]:
def test_NOFON(NOx_fuelem):
   return 0.452*NOx_fuelem + 0.000346234329193855

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
NOx_fuelem_crroefSAr_NOFON = list(map(test_NOFON, NOx_fuelem))
NOx_fuelem_crroefSAr_NOFON

In [ ]:
def test_NrxCF(NOx_fuelem):
   return 31.945605859661*NOx_fuelem - 0.964461220997264
#R_straw = range(1, 6,0.5)
NOx_fuelem_crroefSAr_Nrx = list(map(test_NrxCF, NOx_fuelem))
NOx_fuelem_crroefSAr_Nrx

In [ ]:
def test_NryCF(NOx_fuelem):
   return 0.243010752688172*NOx_fuelem - 0.00733667247641881
#R_straw = range(1, 6,0.5)
NOx_fuelem_crroefSAr_Nry = list(map(test_NryCF, NOx_fuelem))
NOx_fuelem_crroefSAr_Nry

In [ ]:
def test_NaccxCF(NOx_fuelem):
   return 0.971756405495065 - 31.945605859661*NOx_fuelem
#R_straw = range(1, 6,0.5)
NOx_fuelem_crroefSAr_Naccx = list(map(test_NaccxCF, NOx_fuelem))
NOx_fuelem_crroefSAr_Naccx

In [ ]:
def test_NaccyCF(NOx_fuelem):
   return 0.00739216706567775 - 0.243010752688172*NOx_fuelem
#R_straw = range(1, 6,0.5)
NOx_fuelem_crroefSAr_Naccy = list(map(test_NaccyCF, NOx_fuelem))
NOx_fuelem_crroefSAr_Naccy

##### DR_feed

In [ ]:
DR_feed = np.linspace(305.556,638.889,1001,endpoint=True).tolist()
DR_feed

In [ ]:
def test_GWPCF(DR_feed):
   return 2.18541300444561e-5*sqrt(5)*sqrt(DR_feed) + 0.0826348005627652

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
DR_feed_crroefSAr_GWP = list(map(test_GWPCF, DR_feed))
DR_feed_crroefSAr_GWP

In [ ]:
def test_N2xCF(DR_feed):
   return (-8.43492738557956e-8*sqrt(5)*sqrt(DR_feed) - 9.63462803288333e-5 - 0.001328096/DR_feed)/(8.43492738557956e-8*sqrt(5)*sqrt(DR_feed) + 0.0141421771397644 + 0.001328096/DR_feed)
#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
DR_feed_crroefSAr_N2x = list(map(test_N2xCF, DR_feed))
DR_feed_crroefSAr_N2x

In [ ]:
def test_N2yCF(DR_feed):
   return -4.53490719654815e-8*sqrt(5)*sqrt(DR_feed) - 5.17990754456093e-5 - 0.000714030107526882/DR_feed
#R_straw = range(1, 6,0.5)
DR_feed_crroefSAr_N2y = list(map(test_N2yCF, DR_feed))
DR_feed_crroefSAr_N2y

In [ ]:
def test_NrxCF(DR_feed):
   return -0.013497276806139/(8.43492738557956e-8*sqrt(5)*sqrt(DR_feed) + 0.0141421771397644 + 0.001328096/DR_feed)
#R_straw = range(1, 6,0.5)
DR_feed_crroefSAr_Nrx = list(map(test_NrxCF, DR_feed))
DR_feed_crroefSAr_Nrx

In [ ]:
def test_NaccxCF(DR_feed):
   return (8.43492738557956e-8*sqrt(5)*sqrt(DR_feed) + 0.0135936230864678 + 0.001328096/DR_feed)/(8.43492738557956e-8*sqrt(5)*sqrt(DR_feed) + 0.0141421771397644 + 0.001328096/DR_feed)
#R_straw = range(1, 6,0.5)
DR_feed_crroefSAr_Naccx = list(map(test_NaccxCF, DR_feed))
DR_feed_crroefSAr_Naccx

In [ ]:
def test_NaccyCF(DR_feed):
   return 4.53490719654815e-8*sqrt(5)*sqrt(DR_feed) + 0.00730839950885367 + 0.000714030107526882/DR_feed
#R_straw = range(1, 6,0.5)
DR_feed_crroefSAr_Naccy = list(map(test_NaccyCF, DR_feed))
DR_feed_crroefSAr_Naccy

##### ER_N2O

In [ ]:
ER_N2O = np.linspace(1.12,1.411,1001,endpoint=True).tolist()
ER_N2O

In [ ]:
def test_NOFON(ER_N2O):
   return 0.0716898317483541*(5.97e-5*ER_N2O + 0.0001393)/(0.3*ER_N2O + 0.7) + 0.000480902052675933

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
ER_N2O_crroefSAr_NOFON = list(map(test_NOFON, ER_N2O))
ER_N2O_crroefSAr_NOFON

In [ ]:
def test_GWPCF(ER_N2O):
   return 0.0716898317483541*(0.02796*ER_N2O + 0.06524)/(0.3*ER_N2O + 0.7) + 0.0770460147460414

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
ER_N2O_crroefSAr_GWP = list(map(test_GWPCF, ER_N2O))
ER_N2O_crroefSAr_GWP

In [ ]:
def test_N2xCF(ER_N2O):
   return 13.5113575762733*(8.388625e-5*ER_N2O + 0.00019533875)/(0.3*ER_N2O + 0.7) - 0.0110682802958537
#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
ER_N2O_crroefSAr_N2x = list(map(test_N2xCF, ER_N2O))
ER_N2O_crroefSAr_N2x

In [ ]:
def test_N2yCF(ER_N2O):
   return 0.102781120786171*(8.388625e-5*ER_N2O + 0.00019533875)/(0.3*ER_N2O + 0.7) - 8.41965914647268e-5
#R_straw = range(1, 6,0.5)
ER_N2O_crroefSAr_N2y = list(map(test_N2yCF, ER_N2O))
ER_N2O_crroefSAr_N2y

In [ ]:
def test_NrxCF(ER_N2O):
   return 5.0667590911025*(5.97e-5*ER_N2O + 0.0001393)/(0.3*ER_N2O + 0.7) - 0.954943428925635
#R_straw = range(1, 6,0.5)
ER_N2O_crroefSAr_Nrx = list(map(test_NrxCF, ER_N2O))
ER_N2O_crroefSAr_Nrx

In [ ]:
def test_NryCF(ER_N2O):
   return 0.038542920294814*(5.97e-5*ER_N2O + 0.0001393)/(0.3*ER_N2O + 0.7) - 0.00726427047454673
#R_straw = range(1, 6,0.5)
ER_N2O_crroefSAr_Nry = list(map(test_NryCF, ER_N2O))
ER_N2O_crroefSAr_Nry

In [ ]:
def test_NaccxCF(ER_N2O):
   return -5.0667590911025*(5.97e-5*ER_N2O + 0.0001393)/(0.3*ER_N2O + 0.7) - 13.5113575762733*(8.388625e-5*ER_N2O + 0.00019533875)/(0.3*ER_N2O + 0.7) + 0.966011709221488
#R_straw = range(1, 6,0.5)
ER_N2O_crroefSAr_Naccx = list(map(test_NaccxCF, ER_N2O))
ER_N2O_crroefSAr_Naccx

In [ ]:
def test_NaccyCF(ER_N2O):
   return -0.038542920294814*(5.97e-5*ER_N2O + 0.0001393)/(0.3*ER_N2O + 0.7) - 0.102781120786171*(8.388625e-5*ER_N2O + 0.00019533875)/(0.3*ER_N2O + 0.7) + 0.00734846706601146
#R_straw = range(1, 6,0.5)
ER_N2O_crroefSAr_Naccy = list(map(test_NaccyCF, ER_N2O))
ER_N2O_crroefSAr_Naccy

##### D_train_jetfuel

In [ ]:
D_train_jetfuel = np.linspace(716,810,1001,endpoint=True).tolist()
D_train_jetfuel

In [ ]:
def test_N2xCF(D_train_jetfuel):
   return (-8.79e-9*D_train_jetfuel - 9.66362260216231e-5)/(8.79e-9*D_train_jetfuel + 0.0141424670854572)
#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
D_train_jetfuel_crroefSAr_N2x = list(map(test_N2xCF, D_train_jetfuel))
D_train_jetfuel_crroefSAr_N2x

In [ ]:
def test_N2yCF(D_train_jetfuel):
   return -4.7258064516129e-9*D_train_jetfuel - 5.19549602266791e-5
#R_straw = range(1, 6,0.5)
D_train_jetfuel_crroefSAr_N2y = list(map(test_N2yCF, D_train_jetfuel))
D_train_jetfuel_crroefSAr_N2y

In [ ]:
def test_NrxCF(D_train_jetfuel):
   return -0.013497276806139/(8.79e-9*D_train_jetfuel + 0.0141424670854572)
#R_straw = range(1, 6,0.5)
D_train_jetfuel_crroefSAr_Nrx = list(map(test_NrxCF, D_train_jetfuel))
D_train_jetfuel_crroefSAr_Nrx

In [ ]:
def test_NaccxCF(D_train_jetfuel):
   return (8.79e-9*D_train_jetfuel + 0.0135939130321606)/(8.79e-9*D_train_jetfuel + 0.0141424670854572)
#R_straw = range(1, 6,0.5)
D_train_jetfuel_crroefSAr_Naccx = list(map(test_NaccxCF, D_train_jetfuel))
D_train_jetfuel_crroefSAr_Naccx

In [ ]:
def test_NaccyCF(D_train_jetfuel):
   return 4.7258064516129e-9*D_train_jetfuel + 0.00730855539363474
#R_straw = range(1, 6,0.5)
D_train_jetfuel_crroefSAr_Naccy = list(map(test_NaccyCF, D_train_jetfuel))
D_train_jetfuel_crroefSAr_Naccy

##### NiW_catalyst

In [ ]:
NiW_catalyst = np.linspace(1,5,1001,endpoint=True).tolist()
NiW_catalyst

In [ ]:
def test_NOFON(NiW_catalyst):
   return 8.175e-7*NiW_catalyst + 0.000492715829193855

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
NiW_catalyst_crroefSAr_NOFON = list(map(test_NOFON, NiW_catalyst))
NiW_catalyst_crroefSAr_NOFON

In [ ]:
def test_N2xCF(NiW_catalyst):
   return (-6.175e-7*NiW_catalyst - 0.000101367436021623)/(6.175e-7*NiW_catalyst + 0.0141471982954572)
#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
NiW_catalyst_crroefSAr_N2x = list(map(test_N2xCF,NiW_catalyst))
NiW_catalyst_crroefSAr_N2x

In [ ]:
def test_N2yCF(NiW_catalyst):
   return -3.31989247311828e-7*NiW_catalyst - 5.44986215170017e-5
#R_straw = range(1, 6,0.5)
NiW_catalyst_crroefSAr_N2y = list(map(test_N2yCF, NiW_catalyst))
NiW_catalyst_crroefSAr_N2y

In [ ]:
def test_NrxCF(NiW_catalyst):
   return (8.175e-7*NiW_catalyst - 0.013499729306139)/(6.175e-7*NiW_catalyst + 0.0141471982954572)
#R_straw = range(1, 6,0.5)
NiW_catalyst_crroefSAr_Nrx = list(map(test_NrxCF, NiW_catalyst))
NiW_catalyst_crroefSAr_Nrx

In [ ]:
def test_NryCF(NiW_catalyst):
   return 4.39516129032258e-7*NiW_catalyst - 0.00725791898179516
#R_straw = range(1, 6,0.5)
NiW_catalyst_crroefSAr_Nry = list(map(test_NryCF, NiW_catalyst))
NiW_catalyst_crroefSAr_Nry

In [ ]:
def test_NaccxCF(NiW_catalyst):
   return (0.0136010967421606 - 2.0e-7*NiW_catalyst)/(6.175e-7*NiW_catalyst + 0.0141471982954572)#R_straw = range(1, 6,0.5)
NiW_catalyst_crroefSAr_Naccx = list(map(test_NaccxCF, NiW_catalyst))
NiW_catalyst_crroefSAr_Naccx

In [ ]:
def test_NaccyCF(NiW_catalyst):
   return 0.00731241760331216 - 1.0752688172043e-7*NiW_catalyst
#R_straw = range(1, 6,0.5)
NiW_catalyst_crroefSAr_Naccy = list(map(test_NaccyCF, NiW_catalyst))
NiW_catalyst_crroefSAr_Naccy

##### P_steel

In [ ]:
P_steel = np.linspace(450.629,501.526,1001,endpoint=True).tolist()
P_steel

In [ ]:
def test_N2xCF(P_steel):
   return (-5.488e-9*P_steel - 0.000100563744021623)/(5.488e-9*P_steel + 0.0141463946034572)
#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
P_steel_crroefSAr_N2x = list(map(test_N2xCF,P_steel))
P_steel_crroefSAr_N2x

In [ ]:
def test_N2yCF(P_steel):
   return -2.9505376344086e-9*P_steel - 5.40665290438834e-5
#R_straw = range(1, 6,0.5)
P_steel_crroefSAr_N2y = list(map(test_N2yCF, P_steel))
P_steel_crroefSAr_N2y

In [ ]:
def test_NrxCF(P_steel):
   return -0.013497276806139/(5.488e-9*P_steel + 0.0141463946034572)
#R_straw = range(1, 6,0.5)
P_steel_crroefSAr_Nrx = list(map(test_NrxCF, P_steel))
P_steel_crroefSAr_Nrx

In [ ]:
def test_NaccxCF(P_steel):
   return (5.488e-9*P_steel + 0.0135978405501606)/(5.488e-9*P_steel + 0.0141463946034572)
#R_straw = range(1, 6,0.5)
P_steel_crroefSAr_Naccx = list(map(test_NaccxCF, P_steel))
P_steel_crroefSAr_Naccx

In [ ]:
def test_NaccyCF(P_steel):
   return 2.9505376344086e-9*P_steel + 0.00731066696245194



#R_straw = range(1, 6,0.5)
P_steel_crroefSAr_Naccy = list(map(test_NaccyCF, P_steel))
P_steel_crroefSAr_Naccy

##### Life_FTplant

In [ ]:
Life_FTplant = np.linspace(13,17,1001,endpoint=True).tolist()
Life_FTplant

In [ ]:
def test_N2xCF(Life_FTplant):
   return (-0.000100563744021623 - 3.984288e-5/Life_FTplant)/(0.0141463946034572 + 3.984288e-5/Life_FTplant)
#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
Life_FTplant_crroefSAr_N2x = list(map(test_N2xCF,Life_FTplant))
Life_FTplant_crroefSAr_N2x

In [ ]:
def test_N2yCF(Life_FTplant):
   return  -5.40665290438834e-5 - 2.14209032258065e-5/Life_FTplant
#R_straw = range(1, 6,0.5)
Life_FTplant_crroefSAr_N2y = list(map(test_N2yCF, Life_FTplant))
Life_FTplant_crroefSAr_N2y

In [ ]:
def test_NrxCF(Life_FTplant):
   return -0.013497276806139/(0.0141463946034572 + 3.984288e-5/Life_FTplant)
#R_straw = range(1, 6,0.5)
Life_FTplant_crroefSAr_Nrx = list(map(test_NrxCF, Life_FTplant))
Life_FTplant_crroefSAr_Nrx

In [ ]:
def test_NaccxCF(Life_FTplant):
   return (0.0135978405501606 + 3.984288e-5/Life_FTplant)/(0.0141463946034572 + 3.984288e-5/Life_FTplant)
#R_straw = range(1, 6,0.5)
Life_FTplant_crroefSAr_Naccx = list(map(test_NaccxCF,Life_FTplant))
Life_FTplant_crroefSAr_Naccx

In [ ]:
def test_NaccyCF(Life_FTplant):
   return 0.00731066696245194 + 2.14209032258065e-5/Life_FTplant
#R_straw = range(1, 6,0.5)
Life_FTplant_crroefSAr_Naccy = list(map(test_NaccyCF, Life_FTplant))
Life_FTplant_crroefSAr_Naccy

##### ER_NO3

In [ ]:
ER_NO3 = np.linspace(1.364,2.173,1001,endpoint=True).tolist()
ER_NO3

In [ ]:
def test_NOFON(ER_NO3):
   return 0.0716898317483541*(0.00039*ER_NO3 + 0.00091)/(0.3*ER_NO3 + 0.7) + 0.000401971547920995

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
ER_NO3_crroefSAr_NOFON = list(map(test_NOFON, ER_NO3))
ER_NO3_crroefSAr_NOFON

In [ ]:
def test_NrxCF(ER_NO3):
   return 5.0667590911025*(0.00039*ER_NO3 + 0.00091)/(0.3*ER_NO3 + 0.7) - 0.960521930684938
#R_straw = range(1, 6,0.5)
ER_NO3_crroefSAr_Nrx = list(map(test_NrxCF, ER_NO3))
ER_NO3_crroefSAr_Nrx

In [ ]:
def test_NryCF(ER_NO3):
   return 0.038542920294814*(0.00039*ER_NO3 + 0.00091)/(0.3*ER_NO3 + 0.7) - 0.00730670622979132
#R_straw = range(1, 6,0.5)
ER_NO3_crroefSAr_Nry = list(map(test_NryCF, ER_NO3))
ER_NO3_crroefSAr_Nry

In [ ]:
def test_NaccxCF(ER_NO3):
   return -5.0667590911025*(0.00039*ER_NO3 + 0.00091)/(0.3*ER_NO3 + 0.7) + 0.96781711518274
#R_straw = range(1, 6,0.5)
ER_NO3_crroefSAr_Naccx = list(map(test_NaccxCF,ER_NO3))
ER_NO3_crroefSAr_Naccx

In [ ]:
def test_NaccyCF(ER_NO3):
   return -0.038542920294814*(0.00039*ER_NO3 + 0.00091)/(0.3*ER_NO3 + 0.7) + 0.00736220081905026
#R_straw = range(1, 6,0.5)
ER_NO3_crroefSAr_Naccy = list(map(test_NaccyCF, ER_NO3))
ER_NO3_crroefSAr_Naccy

##### ER_NH3

In [ ]:
ER_NH3 = np.linspace(0.743,0.946,1001,endpoint=True).tolist()
ER_NH3

In [ ]:
def test_NOFON(ER_NH3):
   return 0.0716898317483541*(0.0001725*ER_NH3 + 0.0004025)/(0.3*ER_NH3 + 0.7) + 0.000453946675938552

#test_GWPCF=0.147*(0.151*R_straw + 1.1821055)/(R_straw + 1.2105) - 0.0112032821 + 0.00100148205176129/sqrt(R_straw)
#R_straw = range(1, 6,0.5)
ER_NH3_crroefSAr_NOFON = list(map(test_NOFON, ER_NH3))
ER_NH3_crroefSAr_NOFON

In [ ]:
def test_NrxCF(ER_NH3):
   return 5.0667590911025*(0.0001725*ER_NH3 + 0.0004025)/(0.3*ER_NH3 + 0.7) - 0.956848530343889
#R_straw = range(1, 6,0.5)
ER_NH3_crroefSAr_Nrx = list(map(test_NrxCF, ER_NH3))
ER_NH3_crroefSAr_Nrx

In [ ]:
def test_NryCF(ER_NH3):
   return 0.038542920294814*(0.0001725*ER_NH3 + 0.0004025)/(0.3*ER_NH3 + 0.7) - 0.00727876261257758
#R_straw = range(1, 6,0.5)
ER_NH3_crroefSAr_Nry = list(map(test_NryCF, ER_NH3))
ER_NH3_crroefSAr_Nry

In [ ]:
def test_NaccxCF(ER_NH3):
   return -5.0667590911025*(0.0001725*ER_NH3 + 0.0004025)/(0.3*ER_NH3 + 0.7) + 0.964143714841691
#R_straw = range(1, 6,0.5)
ER_NH3_crroefSAr_Naccx = list(map(test_NaccxCF,ER_NH3))
ER_NH3_crroefSAr_Naccx

In [ ]:
def test_NaccyCF(ER_NH3):
   return -0.038542920294814*(0.0001725*ER_NH3 + 0.0004025)/(0.3*ER_NH3 + 0.7) + 0.00733425720183652
#R_straw = range(1, 6,0.5)
ER_NH3_crroefSAr_Naccy = list(map(test_NaccyCF, ER_NH3))
ER_NH3_crroefSAr_Naccy

In [ ]:
df_corref_res = pd.DataFrame()

df_corref_res['R_straw_crroefSAr_GWP']=R_straw_crroefSAr_GWP
df_corref_res['R_straw_crroefSAr_WF']=R_straw_crroefSAr_WF


df_corref_res['R_cob_crroefSAr_GWP']=R_cob_crroefSAr_GWP
df_corref_res['R_cob_crroefSAr_WF']=R_cob_crroefSAr_WF

df_corref_res['R_Straw_return_crroefSAr_GWP']=R_Straw_return_crroefSAr_GWP

df_corref_res['CO2_maizep_total_crroefSAr_GWP']=CO2_maizep_total_crroefSAr_GWP

df_corref_res['Elec_C_crroefSAr_GWP']=Elec_C_crroefSAr_GWP

df_corref_res['Diesel_C_crroefSAr_GWP']=Diesel_C_crroefSAr_GWP

df_corref_res['Elec_milling_crroefSAr_GWP']=Elec_milling_crroefSAr_GWP

df_corref_res['R_biojet_straw_crroefSAr_GWP']=R_biojet_straw_crroefSAr_GWP
df_corref_res['R_biojet_straw_crroefSAr_WF']=R_biojet_straw_crroefSAr_WF

df_corref_res['DR_feed_crroefSAr_GWP']=DR_feed_crroefSAr_GWP

df_corref_res['ER_N2O_crroefSAr_GWP']=ER_N2O_crroefSAr_GWP



df_corref_res['R_straw_crroefSAr_N2x']=R_straw_crroefSAr_N2x
df_corref_res['R_straw_crroefSAr_N2y']=R_straw_crroefSAr_N2y
df_corref_res['R_straw_crroefSAr_Nrx']=R_straw_crroefSAr_Nrx
df_corref_res['R_straw_crroefSAr_Nry']=R_straw_crroefSAr_Nry
df_corref_res['R_straw_crroefSAr_Naccx']=R_straw_crroefSAr_Naccx
df_corref_res['R_straw_crroefSAr_Naccy']=R_straw_crroefSAr_Naccy

df_corref_res['R_cob_crroefSAr_N2x']=R_cob_crroefSAr_N2x
df_corref_res['R_cob_crroefSAr_N2y']=R_cob_crroefSAr_N2y
df_corref_res['R_cob_crroefSAr_Nrx']=R_cob_crroefSAr_Nrx
df_corref_res['R_cob_crroefSAr_Nry']=R_cob_crroefSAr_Nry
df_corref_res['R_cob_crroefSAr_Naccx']=R_cob_crroefSAr_Naccx
df_corref_res['R_cob_crroefSAr_Naccy']=R_cob_crroefSAr_Naccy

df_corref_res['ER_NO2_N2_crroefSAr_N2x']=ER_NO2_N2_crroefSAr_N2x
df_corref_res['ER_NO2_N2_crroefSAr_N2y']=ER_NO2_N2_crroefSAr_N2y
df_corref_res['ER_NO2_N2_crroefSAr_Naccx']=ER_NO2_N2_crroefSAr_Naccx
df_corref_res['ER_NO2_N2_crroefSAr_Naccy']=ER_NO2_N2_crroefSAr_Naccy

df_corref_res['R_Straw_return_crroefSAr_N2x']=R_Straw_return_crroefSAr_N2x
df_corref_res['R_Straw_return_crroefSAr_N2y']=R_Straw_return_crroefSAr_N2y
df_corref_res['R_Straw_return_crroefSAr_Nrx']=R_Straw_return_crroefSAr_Nrx
df_corref_res['R_Straw_return_crroefSAr_Nry']=R_Straw_return_crroefSAr_Nry
df_corref_res['R_Straw_return_crroefSAr_Naccx']=R_Straw_return_crroefSAr_Naccx
df_corref_res['R_Straw_return_crroefSAr_Naccy']=R_Straw_return_crroefSAr_Naccy

df_corref_res['Elec_C_crroefSAr_N2x']=Elec_C_crroefSAr_N2x
df_corref_res['Elec_C_crroefSAr_N2y']=Elec_C_crroefSAr_N2y
df_corref_res['Elec_C_crroefSAr_Nrx']=Elec_C_crroefSAr_Nrx
df_corref_res['Elec_C_crroefSAr_Nry']=Elec_C_crroefSAr_Nry
df_corref_res['Elec_C_crroefSAr_Naccx']=Elec_C_crroefSAr_Naccx
df_corref_res['Elec_C_crroefSAr_Naccy']=Elec_C_crroefSAr_Naccy

df_corref_res['Diesel_C_crroefSAr_N2x']=Diesel_C_crroefSAr_N2x
df_corref_res['Diesel_C_crroefSAr_N2y']=Diesel_C_crroefSAr_N2y
df_corref_res['Diesel_C_crroefSAr_Nrx']=Diesel_C_crroefSAr_Nrx
df_corref_res['Diesel_C_crroefSAr_Nry']=Diesel_C_crroefSAr_Nry
df_corref_res['Diesel_C_crroefSAr_Naccx']=Diesel_C_crroefSAr_Naccx
df_corref_res['Diesel_C_crroefSAr_Naccy']=Diesel_C_crroefSAr_Naccy

df_corref_res['Elec_milling_crroefSAr_N2x']=Elec_milling_crroefSAr_N2x
df_corref_res['Elec_milling_crroefSAr_N2y']=Elec_milling_crroefSAr_N2y
df_corref_res['Elec_milling_crroefSAr_Nrx']=Elec_milling_crroefSAr_Nrx
df_corref_res['Elec_milling_crroefSAr_Nry']=Elec_milling_crroefSAr_Nry
df_corref_res['Elec_milling_crroefSAr_Naccx']=Elec_milling_crroefSAr_Naccx
df_corref_res['Elec_milling_crroefSAr_Naccy']=Elec_milling_crroefSAr_Naccy

df_corref_res['R_biojet_straw_crroefSAr_N2x']=R_biojet_straw_crroefSAr_N2x
df_corref_res['R_biojet_straw_crroefSAr_N2y']=R_biojet_straw_crroefSAr_N2y
df_corref_res['R_biojet_straw_crroefSAr_Nrx']=R_biojet_straw_crroefSAr_Nrx
df_corref_res['R_biojet_straw_crroefSAr_Nry']=R_biojet_straw_crroefSAr_Nry
df_corref_res['R_biojet_straw_crroefSAr_Naccx']=R_biojet_straw_crroefSAr_Naccx
df_corref_res['R_biojet_straw_crroefSAr_Naccy']=R_biojet_straw_crroefSAr_Naccy


df_corref_res['CoRu_catalyst_crroefSAr_N2x']=CoRu_catalyst_crroefSAr_N2x
df_corref_res['CoRu_catalyst_crroefSAr_N2y']=CoRu_catalyst_crroefSAr_N2y
df_corref_res['CoRu_catalyst_crroefSAr_Nrx']=CoRu_catalyst_crroefSAr_Nrx
df_corref_res['CoRu_catalyst_crroefSAr_Naccx']=CoRu_catalyst_crroefSAr_Naccx
df_corref_res['CoRu_catalyst_crroefSAr_Naccy']=CoRu_catalyst_crroefSAr_Naccy

df_corref_res['NOx_fuelem_crroefSAr_Nrx']=NOx_fuelem_crroefSAr_Nrx
df_corref_res['NOx_fuelem_crroefSAr_Nry']=NOx_fuelem_crroefSAr_Nry
df_corref_res['NOx_fuelem_crroefSAr_Naccx']=NOx_fuelem_crroefSAr_Naccx
df_corref_res['NOx_fuelem_crroefSAr_Naccy']=NOx_fuelem_crroefSAr_Naccy

df_corref_res['DR_feed_crroefSAr_N2x']=DR_feed_crroefSAr_N2x
df_corref_res['DR_feed_crroefSAr_N2y']=DR_feed_crroefSAr_N2y
df_corref_res['DR_feed_crroefSAr_Nrx']=DR_feed_crroefSAr_Nrx
df_corref_res['DR_feed_crroefSAr_Naccx']=DR_feed_crroefSAr_Naccx
df_corref_res['DR_feed_crroefSAr_Naccy']=DR_feed_crroefSAr_Naccy

df_corref_res['ER_N2O_crroefSAr_N2x']=ER_N2O_crroefSAr_N2x
df_corref_res['ER_N2O_crroefSAr_N2y']=ER_N2O_crroefSAr_N2y
df_corref_res['ER_N2O_crroefSAr_Nrx']=ER_N2O_crroefSAr_Nrx
df_corref_res['ER_N2O_crroefSAr_Nry']=ER_N2O_crroefSAr_Nry
df_corref_res['ER_N2O_crroefSAr_Naccx']=ER_N2O_crroefSAr_Naccx
df_corref_res['ER_N2O_crroefSAr_Naccy']=ER_N2O_crroefSAr_Naccy

df_corref_res['D_train_jetfuel_crroefSAr_N2x']=D_train_jetfuel_crroefSAr_N2x
df_corref_res['D_train_jetfuel_crroefSAr_N2y']=D_train_jetfuel_crroefSAr_N2y
df_corref_res['D_train_jetfuel_crroefSAr_Nrx']=D_train_jetfuel_crroefSAr_Nrx
df_corref_res['D_train_jetfuel_crroefSAr_Naccx']=D_train_jetfuel_crroefSAr_Naccx
df_corref_res['D_train_jetfuel_crroefSAr_Naccy']=D_train_jetfuel_crroefSAr_Naccy

df_corref_res['NiW_catalyst_crroefSAr_N2x']=NiW_catalyst_crroefSAr_N2x
df_corref_res['NiW_catalyst_crroefSAr_N2y']=NiW_catalyst_crroefSAr_N2y
df_corref_res['NiW_catalyst_crroefSAr_Nrx']=NiW_catalyst_crroefSAr_Nrx
df_corref_res['NiW_catalyst_crroefSAr_Nry']=NiW_catalyst_crroefSAr_Nry
df_corref_res['NiW_catalyst_crroefSAr_Naccx']=NiW_catalyst_crroefSAr_Naccx
df_corref_res['NiW_catalyst_crroefSAr_Naccy']=NiW_catalyst_crroefSAr_Naccy

df_corref_res['P_steel_crroefSAr_N2x']=P_steel_crroefSAr_N2x
df_corref_res['P_steel_crroefSAr_N2y']=P_steel_crroefSAr_N2y
df_corref_res['P_steel_crroefSAr_Nrx']=P_steel_crroefSAr_Nrx
df_corref_res['P_steel_crroefSAr_Naccx']=P_steel_crroefSAr_Naccx
df_corref_res['P_steel_crroefSAr_Naccy']=P_steel_crroefSAr_Naccy

df_corref_res['Life_FTplant_crroefSAr_N2x']=Life_FTplant_crroefSAr_N2x
df_corref_res['Life_FTplant_crroefSAr_N2y']=Life_FTplant_crroefSAr_N2y
df_corref_res['Life_FTplant_crroefSAr_Nrx']=Life_FTplant_crroefSAr_Nrx
df_corref_res['Life_FTplant_crroefSAr_Naccx']=Life_FTplant_crroefSAr_Naccx
df_corref_res['Life_FTplant_crroefSAr_Naccy']=Life_FTplant_crroefSAr_Naccy

df_corref_res['ER_NO3_crroefSAr_Nrx']=ER_NO3_crroefSAr_Nrx
df_corref_res['ER_NO3_crroefSAr_Nry']=ER_NO3_crroefSAr_Nry
df_corref_res['ER_NO3_crroefSAr_Naccx']=ER_NO3_crroefSAr_Naccx
df_corref_res['ER_NO3_crroefSAr_Naccy']=ER_NO3_crroefSAr_Naccy

df_corref_res['ER_NH3_crroefSAr_Nrx']=ER_NH3_crroefSAr_Nrx
df_corref_res['ER_NH3_crroefSAr_Nry']=ER_NH3_crroefSAr_Nry
df_corref_res['ER_NH3_crroefSAr_Naccx']=ER_NH3_crroefSAr_Naccx
df_corref_res['ER_NH3_crroefSAr_Naccy']=ER_NH3_crroefSAr_Naccy

df_corref_res.iloc[:, 13:109]*=100




df_corref_res['Zero']=[0]*(1001)
df_corref_res['GWP_LC_D']=[0.083727507064988]*(1001)
df_corref_res['WF_LC_D']=[0.53050475493782]*(1001)
df_corref_res['N2x_LC_D']=[-0.00729518449780133*100]*(1001)
df_corref_res['N2y_LC_D']=[-0.0000554945892589371*100]*(1001)
df_corref_res['Nrx_LC_D']=[-0.953935143866505*100]*(1001)
df_corref_res['Nry_LC_D']=[-0.00725660043340807*100]*(1001)
df_corref_res['Naccx_LC_D']=[0.961230328364307*100]*(1001)
df_corref_res['Naccy_LC_D']=[0.007312095022667*100]*(1001)

# df = df.apply(pd.to_numeric)
#df_corref_res["DR_feed_crroefSAr_GWP"] = pd.to_numeric(df_corref_res["DR_feed_crroefSAr_GWP"].iloc[1:1000, :])
df_corref_res 

In [ ]:
df_corref_resON = pd.DataFrame()

df_corref_resON['R_straw_crroefSAr_NIFON']=R_straw_crroefSAr_NIFON
df_corref_resON['R_straw_crroefSAr_NOFON']=R_straw_crroefSAr_NOFON

df_corref_resON['R_cob_crroefSAr_NIFON']=R_cob_crroefSAr_NIFON
df_corref_resON['R_cob_crroefSAr_NOFON']=R_cob_crroefSAr_NOFON

df_corref_resON['R_Straw_return_crroefSAr_NOFON']=R_Straw_return_crroefSAr_NOFON

df_corref_resON['Elec_C_crroefSAr_NOFON']=Elec_C_crroefSAr_NOFON

df_corref_resON['Diesel_C_crroefSAr_NOFON']=Diesel_C_crroefSAr_NOFON

df_corref_resON['Elec_milling_crroefSAr_NOFON']=Elec_milling_crroefSAr_NOFON

df_corref_resON['R_biojet_straw_crroefSAr_NIFON']=R_biojet_straw_crroefSAr_NIFON
df_corref_resON['R_biojet_straw_crroefSAr_NOFON']=R_biojet_straw_crroefSAr_NOFON

df_corref_resON['NOx_fuelem_crroefSAr_NOFON']=NOx_fuelem_crroefSAr_NOFON

df_corref_resON['ER_N2O_crroefSAr_NOFON']=ER_N2O_crroefSAr_NOFON

df_corref_resON['NiW_catalyst_crroefSAr_NOFON']=NiW_catalyst_crroefSAr_NOFON

df_corref_resON['ER_NO3_crroefSAr_NOFON']=ER_NO3_crroefSAr_NOFON

df_corref_resON['ER_NH3_crroefSAr_NOFON']=ER_NH3_crroefSAr_NOFON


df_corref_resON['NIF_ON_LC_D']=[0.0139924451353328]*(1001)


# df = df.apply(pd.to_numeric)
#df_corref_res["DR_feed_crroefSAr_GWP"] = pd.to_numeric(df_corref_res["DR_feed_crroefSAr_GWP"].iloc[1:1000, :])
df_corref_resON 

In [ ]:
#df_corref_resON.to_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\UA and SA_each paramaters\ON LCI paramaters sensitivity result.xlsx', sheet_name='ON_LCI_paras_corref', index = True)

#### LCI paramaters sensitivity result import after calculation

In [ ]:
Nry_LC_D = df_corref_res['Nry_LC_D'].to_list()
Nry_LC_D

In [ ]:
NIF_ON_LC_D = df_corref_resON['NIF_ON_LC_D'].to_list()
NIF_ON_LC_D

In [ ]:
df_corref_res2= pd.read_excel(r'C:\Users\DELL\学术\N sustainability\bio-jet fuel production\UA and SA_each paramaters\LCI paramaters sensitivity result_v4.xlsx', sheet_name='LCI_paras_corref') # , skiprows=[0]
df_corref_res2.drop('Unnamed: 0', axis=1, inplace=True)
df_corref_res2

#### sustainability indicators sensitivity result plot

In [ ]:
import matplotlib.pyplot as plt
import numpy
import os
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

plt.rcParams['text.usetex'] =  False #     True
plt.rcParams['axes.unicode_minus'] = False

plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['mathtext.it'] = 'STIXGeneral:italic'
plt.rcParams['mathtext.bf'] = 'STIXGeneral:italic:bold'


plt.figure(figsize=(9,6))

# Benchmark Mean values 
#plt.axvline(x=0, color='k',linestyle='--', linewidth=0.5) 
#plt.axhline(y=0, color='k', linestyle='--', linewidth=0.5) 

# plotting box plot legend 
#box0=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'red', marker = 'None', s = 100)
#box1=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'yellowgreen', marker = '$▬$', s = 100)
#box2=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'deepskyblue', marker = '$▬$', s = 100)
#box3=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'orange', marker = '$▬$', s = 100)
#box4=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'blueviolet', marker = '$▬$', s = 100)

# plotting standard deviation curve
#SDc0=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'red', marker='None', s = 500)
#SDc1=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'lime', marker = '_', s = 500)
#SDc2=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'blue', marker = '_', s = 500)
#SDc3=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'yellow', marker = '_', s = 500)
#SDc4=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'lightpink', marker = '_', s = 500)

markersh = ["1","x","v","2","^","p","3","<","*","+","4","h"]
#  1	x	v	2	^	p	3	<	*	+	4	h	>	P	X	d	o	|	_

# plotting point shape legend 
ps1 =plt.scatter(x=R_cob_crroefSAr_GWP[120], y=R_cob_crroefSAr_WF[120], c = 'black',marker='*', s = marker_S-30)
ps2 =plt.scatter(x=R_cob_crroefSAr_GWP[120], y=R_cob_crroefSAr_WF[120], c = 'black',marker='1', s = marker_S-30)
ps3 =plt.scatter(x=R_cob_crroefSAr_GWP[120], y=R_cob_crroefSAr_WF[120], c = 'black',marker='x', s = marker_S-30)
ps4 =plt.scatter(x=R_cob_crroefSAr_GWP[120], y=R_cob_crroefSAr_WF[120], c = 'black',marker='^', s = marker_S-30)
ps5 =plt.scatter(x=R_cob_crroefSAr_GWP[120], y=R_cob_crroefSAr_WF[120], c = 'black',marker='<', s = marker_S-30)
ps6 =plt.scatter(x=R_cob_crroefSAr_GWP[120], y=R_cob_crroefSAr_WF[120], c = 'black',marker='p', s = marker_S-30)
ps7 =plt.scatter(x=R_cob_crroefSAr_GWP[120], y=R_cob_crroefSAr_WF[120], c = 'black',marker='3', s = marker_S-30)
ps8 =plt.scatter(x=R_cob_crroefSAr_GWP[120], y=R_cob_crroefSAr_WF[120], c = 'black',marker='2', s = marker_S-30)
ps9 =plt.scatter(x=R_cob_crroefSAr_GWP[120], y=R_cob_crroefSAr_WF[120], c = 'black',marker='h', s = marker_S-30)
ps10=plt.scatter(x=R_cob_crroefSAr_GWP[120], y=R_cob_crroefSAr_WF[120], c = 'black',marker='>', s = marker_S-30)

legend1=plt.legend([ps1,ps2,ps3,ps4,ps5,ps6,ps7,ps8,ps9,ps10], #r"$\mathit{F_{\alpha}}$"
                   ['$\mathit{R_{straw-fuel}}$', '$\mathit{R_{straw}}$','$\mathit{R_{cob}}$',
                    '$\mathit{CO_{2-maize}}$','$\mathit{Elec_{treat}}$','$\mathit{Elec_{pre-treat}}$',
                    '$\mathit{Diesel_{pre-treat}}$','$\mathit{R_{straw-soil}}$','$\mathit{R_{feed}}$',
                    '$\mathit{ER_{N_{2}O}}$',
                   ],
           loc='best' ,prop={'family':'Times New Roman', 'size':17}, frameon=False, title="Life cycle\nparamaters",title_fontsize=18, bbox_to_anchor=(1.32, 0.95),
                      )



#legend2=plt.legend([ps0],['Paramaters value'],
#           loc='best', fontsize=16, frameon=False, bbox_to_anchor=(1.35, 0.95),
#                      )
    
#plt.gca().add_artist(legend1)




marker_S =60+30
# plotting GWP amd WF           
pcb1 = plt.scatter(data=df_corref_res2, x='R_biojet_straw_crroefSAr_GWP', y='R_biojet_straw_crroefSAr_WF',s=marker_S, marker='*',linewidth=0,cmap='RdPu',c=R_straw, label=None)
sns.scatterplot(data=df_corref_res2, x='R_straw_crroefSAr_GWP', y='R_straw_crroefSAr_WF',s=marker_S, marker='1',cmap='RdPu',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='R_cob_crroefSAr_GWP', y='R_cob_crroefSAr_WF',s=marker_S, marker='x',linewidth=1,cmap='RdPu',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='CO2_maizep_total_crroefSAr_GWP', y='WF_LC_D',s=marker_S, marker='^',linewidth=0,cmap='RdPu',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='Elec_milling_crroefSAr_GWP', y='WF_LC_D',s=marker_S, marker='<',linewidth=0,cmap='RdPu',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='Elec_C_crroefSAr_GWP', y='WF_LC_D',s=marker_S+25, marker='p',linewidth=0,cmap='RdPu',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='Diesel_C_crroefSAr_GWP', y='WF_LC_D',s=marker_S+25, marker='3',linewidth=1,cmap='RdPu',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='R_Straw_return_crroefSAr_GWP', y='WF_LC_D',s=marker_S, marker='2',linewidth=1,cmap='RdPu',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='DR_feed_crroefSAr_GWP', y='WF_LC_D',s=marker_S-30, marker='h',linewidth=0,cmap='RdPu',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='ER_N2O_crroefSAr_GWP', y='WF_LC_D',s=marker_S-25, marker='>',linewidth=0,cmap='RdPu',c=R_straw)

#pcb1._legend.remove()
#plt.scatter(x, y, c=x, s=500, cmap='gray_r')
#pcb1.get_legend().set_visible(False)
#plt.text(x=1500, y=-4.2, s="Quadrant 4: NIF<0",fontsize=18, color='black') #alpha=0.7,
#plt.text(x=1500, y=-1.95, s="Quadrant 3: NIF>0",fontsize=18, color='black')
plt.text(x=0.1005, y=0.2998, s="Highest parameter value",fontsize=16, color='black')
plt.text(x=0.0605, y=0.2998, s="Lowest parameter value",fontsize=16, color='black') 


#sns.scatterplot(data=df_corref_res, x='R_straw_crroefSAr_GWP', y='R_straw_crroefSAr_WF',color='green',s=25)
#sns.scatterplot(data=df_NFI_PD, x='NFI_ON_x(per)', y='NFI_ON_y(per)',color='blue',s=25, marker='>')
#sns.scatterplot(data=df_NFI_PD, x='NFI_Nacc_x(per)', y='NFI_Nacc_y(per)',color='red',s=25, marker=',')




#legend1=plt.legend(['Inert nitrogen flow index', 'Reactive nitrogen flow index', 'Nitrogen accumulation index', 'X axis: EF v3.0 climate change total \nY aixs EF v3.0 deprivation-weighted water consumption'],
#           loc='best', fontsize=16, frameon=False, title="Indicators",title_fontsize=17, bbox_to_anchor=(0.62, -0.17))




plt.title('Carbon, water footprint sensitivity to life cycle parameters', fontsize=22,fontname="Times New Roman")
plt.xlabel('Carbon footprint (kg CO$_{2-eq}$)', fontsize=20,fontname="Times New Roman")
plt.ylabel('Water footprint (m$^3$ world eq. water deprived)', fontsize=20,fontname="Times New Roman")

axf = plt.gca()
fig = plt.gca()
#axf.yaxis.set_label_coords(-0.06, 0.51) # first x second y
#axf.xaxis.set_label_coords(0.51, -0.025)
# Relative percentage of N flow change or N accumulation (%)\n 
# Magnitude of N flow change or N accumulation (%)\n 

# rotation=45,horizontalalignment="right"
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
mpl.rc('font',family='Times New Roman')

#plt.legend(loc='best', fontsize=16) 
#for i in range(hdi_df.shape[0]):
#          plt.text(hdi_df.gni_pc[i], y=hdi_df.life_ex[i], s=hdi_df.Country[i], alpha=0.8)




#plt.yscale('log')
#plt.yscale('symlog')
#plt.xscale('log')
#plt.xscale('symlog')


#plt.ylim(ymin = 0)
#panel.ellipse(NFI_IN_x_per, NFI_IN_y_per, lwd = 2, col="green", robust=FALSE,  level=0.65)

# draw standard deviation curve
nstd = 1
ax = plt.subplot(111)

x1=NFI_IN_x_per
y1=NFI_IN_y_per

cov = np.cov(x1, y1)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w1, h1 = 2 * nstd * np.sqrt(vals)
ell1 = Ellipse(xy=(np.mean(x1), np.mean(y1)),
              width=w1, height=h1,
              angle=theta, color='lime', linewidth = 2)
ell1.set_facecolor('none')
#ax.add_artist(ell1)

x2=NFI_ON_x_per
y2=NFI_ON_y_per

cov = np.cov(x2, y2)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w2, h2 = 2 * nstd * np.sqrt(vals)
ell2 = Ellipse(xy=(np.mean(x2), np.mean(y2)),
              width=w2, height=h2,
              angle=theta, color='blue')
ell2.set_facecolor('none')
#ax.add_artist(ell2)

#sns.scatterplot(x='-7.81376479400426', y='-987.4604094144358',color='yellow',s=100)


x3=NFI_Nacc_x_per
y3=NFI_Nacc_y_per

cov = np.cov(x3, y3)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w3, h3 = 2 * nstd * np.sqrt(vals)
ell3 = Ellipse(xy=(np.mean(x3), np.mean(y3)),
              width=w3, height=h3,
              angle=theta, color='yellow') #,lw=1.5
ell3.set_facecolor('none')
#ax.add_artist(ell3)

#ax = sns.regplot(x=x1, y=y1, ci=95,scatter=False,color='lime')
#ax = sns.regplot(x=x2, y=y2, ci=95,scatter=False,color='blue')
#ax = sns.regplot(x=x3, y=y3, ci=95,scatter=False,color='red')



# Percentile and PD scale


#plt.plot([100,0],[0,0],'green')


x4=MC_GWP_scores
y4=MC_water_scores

cov = np.cov(x4, y4)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w4, h4 = 2 * nstd * np.sqrt(vals)
ell4 = Ellipse(xy=(np.mean(x4), np.mean(y4)),
              width=w4, height=h4,
              angle=theta, color='lightpink',lw=2)
ell4.set_facecolor('none')
#ax.add_artist(ell4)

#im = plt.imshow(data=df_corref_res, interpolation='nearest', cmap="PuRd")
#cax = fig.add_axes([0.27, 0.8, 0.5, 0.05])

axins = inset_axes(axf,
                    width="100%",  
                    height="5%",
                    loc='lower center',
                    borderpad=-7
                   )

cb1=plt.colorbar(pcb1,orientation='horizontal',cax=axins) #
cb1.set_ticks([])
#x4_ave = np.average(x4)
#y4_ave = np.average(y4)
#Mean4=plt.scatter(x4_ave, y4_ave, c = 'purple', marker = '$☉$', s = 200)

for label in cb1.ax.xaxis.get_ticklabels()[::1]:
    label.set_visible(False)

mpl.rcParams["figure.dpi"] = 900

plt.show()


#output_path = os.path.join('/Users/DELL/学术/N sustainability/bio-jet fuel production',str(NFI_MC_nexus)+'.png')
#plt.savefig()


In [ ]:
import matplotlib.pyplot as plt
import numpy
import os

plt.rcParams['text.usetex'] =  False #     True
plt.rcParams['axes.unicode_minus'] = False

plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['mathtext.it'] = 'STIXGeneral:italic'
plt.rcParams['mathtext.bf'] = 'STIXGeneral:italic:bold'

plt.figure(figsize=(9,6))

# Benchmark Mean values 
#plt.axvline(x=0, color='k',linestyle='--', linewidth=0.5) 
#plt.axhline(y=0, color='k', linestyle='--', linewidth=0.5) 

# plotting box plot legend 
#box0=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'red', marker = 'None', s = 100)
#box1=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'yellowgreen', marker = '$▬$', s = 100)
#box2=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'deepskyblue', marker = '$▬$', s = 100)
#box3=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'orange', marker = '$▬$', s = 100)
#box4=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'blueviolet', marker = '$▬$', s = 100)

# plotting standard deviation curve
#SDc0=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'red', marker='None', s = 500)
#SDc1=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'lime', marker = '_', s = 500)
#SDc2=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'blue', marker = '_', s = 500)
#SDc3=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'yellow', marker = '_', s = 500)
#SDc4=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'lightpink', marker = '_', s = 500)

markersh = ["1","x","v","2","^","p","3","<","*","+","4","h"]
#  1	x	v	2	^	p	3	<	*	+	4	h	>	P	X	d	o	|	_
# plotting point shape legend 
ps0 =plt.scatter(x=Diesel_C_crroefSAr_N2x[120]*100, y=Diesel_C_crroefSAr_N2y[120]*100, c = 'black',marker='1', s = marker_S-30)
ps1 =plt.scatter(x=Diesel_C_crroefSAr_N2x[120]*100, y=Diesel_C_crroefSAr_N2y[120]*100, c = 'black',marker='x', s = marker_S-30)
ps2 =plt.scatter(x=Diesel_C_crroefSAr_N2x[120]*100, y=Diesel_C_crroefSAr_N2y[120]*100, c = 'black',marker='v', s = marker_S-30)
ps3 =plt.scatter(x=Diesel_C_crroefSAr_N2x[120]*100, y=Diesel_C_crroefSAr_N2y[120]*100, c = 'black',marker='2', s = marker_S-30)
ps4 =plt.scatter(x=Diesel_C_crroefSAr_N2x[120]*100, y=Diesel_C_crroefSAr_N2y[120]*100, c = 'black',marker='p', s = marker_S-30)
ps5 =plt.scatter(x=Diesel_C_crroefSAr_N2x[120]*100, y=Diesel_C_crroefSAr_N2y[120]*100, c = 'black',marker='3', s = marker_S-30)
ps6 =plt.scatter(x=Diesel_C_crroefSAr_N2x[120]*100, y=Diesel_C_crroefSAr_N2y[120]*100, c = 'black',marker='<', s = marker_S-30)
ps7 =plt.scatter(x=Diesel_C_crroefSAr_N2x[120]*100, y=Diesel_C_crroefSAr_N2y[120]*100, c = 'black',marker='*', s = marker_S-30)
ps8 =plt.scatter(x=Diesel_C_crroefSAr_N2x[120]*100, y=Diesel_C_crroefSAr_N2y[120]*100, c = 'black',marker='+', s = marker_S-30)
ps9 =plt.scatter(x=Diesel_C_crroefSAr_N2x[120]*100, y=Diesel_C_crroefSAr_N2y[120]*100, c = 'black',marker='h', s = marker_S-30)
ps10=plt.scatter(x=Diesel_C_crroefSAr_N2x[120]*100, y=Diesel_C_crroefSAr_N2y[120]*100, c = 'black',marker='>', s = marker_S-30)
ps11=plt.scatter(x=Diesel_C_crroefSAr_N2x[120]*100, y=Diesel_C_crroefSAr_N2y[120]*100, c = 'black',marker='P', s = marker_S-30)
ps12=plt.scatter(x=Diesel_C_crroefSAr_N2x[120]*100, y=Diesel_C_crroefSAr_N2y[120]*100, c = 'black',marker='X', s = marker_S-30)
ps13=plt.scatter(x=Diesel_C_crroefSAr_N2x[120]*100, y=Diesel_C_crroefSAr_N2y[120]*100, c = 'black',marker='d', s = marker_S-30)
ps14=plt.scatter(x=Diesel_C_crroefSAr_N2x[120]*100, y=Diesel_C_crroefSAr_N2y[120]*100, c = 'black',marker='o', s = marker_S-30)



legend1=plt.legend([ps0,ps1,ps2,ps3,ps4,ps5,ps6,ps7,ps8,ps9,ps10,ps11,ps12,ps13,ps14], #r"$\mathit{F_{\alpha}}$"
                   ['$\mathit{R_{straw}}$', '$\mathit{R_{cob}}$','$\mathit{R_{N_{2}O-N_{2}}}$',
                    '$\mathit{R_{straw-soil}}$','$\mathit{Elec_{pre-treat}}$','$\mathit{Diesel_{pre-treat}}$',
                    '$\mathit{Elec_{treat}}$','$\mathit{R_{straw-fuel}}$','$\mathit{R_{Co-Ru}}$',
                    '$\mathit{R_{feed}}$','$\mathit{ER_{N_{2}O}}$','$\mathit{D_{railway}}$',
                    '$\mathit{R_{Ni-W}}$','$\mathit{P_{steel}}$','$\mathit{Life_{plant}}$'
                   ],
           loc='best' ,prop={'family':'Times New Roman', 'size':17}, frameon=False, title="Life cycle\nparamaters",title_fontsize=18, bbox_to_anchor=(1.32, 1.08),
                      )



marker_S =60+40
# plotting NFI N2          
sns.scatterplot(data=df_corref_res2, x='R_biojet_straw_crroefSAr_N2x', y='R_biojet_straw_crroefSAr_N2y',s=marker_S, marker='*',linewidth=0,cmap='BuGn',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='ER_NO2_N2_crroefSAr_N2x', y='ER_NO2_N2_crroefSAr_N2y',s=marker_S, marker='v',linewidth=0,cmap='BuGn',c=R_straw)
pcb1 = plt.scatter(data=df_corref_res2, x='R_straw_crroefSAr_N2x', y='R_straw_crroefSAr_N2y',s=marker_S, marker='1',cmap='BuGn',c=R_straw, label=None)
sns.scatterplot(data=df_corref_res2, x='R_cob_crroefSAr_N2x', y='R_cob_crroefSAr_N2y',s=marker_S+25, marker='x',linewidth=1,cmap='BuGn',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='CoRu_catalyst_crroefSAr_N2x', y='CoRu_catalyst_crroefSAr_N2y',s=marker_S+60, marker='+',linewidth=1,cmap='BuGn',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='Elec_milling_crroefSAr_N2x', y='Elec_milling_crroefSAr_N2y',s=marker_S+50, marker='<',linewidth=0,cmap='BuGn',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='Elec_C_crroefSAr_N2x', y='Elec_C_crroefSAr_N2y',s=marker_S+95, marker='p',linewidth=0,cmap='BuGn',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='Diesel_C_crroefSAr_N2x', y='Diesel_C_crroefSAr_N2y',s=marker_S+35, marker='3',linewidth=1,cmap='BuGn',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='NiW_catalyst_crroefSAr_N2x', y='NiW_catalyst_crroefSAr_N2y',s=marker_S, marker='X',linewidth=0,cmap='BuGn',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='R_Straw_return_crroefSAr_N2x', y='R_Straw_return_crroefSAr_N2y',s=marker_S+5, marker='2',linewidth=1,cmap='BuGn',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='D_train_jetfuel_crroefSAr_N2x', y='D_train_jetfuel_crroefSAr_N2y',s=marker_S, marker='P',linewidth=0,cmap='BuGn',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='DR_feed_crroefSAr_N2x', y='DR_feed_crroefSAr_N2y',s=marker_S-15, marker='h',linewidth=0,cmap='BuGn',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='Life_FTplant_crroefSAr_N2x', y='Life_FTplant_crroefSAr_N2y',s=marker_S-40, marker='o',linewidth=0,cmap='BuGn',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='ER_N2O_crroefSAr_N2x', y='ER_N2O_crroefSAr_N2y',s=marker_S-25, marker='>',linewidth=0,cmap='BuGn',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='P_steel_crroefSAr_N2x', y='P_steel_crroefSAr_N2y',s=marker_S-25, marker='d',linewidth=0,cmap='BuGn',c=R_straw)





#plt.scatter(x, y, c=x, s=500, cmap='gray_r')


plt.text(x=-0.553, y=-0.008415, s="Highest parameter value",fontsize=16, color='black')
plt.text(x=-0.913, y=-0.008415, s="Lowest parameter value",fontsize=16, color='black') 


#sns.scatterplot(data=df_corref_res, x='R_straw_crroefSAr_GWP', y='R_straw_crroefSAr_WF',color='green',s=25)
#sns.scatterplot(data=df_NFI_PD, x='NFI_ON_x(per)', y='NFI_ON_y(per)',color='blue',s=25, marker='>')
#sns.scatterplot(data=df_NFI_PD, x='NFI_Nacc_x(per)', y='NFI_Nacc_y(per)',color='red',s=25, marker=',')


# plotting point shape legend 
#ps0=plt.scatter(x=NFI_IN_x_per[0], y=NFI_IN_y_per[0], c = 'red',marker='None', s = 21)
#ps1=plt.scatter(x=NFI_IN_x_per[0], y=NFI_IN_y_per[0], c = 'green', s = 21)
#ps2=plt.scatter(x=NFI_ON_x_per[0], y=NFI_IN_y_per[0], c = 'blue', marker = '>', s = 21)
#ps3=plt.scatter(x=NFI_Nacc_x_per[0], y=NFI_Nacc_y_per[0], c = 'red', marker = ',', s = 21)
#ps4=plt.scatter(x=MC_GWP_scores[0], y=MC_water_scores[0], c = 'fuchsia', marker = 's', s = 21)



#legend1=plt.legend(['Inert nitrogen flow index', 'Reactive nitrogen flow index', 'Nitrogen accumulation index', 'X axis: EF v3.0 climate change total \nY aixs EF v3.0 deprivation-weighted water consumption'],
#           loc='best', fontsize=16, frameon=False, title="Indicators",title_fontsize=17, bbox_to_anchor=(0.62, -0.17))

#legend1=plt.legend([ps1,SDc1,Mean1],['Monte Carlo sampling outcome distribution', 'Standard deviation curve of sampling result','Arithmetic mean of sampling outcome'],
#           loc='best', fontsize=16, frameon=False, title="Inert nitrogen flow index",title_fontsize=17, bbox_to_anchor=(0.62, -0.17),
#                      )


plt.title('Inert N flow indicator sensitivity to life cycle parameters', fontsize=22,fontname="Times New Roman")
plt.xlabel('Relative percentage of inert N flow change (%)', fontsize=20,fontname="Times New Roman")
plt.ylabel('Magnitude of inert N flow change (%)', fontsize=20,fontname="Times New Roman")

axf = plt.gca()
#axf.yaxis.set_label_coords(-0.06, 0.5) # first x second y
#axf.xaxis.set_label_coords(0.49, -0.03)

# Relative percentage of N flow change or N accumulation (%)\n 
# Magnitude of N flow change or N accumulation (%)\n 

# rotation=45,horizontalalignment="right"
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
mpl.rc('font',family='Times New Roman')

#plt.legend(loc='best', fontsize=16) 
#for i in range(hdi_df.shape[0]):
#          plt.text(hdi_df.gni_pc[i], y=hdi_df.life_ex[i], s=hdi_df.Country[i], alpha=0.8)




#plt.yscale('log')
#plt.yscale('symlog')
#plt.xscale('log')
#plt.xscale('symlog')
#plt.gca().invert_yaxis()

#plt.ylim(ymin = 0)
#panel.ellipse(NFI_IN_x_per, NFI_IN_y_per, lwd = 2, col="green", robust=FALSE,  level=0.65)

# draw standard deviation curve
nstd = 1
ax = plt.subplot(111)

x1=NFI_IN_x_per
y1=NFI_IN_y_per

cov = np.cov(x1, y1)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w1, h1 = 2 * nstd * np.sqrt(vals)
ell1 = Ellipse(xy=(np.mean(x1), np.mean(y1)),
              width=w1, height=h1,
              angle=theta, color='lime', linewidth = 2)
ell1.set_facecolor('none')
#ax.add_artist(ell1)

x2=NFI_ON_x_per
y2=NFI_ON_y_per

cov = np.cov(x2, y2)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w2, h2 = 2 * nstd * np.sqrt(vals)
ell2 = Ellipse(xy=(np.mean(x2), np.mean(y2)),
              width=w2, height=h2,
              angle=theta, color='blue')
ell2.set_facecolor('none')
#ax.add_artist(ell2)

#sns.scatterplot(x='-7.81376479400426', y='-987.4604094144358',color='yellow',s=100)


x3=NFI_Nacc_x_per
y3=NFI_Nacc_y_per

cov = np.cov(x3, y3)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w3, h3 = 2 * nstd * np.sqrt(vals)
ell3 = Ellipse(xy=(np.mean(x3), np.mean(y3)),
              width=w3, height=h3,
              angle=theta, color='yellow') #,lw=1.5
ell3.set_facecolor('none')
#ax.add_artist(ell3)

#ax = sns.regplot(x=x1, y=y1, ci=95,scatter=False,color='lime')
#ax = sns.regplot(x=x2, y=y2, ci=95,scatter=False,color='blue')
#ax = sns.regplot(x=x3, y=y3, ci=95,scatter=False,color='red')



# Percentile and PD scale


#plt.plot([100,0],[0,0],'green')


x4=MC_GWP_scores
y4=MC_water_scores

cov = np.cov(x4, y4)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w4, h4 = 2 * nstd * np.sqrt(vals)
ell4 = Ellipse(xy=(np.mean(x4), np.mean(y4)),
              width=w4, height=h4,
              angle=theta, color='lightpink',lw=2)
ell4.set_facecolor('none')
#ax.add_artist(ell4)

axins = inset_axes(axf,
                    width="100%",  
                    height="5%",
                    loc='lower center',
                    borderpad=-7.1
                   )

cb1=plt.colorbar(pcb1,orientation='horizontal',cax=axins) #
cb1.set_ticks([])
#x4_ave = np.average(x4)
#y4_ave = np.average(y4)
#Mean4=plt.scatter(x4_ave, y4_ave, c = 'purple', marker = '$☉$', s = 200)

#for label in cb1.ax.xaxis.get_ticklabels()[::1]:
#    label.set_visible(False)

mpl.rcParams["figure.dpi"] = 900

plt.show()


#output_path = os.path.join('/Users/DELL/学术/N sustainability/bio-jet fuel production',str(NFI_MC_nexus)+'.png')
#plt.savefig()


In [ ]:
import matplotlib.pyplot as plt
import numpy
import os

plt.rcParams['text.usetex'] =  False #     True
plt.rcParams['axes.unicode_minus'] = False

plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['mathtext.it'] = 'STIXGeneral:italic'
plt.rcParams['mathtext.bf'] = 'STIXGeneral:italic:bold'

plt.figure(figsize=(9,6))

# Benchmark Mean values 
#plt.axvline(x=0, color='k',linestyle='--', linewidth=0.5) 
#plt.axhline(y=0, color='k', linestyle='--', linewidth=0.5) 

# plotting box plot legend 
#box0=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'red', marker = 'None', s = 100)
#box1=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'yellowgreen', marker = '$▬$', s = 100)
#box2=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'deepskyblue', marker = '$▬$', s = 100)
#box3=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'orange', marker = '$▬$', s = 100)
#box4=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'blueviolet', marker = '$▬$', s = 100)

# plotting standard deviation curve
#SDc0=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'red', marker='None', s = 500)
#SDc1=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'lime', marker = '_', s = 500)
#SDc2=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'blue', marker = '_', s = 500)
#SDc3=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'yellow', marker = '_', s = 500)
#SDc4=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'lightpink', marker = '_', s = 500)

markersh = ["1","x","v","2","^","p","3","<","*","+","4","h"]
#  1	x	v	2	^	p	3	<	*	+	4	h	>	P	X	d	o	|	_
# plotting point shape legend 
ps0 =plt.scatter(x=P_steel_crroefSAr_Nrx[120]*100, y=Nry_LC_D[120], c = 'black',marker='1', s = marker_S-20)
ps1 =plt.scatter(x=P_steel_crroefSAr_Nrx[120]*100, y=Nry_LC_D[120], c = 'black',marker='x', s = marker_S-20)
ps2 =plt.scatter(x=P_steel_crroefSAr_Nrx[120]*100, y=Nry_LC_D[120], c = 'black',marker='2', s = marker_S-20)
ps3 =plt.scatter(x=P_steel_crroefSAr_Nrx[120]*100, y=Nry_LC_D[120], c = 'black',marker='p', s = marker_S-20)
ps4 =plt.scatter(x=P_steel_crroefSAr_Nrx[120]*100, y=Nry_LC_D[120], c = 'black',marker='3', s = marker_S-20)
ps5 =plt.scatter(x=P_steel_crroefSAr_Nrx[120]*100, y=Nry_LC_D[120], c = 'black',marker='<', s = marker_S-20)
ps6 =plt.scatter(x=P_steel_crroefSAr_Nrx[120]*100, y=Nry_LC_D[120], c = 'black',marker='*', s = marker_S-20)
ps7 =plt.scatter(x=P_steel_crroefSAr_Nrx[120]*100, y=Nry_LC_D[120], c = 'black',marker='+', s = marker_S-20)
ps8 =plt.scatter(x=P_steel_crroefSAr_Nrx[120]*100, y=Nry_LC_D[120], c = 'black',marker='4', s = marker_S-20)
ps9 =plt.scatter(x=P_steel_crroefSAr_Nrx[120]*100, y=Nry_LC_D[120], c = 'black',marker='h', s = marker_S-20)
ps10=plt.scatter(x=P_steel_crroefSAr_Nrx[120]*100, y=Nry_LC_D[120], c = 'black',marker='>', s = marker_S-20)
ps11=plt.scatter(x=P_steel_crroefSAr_Nrx[120]*100, y=Nry_LC_D[120], c = 'black',marker='P', s = marker_S-20)
ps12=plt.scatter(x=P_steel_crroefSAr_Nrx[120]*100, y=Nry_LC_D[120], c = 'black',marker='X', s = marker_S-20)
ps13=plt.scatter(x=P_steel_crroefSAr_Nrx[120]*100, y=Nry_LC_D[120], c = 'black',marker='d', s = marker_S-20)
ps14=plt.scatter(x=P_steel_crroefSAr_Nrx[120]*100, y=Nry_LC_D[120], c = 'black',marker='o', s = marker_S-20)
#ps15=plt.scatter(x=P_steel_crroefSAr_Nrx[120]*100, y=Nry_LC_D[120], c = 'black',marker='|', s = marker_S-30)
#ps16=plt.scatter(x=P_steel_crroefSAr_Nrx[120]*100, y=Nry_LC_D[120], c = 'black',marker='_', s = marker_S-30)
# ,ps15,ps16

legend1=plt.legend([ps0,ps1,ps2,ps3,ps4,ps5,ps6,ps7,ps8,ps9,ps10,ps11,ps12,ps13,ps14], #r"$\mathit{F_{\alpha}}$"
                   ['$\mathit{R_{straw}}$', '$\mathit{R_{cob}}$','$\mathit{R_{straw-soil}}$',
                    '$\mathit{Elec_{pre-treat}}$','$\mathit{Diesel_{pre-treat}}$','$\mathit{Elec_{treat}}$',
                    '$\mathit{R_{straw-fuel}}$','$\mathit{R_{Co-Ru}}$','$\mathit{NO_{x-fuel}}$',
                    '$\mathit{R_{feed}}$','$\mathit{ER_{N_{2}O}}$','$\mathit{D_{railway}}$',
                    '$\mathit{R_{Ni-W}}$','$\mathit{P_{steel}}$','$\mathit{Life_{plant}}$',
                    #'$\mathit{ER_{NO_{3}}}$','$\mathit{ER_{NH_{3}}}$'
                   ],
           loc='best' ,prop={'family':'Times New Roman', 'size':17}, frameon=False, title="Life cycle\nparamaters",title_fontsize=18, bbox_to_anchor=(1.32, 1.08),
                      )



marker_S =60+80
# plotting NFI Nr          
sns.scatterplot(data=df_corref_res2, x='NOx_fuelem_crroefSAr_Nrx', y='NOx_fuelem_crroefSAr_Nry',s=marker_S-50, marker='4',linewidth=1,cmap='GnBu',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='R_biojet_straw_crroefSAr_Nrx', y='R_biojet_straw_crroefSAr_Nry',s=marker_S, marker='*',linewidth=0,cmap='GnBu',c=R_straw)
pcb1 = plt.scatter(data=df_corref_res2, x='R_straw_crroefSAr_Nrx', y='R_straw_crroefSAr_Nry',s=marker_S, marker='1',cmap='GnBu',c=R_straw, label=None)
sns.scatterplot(data=df_corref_res2, x='R_cob_crroefSAr_Nrx', y='R_cob_crroefSAr_Nry',s=marker_S+30, marker='x',linewidth=1,cmap='GnBu',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='Elec_milling_crroefSAr_Nrx', y='Elec_milling_crroefSAr_Nry',s=marker_S+60, marker='<',linewidth=0,cmap='GnBu',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='CoRu_catalyst_crroefSAr_Nrx', y='Nry_LC_D',s=marker_S+30, marker='+',linewidth=1,cmap='GnBu',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='R_Straw_return_crroefSAr_Nrx', y='R_Straw_return_crroefSAr_Nry',s=marker_S+5, marker='2',linewidth=1,cmap='GnBu',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='Elec_C_crroefSAr_Nrx', y='Elec_C_crroefSAr_Nry',s=marker_S+95, marker='p',linewidth=0,cmap='GnBu',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='NiW_catalyst_crroefSAr_Nrx', y='NiW_catalyst_crroefSAr_Nry',s=marker_S, marker='X',linewidth=0,cmap='GnBu',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='Diesel_C_crroefSAr_Nrx', y='Diesel_C_crroefSAr_Nry',s=marker_S-20, marker='3',linewidth=1,cmap='GnBu',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='D_train_jetfuel_crroefSAr_Nrx', y='Nry_LC_D',s=marker_S, marker='P',linewidth=0,cmap='GnBu',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='DR_feed_crroefSAr_Nrx', y='Nry_LC_D',s=marker_S-30, marker='h',linewidth=0,cmap='GnBu',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='Life_FTplant_crroefSAr_Nrx', y='Nry_LC_D',s=marker_S-70, marker='o',linewidth=0,cmap='GnBu',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='ER_N2O_crroefSAr_Nrx', y='ER_N2O_crroefSAr_Nry',s=marker_S-70, marker='>',linewidth=0,cmap='GnBu',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='P_steel_crroefSAr_Nrx', y='Nry_LC_D',s=marker_S-95, marker='d',linewidth=0,cmap='GnBu',c=R_straw)
#sns.scatterplot(data=df_corref_res, x='ER_NO3_crroefSAr_Nrx', y='ER_NO3_crroefSAr_Nry',s=marker_S+50, marker='|',linewidth=1,cmap='GnBu',c=R_straw)
#sns.scatterplot(data=df_corref_res, x='ER_NH3_crroefSAr_Nrx', y='ER_NH3_crroefSAr_Nry',s=marker_S+50, marker='_',linewidth=1,cmap='GnBu',c=R_straw)

#plt.scatter(x, y, c=x, s=500, cmap='gray_r')


plt.text(x=-95.115, y=-1.057, s="Highest parameter value",fontsize=16, color='black')
plt.text(x=-95.688, y=-1.057, s="Lowest parameter value",fontsize=16, color='black') 


#sns.scatterplot(data=df_corref_res, x='R_straw_crroefSAr_GWP', y='R_straw_crroefSAr_WF',color='green',s=25)
#sns.scatterplot(data=df_NFI_PD, x='NFI_ON_x(per)', y='NFI_ON_y(per)',color='blue',s=25, marker='>')
#sns.scatterplot(data=df_NFI_PD, x='NFI_Nacc_x(per)', y='NFI_Nacc_y(per)',color='red',s=25, marker=',')


# plotting point shape legend 
#ps0=plt.scatter(x=NFI_IN_x_per[0], y=NFI_IN_y_per[0], c = 'red',marker='None', s = 21)
#ps1=plt.scatter(x=NFI_IN_x_per[0], y=NFI_IN_y_per[0], c = 'green', s = 21)
#ps2=plt.scatter(x=NFI_ON_x_per[0], y=NFI_IN_y_per[0], c = 'blue', marker = '>', s = 21)
#ps3=plt.scatter(x=NFI_Nacc_x_per[0], y=NFI_Nacc_y_per[0], c = 'red', marker = ',', s = 21)
#ps4=plt.scatter(x=MC_GWP_scores[0], y=MC_water_scores[0], c = 'fuchsia', marker = 's', s = 21)



#legend1=plt.legend(['Inert nitrogen flow index', 'Reactive nitrogen flow index', 'Nitrogen accumulation index', 'X axis: EF v3.0 climate change total \nY aixs EF v3.0 deprivation-weighted water consumption'],
#           loc='best', fontsize=16, frameon=False, title="Indicators",title_fontsize=17, bbox_to_anchor=(0.62, -0.17))

#legend1=plt.legend([ps1,SDc1,Mean1],['Monte Carlo sampling outcome distribution', 'Standard deviation curve of sampling result','Arithmetic mean of sampling outcome'],
#           loc='best', fontsize=16, frameon=False, title="Inert nitrogen flow index",title_fontsize=17, bbox_to_anchor=(0.62, -0.17),
#                      )


plt.title('Reactive N flow indicator sensitivity to life cycle parameters', fontsize=22,fontname="Times New Roman") #,y=1.04
plt.xlabel('Relative percentage of reactive N flow change (%)', fontsize=20,fontname="Times New Roman")
plt.ylabel('Magnitude of reactive N flow change (%)', fontsize=20,fontname="Times New Roman")

axf = plt.gca()
#axf.yaxis.set_label_coords(-0.043, 0.5) # first x second y
#axf.xaxis.set_label_coords(0.49, -0.03)

# Relative percentage of N flow change or N accumulation (%)\n 
# Magnitude of N flow change or N accumulation (%)\n 

# rotation=45,horizontalalignment="right"
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
mpl.rc('font',family='Times New Roman')

#plt.legend(loc='best', fontsize=16) 
#for i in range(hdi_df.shape[0]):
#          plt.text(hdi_df.gni_pc[i], y=hdi_df.life_ex[i], s=hdi_df.Country[i], alpha=0.8)




#plt.yscale('log')
#plt.yscale('symlog')
#plt.xscale('log')
#plt.xscale('symlog')
#plt.gca().invert_yaxis()

#plt.ylim(ymin = 0)
#panel.ellipse(NFI_IN_x_per, NFI_IN_y_per, lwd = 2, col="green", robust=FALSE,  level=0.65)

# draw standard deviation curve
nstd = 1
ax = plt.subplot(111)

x1=NFI_IN_x_per
y1=NFI_IN_y_per

cov = np.cov(x1, y1)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w1, h1 = 2 * nstd * np.sqrt(vals)
ell1 = Ellipse(xy=(np.mean(x1), np.mean(y1)),
              width=w1, height=h1,
              angle=theta, color='lime', linewidth = 2)
ell1.set_facecolor('none')
#ax.add_artist(ell1)

x2=NFI_ON_x_per
y2=NFI_ON_y_per

cov = np.cov(x2, y2)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w2, h2 = 2 * nstd * np.sqrt(vals)
ell2 = Ellipse(xy=(np.mean(x2), np.mean(y2)),
              width=w2, height=h2,
              angle=theta, color='blue')
ell2.set_facecolor('none')
#ax.add_artist(ell2)

#sns.scatterplot(x='-7.81376479400426', y='-987.4604094144358',color='yellow',s=100)


x3=NFI_Nacc_x_per
y3=NFI_Nacc_y_per

cov = np.cov(x3, y3)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w3, h3 = 2 * nstd * np.sqrt(vals)
ell3 = Ellipse(xy=(np.mean(x3), np.mean(y3)),
              width=w3, height=h3,
              angle=theta, color='yellow') #,lw=1.5
ell3.set_facecolor('none')
#ax.add_artist(ell3)

#ax = sns.regplot(x=x1, y=y1, ci=95,scatter=False,color='lime')
#ax = sns.regplot(x=x2, y=y2, ci=95,scatter=False,color='blue')
#ax = sns.regplot(x=x3, y=y3, ci=95,scatter=False,color='red')



# Percentile and PD scale


#plt.plot([100,0],[0,0],'green')


x4=MC_GWP_scores
y4=MC_water_scores

cov = np.cov(x4, y4)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w4, h4 = 2 * nstd * np.sqrt(vals)
ell4 = Ellipse(xy=(np.mean(x4), np.mean(y4)),
              width=w4, height=h4,
              angle=theta, color='lightpink',lw=2)
ell4.set_facecolor('none')
#ax.add_artist(ell4)

axins = inset_axes(axf,
                    width="100%",  
                    height="5%",
                    loc='lower center',
                    borderpad=-7
                
                   )

cb1=plt.colorbar(pcb1,orientation='horizontal',cax=axins) #
cb1.set_ticks([])
#x4_ave = np.average(x4)
#y4_ave = np.average(y4)
#Mean4=plt.scatter(x4_ave, y4_ave, c = 'purple', marker = '$☉$', s = 200)

#for label in cb1.ax.xaxis.get_ticklabels()[::1]:
#    label.set_visible(False)

mpl.rcParams["figure.dpi"] = 900

plt.show()


#output_path = os.path.join('/Users/DELL/学术/N sustainability/bio-jet fuel production',str(NFI_MC_nexus)+'.png')
#plt.savefig()


In [ ]:
import matplotlib.pyplot as plt
import numpy
import os

plt.rcParams['text.usetex'] =  False #     True
plt.rcParams['axes.unicode_minus'] = False

plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['mathtext.it'] = 'STIXGeneral:italic'
plt.rcParams['mathtext.bf'] = 'STIXGeneral:italic:bold'

plt.figure(figsize=(9,6))

# Benchmark Mean values 
#plt.axvline(x=0, color='k',linestyle='--', linewidth=0.5) 
#plt.axhline(y=0, color='k', linestyle='--', linewidth=0.5) 

# plotting box plot legend 
#box0=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'red', marker = 'None', s = 100)
#box1=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'yellowgreen', marker = '$▬$', s = 100)
#box2=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'deepskyblue', marker = '$▬$', s = 100)
#box3=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'orange', marker = '$▬$', s = 100)
#box4=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'blueviolet', marker = '$▬$', s = 100)

# plotting standard deviation curve
#SDc0=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'red', marker='None', s = 500)
#SDc1=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'lime', marker = '_', s = 500)
#SDc2=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'blue', marker = '_', s = 500)
#SDc3=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'yellow', marker = '_', s = 500)
#SDc4=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'lightpink', marker = '_', s = 500)

markersh = ["1","x","v","2","^","p","3","<","*","+","4","h"]
#  1	x	v	2	^	p	3	<	*	+	4	h	>	P	X	d	o	|	_
# plotting point shape legend 
ps0 =plt.scatter(x=R_cob_crroefSAr_Naccx[120]*100, y=R_cob_crroefSAr_Naccy[120]*100, c = 'black',marker='1', s = marker_S-30)
ps1 =plt.scatter(x=R_cob_crroefSAr_Naccx[120]*100, y=R_cob_crroefSAr_Naccy[120]*100, c = 'black',marker='x', s = marker_S-30)
ps2=plt.scatter(x=R_cob_crroefSAr_Naccx[120]*100, y=R_cob_crroefSAr_Naccy[120]*100, c = 'black',marker='v', s = marker_S-30)
ps3 =plt.scatter(x=R_cob_crroefSAr_Naccx[120]*100, y=R_cob_crroefSAr_Naccy[120]*100, c = 'black',marker='2', s = marker_S-30)
ps4 =plt.scatter(x=R_cob_crroefSAr_Naccx[120]*100, y=R_cob_crroefSAr_Naccy[120]*100, c = 'black',marker='p', s = marker_S-30)
ps5 =plt.scatter(x=R_cob_crroefSAr_Naccx[120]*100, y=R_cob_crroefSAr_Naccy[120]*100, c = 'black',marker='3', s = marker_S-30)
ps6 =plt.scatter(x=R_cob_crroefSAr_Naccx[120]*100, y=R_cob_crroefSAr_Naccy[120]*100, c = 'black',marker='<', s = marker_S-30)
ps7 =plt.scatter(x=R_cob_crroefSAr_Naccx[120]*100, y=R_cob_crroefSAr_Naccy[120]*100, c = 'black',marker='*', s = marker_S-30)
ps8 =plt.scatter(x=R_cob_crroefSAr_Naccx[120]*100, y=R_cob_crroefSAr_Naccy[120]*100, c = 'black',marker='+', s = marker_S-30)
ps9 =plt.scatter(x=R_cob_crroefSAr_Naccx[120]*100, y=R_cob_crroefSAr_Naccy[120]*100, c = 'black',marker='4', s = marker_S-30)
ps10 =plt.scatter(x=R_cob_crroefSAr_Naccx[120]*100, y=R_cob_crroefSAr_Naccy[120]*100, c = 'black',marker='h', s = marker_S-30)
ps11=plt.scatter(x=R_cob_crroefSAr_Naccx[120]*100, y=R_cob_crroefSAr_Naccy[120]*100, c = 'black',marker='>', s = marker_S-30)
ps12=plt.scatter(x=R_cob_crroefSAr_Naccx[120]*100, y=R_cob_crroefSAr_Naccy[120]*100, c = 'black',marker='P', s = marker_S-30)
ps13=plt.scatter(x=R_cob_crroefSAr_Naccx[120]*100, y=R_cob_crroefSAr_Naccy[120]*100, c = 'black',marker='X', s = marker_S-30)
ps14=plt.scatter(x=R_cob_crroefSAr_Naccx[120]*100, y=R_cob_crroefSAr_Naccy[120]*100, c = 'black',marker='d', s = marker_S-30)
ps15=plt.scatter(x=R_cob_crroefSAr_Naccx[120]*100, y=R_cob_crroefSAr_Naccy[120]*100, c = 'black',marker='o', s = marker_S-30)
#ps16=plt.scatter(x=P_steel_crroefSAr_Nrx[120]*100, y=Nry_LC_D[120], c = 'black',marker='_', s = marker_S-30)
# ,ps15,ps16

legend1=plt.legend([ps0,ps1,ps2,ps3,ps4,ps5,ps6,ps7,ps8,ps9,ps10,ps11,ps12,ps13,ps14,ps15], #r"$\mathit{F_{\alpha}}$"
                   ['$\mathit{R_{straw}}$', '$\mathit{R_{cob}}$','$\mathit{R_{N_{2}O-N_{2}}}$','$\mathit{R_{straw-soil}}$',
                    '$\mathit{Elec_{pre-treat}}$','$\mathit{Diesel_{pre-treat}}$','$\mathit{Elec_{treat}}$',
                    '$\mathit{R_{straw-fuel}}$','$\mathit{R_{Co-Ru}}$','$\mathit{NO_{x-fuel}}$',
                    '$\mathit{R_{feed}}$','$\mathit{ER_{N_{2}O}}$','$\mathit{D_{railway}}$',
                    '$\mathit{R_{Ni-W}}$','$\mathit{P_{steel}}$','$\mathit{Life_{plant}}$',
                  #'$\mathit{ER_{NO_{3}}}$','$\mathit{ER_{NH_{3}}}$'
                   ],
           loc='best' ,prop={'family':'Times New Roman', 'size':17}, frameon=False, title="Life cycle\nparamaters",title_fontsize=18, bbox_to_anchor=(1.335, 1.09),
                      )



marker_S =80+80
# plotting NFI Nacc          
sns.scatterplot(data=df_corref_res2, x='ER_NO2_N2_crroefSAr_Naccx', y='ER_NO2_N2_crroefSAr_Naccy',s=marker_S+90, marker='v',linewidth=0,cmap='OrRd',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='NOx_fuelem_crroefSAr_Naccx', y='NOx_fuelem_crroefSAr_Naccy',s=marker_S-50, marker='4',linewidth=1,cmap='OrRd',c=R_straw)
pcb1 = plt.scatter(data=df_corref_res2, x='R_straw_crroefSAr_Naccx', y='R_straw_crroefSAr_Naccy',s=marker_S, marker='1',cmap='OrRd',c=R_straw, label=None)
sns.scatterplot(data=df_corref_res2, x='R_cob_crroefSAr_Naccx', y='R_cob_crroefSAr_Naccy',s=marker_S+30, marker='x',linewidth=1,cmap='OrRd',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='R_biojet_straw_crroefSAr_Naccx', y='R_biojet_straw_crroefSAr_Naccy',s=marker_S, marker='*',linewidth=0,cmap='OrRd',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='R_Straw_return_crroefSAr_Naccx', y='R_Straw_return_crroefSAr_Naccy',s=marker_S+5, marker='2',linewidth=1,cmap='OrRd',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='Elec_milling_crroefSAr_Naccx', y='Elec_milling_crroefSAr_Naccy',s=marker_S+75, marker='<',linewidth=0,cmap='OrRd',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='Elec_C_crroefSAr_Naccx', y='Elec_C_crroefSAr_Naccy',s=marker_S+35, marker='p',linewidth=0,cmap='OrRd',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='NiW_catalyst_crroefSAr_Naccx', y='NiW_catalyst_crroefSAr_Naccy',s=marker_S, marker='X',linewidth=0,cmap='OrRd',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='Diesel_C_crroefSAr_Naccx', y='Diesel_C_crroefSAr_Naccy',s=marker_S-20, marker='3',linewidth=1,cmap='OrRd',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='D_train_jetfuel_crroefSAr_Naccx', y='D_train_jetfuel_crroefSAr_Naccy',s=marker_S, marker='P',linewidth=0,cmap='OrRd',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='CoRu_catalyst_crroefSAr_Naccx', y='CoRu_catalyst_crroefSAr_Naccy',s=marker_S-15, marker='+',linewidth=1,cmap='OrRd',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='DR_feed_crroefSAr_Naccx', y='DR_feed_crroefSAr_Naccy',s=marker_S-40, marker='h',linewidth=0,cmap='OrRd',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='Life_FTplant_crroefSAr_Naccx', y='Life_FTplant_crroefSAr_Naccy',s=marker_S-90, marker='o',linewidth=0,cmap='OrRd',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='ER_N2O_crroefSAr_Naccx', y='ER_N2O_crroefSAr_Naccy',s=marker_S-70, marker='>',linewidth=0,cmap='OrRd',c=R_straw)
sns.scatterplot(data=df_corref_res2, x='P_steel_crroefSAr_Naccx', y='P_steel_crroefSAr_Naccy',s=marker_S-115, marker='d',linewidth=0,cmap='OrRd',c=R_straw)

#plt.scatter(x, y, c=x, s=500, cmap='gray_r')


plt.text(x=96.2105, y=0.432, s="Highest parameter value",fontsize=16, color='black')
plt.text(x=95.733, y=0.432, s="Lowest parameter value",fontsize=16, color='black') 


#sns.scatterplot(data=df_corref_res, x='R_straw_crroefSAr_GWP', y='R_straw_crroefSAr_WF',color='green',s=25)
#sns.scatterplot(data=df_NFI_PD, x='NFI_ON_x(per)', y='NFI_ON_y(per)',color='blue',s=25, marker='>')
#sns.scatterplot(data=df_NFI_PD, x='NFI_Nacc_x(per)', y='NFI_Nacc_y(per)',color='red',s=25, marker=',')


# plotting point shape legend 
#ps0=plt.scatter(x=NFI_IN_x_per[0], y=NFI_IN_y_per[0], c = 'red',marker='None', s = 21)
#ps1=plt.scatter(x=NFI_IN_x_per[0], y=NFI_IN_y_per[0], c = 'green', s = 21)
#ps2=plt.scatter(x=NFI_ON_x_per[0], y=NFI_IN_y_per[0], c = 'blue', marker = '>', s = 21)
#ps3=plt.scatter(x=NFI_Nacc_x_per[0], y=NFI_Nacc_y_per[0], c = 'red', marker = ',', s = 21)
#ps4=plt.scatter(x=MC_GWP_scores[0], y=MC_water_scores[0], c = 'fuchsia', marker = 's', s = 21)



#legend1=plt.legend(['Inert nitrogen flow index', 'Reactive nitrogen flow index', 'Nitrogen accumulation index', 'X axis: EF v3.0 climate change total \nY aixs EF v3.0 deprivation-weighted water consumption'],
#           loc='best', fontsize=16, frameon=False, title="Indicators",title_fontsize=17, bbox_to_anchor=(0.62, -0.17))

#legend1=plt.legend([ps1,SDc1,Mean1],['Monte Carlo sampling outcome distribution', 'Standard deviation curve of sampling result','Arithmetic mean of sampling outcome'],
#           loc='best', fontsize=16, frameon=False, title="Inert nitrogen flow index",title_fontsize=17, bbox_to_anchor=(0.62, -0.17),
#                      )


plt.title('N accumulation indicator sensitivity to life cycle parameters', fontsize=22,fontname="Times New Roman") #,y=1.04
plt.xlabel('Relative percentage of N accumulation (%)', fontsize=20,fontname="Times New Roman")
plt.ylabel('Magnitude of N accumulation (%)', fontsize=20,fontname="Times New Roman")

axf = plt.gca()
#axf.yaxis.set_label_coords(-0.043, 0.5) # first x second y
#axf.xaxis.set_label_coords(0.49, -0.03)

# Relative percentage of N flow change or N accumulation (%)\n 
# Magnitude of N flow change or N accumulation (%)\n 

# rotation=45,horizontalalignment="right"
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
mpl.rc('font',family='Times New Roman')

#plt.legend(loc='best', fontsize=16) 
#for i in range(hdi_df.shape[0]):
#          plt.text(hdi_df.gni_pc[i], y=hdi_df.life_ex[i], s=hdi_df.Country[i], alpha=0.8)




#plt.yscale('log')
#plt.yscale('symlog')
#plt.xscale('log')
#plt.xscale('symlog')
#plt.gca().invert_yaxis()

#plt.ylim(ymin = 0)
#panel.ellipse(NFI_IN_x_per, NFI_IN_y_per, lwd = 2, col="green", robust=FALSE,  level=0.65)

# draw standard deviation curve
nstd = 1
ax = plt.subplot(111)

x1=NFI_IN_x_per
y1=NFI_IN_y_per

cov = np.cov(x1, y1)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w1, h1 = 2 * nstd * np.sqrt(vals)
ell1 = Ellipse(xy=(np.mean(x1), np.mean(y1)),
              width=w1, height=h1,
              angle=theta, color='lime', linewidth = 2)
ell1.set_facecolor('none')
#ax.add_artist(ell1)

x2=NFI_ON_x_per
y2=NFI_ON_y_per

cov = np.cov(x2, y2)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w2, h2 = 2 * nstd * np.sqrt(vals)
ell2 = Ellipse(xy=(np.mean(x2), np.mean(y2)),
              width=w2, height=h2,
              angle=theta, color='blue')
ell2.set_facecolor('none')
#ax.add_artist(ell2)

#sns.scatterplot(x='-7.81376479400426', y='-987.4604094144358',color='yellow',s=100)


x3=NFI_Nacc_x_per
y3=NFI_Nacc_y_per

cov = np.cov(x3, y3)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w3, h3 = 2 * nstd * np.sqrt(vals)
ell3 = Ellipse(xy=(np.mean(x3), np.mean(y3)),
              width=w3, height=h3,
              angle=theta, color='yellow') #,lw=1.5
ell3.set_facecolor('none')
#ax.add_artist(ell3)

#ax = sns.regplot(x=x1, y=y1, ci=95,scatter=False,color='lime')
#ax = sns.regplot(x=x2, y=y2, ci=95,scatter=False,color='blue')
#ax = sns.regplot(x=x3, y=y3, ci=95,scatter=False,color='red')



# Percentile and PD scale


#plt.plot([100,0],[0,0],'green')


x4=MC_GWP_scores
y4=MC_water_scores

cov = np.cov(x4, y4)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w4, h4 = 2 * nstd * np.sqrt(vals)
ell4 = Ellipse(xy=(np.mean(x4), np.mean(y4)),
              width=w4, height=h4,
              angle=theta, color='lightpink',lw=2)
ell4.set_facecolor('none')
#ax.add_artist(ell4)

axins = inset_axes(axf,
                    width="100%",  
                    height="5%",
                    loc='lower center',
                    borderpad=-7
                
                   )

cb1=plt.colorbar(pcb1,orientation='horizontal',cax=axins) #
cb1.set_ticks([])
#x4_ave = np.average(x4)
#y4_ave = np.average(y4)
#Mean4=plt.scatter(x4_ave, y4_ave, c = 'purple', marker = '$☉$', s = 200)

#for label in cb1.ax.xaxis.get_ticklabels()[::1]:
#    label.set_visible(False)

mpl.rcParams["figure.dpi"] = 900

plt.show()


#output_path = os.path.join('/Users/DELL/学术/N sustainability/bio-jet fuel production',str(NFI_MC_nexus)+'.png')
#plt.savefig()


In [ ]:
import matplotlib.pyplot as plt
import numpy
import os

plt.rcParams['text.usetex'] =  False #     True
plt.rcParams['axes.unicode_minus'] = False

plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['mathtext.it'] = 'STIXGeneral:italic'
plt.rcParams['mathtext.bf'] = 'STIXGeneral:italic:bold'

plt.figure(figsize=(9,6))

# Benchmark Mean values 
#plt.axvline(x=0, color='k',linestyle='--', linewidth=0.5) 
#plt.axhline(y=0, color='k', linestyle='--', linewidth=0.5) 

# plotting box plot legend 
#box0=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'red', marker = 'None', s = 100)
#box1=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'yellowgreen', marker = '$▬$', s = 100)
#box2=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'deepskyblue', marker = '$▬$', s = 100)
#box3=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'orange', marker = '$▬$', s = 100)
#box4=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'blueviolet', marker = '$▬$', s = 100)

# plotting standard deviation curve
#SDc0=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'red', marker='None', s = 500)
#SDc1=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'lime', marker = '_', s = 500)
#SDc2=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'blue', marker = '_', s = 500)
#SDc3=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'yellow', marker = '_', s = 500)
#SDc4=plt.scatter(x=CNW_PC_ave[0], y=CNW_PC_ave[1], c = 'lightpink', marker = '_', s = 500)

markersh = ["1","x","v","2","^","p","3","<","*","+","4","h"]
#  1	x	v	2	^	p	3	<	*	+	4	h	>	P	X	d	o	|	_
# plotting point shape legend 
ps0 =plt.scatter(x=R_straw_crroefSAr_NIFON[300], y=R_straw_crroefSAr_NOFON[300], c = 'black',marker='1', s = marker_S-30)
ps1 =plt.scatter(x=R_straw_crroefSAr_NIFON[300], y=R_straw_crroefSAr_NOFON[300], c = 'black',marker='x', s = marker_S-30)
ps2 =plt.scatter(x=R_straw_crroefSAr_NIFON[300], y=R_straw_crroefSAr_NOFON[300], c = 'black',marker='2', s = marker_S-30)
ps3 =plt.scatter(x=R_straw_crroefSAr_NIFON[300], y=R_straw_crroefSAr_NOFON[300], c = 'black',marker='p', s = marker_S-30)
ps4 =plt.scatter(x=R_straw_crroefSAr_NIFON[300], y=R_straw_crroefSAr_NOFON[300], c = 'black',marker='3', s = marker_S-30)
ps5 =plt.scatter(x=R_straw_crroefSAr_NIFON[300], y=R_straw_crroefSAr_NOFON[300], c = 'black',marker='<', s = marker_S-30)
ps6 =plt.scatter(x=R_straw_crroefSAr_NIFON[300], y=R_straw_crroefSAr_NOFON[300], c = 'black',marker='*', s = marker_S-30)
ps7 =plt.scatter(x=R_straw_crroefSAr_NIFON[300], y=R_straw_crroefSAr_NOFON[300], c = 'black',marker='4', s = marker_S-30)
ps8=plt.scatter(x=R_straw_crroefSAr_NIFON[300], y=R_straw_crroefSAr_NOFON[300], c = 'black',marker='>', s = marker_S-30)
ps9=plt.scatter(x=R_straw_crroefSAr_NIFON[300], y=R_straw_crroefSAr_NOFON[300], c = 'black',marker='X', s = marker_S-30)
#ps10=plt.scatter(x=P_steel_crroefSAr_Nrx[120]*100, y=Nry_LC_D[120], c = 'black',marker='_', s = marker_S-30)
# ,ps15,ps16
#,ps10,ps11,ps12,ps13,ps14,ps15

legend1=plt.legend([ps0,ps1,ps2,ps3,ps4,ps5,ps6,ps7,ps8,ps9], #r"$\mathit{F_{\alpha}}$"
                   ['$\mathit{R_{straw}}$', '$\mathit{R_{cob}}$','$\mathit{R_{straw-soil}}$',
                    '$\mathit{Elec_{pre-treat}}$','$\mathit{Diesel_{pre-treat}}$','$\mathit{Elec_{treat}}$',
                    '$\mathit{R_{straw-fuel}}$','$\mathit{NO_{x-fuel}}$','$\mathit{ER_{N_{2}O}}$',
                    '$\mathit{R_{Ni-W}}$'
                    
                   # '$\mathit{R_{Co-Ru}}$', '$\mathit{R_{feed}}$','$\mathit{D_{railway}}$',
                   #'$\mathit{P_{steel}}$','$\mathit{Life_{plant}}$',
                  #'$\mathit{ER_{NO_{3}}}$','$\mathit{ER_{NH_{3}}}$'
                   ],
           loc='best' ,prop={'family':'Times New Roman', 'size':17}, frameon=False, title="Life cycle\nparamaters",title_fontsize=18, bbox_to_anchor=(1.32, 0.91),
                      )



marker_S =60+80
# plotting NOF and NIF ON          
sns.scatterplot(data=df_corref_resON, x='NIF_ON_LC_D', y='NOx_fuelem_crroefSAr_NOFON',s=marker_S-50, marker='4',linewidth=1,cmap='Wistia',c=R_straw)
pcb1 = plt.scatter(data=df_corref_resON, x='R_straw_crroefSAr_NIFON', y='R_straw_crroefSAr_NOFON',s=marker_S, marker='1',cmap='Wistia',c=R_straw, label=None)
sns.scatterplot(data=df_corref_resON, x='R_biojet_straw_crroefSAr_NIFON', y='R_biojet_straw_crroefSAr_NOFON',s=marker_S, marker='*',linewidth=0,cmap='Wistia',c=R_straw)
sns.scatterplot(data=df_corref_resON, x='R_cob_crroefSAr_NIFON', y='R_cob_crroefSAr_NOFON',s=marker_S+30, marker='x',linewidth=1,cmap='Wistia',c=R_straw)
sns.scatterplot(data=df_corref_resON, x='NIF_ON_LC_D', y='R_Straw_return_crroefSAr_NOFON',s=marker_S+5, marker='2',linewidth=1,cmap='Wistia',c=R_straw)
sns.scatterplot(data=df_corref_resON, x='NIF_ON_LC_D', y='Elec_milling_crroefSAr_NOFON',s=marker_S+75, marker='<',linewidth=0,cmap='Wistia',c=R_straw)
sns.scatterplot(data=df_corref_resON, x='NIF_ON_LC_D', y='NiW_catalyst_crroefSAr_NOFON',s=marker_S, marker='X',linewidth=0,cmap='Wistia',c=R_straw)
sns.scatterplot(data=df_corref_resON, x='NIF_ON_LC_D', y='Elec_C_crroefSAr_NOFON',s=marker_S+10, marker='p',linewidth=0,cmap='Wistia',c=R_straw)
sns.scatterplot(data=df_corref_resON, x='NIF_ON_LC_D', y='Diesel_C_crroefSAr_NOFON',s=marker_S-40, marker='3',linewidth=1,cmap='Wistia',c=R_straw)
sns.scatterplot(data=df_corref_resON, x='NIF_ON_LC_D', y='ER_N2O_crroefSAr_NOFON',s=marker_S-70, marker='>',linewidth=0,cmap='Wistia',c=R_straw)

#plt.scatter(x, y, c=x, s=500, cmap='gray_r')


plt.text(x=0.0161, y=0.000358, s="Highest parameter value",fontsize=16, color='black')
plt.text(x=0.01007, y=0.000358, s="Lowest parameter value",fontsize=16, color='black') 


#sns.scatterplot(data=df_corref_res, x='R_straw_crroefSAr_GWP', y='R_straw_crroefSAr_WF',color='green',s=25)
#sns.scatterplot(data=df_NFI_PD, x='NFI_ON_x(per)', y='NFI_ON_y(per)',color='blue',s=25, marker='>')
#sns.scatterplot(data=df_NFI_PD, x='NFI_Nacc_x(per)', y='NFI_Nacc_y(per)',color='red',s=25, marker=',')


# plotting point shape legend 
#ps0=plt.scatter(x=NFI_IN_x_per[0], y=NFI_IN_y_per[0], c = 'red',marker='None', s = 21)
#ps1=plt.scatter(x=NFI_IN_x_per[0], y=NFI_IN_y_per[0], c = 'green', s = 21)
#ps2=plt.scatter(x=NFI_ON_x_per[0], y=NFI_IN_y_per[0], c = 'blue', marker = '>', s = 21)
#ps3=plt.scatter(x=NFI_Nacc_x_per[0], y=NFI_Nacc_y_per[0], c = 'red', marker = ',', s = 21)
#ps4=plt.scatter(x=MC_GWP_scores[0], y=MC_water_scores[0], c = 'fuchsia', marker = 's', s = 21)



#legend1=plt.legend(['Inert nitrogen flow index', 'Reactive nitrogen flow index', 'Nitrogen accumulation index', 'X axis: EF v3.0 climate change total \nY aixs EF v3.0 deprivation-weighted water consumption'],
#           loc='best', fontsize=16, frameon=False, title="Indicators",title_fontsize=17, bbox_to_anchor=(0.62, -0.17))

#legend1=plt.legend([ps1,SDc1,Mean1],['Monte Carlo sampling outcome distribution', 'Standard deviation curve of sampling result','Arithmetic mean of sampling outcome'],
#           loc='best', fontsize=16, frameon=False, title="Inert nitrogen flow index",title_fontsize=17, bbox_to_anchor=(0.62, -0.17),
#                      )


plt.title('Reactive N input and output \n footprint sensitivity to life cycle parameters', fontsize=22,fontname="Times New Roman") #,y=1.04
plt.xlabel('System reactive N input footprint (kg N/MJ fuel)', fontsize=20,fontname="Times New Roman")
plt.ylabel('System reactive N output footprint \n (kg N/MJ fuel)', fontsize=20,fontname="Times New Roman")

axf = plt.gca()
#axf.yaxis.set_label_coords(-0.043, 0.5) # first x second y
#axf.xaxis.set_label_coords(0.49, -0.03)

# Relative percentage of N flow change or N accumulation (%)\n 
# Magnitude of N flow change or N accumulation (%)\n 

# rotation=45,horizontalalignment="right"
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
mpl.rc('font',family='Times New Roman')

#plt.legend(loc='best', fontsize=16) 
#for i in range(hdi_df.shape[0]):
#          plt.text(hdi_df.gni_pc[i], y=hdi_df.life_ex[i], s=hdi_df.Country[i], alpha=0.8)




#plt.yscale('log')
#plt.yscale('symlog')
#plt.xscale('log')
#plt.xscale('symlog')
#plt.gca().invert_yaxis()

#plt.ylim(ymin = 0)
#panel.ellipse(NFI_IN_x_per, NFI_IN_y_per, lwd = 2, col="green", robust=FALSE,  level=0.65)

# draw standard deviation curve
nstd = 1
ax = plt.subplot(111)

x1=NFI_IN_x_per
y1=NFI_IN_y_per

cov = np.cov(x1, y1)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w1, h1 = 2 * nstd * np.sqrt(vals)
ell1 = Ellipse(xy=(np.mean(x1), np.mean(y1)),
              width=w1, height=h1,
              angle=theta, color='lime', linewidth = 2)
ell1.set_facecolor('none')
#ax.add_artist(ell1)

x2=NFI_ON_x_per
y2=NFI_ON_y_per

cov = np.cov(x2, y2)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w2, h2 = 2 * nstd * np.sqrt(vals)
ell2 = Ellipse(xy=(np.mean(x2), np.mean(y2)),
              width=w2, height=h2,
              angle=theta, color='blue')
ell2.set_facecolor('none')
#ax.add_artist(ell2)

#sns.scatterplot(x='-7.81376479400426', y='-987.4604094144358',color='yellow',s=100)


x3=NFI_Nacc_x_per
y3=NFI_Nacc_y_per

cov = np.cov(x3, y3)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w3, h3 = 2 * nstd * np.sqrt(vals)
ell3 = Ellipse(xy=(np.mean(x3), np.mean(y3)),
              width=w3, height=h3,
              angle=theta, color='yellow') #,lw=1.5
ell3.set_facecolor('none')
#ax.add_artist(ell3)

#ax = sns.regplot(x=x1, y=y1, ci=95,scatter=False,color='lime')
#ax = sns.regplot(x=x2, y=y2, ci=95,scatter=False,color='blue')
#ax = sns.regplot(x=x3, y=y3, ci=95,scatter=False,color='red')



# Percentile and PD scale


#plt.plot([100,0],[0,0],'green')


x4=MC_GWP_scores
y4=MC_water_scores

cov = np.cov(x4, y4)
vals, vecs = eigsorted(cov)
theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
w4, h4 = 2 * nstd * np.sqrt(vals)
ell4 = Ellipse(xy=(np.mean(x4), np.mean(y4)),
              width=w4, height=h4,
              angle=theta, color='lightpink',lw=2)
ell4.set_facecolor('none')
#ax.add_artist(ell4)

axins = inset_axes(axf,
                    width="100%",  
                    height="5%",
                    loc='lower center',
                    borderpad=-7
                
                   )

cb1=plt.colorbar(pcb1,orientation='horizontal',cax=axins) #
cb1.set_ticks([])
#x4_ave = np.average(x4)
#y4_ave = np.average(y4)
#Mean4=plt.scatter(x4_ave, y4_ave, c = 'purple', marker = '$☉$', s = 200)

#for label in cb1.ax.xaxis.get_ticklabels()[::1]:
#    label.set_visible(False)

mpl.rcParams["figure.dpi"] = 900

plt.show()


#output_path = os.path.join('/Users/DELL/学术/N sustainability/bio-jet fuel production',str(NFI_MC_nexus)+'.png')
#plt.savefig()


#### back up code

In [ ]:
simpl_lambdas=sobol_simplify_model(ME_fueltMJ_LCI, biojet_method, min_ratio=1,var_params=para_FTbiojet_list, n=1000)

In [ ]:
compare_simplified(ME_fueltMJ_LCI, biojet_method,simpl_lambdas)

In [ ]:
simpl_lambdas